Сверху currenDate для распределения первый месяц текущего квартала 
При формировании f0 current date ставим по текщему месяцу

In [1]:
import pandas as pd
import datetime
import numpy as np
import os
from pyexcelerate import Workbook
import calendar
pd.set_option('display.max_columns', None)

# pd.set_option('display.max_rows', None)

def save_real_csv(datafr,cols_text,path_save):get_in_backup
    df1 = datafr.copy()
    for i in cols_text:
        df1[i] = '='+ '"'+df1[i].astype('str')+'"'
    df1.to_csv(path_save,encoding='utf-8-sig',sep = ';',decimal = ',')   
    df1 = pd.DataFrame()


def rev_to_dict(ind,find):
#     ind = ind.astype('str')комплекс
    df = pd.DataFrame(index=ind,data=find.values,columns=[find.name])
    df = df.groupby(df.index).first()
    dictt =  df.to_dict('index')
    return dictt


def save_ex(path,df):
    writer = pd.ExcelWriter(path,engine='openpyxl')
    df.to_excel(writer,sheet_name = 'sheet1')
    writer.save()
    
def save_pyex(path,df):
    wb = Workbook()
    wb.new_sheet('sheet1',data =  [df.columns.tolist(),]+df.values.tolist())
    wb.save(path)
    

    
# extract    
def formula_vivod_po_doc_potrebn(x):
    if x['ЦС/ПЗ']=='ПЗ' and x['Дата потребности']<date_start_year:
        return 'Отдельная отработка'
    elif x['ЦС/ПЗ']=='ПЗ'and (x['Вид документа потребности'] in ['AR', 'SB','UB02','Z1' , 'Z3']):
        return 'OK'
    elif x['ЦС/ПЗ']=='ПЗ'and  x['Вид документа потребности']==0 and x['Наименование типа элемента ППМ пополн'] in ['Запас','Зпс/Кл','ЗпсПрк']:
        return 'OK'
    elif x['ЦС/ПЗ']=='ЦС' and (x['Вид документа потребности'] in ['04', 'UB01','UB03','UB04' , 'UB06','Z1','ZUB1','ZUB2','ZUB4']):
        return 'OK'  
    elif x['ЦС/ПЗ']=='ЦС' and x['Вид документа потребности']==0 and x['Краткое наименование элемента ППМ']=='СтрЗап':
        return 'OK'
    else:
        return'Отдельная отработка'
# впр    
def insert_vpr(x,dictt,name):
    try:
        return dictt[x][name]
    except KeyError:
        return 0
   # extract 
def formula_vivod_po_doc_popolneniya(x):   
    if   x['ЦС/ПЗ']=='ПЗ' and x['Вид документа пополнения'] in ['UB03','ZUB1','ZUB2','ZUB4']:
        return 'OK'
    elif x['ЦС/ПЗ']=='ПЗ' and x['Наименование типа элемента ППМ пополн'] in ['Запас','Зпс/Кл','ЗпсПрк']:
        return 'OK'
    elif x['ЦС/ПЗ']=='ПЗ' and  (x['Вид документа пополнения'] in ['UB01','UB04','UB05','NB01','NB02'] and x['Недропользование']!=0 ):
        return 'OK'
    elif x['ЦС/ПЗ']=='ЦС' and x['Вид документа пополнения'] in ['NB01','NB02','UB01','UB05','UB03','UB04','UB06','ZNB1','ZNB2','ZNB3','ZNB5','ZUB1','ZUB4']:
        return 'OK'
    elif  x['ЦС/ПЗ']=='ЦС' and x['Вид документа пополнения'] ==0 and x['Наименование типа элемента ППМ пополн'] in ['Запас','Зпс/Кл','ЗпсПрк','СклдЗп']:
        return 'OK'
    else:
        return 'Отдельная отработка'
    
    
def modul(x):
    if x['ЦС/ПЗ']=='ПЗ' and x['Вид документа потребности'] in ['UB02','Z1']:
        return 'Прочие'
    elif x['ЦС/ПЗ']=='ПЗ' and x['Вид документа потребности']=='AR' and list(str(x['Заказ']))[0]=='1':
        return 'ТОРО'
    elif x['ЦС/ПЗ']=='ПЗ' and (x['Вид документа потребности']=='SB' or (x['Вид документа потребности']=='AR' and list(str(x['Заказ']))[0]=='2')):
        return 'PP'
    elif x['ЦС/ПЗ']=='ЦС':
        return 0
    elif x['ЦС/ПЗ']==0:
        return 0
    else:
        return 'ХЗ'
    
# формулы остатков    
def Q_E_ALL(x):
    if x['Блокированный запас']>0:
        return 'Block'
    elif x['Особый запас']in ['E','Q']:
        return 'E+Q'
    else:
        return 'All'
    
def ukrup_zakaz(x):
    try:
        if x['ЦС/ПЗ']=='ПЗ':
            return spr_ost_CUPZ_dict[x['Завод+Склад']]['Заказчик']
        elif x['ЦС/ПЗ']=='ЦС':
            ar = x['Завод+Склад']+' '+ x['Партия']
            return spr_ost_CUOCS_dict[ar]['Заказчик']      
    except KeyError:
        pass
    
    
# формулы расхода  
def Q_E_ALL_rash(x):
    if x['Особый запас']in ['E','Q']:
        return 'E+Q'
    else:
        return 'All'  
    
# разбиение Данных на файлы    
def saveelse_start(path,df_filter,name_col_filter,arr_datafrs):
      for df in arr_datafrs:  
        for i in df_filter.columns:
            data = df[df[name_col_filter].isin(df_filter[i])]
            data.to_csv(path+str(df.name)+ str(i)+'.csv',encoding='utf-8-sig',sep = ';')  
            
            
# превращает цифры строками  в нормальные цифры
def replace_float(df,name_cols):
    try:
        for col_n in name_cols:
            mas = df[col_n].apply(lambda x: x.replace('.',''))
            mas1 = mas.apply(lambda x: x.replace(',','.'))
            df[col_n] =  mas1.astype('float')
    except:
        pass

#     получение данных из словаря
def ret_data_dict(filter_,dictionary):
    try:
        a = dictionary[filter_]
        return a
    except:
        return pd.DataFrame()  
    
from threading import Thread
directory_mc = 'C:\\Users\\User\\Desktop\\kazzinc\\Uploads1\\для ZMM_MATCATAL\\'

# Блок для поточного чтения файлов

import time
import func_read_ex
from multiprocessing import Pool
def exlsx_f(directory):
    start_time = time.time()
    pool = Pool(processes=5)

    files = os.listdir(directory)
    print(files)
    file_list = [(filename,directory) for filename in files if filename.endswith('.XLSX')]
    df_list = pool.map(func_read_ex.read_csv, file_list)
    df = pd.concat(df_list)
    print("--- %s seconds ---" % (time.time() - start_time))
    pool.close()
    return df



def save_backup(path_backup,arrs):
    try:
        backup = pd.HDFStore(path_backup)
        for arr in arrs:
            backup[str(arr.name)] = arr
        backup.close()   
    except:
        backup.close()
    
def get_in_backup(path_backup,name):
    try:
        backup = pd.HDFStore(path_backup)
        ar  = backup[name]
        backup.close()
        return ar
    except:
        backup.close()
        
def category_of_zapas(x):
    if x['Особый запас'] in ['Q']:
        return 'Проекты'
    elif x['Завод']=='1505' or (x['ЦС/ПЗ'] =='ЦС' and x['СтатусМтрл на заводе']=='2'):
        return 'Неликвиды'
    elif x['Завод']=='1516' or x['Завод']=='3556' or x['Завод']=='3566' or x['Склад'][:2]=='58':
        return 'Сервисный контракт'
    elif x['Комплекс']=='ГО':
        return 'ГО'
    elif x['Cвободный запас']>0 and x['Блокированный запас']==0:
        return 'Для распределения и нет блокированного запаса в строке'
    elif x['Cвободный запас']>0 and x['Блокированный запас']>0:
        return 'Для распределения и есть блокированный запас в строке'
    elif x['Cвободный запас']==0:
        return 'Свободного остатка нет' 
    
def func_cat_tmc(x):
    if x['Блокированный запас']>0:
        return 'ТНСЗ'
    elif x['Признак Q остатка']==True or x['Признак Q прихода']==True or x['Признак Q расхода']==True:
        return 'Проект'
    elif x['Комплекс'].find('Service') !=-1:
        return 'Сервисный контракт'
    elif x['Комплекс']=='ГО':
        return 'ГО'
    elif x['Комплекс'] =='НЕЛИКВИДЫ':
        return 'Неликвиды'
    elif x['Комплекс']=='СПЕЦОДЕЖДА':
        return 'Спецодежда'
    elif x['Количество потребности']>0:
        return 'Текущая потребность'
    elif x['Количество потребности']==0:
        return 'Без ПП'
    else:
        return 0    
    
def func_cat_tmc_new(x,current_date,dic):
    if (x['Количество потребности']>0 and current_date.year == x['Дата потребности'].year) or \
    (x['Категория ТМЦ'] in['ТНСЗ','страховой запас','Проект','ГО','Спецодежда']):
        return 'Востребованный'
    elif x['Количество потребности']>0 and x['Дата потребности'].year>current_date.year and x['Распределение_остатака']>0:
        return 'Среднесрочный'
    elif x['Комплекс'] =='НЕЛИКВИДЫ':
        return 'Неликвиды'
    elif dic[x['Завод+НН']]==0:
        return 'Невостребованный'
    else:
        return 'Прочие'    
    



def parallel_calc_fast(cons,KONH,current_date,KONP,MARM,pl_kurs):
    if len([x for x in cons.columns if x=='Цена'])>0:
        cons.drop(['Цена'],axis = 1,inplace = True)
    data_frame = cons.copy()
    cons = cons[['Код округа','Материал','Завод','Базовая ЕИ']].drop_duplicates()
    cons.reset_index(inplace = True,drop = True)
    start_time = time.time()
    pool = Pool(processes=10)
    cons['Цена']=0
    cons.rename(columns={"Код округа": "округ"},inplace=True)
    KONH = KONH.reset_index(drop=True)
#     KONH_po = KONH.reset_index().set_index(['Материал',pd.Index(KONH['Действ. по'].apply(lambda x:str(x.year)))])
    KONH['key2'] = KONH['Материал']+KONH['Действ. по'].apply(lambda x: str(x.year))
#     KONH_po = KONH.reset_index().set_index(['key2'])

#     KONH_s = KONH.reset_index().set_index(['Материал',pd.Index(KONH['Действ. с'].apply(lambda x:str(x.year)))])
    KONH['key1'] = KONH['Материал']+KONH['Действ. с'].apply(lambda x: str(x.year))
#     KONH_s = KONH.reset_index().set_index(['key1'])

    KONH.rename(columns={"Номер записи условий": 'Номер_записи_условий'},inplace=True)
    KONH['sort'] = KONH['Номер_записи_условий'].astype('int')
    KONH.sort_values('sort',inplace = True,ascending = False)
    KONH_po = KONH.groupby('key2')
    KONH_s = KONH.groupby('key1')
    
    
    
    
#     KONP = KONP.set_index(['Номер записи условий'])
    KONP = KONP.groupby('Номер записи условий')
    MARM['key'] = MARM['Материал']+MARM['Альтернативная ЕИ']
#     MARM =  MARM.set_index(['Материал','Альтернативная ЕИ'])
    MARM = MARM.groupby('key')
    arrs =  np.array_split(cons,20)
    arrs =  [(arr,KONH_po,KONH_s,current_date,KONP,MARM,pl_kurs) for arr in arrs]
    df_list = pool.map(func_read_ex.calc_price_super_fast, arrs)
    df = pd.concat(df_list)
    print("--- %s seconds ---" % (time.time() - start_time))
    pool.close()
    df.rename(columns={"округ": "Код округа"},inplace=True)
#     df = pd.concat([df.set_index(['Код округа','Материал','Завод','Базовая ЕИ']),data_frame.set_index(['Код округа','Материал','Завод','Базовая ЕИ'])],axis = 1)
    df = data_frame.merge(df,left_on=['Код округа','Материал','Завод','Базовая ЕИ'],right_on=['Код округа','Материал','Завод','Базовая ЕИ'],how='outer')
    
    df.reset_index(inplace = True,drop = True)
    
    df.drop(['номер записи условий','цена без учета коэф.пересчата','единица измерения закупки'],axis = 1,inplace = True)
    return df 


def kopmpl_ost__(TWLAD,ADRC,df,T001W):
    
    def uu(x):
        if str(x).find('FREE')==0:
            return x[4:6]
        else:
            return str(x)[:2]
    
    start_time = time.time()
    to_str_ost = ['Завод','Склад','Номер адреса']
    for i in to_str_ost:
        TWLAD[i] = TWLAD[i].apply(lambda x: str(x).replace('.0','')).astype('str')
        TWLAD[i] = TWLAD[i].apply(lambda x: '' if x=='0' else x)
    to_str_ost1 = ['Номер адреса']   
    for i in to_str_ost1:
        ADRC[i] = ADRC[i].apply(lambda x: str(x).replace('.0','')).astype('str')
        ADRC[i] = ADRC[i].apply(lambda x: '' if x=='0' else x)
    to_str_ost1 = ['Закуп. организация','Завод']   
    for i in to_str_ost1:
        T001W[i] = T001W[i].apply(lambda x: str(x).replace('.0','')).astype('str')
        T001W[i] = T001W[i].apply(lambda x: '' if x=='0' else x)
    
    
    ADRC = ADRC.set_index('Номер адреса')
    df['Комплекс'] = 0 
    
    for ind in TWLAD.index:
        try:
            df_master = ADRC.xs(TWLAD.at[ind,'Номер адреса'])

            df.loc[(df['Завод']==TWLAD.at[ind,'Завод']) & (df['Склад']==TWLAD.at[ind,'Склад']) &(df['ЦС/ПЗ']=='ПЗ'),'Комплекс'] =df_master['Критерий поиска 2']
            df.loc[(df['Завод']==TWLAD.at[ind,'Завод'])  &(df['ЦС/ПЗ']=='ПЗ')&(df['Комплекс']==0),'Комплекс'] =df_master['Критерий поиска 2'] 
        except KeyError:
            df_master = pd.DataFrame()
            
    yyy  =ostatok['Партия'].apply(lambda x:x[:1].isalpha() and x[-4:].isalpha()==False)
    for ind in TWLAD.index:
        try:
            
            df_master = ADRC.xs(TWLAD.at[ind,'Номер адреса'])
        
        
            df.loc[(df[yyy]['Партия'].str[-4:]==TWLAD.at[ind,'Завод'])  &(df['ЦС/ПЗ']=='ЦС'),'Комплекс'] =df_master['Критерий поиска 2']
            
            
#             df.loc[(df['Завод']==TWLAD.at[ind,'Завод']) & (df['Склад']==TWLAD.at[ind,'Склад']) &(df['ЦС/ПЗ']=='ЦС')&(df['Комплекс']==0),'Комплекс'] =df_master['Критерий поиска 2']

        except KeyError:
            df_master = pd.DataFrame()
            
    for ind in T001W.index:

        t = df[(df['Партия'].str[-4:]==T001W.at[ind,'Закуп. организация'])&(df['Партия'].apply(lambda x: uu(x))==T001W.at[ind,'Код округа'])&(df['Комплекс']==0)&(df['ЦС/ПЗ']=='ЦС')]

        if t.shape[0]>0:
            number_address =  TWLAD[TWLAD['Завод']==T001W.at[ind,'Завод']]['Номер адреса'].values[0]
            try:

                df_master = ADRC.xs(str(number_address))
                df.loc[t.index,'Комплекс'] =df_master['Критерий поиска 2']
            except KeyError:
                df_master = pd.DataFrame() 
    
    for ind in TWLAD.index:
        try:
            df_master = ADRC.xs(TWLAD.at[ind,'Номер адреса'])
            
            df.loc[(df['Завод']==TWLAD.at[ind,'Завод'])  &(df['ЦС/ПЗ']=='ЦС')&(df['Комплекс']==0),'Комплекс'] =df_master['Критерий поиска 2'] 
        except KeyError:
            df_master = pd.DataFrame()
            
    df.fillna(0,inplace = True)        
    df.loc[(df['Комплекс']==0),'Комплекс'] ='Прочее'
    print("--- %s seconds ---" % (time.time() - start_time))  
    
    return df

In [ ]:
# управление механикой остатков(вызывать ее)
def ostat(ost_st, potr_st, current_date, filters,all_months,col_name_0,col_name_1,col_name_2,col_name_3,col_name_4,col_name_5):
    
       
    
#     cons = potr_st.sort_values(['Область ППМ','сортировка по модулю'])
    pz_ost = ost_st[ost_st['ЦС/ПЗ']=='ПЗ']
    cons = potr_st.set_index(['Завод+НН','del'])
    
    if all_months ==True:
        dat = cons[cons['month']>=current_date].sort_values('Дата потребности')
                
    else:   
        dat = cons[cons['month']>current_date].sort_values('Дата потребности')
    
    # 15,2,1
    for pz_slave_row in pz_ost[pz_ost[col_name_0]!=0].index:
#         print(pz_ost.loc[pz_slave_row,'Материал'])
        try: 
            df_master = dat.xs(pz_ost.loc[pz_slave_row,'Завод+НН'])
        except KeyError:
            df_master = pd.DataFrame()
        
        if len(df_master)>0:
            mex_ost(df_master, pz_ost, pz_slave_row, ost_st, potr_st,col_name_0,col_name_1,col_name_2,col_name_3,col_name_4,col_name_5)

            
            
    pz_ost = ost_st[ost_st['ЦС/ПЗ']=='ЦС']
    cons = potr_st.reset_index().set_index(['Материал','Заказчик + HH','del'])
    
    if all_months ==True:
        dat = cons[cons['month']>=current_date].sort_values('Дата потребности')
                
    else:   
        dat = cons[cons['month']>current_date].sort_values('Дата потребности')
    
    for pz_slave_row in pz_ost[pz_ost[col_name_0]!=0].index:
        if ost_st.at[pz_slave_row, 'Комплекс'] == 'ALL1000':
            
            try:
                df_master = dat.xs(pz_ost.at[pz_slave_row, 'Материал'])
            except KeyError:
                df_master = pd.DataFrame()    
               
            if len(df_master) > 0:
                
                # отсекаем только all1000 и сортируем
                df_master = df_master[df_master['Комплекс'].isin(filters['one'])].sort_values('Дата потребности')
                mex_ost(df_master, pz_ost, pz_slave_row, ost_st, potr_st,col_name_0,col_name_1,col_name_2,col_name_3,col_name_4,col_name_5)
                
        else:
            
            try:
                df_master = dat.xs(pz_ost.at[pz_slave_row, 'Заказчик + HH'],level = 1)
            except KeyError:
                df_master = pd.DataFrame()    
            
            
            if len(df_master) > 0:
                
                df_master = df_master.sort_values('Дата потребности')
                
                mex_ost(df_master, pz_ost, pz_slave_row, ost_st, potr_st,col_name_0,col_name_1,col_name_2,col_name_3,col_name_4,col_name_5)
                      
                 
            #             распределяем
             
        
    
   
    

# мехника распределения остатков 
# col_name_0 название столбца откуда рпспределяем
# col_name_1 навание с толбца куда распределяем
# col_name_2 название столбца с количеством потребности
# col_name_5 название столбца куда распределяем сумму
# Для цены:
# col_name_3 название столбца с суммой потребности
# col_name_4 название столбца с начальным кол-вом  потребности
def mex_ost(df_master, pz_ost, pz_slave_row, ost_st, potr_st,col_name_0,col_name_1,col_name_2,col_name_3,col_name_4,col_name_5):
    df_master = df_master.reset_index().set_index(['index'])
#     print(df_master.index)
    ostatok = pz_ost.at[pz_slave_row, col_name_0]
    len_it = len(df_master) - 1
    for ind, row_master_index in enumerate(df_master.index):
#         print(df_master)
        if len_it == ind:
            if pz_ost.at[pz_slave_row, col_name_3]==0 or pz_ost.at[pz_slave_row, col_name_4]==0:
                price = 0
            else:
                price = pz_ost.at[pz_slave_row, col_name_3]/pz_ost.at[pz_slave_row, col_name_4]
            summa = ostatok*price
            
            potr_st.at[row_master_index, col_name_5] = potr_st.at[row_master_index, col_name_5] + summa
            potr_st.at[row_master_index, col_name_1] = potr_st.at[row_master_index, col_name_1] + ostatok

            pz_ost.at[pz_slave_row, col_name_0] = 0
            ost_st.at[pz_slave_row, col_name_0] = 0
            # print(df_master[['Количество потребности','Дата потребности','Распределение_остатака']])
            ostatok = 0
            return
        
        elif (potr_st.at[row_master_index, col_name_2] > potr_st.at[row_master_index, col_name_1]) and (
                potr_st.at[row_master_index, col_name_2] >= ostatok):
            if ostatok+potr_st.at[row_master_index, col_name_1]>potr_st.at[row_master_index, col_name_2]:
                a=0
                if pz_ost.at[pz_slave_row, col_name_3]==0 or pz_ost.at[pz_slave_row, col_name_4]==0:
                    price = 0
                else:    
                    price = pz_ost.at[pz_slave_row, col_name_3]/pz_ost.at[pz_slave_row, col_name_4]
                
                vzyali = potr_st.at[row_master_index, col_name_2]-potr_st.at[row_master_index, col_name_1]
                
                summa = vzyali*price
                
                potr_st.at[row_master_index, col_name_5] = potr_st.at[row_master_index, col_name_5] + summa
                potr_st.at[row_master_index, col_name_1] = potr_st.at[row_master_index, col_name_1] + vzyali

                pz_ost.at[pz_slave_row, col_name_0] = pz_ost.at[pz_slave_row, col_name_0]-vzyali
                ost_st.at[pz_slave_row, col_name_0] = ost_st.at[pz_slave_row, col_name_0]-vzyali
                ostatok =  ostatok -vzyali

            else:
                if pz_ost.at[pz_slave_row, col_name_3]==0 or pz_ost.at[pz_slave_row, col_name_4]==0:
                    price = 0
                else:
                    price = pz_ost.at[pz_slave_row, col_name_3]/pz_ost.at[pz_slave_row, col_name_4]
                summa = ostatok*price
                potr_st.at[row_master_index, col_name_5] = potr_st.at[row_master_index, col_name_5] + summa
                potr_st.at[row_master_index, col_name_1] = potr_st.at[row_master_index, col_name_1] + ostatok

                pz_ost.at[pz_slave_row, col_name_0] = 0
                ost_st.at[pz_slave_row, col_name_0] = 0
                ostatok = 0
                # print(df_master[['Количество потребности', 'Дата потребности', 'Распределение_остатака']])
                return

        elif (potr_st.at[row_master_index, col_name_2] > potr_st.at[row_master_index, col_name_1]) and (potr_st.at[row_master_index, col_name_2] < ostatok):
            if pz_ost.at[pz_slave_row, col_name_3]==0 or pz_ost.at[pz_slave_row, col_name_4]==0:
                price = 0
            else:
                price = pz_ost.at[pz_slave_row, col_name_3]/pz_ost.at[pz_slave_row, col_name_4]
       
            vzyali = potr_st.at[row_master_index, col_name_2]-potr_st.at[row_master_index, col_name_1]      
            summa = vzyali*price
            potr_st.at[row_master_index, col_name_5] = potr_st.at[row_master_index, col_name_5] + summa
            potr_st.at[row_master_index, col_name_1] = potr_st.at[row_master_index, col_name_1] + (potr_st.at[row_master_index, col_name_2] -potr_st.at[row_master_index, col_name_1])

            pz_ost.at[pz_slave_row, col_name_0] = pz_ost.at[pz_slave_row, col_name_0] - vzyali
            ost_st.at[pz_slave_row, col_name_0] = ost_st.at[pz_slave_row, col_name_0] - vzyali
            ostatok = ostatok - vzyali
            # print(df_master[['Количество потребности', 'Дата потребности', 'Распределение_остатака']])

In [ ]:
# управление механикой остатков(вызывать ее)         
def rashod_raspr(ost_st,potr_st,current_date,filters,all_months,col_name_0,col_name_1,col_name_2,col_name_3,col_name_4,col_name_5):

    pz_ost =  ost_st
    # пз
    cons = potr_st.set_index(['Завод+НН','del'])
    
    if all_months==True:
        dat = cons[cons['month']>=current_date].sort_values('Дата потребности')
    else:
        dat = cons[cons['month']>current_date].sort_values('Дата потребности')
    
    
    for pz_slave_row in pz_ost[pz_ost[col_name_0]!=0].index:
        
        try: 
            df_master = dat.xs(pz_ost.loc[pz_slave_row,'Завод+НН'])
        except KeyError:
            df_master = pd.DataFrame()

        if len(df_master)>0:
            df_master = df_master.sort_values('Дата потребности')
            # функция
            mex_rashod(df_master, pz_ost, pz_slave_row, ost_st, potr_st,col_name_0,col_name_1,col_name_2,col_name_3,col_name_4,col_name_5)


# col_name_0 название столбца откуда рпспределяем
# col_name_1 навание с толбца куда распределяем
# col_name_2 название столбца с количеством потребности
# col_name_5 название столбца куда распределяем сумму
# Для цены:
# col_name_3 название столбца с суммой потребности
# col_name_4 название столбца с начальным кол-вом  потребности
def mex_rashod(df_master, pz_ost, pz_slave_row, ost_st, potr_st,col_name_0,col_name_1,col_name_2,col_name_3,col_name_4,col_name_5):         
    df_master = df_master.reset_index().set_index(['index'])
    ostatok = pz_ost.at[pz_slave_row, col_name_0]
    len_it = len(df_master) - 1
    for ind, row_master_index in enumerate(df_master.index):

        if len_it == ind:
            price = ost_st.at[pz_slave_row, col_name_3] / ost_st.at[pz_slave_row, col_name_4]
            summa = ostatok * price

            potr_st.at[row_master_index, col_name_5] = potr_st.at[row_master_index, col_name_5] + summa


            potr_st.at[row_master_index, col_name_1] = potr_st.at[row_master_index, col_name_1] + ostatok

            pz_ost.at[pz_slave_row, col_name_0] = 0
            ost_st.at[pz_slave_row, col_name_0] = 0
            # print(df_master[['Количество потребности','Дата потребности','Распределение_остатака']])
            ostatok = 0
            return

        elif (potr_st.at[row_master_index, col_name_2] > potr_st.at[row_master_index, col_name_1]) and (
                potr_st.at[row_master_index, col_name_2] >= ostatok):
            if ostatok+potr_st.at[row_master_index, col_name_1]>potr_st.at[row_master_index, col_name_2]:
                a=0
                price = ost_st.at[pz_slave_row, col_name_3] / ost_st.at[pz_slave_row, col_name_4]
                vzyali = potr_st.at[row_master_index, col_name_2]-potr_st.at[row_master_index, col_name_1]
                summa = vzyali * price

                potr_st.at[row_master_index, col_name_5] = potr_st.at[row_master_index, col_name_5] + summa


                potr_st.at[row_master_index, col_name_1] = potr_st.at[row_master_index, col_name_1] + vzyali

                pz_ost.at[pz_slave_row, col_name_0] = pz_ost.at[pz_slave_row, col_name_0]-vzyali
                ost_st.at[pz_slave_row, col_name_0] = ost_st.at[pz_slave_row, col_name_0]-vzyali
                ostatok =  ostatok -vzyali

            else:
                price = ost_st.at[pz_slave_row, col_name_3] / ost_st.at[pz_slave_row, col_name_4]
                summa = ostatok * price
                potr_st.at[row_master_index, col_name_5] = potr_st.at[row_master_index, col_name_5] + summa

                potr_st.at[row_master_index, col_name_1] = potr_st.at[row_master_index, col_name_1] + ostatok

                pz_ost.at[pz_slave_row, col_name_0] = 0
                ost_st.at[pz_slave_row, col_name_0] = 0
                ostatok = 0
                # print(df_master[['Количество потребности', 'Дата потребности', 'Распределение_остатака']])
                return

        elif (potr_st.at[row_master_index, col_name_2] > potr_st.at[row_master_index, col_name_1]) and (potr_st.at[row_master_index, col_name_2] < ostatok):
            price = ost_st.at[pz_slave_row, col_name_3] / ost_st.at[pz_slave_row, col_name_4]
            vzyali = potr_st.at[row_master_index, col_name_2] - potr_st.at[row_master_index, col_name_1]
            summa = vzyali * price
            potr_st.at[row_master_index, col_name_5] = potr_st.at[row_master_index, col_name_5] + summa


            potr_st.at[row_master_index, col_name_1] = potr_st.at[row_master_index, col_name_1] + (potr_st.at[row_master_index, col_name_2] -potr_st.at[row_master_index, col_name_1])

            pz_ost.at[pz_slave_row, col_name_0] = pz_ost.at[pz_slave_row, col_name_0] - vzyali
            ost_st.at[pz_slave_row, col_name_0] = ost_st.at[pz_slave_row, col_name_0] - vzyali
            ostatok = ostatok - vzyali
            # print(df_master[['Количество потребности', 'Дата потребности', 'Распределение_остатака']])

In [ ]:
# управление механикой остатков(вызывать ее)         
def prihod_raspr(ost_st,potr_st,current_date,filters,all_months,col_name_0,col_name_1,col_name_2,col_name_3,col_name_4,col_name_5):
#     группировка остатков по 

    pz_ost = ost_st
    # пз
    cons = potr_st.set_index(['Заказчик + HH','del']).sort_values('Дата потребности')
    
    
    if all_months==True:
        dat = cons[cons['month']>=current_date]
    else:
        dat = cons[cons['month']>current_date]
        
    for pz_slave_row in pz_ost[pz_ost[col_name_0]!=0].index:
        try: 
            df_master = dat.xs(pz_ost.loc[pz_slave_row,'Комплекс+НН'])
        except KeyError:
            df_master = pd.DataFrame()
        
        if len(df_master)>0:
            
            
            # функция
            mex_prihod(df_master,pz_ost,pz_slave_row,ost_st,potr_st,col_name_0,col_name_1,col_name_2,col_name_3,col_name_4,col_name_5)
            
            
    cons = potr_st.set_index(['Материал','del'])
    
    if all_months==True:
        dat = cons[cons['month']>=current_date]
    else:
        dat = cons[cons['month']>current_date]
    
    
    for pz_slave_row in pz_ost[pz_ost[col_name_0]!=0].index:
        
        if pz_ost.at[pz_slave_row,'Комплекс']=='ALL1000':
            try: 
                df_master = dat.xs(pz_ost.loc[pz_slave_row,'Материал'])
            except KeyError:
                df_master = pd.DataFrame()

            if len(df_master)>0:
                
                df_master = df_master[df_master['Комплекс'].isin(filters['one'])].sort_values('Дата потребности')
                # функция
                mex_prihod(df_master,pz_ost,pz_slave_row,ost_st,potr_st,col_name_0,col_name_1,col_name_2,col_name_3,col_name_4,col_name_5)
    
            
            
            
            
def mex_prihod(df_master, pz_ost, pz_slave_row, ost_st, potr_st,col_name_0,col_name_1,col_name_2,col_name_3,col_name_4,col_name_5):         
    df_master = df_master.reset_index().set_index(['index'])
    ostatok = pz_ost.at[pz_slave_row, col_name_0]
    len_it = len(df_master) - 1
    for ind, row_master_index in enumerate(df_master.index):

        if len_it == ind:
            price = ost_st.at[pz_slave_row, col_name_3] / ost_st.at[pz_slave_row, col_name_4]
            summa = ostatok * price

            potr_st.at[row_master_index, col_name_5] = potr_st.at[row_master_index, col_name_5] + summa


            potr_st.at[row_master_index, col_name_1] = potr_st.at[row_master_index, col_name_1] + ostatok

            pz_ost.at[pz_slave_row, col_name_0] = 0
            ost_st.at[pz_slave_row, col_name_0] = 0
            # print(df_master[['Количество потребности','Дата потребности','Распределение_остатака']])
            ostatok = 0
            return

        elif (potr_st.at[row_master_index, col_name_2] > potr_st.at[row_master_index, col_name_1]) and (
                potr_st.at[row_master_index, col_name_2] >= ostatok):
            if ostatok+potr_st.at[row_master_index, col_name_1]>potr_st.at[row_master_index, col_name_2]:
                a=0
                price = ost_st.at[pz_slave_row, col_name_3] / ost_st.at[pz_slave_row, col_name_4]
                vzyali = potr_st.at[row_master_index, col_name_2]-potr_st.at[row_master_index, col_name_1]
                summa = vzyali * price

                potr_st.at[row_master_index, col_name_5] = potr_st.at[row_master_index, col_name_5] + summa


                potr_st.at[row_master_index, col_name_1] = potr_st.at[row_master_index, col_name_1] + vzyali

                pz_ost.at[pz_slave_row, col_name_0] = pz_ost.at[pz_slave_row, col_name_0]-vzyali
                ost_st.at[pz_slave_row, col_name_0] = ost_st.at[pz_slave_row, col_name_0]-vzyali
                ostatok =  ostatok -vzyali

            else:
                price = ost_st.at[pz_slave_row, col_name_3] / ost_st.at[pz_slave_row, col_name_4]
                summa = ostatok * price
                potr_st.at[row_master_index, col_name_5] = potr_st.at[row_master_index, col_name_5] + summa

                potr_st.at[row_master_index, col_name_1] = potr_st.at[row_master_index, col_name_1] + ostatok

                pz_ost.at[pz_slave_row, col_name_0] = 0
                ost_st.at[pz_slave_row, col_name_0] = 0
                ostatok = 0
                # print(df_master[['Количество потребности', 'Дата потребности', 'Распределение_остатака']])
                return

        elif (potr_st.at[row_master_index, col_name_2] > potr_st.at[row_master_index, col_name_1]) and (potr_st.at[row_master_index, col_name_2] < ostatok):
            price = ost_st.at[pz_slave_row, col_name_3] / ost_st.at[pz_slave_row, col_name_4]
            vzyali = potr_st.at[row_master_index, col_name_2] - potr_st.at[row_master_index, col_name_1]
            summa = vzyali * price
            potr_st.at[row_master_index, col_name_5] = potr_st.at[row_master_index, col_name_5] + summa


            potr_st.at[row_master_index, col_name_1] = potr_st.at[row_master_index, col_name_1] + (potr_st.at[row_master_index, col_name_2] -potr_st.at[row_master_index, col_name_1])

            pz_ost.at[pz_slave_row, col_name_0] = pz_ost.at[pz_slave_row, col_name_0] - vzyali
            ost_st.at[pz_slave_row, col_name_0] = ost_st.at[pz_slave_row, col_name_0] - vzyali
            ostatok = ostatok - vzyali
            # print(df_master[['Количество потребности', 'Дата потребности', 'Распределение_остатака']])

In [ ]:
# управление механикой остатков(вызывать ее)
def avizot_raspr(ost_st, potr_st, current_date, filters,all_months):
    #     группировка остатков по
    if all_months ==True:
        master_zav_nn = dict(tuple(potr_st[potr_st['month'] >= current_date].groupby('Заказчик + HH')))
    else:
        master_zav_nn = dict(tuple(potr_st[potr_st['month'] > current_date].groupby('Заказчик + HH')))
    pz_ost = ost_st
    # пз
#     print(master_zav_nn.keys())
    for pz_slave_row in pz_ost.index:
        df_master = ret_data_dict(pz_ost.at[pz_slave_row, 'Комплекс+Маттериал'], master_zav_nn)
        if df_master is None:
            df_master = pd.DataFrame()
        # если есть потребность у сцепки зав нн
        # pz_ost.at[pz_slave_row, 'осталось'] = 1600
        if len(df_master) > 0 and pz_ost.at[pz_slave_row, 'осталось'] > 0:
            df_master = df_master.sort_values('Дата потребности')
            # функция
            df_master = mex_avizot(df_master, pz_ost, pz_slave_row, ost_st, potr_st)
            master_zav_nn[pz_ost.at[pz_slave_row, 'Комплекс+Маттериал']] = df_master
    if all_months ==True:
        master_zav_nn = dict(tuple(potr_st[potr_st['month'] >= current_date].groupby('Материал')))
    else:
        master_zav_nn = dict(tuple(potr_st[potr_st['month'] > current_date].groupby('Материал')))

    for pz_slave_row in pz_ost.index:
        if pz_ost.at[pz_slave_row, 'Комплекс'] == 'ALL1000':
            df_master = ret_data_dict(pz_ost.at[pz_slave_row, 'Материал'], master_zav_nn)
            if df_master is None:
                df_master = pd.DataFrame()
            # если есть потребность у сцепки зав нн
            # pz_ost.at[pz_slave_row, 'осталось'] = 1600
            if len(df_master) > 0 and pz_ost.at[pz_slave_row, 'осталось'] > 0:
                df_master = df_master[df_master['Комплекс'].isin(filters['one'])].sort_values('Дата потребности')
                # функция
                df_master = mex_avizot(df_master, pz_ost, pz_slave_row, ost_st, potr_st)
                master_zav_nn[pz_ost.at[pz_slave_row, 'Материал']] = df_master


# мехника распределения остатков
def mex_avizot(df_master, pz_ost, pz_slave_row, ost_st, potr_st):
#     print(df_master[['Распределение_остатака', 'Количество потребности', 'avizot']])
    ostatok = pz_ost.at[pz_slave_row, 'Количество пополнения']
    len_it = len(df_master) - 1
    for ind, row_master_index in enumerate(df_master.index):
        # если покрыта вся потребность то ничего не делаем
        #         if df_master.at[row_master_index, 'Количество потребности'] == df_master.at[row_master_index, 'Распределение_прихода']:
        #             pass
        # если это последний элемент нужно записать то что осталось в остатке
        #  1 прибавляем в столбец Распределение_остатака гласного фрейма все то что осталось
        #  2 вставляем в лист остатков в столбец забрали
        #  3 вставлем в маленькие выборки аналогично
        #  4 рассчитываем сумму по цене и прибавляем к имеющейся в ячейке
        if len_it == ind and (df_master.at[row_master_index, 'Количество потребности'] > (df_master.at[row_master_index, 'Распределение_остатака'] + df_master.at[row_master_index, 'avizot'])):
            #             price = ost_st.at[pz_slave_row, 'Сумма во ВВ'] / ost_st.at[pz_slave_row, 'Количество']
            #             summa = ostatok * price
            #             df_master.at[row_master_index, 'Сумма_распреденного_прихода'] = df_master.at[row_master_index, 'Сумма_распреденного_прихода'] + summa
            #             potr_st.at[row_master_index, 'Сумма_распреденного_прихода'] = potr_st.at[row_master_index, 'Сумма_распреденного_прихода'] + summa

            #             df_master.at[row_master_index, 'Распределение_прихода'] = df_master.at[row_master_index, 'Распределение_прихода'] + ostatok
            #             potr_st.at[row_master_index, 'Распределение_прихода'] = potr_st.at[row_master_index, 'Распределение_прихода'] + ostatok


            if (ostatok - (df_master.at[row_master_index, 'Количество потребности'] - df_master.at[row_master_index, 'Распределение_остатака']- df_master.at[row_master_index, 'avizot']))<0:
                df_master.at[row_master_index, 'avizot'] =df_master.at[row_master_index, 'avizot']+ ostatok
                potr_st.at[row_master_index, 'avizot'] = potr_st.at[row_master_index, 'avizot']+ ostatok
                ostatok = 0
                pz_ost.at[pz_slave_row, 'осталось'] = ostatok
                ost_st.at[pz_slave_row, 'осталось'] = ostatok
#                 print(df_master[['Распределение_остатака', 'Количество потребности', 'avizot']])
                return df_master
            else:
                a = df_master.at[row_master_index, 'avizot']
                df_master.at[row_master_index, 'avizot'] =df_master.at[row_master_index, 'avizot']+(df_master.at[row_master_index, 'Количество потребности'] - df_master.at[row_master_index, 'Распределение_остатака']-a)
                potr_st.at[row_master_index, 'avizot'] = potr_st.at[row_master_index, 'avizot']+ (potr_st.at[row_master_index, 'Количество потребности'] - df_master.at[row_master_index, 'Распределение_остатака']-a)
                ostatok = ostatok - (df_master.at[row_master_index, 'Количество потребности'] - df_master.at[row_master_index, 'Распределение_остатака']-a)
#                 print(df_master[['Распределение_остатака', 'Количество потребности', 'avizot']])

                
                



                        # ostatok = 0
            # return df_master
        elif ostatok == 0:
            return df_master
        # если покарыта не вся потребность  и не последняя строка тогда и не хватает чтобы закрыть потребность:
        #  1 прибавляем в столбец  Распределение_остатака гласного фрейма потребностей только остаток
        #  2 зануляем остаток
        #  3 зануляем в листе остатков в столбец осталось
        #  4 вставлем в маленькие выборки аналогично
        #  5 рассчитываем сумму по цене и прибавляем к имеющейся в ячейке

        elif (df_master.at[row_master_index, 'Количество потребности'] > (df_master.at[row_master_index, 'Распределение_остатака'] + df_master.at[row_master_index, 'avizot'])):



            if (ostatok - (df_master.at[row_master_index, 'Количество потребности'] - df_master.at[row_master_index, 'Распределение_остатака']-df_master.at[row_master_index, 'avizot']))<0:
                df_master.at[row_master_index, 'avizot'] =df_master.at[row_master_index, 'avizot']+ ostatok
                potr_st.at[row_master_index, 'avizot'] = potr_st.at[row_master_index, 'avizot']+ ostatok
                ostatok = 0
                pz_ost.at[pz_slave_row, 'осталось'] = 0
                ost_st.at[pz_slave_row, 'осталось'] = 0
#                 print(df_master[['Распределение_остатака', 'Количество потребности', 'avizot']])
                return df_master
            else:
                a = df_master.at[row_master_index, 'avizot']
                df_master.at[row_master_index, 'avizot'] =df_master.at[row_master_index, 'avizot']+(df_master.at[row_master_index, 'Количество потребности'] - df_master.at[row_master_index, 'Распределение_остатака']-a)
                potr_st.at[row_master_index, 'avizot'] = potr_st.at[row_master_index, 'avizot']+ (potr_st.at[row_master_index, 'Количество потребности'] - df_master.at[row_master_index, 'Распределение_остатака']-a)
                ostatok = ostatok - (df_master.at[row_master_index, 'Количество потребности'] - df_master.at[row_master_index, 'Распределение_остатака']-a)
#                 print(df_master[['Распределение_остатака', 'Количество потребности', 'avizot']])
#             return df_master
        if len_it == ind and (df_master.at[row_master_index, 'Количество потребности'] <= (df_master.at[row_master_index, 'Распределение_остатака'] + df_master.at[row_master_index, 'avizot'])):
            df_master.at[row_master_index, 'avizot'] = df_master.at[row_master_index, 'avizot'] + 0
            potr_st.at[row_master_index, 'avizot'] = potr_st.at[row_master_index, 'avizot'] + 0
#             print(df_master[['Распределение_остатака', 'Количество потребности', 'avizot']])
            return df_master


In [ ]:

# управление механикой остатков(вызывать ее)
def prt_pzk(ost_st, potr_st, current_date, filters,all_months):
    #     группировка остатков по
    if all_months ==True:
        master_zav_nn = dict(tuple(potr_st[potr_st['month'] >= current_date].groupby('Заказчик + HH')))
    else:
        master_zav_nn = dict(tuple(potr_st[potr_st['month'] > current_date].groupby('Заказчик + HH')))
    pz_ost = ost_st
    # пз
#     print(master_zav_nn.keys())
    for pz_slave_row in pz_ost.index:
        df_master = ret_data_dict(pz_ost.at[pz_slave_row, 'Комплекс+Маттериал'], master_zav_nn)
        if df_master is None:
            df_master = pd.DataFrame()
        # если есть потребность у сцепки зав нн
        # pz_ost.at[pz_slave_row, 'осталось'] = 1600
        if len(df_master) > 0 and pz_ost.at[pz_slave_row, 'осталось'] > 0:
            df_master = df_master.sort_values('Дата потребности')
            # функция
            df_master = mex_prt_pzk(df_master, pz_ost, pz_slave_row, ost_st, potr_st)
            master_zav_nn[pz_ost.at[pz_slave_row, 'Комплекс+Маттериал']] = df_master
    if all_months ==True:
        master_zav_nn = dict(tuple(potr_st[potr_st['month'] >= current_date].groupby('Материал')))
    else:
        master_zav_nn = dict(tuple(potr_st[potr_st['month'] > current_date].groupby('Материал')))

    for pz_slave_row in pz_ost.index:
        if pz_ost.at[pz_slave_row, 'Комплекс'] == 'ALL1000':
            df_master = ret_data_dict(pz_ost.at[pz_slave_row, 'Материал'], master_zav_nn)
            if df_master is None:
                df_master = pd.DataFrame()
            # если есть потребность у сцепки зав нн
            # pz_ost.at[pz_slave_row, 'осталось'] = 1600
            if len(df_master) > 0 and pz_ost.at[pz_slave_row, 'осталось'] > 0:
                df_master = df_master[df_master['Комплекс'].isin(filters['one'])].sort_values('Дата потребности')
                # функция
                df_master = mex_prt_pzk(df_master, pz_ost, pz_slave_row, ost_st, potr_st)
                master_zav_nn[pz_ost.at[pz_slave_row, 'Материал']] = df_master


# мехника распределения остатков
def mex_prt_pzk(df_master, pz_ost, pz_slave_row, ost_st, potr_st):
#     print(df_master[['Распределение_остатака', 'Количество потребности', 'законтрактовано']])
    ostatok = pz_ost.at[pz_slave_row, 'Количество пополнения']
    len_it = len(df_master) - 1
    for ind, row_master_index in enumerate(df_master.index):

        # если покрыта вся потребность то ничего не делаем
        #         if df_master.at[row_master_index, 'Количество потребности'] == df_master.at[row_master_index, 'Распределение_прихода']:
        #             pass
        # если это последний элемент нужно записать то что осталось в остатке
        #  1 прибавляем в столбец Распределение_остатака гласного фрейма все то что осталось
        #  2 вставляем в лист остатков в столбец забрали
        #  3 вставлем в маленькие выборки аналогично
        #  4 рассчитываем сумму по цене и прибавляем к имеющейся в ячейке
        if len_it == ind and (df_master.at[row_master_index, 'Количество потребности'] > (df_master.at[row_master_index, 'Распределение_остатака'] + df_master.at[row_master_index, 'законтрактовано'])):
            #             price = ost_st.at[pz_slave_row, 'Сумма во ВВ'] / ost_st.at[pz_slave_row, 'Количество']
            #             summa = ostatok * price
            #             df_master.at[row_master_index, 'Сумма_распреденного_прихода'] = df_master.at[row_master_index, 'Сумма_распреденного_прихода'] + summa
            #             potr_st.at[row_master_index, 'Сумма_распреденного_прихода'] = potr_st.at[row_master_index, 'Сумма_распреденного_прихода'] + summa

            #             df_master.at[row_master_index, 'Распределение_прихода'] = df_master.at[row_master_index, 'Распределение_прихода'] + ostatok
            #             potr_st.at[row_master_index, 'Распределение_прихода'] = potr_st.at[row_master_index, 'Распределение_прихода'] + ostatok


            if (ostatok - (df_master.at[row_master_index, 'Количество потребности'] - df_master.at[row_master_index, 'Распределение_остатака']-df_master.at[row_master_index, 'законтрактовано']))<0:
                df_master.at[row_master_index, 'законтрактовано'] =df_master.at[row_master_index, 'законтрактовано']+ ostatok
                potr_st.at[row_master_index, 'законтрактовано'] = potr_st.at[row_master_index, 'законтрактовано']+ ostatok
                ostatok = 0
                pz_ost.at[pz_slave_row, 'осталось'] = ostatok
                ost_st.at[pz_slave_row, 'осталось'] = ostatok
#                 print(df_master[['Распределение_остатака', 'Количество потребности', 'законтрактовано']])
                return df_master
            else:
                a = df_master.at[row_master_index, 'законтрактовано']
                df_master.at[row_master_index, 'законтрактовано'] =df_master.at[row_master_index, 'законтрактовано']+(df_master.at[row_master_index, 'Количество потребности'] - df_master.at[row_master_index, 'Распределение_остатака']-a)
                potr_st.at[row_master_index, 'законтрактовано'] = potr_st.at[row_master_index, 'законтрактовано']+ (potr_st.at[row_master_index, 'Количество потребности'] - df_master.at[row_master_index, 'Распределение_остатака']-a)
                ostatok = ostatok - (df_master.at[row_master_index, 'Количество потребности'] - df_master.at[row_master_index, 'Распределение_остатака']-a)
#                 print(df_master[['Распределение_остатака', 'Количество потребности', 'avizot']])



                        # ostatok = 0
            # return df_master
        elif ostatok == 0:
            return df_master
        # если покарыта не вся потребность  и не последняя строка тогда и не хватает чтобы закрыть потребность:
        #  1 прибавляем в столбец  Распределение_остатака гласного фрейма потребностей только остаток
        #  2 зануляем остаток
        #  3 зануляем в листе остатков в столбец осталось
        #  4 вставлем в маленькие выборки аналогично
        #  5 рассчитываем сумму по цене и прибавляем к имеющейся в ячейке

        elif (df_master.at[row_master_index, 'Количество потребности'] > (df_master.at[row_master_index, 'Распределение_остатака'] + df_master.at[row_master_index, 'законтрактовано'])):



            if (ostatok - (df_master.at[row_master_index, 'Количество потребности'] - df_master.at[row_master_index, 'Распределение_остатака'] - df_master.at[row_master_index, 'законтрактовано']))<0:
                df_master.at[row_master_index, 'законтрактовано'] =df_master.at[row_master_index, 'законтрактовано']+ ostatok
                potr_st.at[row_master_index, 'законтрактовано'] = potr_st.at[row_master_index, 'законтрактовано']+ ostatok
                ostatok = 0
                pz_ost.at[pz_slave_row, 'осталось'] = 0
                ost_st.at[pz_slave_row, 'осталось'] = 0
#                 print(df_master[['Распределение_остатака', 'Количество потребности', 'законтрактовано']])
                return df_master
            else:
                a = df_master.at[row_master_index, 'законтрактовано']
                df_master.at[row_master_index, 'законтрактовано'] =df_master.at[row_master_index, 'законтрактовано']+(df_master.at[row_master_index, 'Количество потребности'] - df_master.at[row_master_index, 'Распределение_остатака']-a)
                potr_st.at[row_master_index, 'законтрактовано'] = potr_st.at[row_master_index, 'законтрактовано']+ (potr_st.at[row_master_index, 'Количество потребности'] - df_master.at[row_master_index, 'Распределение_остатака']-a)
                ostatok = ostatok - (df_master.at[row_master_index, 'Количество потребности'] - df_master.at[row_master_index, 'Распределение_остатака']-a)
#                 print(df_master[['Распределение_остатака', 'Количество потребности', 'avizot']])
#             return df_master
        if len_it == ind and (df_master.at[row_master_index, 'Количество потребности'] <= (df_master.at[row_master_index, 'Распределение_остатака'] + df_master.at[row_master_index, 'законтрактовано'])):
            df_master.at[row_master_index, 'законтрактовано'] = df_master.at[row_master_index, 'законтрактовано'] + 0
            potr_st.at[row_master_index, 'законтрактовано'] = potr_st.at[row_master_index, 'законтрактовано'] + 0
#             print(df_master[['Распределение_остатака', 'Количество потребности', 'законтрактовано']])
            return df_master


In [ ]:
# управление механикой остатков(вызывать ее)
def NB_raspr(ost_st, potr_st, current_date, filters,all_months):
    #     группировка остатков по
    if all_months ==True:
        master_zav_nn = dict(tuple(potr_st[potr_st['month'] >= current_date].groupby('Заказчик + HH')))
    else:
        master_zav_nn = dict(tuple(potr_st[potr_st['month'] > current_date].groupby('Заказчик + HH')))
    pz_ost = ost_st
    # пз
#     print(master_zav_nn.keys())
    for pz_slave_row in pz_ost.index:
        df_master = ret_data_dict(pz_ost.at[pz_slave_row, 'Комплекс+Маттериал'], master_zav_nn)
        if df_master is None:
            df_master = pd.DataFrame()
        # если есть потребность у сцепки зав нн
        # pz_ost.at[pz_slave_row, 'осталось'] = 1600
        if len(df_master) > 0 and pz_ost.at[pz_slave_row, 'осталось'] > 0:
            df_master = df_master.sort_values('Дата потребности')
            # функция
            df_master = mex_NB(df_master, pz_ost, pz_slave_row, ost_st, potr_st)
            master_zav_nn[pz_ost.at[pz_slave_row, 'Комплекс+Маттериал']] = df_master
    if all_months ==True:
        master_zav_nn = dict(tuple(potr_st[potr_st['month'] >= current_date].groupby('Материал')))
    else:
        master_zav_nn = dict(tuple(potr_st[potr_st['month'] > current_date].groupby('Материал')))

    for pz_slave_row in pz_ost.index:
        if pz_ost.at[pz_slave_row, 'Комплекс'] == 'ALL1000':
            df_master = ret_data_dict(pz_ost.at[pz_slave_row, 'Материал'], master_zav_nn)
            if df_master is None:
                df_master = pd.DataFrame()
            # если есть потребность у сцепки зав нн
            # pz_ost.at[pz_slave_row, 'осталось'] = 1600
            if len(df_master) > 0 and pz_ost.at[pz_slave_row, 'осталось'] > 0:
                df_master = df_master[df_master['Комплекс'].isin(filters['one'])].sort_values('Дата потребности')
                # функция
                df_master = mex_NB(df_master, pz_ost, pz_slave_row, ost_st, potr_st)
                master_zav_nn[pz_ost.at[pz_slave_row, 'Материал']] = df_master


# мехника распределения остатков
def mex_NB(df_master, pz_ost, pz_slave_row, ost_st, potr_st):
#     print(df_master[['Распределение_остатака', 'Количество потребности', 'avizot']])
    ostatok = pz_ost.at[pz_slave_row, 'Количество пополнения']
    len_it = len(df_master) - 1
    for ind, row_master_index in enumerate(df_master.index):
        # если покрыта вся потребность то ничего не делаем
        #         if df_master.at[row_master_index, 'Количество потребности'] == df_master.at[row_master_index, 'Распределение_прихода']:
        #             pass
        # если это последний элемент нужно записать то что осталось в остатке
        #  1 прибавляем в столбец Распределение_остатака гласного фрейма все то что осталось
        #  2 вставляем в лист остатков в столбец забрали
        #  3 вставлем в маленькие выборки аналогично
        #  4 рассчитываем сумму по цене и прибавляем к имеющейся в ячейке
        if len_it == ind and (df_master.at[row_master_index, 'Количество потребности'] > (df_master.at[row_master_index, 'Распределение_остатака']+df_master.at[row_master_index, 'законтрактовано']+df_master.at[row_master_index, 'avizot'] + df_master.at[row_master_index, 'NB'])):
            #             price = ost_st.at[pz_slave_row, 'Сумма во ВВ'] / ost_st.at[pz_slave_row, 'Количество']
            #             summa = ostatok * price
            #             df_master.at[row_master_index, 'Сумма_распреденного_прихода'] = df_master.at[row_master_index, 'Сумма_распреденного_прихода'] + summa
            #             potr_st.at[row_master_index, 'Сумма_распреденного_прихода'] = potr_st.at[row_master_index, 'Сумма_распреденного_прихода'] + summa

            #             df_master.at[row_master_index, 'Распределение_прихода'] = df_master.at[row_master_index, 'Распределение_прихода'] + ostatok
            #             potr_st.at[row_master_index, 'Распределение_прихода'] = potr_st.at[row_master_index, 'Распределение_прихода'] + ostatok


            if (ostatok - (df_master.at[row_master_index, 'Количество потребности'] -df_master.at[row_master_index, 'законтрактовано']-df_master.at[row_master_index, 'avizot']- df_master.at[row_master_index, 'Распределение_остатака']-df_master.at[row_master_index, 'NB']))<0:
                df_master.at[row_master_index, 'NB'] =df_master.at[row_master_index, 'NB']+ ostatok
                potr_st.at[row_master_index, 'NB'] = potr_st.at[row_master_index, 'NB']+ ostatok
                ostatok = 0
                pz_ost.at[pz_slave_row, 'осталось'] = ostatok
                ost_st.at[pz_slave_row, 'осталось'] = ostatok
#                 print(df_master[['Количество потребности', 'законтрактовано', 'Распределение_остатака','NB']])
                return df_master
            else:
                a = df_master.at[row_master_index, 'NB']
                df_master.at[row_master_index, 'NB'] = df_master.at[row_master_index, 'NB'] + (df_master.at[row_master_index, 'Количество потребности'] - df_master.at[row_master_index, 'законтрактовано']-df_master.at[row_master_index, 'avizot'] - df_master.at[row_master_index, 'Распределение_остатака']-a)
                potr_st.at[row_master_index, 'NB'] = potr_st.at[row_master_index, 'NB'] + (potr_st.at[row_master_index, 'Количество потребности'] - potr_st.at[row_master_index, 'законтрактовано']-potr_st.at[row_master_index, 'avizot'] - potr_st.at[row_master_index, 'Распределение_остатака']-a)
                ostatok = ostatok - (df_master.at[row_master_index, 'Количество потребности']-potr_st.at[row_master_index, 'Распределение_остатака']-df_master.at[row_master_index, 'законтрактовано']-df_master.at[row_master_index, 'avizot']  - a)
#                 print(df_master[['Количество потребности', 'законтрактовано', 'Распределение_остатака', 'NB']])
                
                
                


                        # ostatok = 0
            # return df_master
        elif ostatok == 0:
            return df_master
        # если покарыта не вся потребность  и не последняя строка тогда и не хватает чтобы закрыть потребность:
        #  1 прибавляем в столбец  Распределение_остатака гласного фрейма потребностей только остаток
        #  2 зануляем остаток
        #  3 зануляем в листе остатков в столбец осталось
        #  4 вставлем в маленькие выборки аналогично
        #  5 рассчитываем сумму по цене и прибавляем к имеющейся в ячейке

        elif (df_master.at[row_master_index, 'Количество потребности'] > (df_master.at[row_master_index, 'Распределение_остатака'] +df_master.at[row_master_index, 'законтрактовано']+df_master.at[row_master_index, 'avizot']+ df_master.at[row_master_index, 'NB'])):



            if (ostatok - (df_master.at[row_master_index, 'Количество потребности'] -df_master.at[row_master_index, 'законтрактовано']-df_master.at[row_master_index, 'avizot']- df_master.at[row_master_index, 'Распределение_остатака']-df_master.at[row_master_index, 'NB']))<0:
                df_master.at[row_master_index, 'NB'] =df_master.at[row_master_index, 'NB']+ ostatok
                potr_st.at[row_master_index, 'NB'] = potr_st.at[row_master_index, 'NB']+ ostatok
                ostatok = 0
                pz_ost.at[pz_slave_row, 'осталось'] = 0
                ost_st.at[pz_slave_row, 'осталось'] = 0
#                 print(df_master[['Количество потребности', 'законтрактовано', 'Распределение_остатака','NB']])
                return df_master
            else:
                a = df_master.at[row_master_index, 'NB']
                df_master.at[row_master_index, 'NB'] = df_master.at[row_master_index, 'NB'] + (df_master.at[row_master_index, 'Количество потребности'] - df_master.at[row_master_index, 'законтрактовано']-df_master.at[row_master_index, 'avizot'] - df_master.at[row_master_index, 'Распределение_остатака']-a)
                potr_st.at[row_master_index, 'NB'] = potr_st.at[row_master_index, 'NB'] + (potr_st.at[row_master_index, 'Количество потребности'] - potr_st.at[row_master_index, 'законтрактовано']-potr_st.at[row_master_index, 'avizot'] - potr_st.at[row_master_index, 'Распределение_остатака']-a)
                ostatok = ostatok - (df_master.at[row_master_index, 'Количество потребности']-potr_st.at[row_master_index, 'Распределение_остатака']-df_master.at[row_master_index, 'законтрактовано']-df_master.at[row_master_index, 'avizot']  - a)
#                 print(df_master[['Количество потребности', 'законтрактовано', 'Распределение_остатака', 'NB']])
#             return df_master
        if len_it == ind and (df_master.at[row_master_index, 'Количество потребности'] <= (df_master.at[row_master_index, 'Распределение_остатака']+df_master.at[row_master_index, 'законтрактовано']+df_master.at[row_master_index, 'avizot'] + df_master.at[row_master_index, 'NB'])):
            df_master.at[row_master_index, 'NB'] = df_master.at[row_master_index, 'NB'] + 0
            potr_st.at[row_master_index, 'NB'] = potr_st.at[row_master_index, 'NB'] + 0
#             print(df_master[['Количество потребности', 'законтрактовано', 'Распределение_остатака','NB']])
            return df_master

In [ ]:
def str_zap_raspr(ost_st, potr_st, current_date, filters,all_months):
    #     группировка остатков по
    if all_months==True:
        master_zav_nn = dict(tuple(potr_st[potr_st['month'] >= current_date].groupby('Заказчик + HH')))
    else:
        master_zav_nn = dict(tuple(potr_st[potr_st['month'] > current_date].groupby('Заказчик + HH')))
    pz_ost = ost_st
    for pz_slave_row in pz_ost.index:
        if pz_ost.at[pz_slave_row, 'Комплекс'] != 'ALL1000':
            df_master = ret_data_dict(pz_ost.at[pz_slave_row, 'Заказчик + HH'], master_zav_nn)
            if df_master is None:
                df_master = pd.DataFrame()
            # если есть потребность у сцепки зав нн
            # pz_ost.at[pz_slave_row, 'осталось'] = 1600
            if len(df_master) > 0 and pz_ost.at[pz_slave_row, 'Количество потребности'] > 0:
                df_master = df_master.sort_values('Дата потребности')
                # функция
                df_master = mexstr_zap_avizot(df_master, pz_ost, pz_slave_row, ost_st, potr_st)
                master_zav_nn[pz_ost.at[pz_slave_row, 'Материал']] = df_master
    if all_months==True:
        master_zav_nn = dict(tuple(potr_st[potr_st['month'] >= current_date].groupby('Материал')))
    else:
        
        master_zav_nn = dict(tuple(potr_st[potr_st['month'] > current_date].groupby('Материал')))

    for pz_slave_row in pz_ost.index:
        if pz_ost.at[pz_slave_row, 'Комплекс'] == 'ALL1000':
            df_master = ret_data_dict(pz_ost.at[pz_slave_row, 'Материал'], master_zav_nn)
            if df_master is None:
                df_master = pd.DataFrame()
            # если есть потребность у сцепки зав нн
            # pz_ost.at[pz_slave_row, 'осталось'] = 1600
            if len(df_master) > 0 and pz_ost.at[pz_slave_row, 'Количество потребности'] > 0:
                df_master = df_master[df_master['Комплекс'].isin(filters['one'])].sort_values('Дата потребности')
                # функция
                df_master = mexstr_zap_avizot(df_master, pz_ost, pz_slave_row, ost_st, potr_st)
                master_zav_nn[pz_ost.at[pz_slave_row, 'Материал']] = df_master
#     potr_st.to_csv('C:\\Users\\User\\Desktop\\kazzinc\\Uploads\\test\\dfsl.csv', encoding='utf-8-sig', sep=';')
#     ost_st.to_csv('C:\\Users\\User\\Desktop\\kazzinc\\Uploads\\test\\ost.csv', encoding='utf-8-sig', sep=';')

# мехника распределения остатков
def mexstr_zap_avizot(df_master, pz_ost, pz_slave_row, ost_st, potr_st):
    arr = []
    ostatok = pz_ost.at[pz_slave_row, 'Количество потребности']
    potr = df_master['Количество потребности'].sum()
    len_it = len(df_master) - 1
    for ind, row_master_index in enumerate(df_master.index):
        # если покрыта вся потребность то ничего не делаем
        #         if df_master.at[row_master_index, 'Количество потребности'] == df_master.at[row_master_index, 'Распределение_прихода']:
        #             pass
        # если это последний элемент нужно записать то что осталось в остатке
        #  1 прибавляем в столбец Распределение_остатака гласного фрейма все то что осталось
        #  2 вставляем в лист остатков в столбец забрали
        #  3 вставлем в маленькие выборки аналогично
        #  4 рассчитываем сумму по цене и прибавляем к имеющейся в ячейке
        if len_it == ind:
            arr.append(df_master.at[row_master_index, 'Количество потребности'] / potr * ostatok)
            #             df_master.at[row_master_index, 'Распределение_прихода'] = df_master.at[row_master_index, 'Распределение_прихода'] + ostatok
            #             potr_st.at[row_master_index, 'Распределение_прихода'] = potr_st.at[row_master_index, 'Распределение_прихода'] + ostatok

            #             pz_ost.at[pz_slave_row, 'осталось'] = 0
            #             ost_st.at[pz_slave_row, 'осталось'] = 0
            #             ostatok = 0

            df_master.loc[df_master.index, 'стрЗапас'] = df_master.loc[df_master.index, 'стрЗапас']+ arr
            potr_st.loc[df_master.index, 'стрЗапас'] = potr_st.loc[df_master.index, 'стрЗапас']+ arr
            pz_ost.at[pz_slave_row, 'осталось'] = 0
            ost_st.at[pz_slave_row, 'осталось'] = 0
            return df_master
        else:
            arr.append(df_master.at[row_master_index, 'Количество потребности'] / potr * ostatok)

In [ ]:
# Даты для распределения
ost_name = 'Остатки на 29.02.2020'
prih_name = 'Приход - февраль 2020 г.'
rash_name = 'Расход - февраль 2020 г.'
folder = '0320'
pl_kurs = 375

date_start_year = datetime.datetime(2020,1,1)
current_date = datetime.datetime(2020,3,1)

# Для непосредственного формирования F0 
current_date_f0 = datetime.datetime(2020,3,1)
first_month_of_kvartal = 1



directory = 'C:\\Users\\Илья\\Desktop\\kazzinc\\Uploads1\\'+folder+'\\'
directory_mc = 'C:\\Users\\Илья\\Desktop\\kazzinc\\Uploads1\\'+folder+'\\для ZMM_MATCATAL\\'




# список месяцев потребности для формирования f0 ДЛЯ 1 месяца и считаем от остатков на начало года
sp = ['ПП кол-во '+str(x)+'/'+str(current_date_f0.year) for x in  range(first_month_of_kvartal, current_date_f0.month+1)]
# список месяцев потребности для формирования f0 ДЛЯ 1 месяца и считаем от текущих остатков
# sp = ['ПП кол-во '+str(x)+'/'+str(current_date_f0.year) for x in  range(first_month_of_kvartal, current_date_f0.month+2)]
print(sp)

In [ ]:
# загрузка из restore


df = get_in_backup(path_backup='backup.h5',name='extract')

# df = pd.read_csv(directory+'CSV.csv',sep = ';',parse_dates=['Дата потребности'],dtype={'Материал':'str','Завод':'str'})
sprav_po_kompl = get_in_backup(path_backup='backup.h5',name='sprav_po_kompl')
sprav_mat_cal = get_in_backup(path_backup='backup.h5',name='sprav_mat_cal')
sprav_mdlv = get_in_backup(path_backup='backup.h5',name='sprav_mdlv')
sprav_nedra = get_in_backup(path_backup='backup.h5',name='sprav_nedra')
central_zavod = get_in_backup(path_backup='backup.h5',name='central_zavod')
group_mat = get_in_backup(path_backup='backup.h5',name='group_mat')
A900 = get_in_backup(path_backup='backup.h5',name='A900')
A902 = get_in_backup(path_backup='backup.h5',name='A902')
KONP = get_in_backup(path_backup='backup.h5',name='KONP')
ZMM_E0078_MDLV = get_in_backup(path_backup='backup.h5',name='ZMM_E0078_MDLV')
filter_extract = get_in_backup(path_backup='backup.h5',name='filter_extract')
filter_else = get_in_backup(path_backup='backup.h5',name='filter_else')
df_str_zap = get_in_backup(path_backup='backup.h5',name='df_str_zap')
ost_nach_perioda = get_in_backup(path_backup='backup.h5',name='ost_nach_perioda')
sprav_mat_cal_last_full = get_in_backup(path_backup='backup.h5',name='sprav_mat_cal_last_full')
sprav_spp = get_in_backup(path_backup='backup.h5',name='sprav_spp')
spr_for_obl_ppm = get_in_backup(path_backup='backup.h5',name='spr_for_obl_ppm')
T001W = get_in_backup(path_backup='backup.h5',name='T001W')
ADRC = get_in_backup(path_backup='backup.h5',name='ADRC')
TWLAD = get_in_backup(path_backup='backup.h5',name='TWLAD')
T001L = get_in_backup(path_backup='backup.h5',name='T001L')
ved_mat = get_in_backup(path_backup='backup.h5',name='ved_mat')
KONH = get_in_backup(path_backup='backup.h5',name='KONH')
MARM = get_in_backup(path_backup='backup.h5',name='MARM')

In [ ]:
# загрузка источников


df = pd.read_csv(directory+'CSV.csv',sep = ';',parse_dates=['Дата потребности'],dtype={'Материал':'str','Завод':'str'})
sprav_po_kompl = pd.read_excel(directory+'Справочник по комплексам.xlsx',sheet_name='Комплекс',dtype={'Область ППМ':'str','Завод':'str'})
sprav_mat_cal = exlsx_f(directory_mc)

# ost_nach_perioda =  pd.read_excel(directory+'ost_nach_perioda.xlsx',sheet_name='sheet1')
# ost_nach_perioda = get_in_backup(path_backup='backup_fine.h5',name='ost_nach_perioda')


In [ ]:
sprav_mdlv = pd.read_excel(directory+'MDLV.xlsx',sheet_name='Sheet1')
sprav_nedra = pd.read_excel(directory+'Справочник недрапользование.xlsx',sheet_name='Недра-ненедра')
central_zavod = pd.read_excel(directory+'Справочник отношения Завода к центральному заводу.xlsx',sheet_name='Sheet1')
group_mat = pd.read_excel(directory+'T023.xlsx',sheet_name='Sheet1',dtype={'Группа материалов':'str'})
A900 = pd.read_excel(directory+'A900.xlsx',sheet_name='Sheet1',dtype={'Номер записи условий':'str'})
A902 = pd.read_excel(directory+'A902.xlsx',sheet_name='Sheet1',dtype={'Номер записи условий':'str'})
KONP = pd.read_excel(directory+'KONP.xlsx',sheet_name='Sheet1',dtype={'Номер записи условий':'str'})
ZMM_E0078_MDLV = pd.read_excel(directory+'ZMM_E0078_MDLV.xlsx',sheet_name='Sheet1',dtype={'Область ППМ':'str','Код округа':'str'})
filter_extract = pd.read_excel(directory+'filters.xlsx',sheet_name='filter_extract')
filter_else = pd.read_excel(directory+'filters.xlsx',sheet_name='filter_else')
# sprav_spp = pd.read_excel(directory+'СПП.xlsx',sheet_name='Sheet1')
spr_for_obl_ppm = pd.read_excel(directory+'MDLG.xlsx',sheet_name='Sheet1')
T001W = pd.read_excel(directory+'T001W.xlsx',sheet_name='Sheet1')

In [ ]:
ADRC = pd.read_excel(directory+'ADRC.xlsx',sheet_name='Sheet1')
TWLAD = pd.read_excel(directory+'TWLAD.xlsx',sheet_name='Sheet1')
T001L = pd.read_excel(directory+'T001L.xlsx',sheet_name='Sheet1')
ved_mat = pd.read_excel(directory+'Выгрузка по ведущему материалу.xlsx',sheet_name='Sheet1')

In [ ]:
KONH = pd.read_excel(directory+'KONH.xlsx',sheet_name='Sheet1')
MARM = pd.read_excel(directory+'MARM.xlsx',sheet_name='Sheet1')

In [ ]:
A901 = pd.read_excel(directory+'A901.xlsx',sheet_name='Sheet1')

In [ ]:
def func_KONH(x):
    if x['Таблица']=='900':return x['Переменный ключ']
    elif x['Таблица']=='901':return x['Переменный ключ'][4:]
    elif x['Таблица']=='902':return x['Переменный ключ'][3:]
    else:return 'пусто'    
    
KONH.fillna(0,inplace=True)
KONP.fillna(0,inplace=True)
MARM.fillna(0,inplace=True)

KONP = KONP[KONP['Индикатор удаления']!='X']

to_str_KONH = ['Номер записи условий','Таблица','Переменный ключ','Вид условия']
for i in to_str_KONH:
    KONH[i] = KONH[i].apply(lambda x: str(x).replace('.0','')).astype('str')
    KONH[i] = KONH[i].apply(lambda x: '' if x=='0' else x)
    
to_str_KONP = ['Номер записи условий','Единица измерения','Вид условия','Валюта условия']
for i in to_str_KONP:
    KONP[i] = KONP[i].apply(lambda x: str(x).replace('.0','')).astype('str')
    KONP[i] = KONP[i].apply(lambda x: '' if x=='0' else x)
    
KONH = KONH[(KONH['Вид условия']=='ZP00')&((KONH['Таблица']=='900')|(KONH['Таблица']=='901')|(KONH['Таблица']=='902'))]   
KONH['Материал'] = KONH.apply(lambda x:func_KONH(x),axis = 1)

KONH['округ'] = KONH.apply(lambda x: x['Переменный ключ'][:2] if x['Таблица']=='902' else 'пусто',axis = 1)
KONH['Завод'] = KONH.apply(lambda x: x['Переменный ключ'][:4] if x['Таблица']=='901' else 'пусто',axis = 1)
KONH.reset_index(drop = True,inplace = True)


to_str_MARM = ['Материал','Альтернативная ЕИ']
for i in to_str_MARM:
    MARM[i] = MARM[i].apply(lambda x: str(x).replace('.0','')).astype('str')
    MARM[i] = MARM[i].apply(lambda x: '' if x=='0' else x)


for d in [A900,A901,A902]:
    d['Номер записи условий'] =  d['Номер записи условий'].apply(lambda x: str(x).replace('.0','')).astype('str')
    
fil = KONH['Номер записи условий'].isin(pd.concat([A900['Номер записи условий'],A901['Номер записи условий'],A902['Номер записи условий']]).drop_duplicates())
KONH = KONH[fil]  


In [ ]:
KONP['Номер записи условий'] = KONP['Номер записи условий'].astype('int').astype('str')

In [ ]:
# |||||||||||||||||||||блок обработка данных листа  extract
# заполняю даты нулевыми если 0
df['Дата потребности'] = df['Дата потребности'].apply(lambda x: '01.01.1900' if x =='00.00.0000' else  x)
df['Дата потребности'] = [datetime.datetime(int(x.split('.')[2]),int(x.split('.')[1]),int(x.split('.')[0])) for x in df['Дата потребности'].values]
# собираю дату с группировкой по месяцу
month = df['Дата потребности'].apply(lambda x:  str(x.year)+'-'+str(x.month)+'-01'+ ' 00:00:00')
month =pd.to_datetime(month) 
# заменяю символ - 
df['Количество потребности'] = df['Количество потребности'].str.replace('-','')
df['Баланс (закрытие потребности)'] = df['Баланс (закрытие потребности)'].str.replace('-','')
# Работаю с числами
df['Вид документа потребности'] = df['Вид документа потребности'].astype('str')
df['Вид документа пополнения'] = df['Вид документа пополнения'].astype('str')
# меняем цифры на нормальные флоат
replace_float(df,['Количество потребности','Количество пополнения','Баланс (закрытие потребности)'])

df.fillna(0,inplace=True)
sprav_po_kompl.fillna(0,inplace=True)
sprav_nedra.fillna(0,inplace=True)
T001W.fillna(0,inplace=True)

In [ ]:
# заполняем дополнительные столбцы в экстракте

# df = df[df['Дата потребности']<=datetime.datetime(2020,2,29)]
df['Завод'] = df['Завод'].apply(lambda x: str(x).replace('.0','')).astype('str')
spr_zav_dict = rev_to_dict(sprav_po_kompl['Завод'],sprav_po_kompl['ЦС/ПЗ'])
df['ЦС/ПЗ'] = df['Завод'].apply(lambda x:  insert_vpr(x,spr_zav_dict,'ЦС/ПЗ'))
df['Номер потребности'] = df['Номер потребности'].apply(lambda x: str(x).replace('.0','')).astype('str')
df['МВЗ'] = df['МВЗ'].apply(lambda x: str(x).replace('.0','')).astype('str')
df['Материал'] = df['Материал'].apply(lambda x: str(x).replace('.0','')).astype('str')

df['Тип материала'] = df['Тип материала'].apply(lambda x: str(x).replace('.0','')).astype('str')
df['БЕ'] = df['БЕ'].apply(lambda x: str(x).replace('.0','')).astype('str')



# недропользование неполнятно
spr_nedra = rev_to_dict(sprav_nedra['Завод + Склад'],sprav_nedra['Завод + Склад'])
# df['Недропользование']= df['Завод'].apply(lambda x: insert_vpr(x,spr_nedra,'Завод'))   
df['Склад потребности'] = df['Склад потребности'].apply(lambda x: str(x).replace('.0','')).astype('str')
df['Завод + Склад'] = df['Завод']+' '+df['Склад потребности']
df['Недропользование'] = df['Завод + Склад'].apply(lambda x: 0 if   insert_vpr(x,spr_nedra,'Завод + Склад')==0 else 1)

# Вывод по докум-ам потребности
df['Вывод по документам потребности'] = df.apply(lambda x: formula_vivod_po_doc_potrebn(x),axis=1 )


df['Вывод по документам пополнения'] = df.apply(lambda x: formula_vivod_po_doc_popolneniya(x),axis=1 )


df['Модуль'] = df.apply(lambda x: modul(x),axis=1 )


df['Завод+НН'] = df['Завод'].astype('str')+' #'+df['Материал'].astype('str')
df['Завод+НН'] = df['Завод+НН'].str.strip()

df['Номер элемента пополнения'] = df['Номер элемента пополнения'].apply(lambda x: str(x).replace('.0',''))

df['Номер элемента пополнения + Номер позиции пополнения'] = df['Номер элемента пополнения'].astype('str')+' '+df['Номер позиции пополнения'].astype('str')
df['Материал'] = df['Материал'].apply(lambda x: '0'+str(x) if  len(str(x))<9 else x)


sprav_mat_cal_last_full = sprav_mat_cal.copy()

to_str_spr_mat = ['СтатусМтрл на заводе','Завод','Материал']

for i in to_str_spr_mat:
    sprav_mat_cal_last_full[i].fillna(0,inplace = True)
    sprav_mat_cal_last_full[i] = sprav_mat_cal_last_full[i].apply(lambda x: str(x).replace('.0','')).astype('str')
    sprav_mat_cal_last_full[i] = sprav_mat_cal_last_full[i].apply(lambda x: '' if x=='0' or x==0 else x)
    sprav_mat_cal[i].fillna(0,inplace = True)
    sprav_mat_cal[i] = sprav_mat_cal[i].apply(lambda x: str(x).replace('.0','')).astype('str')
    sprav_mat_cal[i] = sprav_mat_cal[i].apply(lambda x: '' if x=='0' or x==0 else x)

sprav_mat_cal_last_full['Материал'] = sprav_mat_cal_last_full['Материал'].apply(lambda x: '0'+str(x) if  len(str(x))<9 else x)

sprav_mat_cal['Завод'] = sprav_mat_cal['Завод'].apply(lambda x: str(x).replace('.0','')).astype('str')
sprav_mat_cal['Материал'] = sprav_mat_cal['Материал'].apply(lambda x: str(x).replace('.0','')).astype('str')
sprav_mat_cal['Материал'] = sprav_mat_cal['Материал'].apply(lambda x: '0'+str(x) if  len(str(x))<9 else x)

sprav_mat_cal = sprav_mat_cal[sprav_mat_cal['Материал'].isin(df['Материал'].unique())]



df['Номер элемента пополнения + Номер позиции пополнения + Наименование типа элемента ППМ пополн'] = df['Номер элемента пополнения + Номер позиции пополнения'].astype('str')+ df.apply(lambda x: 'ПртПЗк' if x['Вид документа пополнения']=='UB05'else x['Наименование типа элемента ППМ пополн'] ,axis=1).astype('str')



df['Номер элемента пополнения + Номер позиции пополнения'] = df['Номер элемента пополнения'].astype('str')+' '+df['Номер позиции пополнения'].astype('str')

df['Область ППМ'] = df['Область ППМ'].apply(lambda x: str(x).replace('.0','')).astype('str')

df['Область ППМ'] = df['Область ППМ'].apply(lambda x:"" if x=='0' else x)


df['Завод + Область ППМ'] = df['Завод'].astype('str')+' '+df['Область ППМ'].astype('str')

# чистим
df['Завод + Область ППМ'] =  df['Завод + Область ППМ'].apply(lambda x: str(x).replace('0.0','').replace('.0','')).str.strip()




spr_zav_obl =  rev_to_dict(sprav_mdlv['Область ППМ'].astype('str').str.strip(),sprav_mdlv['Текст ОблПланирПотр'])
df['Имя Завода'] = df.apply(lambda x: insert_vpr(x['Завод'],spr_zav_obl,'Текст ОблПланирПотр')  if x['Область ППМ']=='' else insert_vpr(x['Область ППМ'],spr_zav_obl,'Текст ОблПланирПотр'),axis = 1 )


spr_zav_kompl = rev_to_dict(sprav_po_kompl['Завод + Область ППМ'].astype('str').str.strip(),sprav_po_kompl['Комплекс'])
df['Комплекс'] = df['Завод + Область ППМ'].apply(lambda x: insert_vpr(x,spr_zav_kompl,'Комплекс') )

df['Заказчик + HH'] =df['Комплекс'].astype('str')+' '+df['Материал'].astype('str')


df['month'] = df['Дата потребности'].apply(lambda x: str(x.year)+'-'+str(x.month)+ '-01')
df['month'] = pd.to_datetime(df['month']) 

spr_obl_ppm = rev_to_dict(spr_for_obl_ppm['Завод'].apply(lambda x: str(x).replace('.0','')).astype('str')+spr_for_obl_ppm['Склад'].\
            apply(lambda x: str(x).replace('.0','')).astype('str'),spr_for_obl_ppm['Область ППМ'])
df['Область ППМ']  = df\
                                    .apply(lambda x: x['Завод'] if    insert_vpr(x['Завод']+x['Склад потребности'],spr_obl_ppm,'Область ППМ')==0 else  \
                                     insert_vpr(x['Завод']+x['Склад потребности'],spr_obl_ppm,'Область ППМ'),axis = 1).astype('str')

spr_kod_okruga = rev_to_dict(ZMM_E0078_MDLV['Область ППМ'].apply(lambda x: str(x).replace('.0',''))\
                             .astype('str').str.strip(),ZMM_E0078_MDLV['Код округа'])

spr_kod_okruga1 = rev_to_dict(T001W['Завод'].apply(lambda x: str(x).replace('.0','')).astype('str'),\
            T001W['Код округа'].apply(lambda x: str(x).replace('.0','')).astype('str'))

df['Код округа'] = df.apply(lambda x: insert_vpr(x['Завод'],spr_kod_okruga1,'Код округа') if insert_vpr(x['Область ППМ'],spr_kod_okruga,'Код округа')==0 else insert_vpr(x['Область ППМ'],spr_kod_okruga,'Код округа'),axis = 1).astype('str')

central_zavod['Область ППМ'] = central_zavod['Область ППМ'].astype('str')
spr_obl_ppm_centr_zav = rev_to_dict(central_zavod['Область ППМ'],central_zavod['Центральный Завод'])
df['Центральный завод'] = df['Область ППМ'].apply(lambda x:  insert_vpr(x,spr_obl_ppm_centr_zav,'Центральный Завод'))

spr_obl_ppm_centr_zav = rev_to_dict(sprav_mat_cal['Завод']+' #'+sprav_mat_cal['Материал'],sprav_mat_cal['Краткий текст материала'])
df['КрТекстМатериала'] = df['Завод+НН'].apply(lambda x:  insert_vpr(x,spr_obl_ppm_centr_zav,'Краткий текст материала'))

spr_obl_ppm_baz_ei = rev_to_dict(sprav_mat_cal['Завод']+' #'+sprav_mat_cal['Материал'],sprav_mat_cal['Базовая ЕИ'])
df['Базовая ЕИ'] = df['Завод+НН'].apply(lambda x:  insert_vpr(x,spr_obl_ppm_baz_ei,'Базовая ЕИ'))

spr_obl_ppm_ved_mat = rev_to_dict(sprav_mat_cal['Завод']+' #'+sprav_mat_cal['Материал'],sprav_mat_cal['Ведущий материал'])
df['Признак аналога'] = df['Завод+НН'].apply(lambda x:  insert_vpr(x,spr_obl_ppm_ved_mat,'Ведущий материал'))

spr_obl_ppm_stat_na_zav = rev_to_dict(sprav_mat_cal['Завод']+' #'+sprav_mat_cal['Материал'],sprav_mat_cal['СтатусМтрл на заводе'])
df['СтатусМтрл на заводе'] = df['Завод+НН'].apply(lambda x:  insert_vpr(x,spr_obl_ppm_stat_na_zav,'СтатусМтрл на заводе'))


spr_obl_ppm_ocenka = rev_to_dict(sprav_mat_cal['Материал'],sprav_mat_cal['Класс оценки'])
df['Класс оценки'] = df['Материал'].apply(lambda x:  insert_vpr(x,spr_obl_ppm_ocenka,'Класс оценки'))
df['Класс оценки'] = df['Класс оценки'].apply(lambda x: str(x).replace('.0','')).astype('str')




spr_obl_ppm_name_planov= rev_to_dict(sprav_mat_cal['Завод']+' #'+sprav_mat_cal['Материал'],sprav_mat_cal['Имя плановика (завод)'])
df['ЦЗ+НН'] = df['Центральный завод'].astype('str')+' #'+df['Материал'].astype('str')
df['Плановик'] = df['ЦЗ+НН'].apply(lambda x:  insert_vpr(x,spr_obl_ppm_name_planov,'Имя плановика (завод)'))


spr_group_mat_l1 = rev_to_dict(group_mat[group_mat['Группа материалов'].str.len()==2]['Группа материалов'],group_mat[group_mat['Группа материалов'].str.len()==2]['Обозначение 2 группы материалов'])
df['группа'] = df['Материал'].str[:2].apply(lambda x:  insert_vpr(x,spr_group_mat_l1,'Обозначение 2 группы материалов'))

spr_group_mat_l2 = rev_to_dict(group_mat[group_mat['Группа материалов'].str.len()==4]['Группа материалов'],group_mat[group_mat['Группа материалов'].str.len()==4]['Обозначение 2 группы материалов'])
df['подгруппа'] = df['Материал'].str[:4].apply(lambda x:  insert_vpr(x,spr_group_mat_l2,'Обозначение 2 группы материалов'))


spr_group_mat_l3 = rev_to_dict(group_mat[group_mat['Группа материалов'].str.len()==6]['Группа материалов'],group_mat[group_mat['Группа материалов'].str.len()==6]['Обозначение 2 группы материалов'])
df['подподгруппа'] = df['Материал'].str[:6].apply(lambda x:  insert_vpr(x,spr_group_mat_l3,'Обозначение 2 группы материалов'))

df['ТИП номенклатуры'] = 0.0



# df = price_main__(df,A902,A900,KONP,current_date,pl_kurs)




spr_zav_mat = rev_to_dict(sprav_mat_cal['Завод']+' '+sprav_mat_cal['Материал'],sprav_mat_cal['Плановый срок поставки (завод)'])
df['Центральный завод'] = df['Центральный завод'].astype('str')
for i in df.index:
        try:
            if df.at[i,'Недропользование']==1:df.at[i,'ПлановСрокПоставки'] =  spr_zav_mat[df.at[i,'Завод']+' '+df.at[i,'Материал']]['Плановый срок поставки (завод)']
            else:df.at[i,'ПлановСрокПоставки'] = spr_zav_mat[df.at[i,'Центральный завод']+' '+df.at[i,'Материал']]['Плановый срок поставки (завод)']
        except KeyError:
            pass


df.fillna(0,inplace=True)

# копия для формирования страхового запаса
df_str_zap = df.copy()
# df_str_zap = df_str_zap[df_str_zap['Дата потребности']<=datetime.datetime(2020,2,29)]
df_str_zap.name =  'df_str_zap'
# df = df[df['Дата потребности']<=datetime.datetime(2020,2,29)]

df = df[(df['ЦС/ПЗ']=='ПЗ')&(df['Вывод по документам потребности']=='OK')&(df['Баланс (закрытие потребности)']==0)&(df['Тип ППМ']=='PD')]
df = df[df['Материал'].apply(lambda x:len(x)<=10)]
df = df[df['Особый запас']!='Q']
mat_cat_str_zap = sprav_mat_cal_last_full[sprav_mat_cal_last_full['Страховой запас']>0]
df = pd.concat([df,mat_cat_str_zap[['Материал','Завод','Код округа','Страховой запас']]])






In [ ]:
df = parallel_calc_fast(df,KONH,current_date,KONP,MARM,pl_kurs)



In [ ]:
df.fillna(0,inplace = True)

df['month'] =    df.apply(lambda x: datetime.datetime(2199,1,1)  if x['Страховой запас']>0 else x['month'],axis = 1)
df['Дата потребности'] =    df.apply(lambda x: datetime.datetime(2199,1,1)  if x['Страховой запас']>0 else x['Дата потребности'],axis = 1)
df['ЦС/ПЗ'] =  df.apply(lambda x: 'ПЗ'  if x['Страховой запас']>0 else x['ЦС/ПЗ'],axis = 1)
df['Завод+НН'] = df['Завод'].astype('str')+' #'+df['Материал'].astype('str')
df['Завод+НН'] = df['Завод+НН'].str.strip()
df['Комплекс'] = df.apply(lambda x:  insert_vpr(x['Завод'],spr_zav_kompl,'Комплекс') if x['Комплекс']==0 else x['Комплекс'],axis = 1)
df.reset_index(inplace = True,drop = True)

In [ ]:
# backup
ZMM_E0078_MDLV.name = 'ZMM_E0078_MDLV'
df.name = 'extract'
sprav_po_kompl.name = 'sprav_po_kompl'
sprav_mdlv.name = 'sprav_mdlv'
sprav_nedra.name = 'sprav_nedra'
central_zavod.name = 'central_zavod'
group_mat.name = 'group_mat'
df.name = 'extract'
sprav_mat_cal.name = 'sprav_mat_cal'
KONP.name = 'KONP'
A902.name = 'A902'
A900.name = 'A900'
filter_extract.name = 'filter_extract'
filter_else.name = 'filter_else'
sprav_mat_cal_last_full.name = 'sprav_mat_cal_last_full'

spr_for_obl_ppm.name = 'spr_for_obl_ppm'
T001W.name = 'T001W'
ADRC.name = 'ADRC'
TWLAD.name = 'TWLAD'
T001L.name = 'T001L'
ved_mat.name = 'ved_mat'
KONH.name = 'KONH'
MARM.name = 'MARM'
A901.name = 'A901'

save_backup('backup.h5',[sprav_mat_cal_last_full,sprav_mat_cal,KONP,df,A902,A900,ZMM_E0078_MDLV,sprav_po_kompl,sprav_mdlv,sprav_nedra,central_zavod,group_mat,filter_extract,filter_else,df_str_zap,spr_for_obl_ppm,T001W\
                        ,ADRC,TWLAD,T001L,ved_mat,KONH,MARM,A901])

In [ ]:
df['month'] =    df.apply(lambda x: datetime.datetime(2199,1,1)  if x['Страховой запас']>0 else x['month'],axis = 1)
df['Дата потребности'] =    df.apply(lambda x: datetime.datetime(2199,1,1)  if x['Страховой запас']>0 else x['Дата потребности'],axis = 1)

In [ ]:
spr_zav_kompl

In [ ]:
# ostatok = pd.read_excel(directory+'Выгрузки\\'+ost_name+'.xlsx',sheet_name='Sheet1')
# ostatok['Ст-ть запаса в конце периода'] = ostatok['сумма USD']

In [ ]:
ostatok = pd.read_excel(directory+'Выгрузки\\'+ost_name+'.xlsx',sheet_name='Sheet1')
block_zapas = pd.read_excel(directory+'Блокированный запас.xlsx',sheet_name='Sheet1')
spr_ost = pd.read_excel(directory+'Справочник_остатки.xlsx',sheet_name='Лист1')
# ostatok['Ст-ть запаса в конце периода'] = ostatok['сумма USD']
# to_str_spp = ['СПП-элемент','Место возникновения прибыли']

# for i in to_str_spp:
#     sprav_spp[i] = sprav_spp[i].apply(lambda x: str(x).replace('.0','')).astype('str')
#     sprav_spp[i] = sprav_spp[i].apply(lambda x: '' if x=='0' or x==0 else x)

spr_ost.fillna(0,inplace=True)
ostatok.rename(columns={"Кол-во запаса в конце периода (31.12.2019)": "Кол-во запаса в конце периода"},inplace=True)

spr_ost['Завод'] = spr_ost['Завод'].apply(lambda x: str(x).replace('.0','')).astype('str')
spr_ost['Склад'] = spr_ost['Склад'].apply(lambda x: str(x).replace('.0','')).astype('str')
spr_ost['Партия']  =spr_ost['Партия'].apply(lambda x: str(x).replace('.0','')).astype('str')

ostatok.dropna(thresh=3,inplace=True)
block_zapas.dropna(thresh=2,inplace=True)
ostatok.fillna(0,inplace=True)
# spr__spp = rev_to_dict(sprav_spp['СПП-элемент'],sprav_spp['Место возникновения прибыли'])
# ostatok['Завод'] = ostatok.apply(lambda x: x['Завод'] if  insert_vpr(x['СПП-элемент'],spr__spp,'Место возникновения прибыли')==0 else\
#               insert_vpr(x['СПП-элемент'],spr__spp,'Место возникновения прибыли'),axis = 1)
ostatok['Материал'] = ostatok['Материал'].apply(lambda x: str(x).replace('.0','')).astype('str')
ostatok['Завод'] = ostatok['Завод'].apply(lambda x: str(x).replace('.0','')).astype('str')
# ostatok['Завод'] = ostatok['Завод'].apply(lambda x: '' if x=='0' or x==0 else x)
ostatok['Склад'] = ostatok['Склад'].apply(lambda x: str(x).replace('.0','')).astype('str')
# ostatok['Склад'] = ostatok['Склад'].apply(lambda x: '' if x=='0' or x==0 else x)
ostatok['Материал'] = ostatok['Материал'].apply(lambda x: '0'+str(x) if  len(str(x))<9 else x)
ostatok['БЕ'] = ostatok['БЕ'].apply(lambda x: str(x).replace('.0','')).astype('str')
ostatok['Основной счет'] = ostatok['Завод'].apply(lambda x: str(x).replace('.0','')).astype('str')


ostatok['Партия'] = ostatok['Партия'].astype('str')

spr_komplex_cs_pz = rev_to_dict(sprav_po_kompl['Завод'].astype('str'),sprav_po_kompl['ЦС/ПЗ'])

ostatok['ЦС/ПЗ'] = ostatok['Завод'].apply(lambda x:  insert_vpr(x,spr_komplex_cs_pz,'ЦС/ПЗ'))
ostatok['Завод+Склад'] = ostatok['Завод'].astype('str')+' '+ostatok['Склад'].astype('str')
ostatok['Завод+Склад']  = ostatok['Завод+Склад'].astype('str')


ostatok['Завод+склад+партия+особый запас+НН'] = ostatok['Завод'].astype('str')+' '+ostatok['Склад'].astype('str')+' '+ostatok['Партия'].astype('str')+' '+ostatok['Особый запас'].astype('str')+' '+ostatok['Материал'].astype('str')
ostatok['Завод+Склад+Партия'] = ostatok['Завод'].astype('str')+' '+ostatok['Склад'].astype('str')+' '+ostatok['Партия'].astype('str')




block_zapas.fillna(0,inplace=True)

block_zapas['Номер материала'] = block_zapas['Номер материала'].apply(lambda x: str(x).replace('.0','')).astype('str')
block_zapas['Завод'] = block_zapas['Завод'].apply(lambda x: str(x).replace('.0','')).astype('str')
block_zapas['Склад'] = block_zapas['Склад'].apply(lambda x: str(x).replace('.0','')).astype('str')
block_zapas['Особый запас'] = block_zapas['Особый запас'].astype('int')


block_zapas['Завод+склад+партия+особый запас+НН'] = block_zapas['Завод'].astype('str')+' '+block_zapas['Склад'].astype('str')+' '+block_zapas['Партия'].astype('str')+' '+block_zapas['Особый запас'].astype('str')+' '+block_zapas['Номер материала'].astype('str')
block_zapas.rename(columns={"Номер материала": "Материал",'Краткий текст материала':'КрТекстМатериала'},inplace=True)
block_zapas = block_zapas[['Завод+склад+партия+особый запас+НН','Блокированный запас','Ст-ть/блок. запаса','Материал','Завод','Склад','Партия','Базовая ЕИ','КрТекстМатериала']].groupby(['Завод+склад+партия+особый запас+НН','Материал','Завод','Склад','Партия','Базовая ЕИ','КрТекстМатериала']).agg({'Блокированный запас':'sum','Ст-ть/блок. запаса':'sum'}).reset_index()
ostatok.fillna(0,inplace=True)
block_zapas = block_zapas[block_zapas['Завод+склад+партия+особый запас+НН']!='0 0 0 0 0']

ostatok = pd.merge(ostatok,block_zapas,how='outer')
ostatok.fillna(0,inplace=True)

ostatok = ostatok[ostatok['Кол-во запаса в конце периода']>0]

for index in ostatok.index:
    if ostatok.at[index,'Блокированный запас']>0:
        ostatok.at[index,'Ст-ть/блок. запаса'] = (ostatok.at[index,'Ст-ть запаса в конце периода']/ostatok.at[index,'Кол-во запаса в конце периода'])*ostatok.at[index,'Блокированный запас']
        


replace_float(ostatok,['Кол-во запаса в конце периода','Блокированный запас'])
# ostatok['Cвободный запас'] = ostatok['Кол-во запаса в конце периода']-ostatok['Блокированный запас']
ostatok['Cвободный запас'] = ostatok.apply(lambda x: 0 if (x['Кол-во запаса в конце периода']-x['Блокированный запас'])<=0 else x['Кол-во запаса в конце периода']-x['Блокированный запас'],axis = 1  )
ostatok['Стоимость свободного остатка'] = ostatok.apply(lambda x: 0 if (x['Ст-ть запаса в конце периода']-x['Ст-ть/блок. запаса'])<=0 else x['Ст-ть запаса в конце периода']-x['Ст-ть/блок. запаса'],axis = 1  )
# ostatok['E_Q_ALL'] = ostatok.apply(lambda x: Q_E_ALL(x),axis=1)
# ostatok['Укрупненный Заказчик'] = ostatok.apply(lambda x:ukrup_zakaz(x),axis=1)

# ostatok['Заказчик + HH'] =ostatok['Укрупненный Заказчик'].astype('str')+' '+ostatok['Материал'].astype('str')

# zav_str = ostatok['Завод'].apply(lambda x: str(x).replace('.0','')).astype('str')
ostatok['Завод+НН'] = ostatok['Завод'].astype('str')+' #'+ostatok['Материал'].astype('str')
ostatok['Завод+НН'] = ostatok['Завод+НН'].str.strip()
ostatok['Материал'] = ostatok['Материал'].astype('str')
# ostatok['Неликвиды'] = ostatok.apply(lambda x: '#'+ x['Материал']+x['Укрупненный Заказчик'] if x['Укрупненный Заказчик']=='НЕЛИКВИДЫ' else 0,axis=1)
spr_komplex_ost = rev_to_dict(spr_ost['Завод']+' '+spr_ost['Склад']+' '+spr_ost['Партия'],spr_ost['Комплекс'])
ostatok['Комплекс'] = ostatok['Завод+Склад+Партия'].apply(lambda x: insert_vpr(x,spr_komplex_ost,'Комплекс') )

spr__st_mrlt = rev_to_dict(sprav_mat_cal_last_full['Завод']+' #'+sprav_mat_cal_last_full['Материал'],sprav_mat_cal_last_full['СтатусМтрл на заводе'])
ostatok['СтатусМтрл на заводе'] = ostatok.apply(lambda x:insert_vpr(x['Завод+НН'],spr__st_mrlt,'СтатусМтрл на заводе'),axis = 1 )

# ind_ost = ostatok[(ostatok['СтатусМтрл на заводе']=='2')&(ostatok['ЦС/ПЗ']=='ЦС')].index
# ostatok.loc[ind_ost,'Комплекс'] = 'НЕЛИКВИДЫ'

ostatok['Заказчик + HH'] =ostatok['Комплекс'].astype('str')+' '+ostatok['Материал'].astype('str')

ostatok['Категория запаса'] = ostatok.apply(lambda x:category_of_zapas(x),axis=1)

ostatok['осталось'] = ostatok['Cвободный запас']
# # перед группировкой заполнить none
ostatok.fillna(0,inplace=True)


fil = ostatok.apply(lambda x: True if ((x['Cвободный запас']+x['Блокированный запас']))-x['Кол-во запаса в конце периода']>0 and (x['Кол-во запаса в конце периода']-x['Блокированный запас'])<0 else False  ,axis = 1)
dd = ostatok[fil].apply(lambda x:x['Кол-во запаса в конце периода']  if x['Кол-во запаса в конце периода']>0 else 0,axis = 1)
for x in dd.index: ostatok.at[x,'Блокированный запас'] = dd[x]
    
    
fil = ostatok.apply(lambda x: True if ((x['Стоимость свободного остатка']+x['Ст-ть/блок. запаса']))-x['Ст-ть запаса в конце периода']>0 and (x['Ст-ть запаса в конце периода']-x['Ст-ть/блок. запаса'])<0 else False  ,axis = 1)
dd = ostatok[fil].apply(lambda x:x['Ст-ть запаса в конце периода']  if x['Ст-ть запаса в конце периода']>0 else 0,axis = 1)
for x in dd.index: ostatok.at[x,'Ст-ть/блок. запаса'] = dd[x]    


spis_st = ['Завод+НН','Материал','Завод','Завод+Склад','Склад','Партия','КрТекстМатериала','Валюта','Базовая ЕИ','Особый запас','Основной счет','Комплекс','Заказчик + HH','Категория запаса','Завод+склад+партия+особый запас+НН','ЦС/ПЗ']
ost_start = ostatok.groupby(spis_st).agg({'Cвободный запас':'sum','Кол-во запаса в конце периода':'sum','Ст-ть запаса в конце периода':'sum','Блокированный запас':'sum','осталось':'sum','Стоимость свободного остатка':'sum','Ст-ть/блок. запаса':'sum'}).reset_index()





spr_str_zap__= rev_to_dict(sprav_mat_cal_last_full['Завод']+sprav_mat_cal_last_full['Материал']+sprav_mat_cal_last_full['Код округа'],sprav_mat_cal_last_full['Страховой запас'])
ost_start['страховой запас'] = (ost_start['Завод']+ost_start['Материал']+ost_start['Партия']).apply(lambda x:insert_vpr(x,spr_str_zap__,'Страховой запас') )


ost_start['страховой запас_п'] = ost_start['страховой запас']
ind__ = ost_start[ost_start.apply(lambda x: True if (x['осталось']-x['страховой запас'])<0  else False,axis = 1 )].index
ost_start.loc[ind__,'страховой запас'] = ost_start.loc[ind__,'осталось']
ost_start.loc[ind__,'осталось'] = 0
ost_start['temp_razn_ost_str_zap'] = ost_start.apply(lambda x: x['осталось']-x['страховой запас'] if  (x['осталось']-x['страховой запас'])>0 and x['страховой запас']>0 and len(x['Партия'])==2  else 0,axis = 1)
ost_start['занулить'] = ost_start.apply(lambda x:'да' if  (x['осталось']-x['страховой запас']==0) and x['страховой запас']>0 and len(x['Партия'])==2  else 0,axis = 1)
ind_str_zap = ost_start[ost_start['temp_razn_ost_str_zap']>0].index
ind_str_zanul = ost_start[ost_start['занулить']=='да'].index
ost_start.loc[ind_str_zap,'осталось'] = ost_start.loc[ind_str_zap,'temp_razn_ost_str_zap']
ost_start.loc[ind_str_zanul,'осталось'] = 0
ind_ = ost_start[ost_start.apply(lambda x: True if x['страховой запас']>0 and  len(x['Партия'])>2 else False,axis = 1 )].index
ost_start.loc[ind_,'страховой запас'] = 0




ost_block = ost_start[(ost_start['Блокированный запас']>0)]
ost_block.name = 'ost_block'


ost_start_raspr = ost_start[(ost_start['осталось']>0)&(ost_start['Особый запас']!='Q')]
ost_start_raspr_str_zap = ost_start[(ost_start['страховой запас']>0)&(ost_start['Особый запас']!='Q')]


ost_start_raspr.name = 'ost_start_raspr'
ost_start_raspr_str_zap.name = 'ost_start_raspr_str_zap'


ost_start.name = 'ost_start'
ostatok.name  = 'ostatok'
block_zapas.name = 'block_zapas'
# save_backup('backup.h5',[ost_start,ostatok,block_zapas,ost_start_raspr])

In [ ]:
rashod = pd.read_excel(directory+'Выгрузки\\'+rash_name+'.xlsx',sheet_name='Sheet1')
sprav_rashod = pd.read_excel(directory+'Справодник для расхода.xlsx',sheet_name='Расход')


rashod.fillna(0,inplace=True)
rashod['Материал'] = rashod['Материал'].apply(lambda x: str(x).replace('.0','')).astype('str')

rashod['Материал'] = rashod['Материал'].apply(lambda x: '0'+str(x) if  len(str(x))<9 else x)
rashod['Завод+НН'] = rashod['Завод'].astype('str')+' #'+rashod['Материал'].astype('str')
rashod['Завод'] = rashod['Завод'].apply(lambda x: str(x).replace('.0','')).astype('str')
rashod['Склад'] = rashod['Склад'].apply(lambda x: str(x).replace('.0','')).astype('str')
# rashod['E_Q_ALL'] = rashod.apply(lambda x: Q_E_ALL_rash(x),axis=1)
rashod['Завод+Склад'] = rashod.apply(lambda x:str(x['Завод'])+' '+str(x['Склад']),axis=1)
rashod['Материал'] = rashod['Материал'].astype('str')

rashod_dict =  rev_to_dict(sprav_rashod['Завод+склад'],sprav_rashod['Комплекс'])
rashod['Комплекс'] = rashod['Завод+Склад'].apply(lambda x:insert_vpr(x,rashod_dict,name='Комплекс') )

rashod['Вид движения'] = rashod['Вид движения'].astype('str')
rashod  = rashod[rashod.apply(lambda x: list(x['Материал'])[-1]!='R'or x['Вид движения']!='262' ,axis = 1)] 
# # # # перед группировкой заполнить none






rashod.fillna(0,inplace=True)
spis_st_rash = ['Комплекс','Завод','Склад','Материал','Завод+НН','Особый запас']
rashod_start = rashod.groupby(spis_st_rash).agg({'Кол-во в ЕИ ввода':'sum','Сумма во ВВ':'sum'}).reset_index()

rashod_start['Кол-во в ЕИ ввода'] = rashod_start['Кол-во в ЕИ ввода']*-1

rashod_start['Сумма во ВВ'] = rashod_start['Сумма во ВВ']*-1

rashod_start = rashod_start[(rashod_start['Кол-во в ЕИ ввода']>0)&(rashod_start['Сумма во ВВ']>0)&(rashod_start['Комплекс']!='Не участвует в распределении')]



rashod_start_raspr = rashod_start[rashod_start['Особый запас']!='Q']

rashod_start.name = 'rashod_start'

In [ ]:
prihod = pd.read_excel(directory+'Выгрузки\\'+prih_name+'.xlsx',sheet_name='Sheet1')
sprav_prihod = pd.read_excel(directory+'Справодник для прихода.xlsx',sheet_name='Приход')
prihod.fillna(0,inplace=True)

prihod['Завод'] = prihod['Завод'].apply(lambda x: str(x).replace('.0','')).astype('str')
prihod['Склад'] = prihod['Склад'].apply(lambda x: str(x).replace('.0','')).astype('str')
prihod['Материал'] = prihod['Материал'].astype('str')
prihod['Материал'] = prihod['Материал'].apply(lambda x: str(x).replace('.0','')).astype('str')
prihod['Материал'] = prihod['Материал'].apply(lambda x: '0'+str(x) if  len(str(x))<9 else x)
prihod['Завод+НН'] =prihod['Завод'].astype('str')+' #'+ prihod['Материал'].astype('str')
prihod['Завод+Склад'] = prihod.apply(lambda x:str(x['Завод'])+' '+str(x['Склад']),axis=1)
prihod_dict =  rev_to_dict(sprav_prihod['Завод+склад'],sprav_prihod['Комплекс'])
prihod['Комплекс'] = prihod['Завод+Склад'].apply(lambda x:insert_vpr(x,prihod_dict,name='Комплекс') )
prihod['Комплекс+НН'] = prihod['Комплекс'].astype('str')+' '+prihod['Материал'].astype('str')


spis_st_prihod = ['Материал','Комплекс','Комплекс+НН','Особый запас','Завод+НН']
# # перед группировкой заполнить none
prihod.fillna(0,inplace=True)
prihod_start = prihod.groupby(spis_st_prihod).agg({'Количество':'sum','Сумма во ВВ':'sum'}).reset_index()
prihod_start = prihod_start[(prihod_start['Количество']>0) & (prihod_start['Сумма во ВВ']>0)&(prihod_start['Комплекс']!='Не участвует в распределении')]
prihod_start_raspr = prihod_start[prihod_start['Особый запас']!='Q']
prihod_start.name = 'prihod_start'


In [ ]:
NB = pd.read_excel(directory+'NB.xlsx',sheet_name='Sheet1')
NB = NB[~NB['Обозначение индикатора деблокирования'].isin(['блокиров.'])]
spr_nb = pd.read_excel(directory+'Справодник для NB.xlsx',sheet_name='NB')
NB.fillna(0,inplace = True)
spr_nb.fillna(0,inplace = True)
NB['Количество пополнения'] = NB['ЗатребованКолич']-NB['Заказанное к-во']
NB['Завод'] = NB['Завод'].apply(lambda x: str(x).replace('.0','')).astype('str')
NB['Склад'] = NB['Склад'].apply(lambda x: str(x).replace('.0','')).astype('str')

NB['Заказ на поставку'] = NB['Заказ на поставку'].apply(lambda x: str(x).replace('.0','')).astype('str')
NB['Позиция ЗкзНаПостав'] = NB['Позиция ЗкзНаПостав'].apply(lambda x: str(x).replace('.0','')).astype('str')

NB['Материал'] = NB['Материал'].apply(lambda x: str(x).replace('.0','')).astype('str')
NB['Материал'] = NB['Материал'].apply(lambda x: '0'+str(x) if  len(str(x))<9 else x)
NB['Склад'] = NB['Склад'].apply(lambda x: '' if x=='0' else x)
NB['Завод+Склад'] = NB['Завод']+' '+NB['Склад']
# NB['Завод+Склад'] = NB['Завод+Склад'].str.strip()
spr_nb1 = rev_to_dict(spr_nb['Завод + Склад'],spr_nb['Комплекс'])
NB['Комплекс'] = NB['Завод+Склад'].apply(lambda x: insert_vpr(x,spr_nb1,'Комплекс'))
NB['Комплекс'] = NB['Комплекс'].apply(lambda x: str(x).replace('.0','')).astype('str')
NB['Комплекс+Маттериал'] = NB['Комплекс']+' '+NB['Материал']
NB['Количество пополнения'] = NB['Количество пополнения'].apply(lambda x:0 if x<0 else x)
NB = NB[NB['Обозначение индикатора деблокирования']!='Тендер завершен']

In [ ]:
ZNB = pd.read_excel(directory+'ZNB.xlsx',sheet_name='Sheet1')
UB05 = pd.read_excel(directory+'UB05.xlsx',sheet_name='Sheet1')
spr_znb = pd.read_excel(directory+'Справодник для ZNB.xlsx',sheet_name='ZNB')
ZNB.fillna(0,inplace=True)
ZNB['Материал'] = ZNB['Материал'].apply(lambda x: str(x).replace('.0','')).astype('str')
ZNB['Материал'] = ZNB['Материал'].apply(lambda x: '0'+str(x) if  len(str(x))<9 else x)
ZNB['Документ закупки'] = ZNB['Документ закупки'].apply(lambda x: str(x).replace('.0','')).astype('str')
ZNB['Позиция'] = ZNB['Позиция'].apply(lambda x: str(x).replace('.0','')).astype('str')
ZNB['Документ закупки + Позиция'] = ZNB['Документ закупки'].astype('str')+' '+ZNB['Позиция'].astype('str')
UB05['Материал'] = UB05['Материал'].apply(lambda x: str(x).replace('.0','')).astype('str')
UB05['Материал'] = UB05['Материал'].apply(lambda x: '0'+str(x) if  len(str(x))<9 else x)
UB05['Документ закупки'] = UB05['Документ закупки'].apply(lambda x: str(x).replace('.0','')).astype('str')
UB05['Позиция'] = UB05['Позиция'].apply(lambda x: str(x).replace('.0','')).astype('str')
UB05['Документ закупки + позиция'] = UB05['Документ закупки'].astype('str')+' '+UB05['Позиция'].astype('str')
UB05['Заявка'] = UB05['Заявка'].apply(lambda x: str(x).replace('.0','')).astype('str')
UB05['Позиция заявки'] = UB05['Позиция заявки'].apply(lambda x: str(x).replace('.0','')).astype('str')
UB05['Заявка + Позиция заявки'] = UB05['Заявка'].astype('str')+' '+UB05['Позиция заявки'].astype('str')
spr_UB05 = rev_to_dict(UB05['Документ закупки + позиция'],UB05['Заявка + Позиция заявки'])
ZNB['Заявка + Позиция (для UB05 по ДП)'] = ZNB['Документ закупки + Позиция'].apply(lambda x:  insert_vpr(x,spr_UB05,'Заявка + Позиция заявки'))
ZNB['Номер элемента пополнения + Номер позиции пополнения + Наименование типа элемента ППМ пополн'] = ZNB.apply(lambda x: x['Документ закупки + Позиция'] if  x['Заявка + Позиция (для UB05 по ДП)']==0 else x['Заявка + Позиция (для UB05 по ДП)'] ,axis = 1)+'АвизОт'
tab_avizot  = df_str_zap.groupby('Номер элемента пополнения + Номер позиции пополнения + Наименование типа элемента ППМ пополн').agg({'Количество пополнения':'sum'})
ZNB= pd.merge(ZNB,tab_avizot,how='left',on=['Номер элемента пополнения + Номер позиции пополнения + Наименование типа элемента ППМ пополн'])
ZNB['Склад'] = ZNB['Склад'].apply(lambda x: str(x).replace('.0','')).astype('str')
ZNB['Склад'] = ZNB['Склад'].apply(lambda x: '' if x==0 or x =='0' else x)
ZNB['Завод'] = ZNB['Завод'].apply(lambda x: str(x).replace('.0','')).astype('str')
spr_nb_ = rev_to_dict(NB['Заказ на поставку']+' '+NB['Позиция ЗкзНаПостав'],NB['Завод+Склад'])
ZNB['Завод+Склад'] = ZNB.apply(lambda x:x['Завод']+' '+x['Склад']  \
          if  insert_vpr(x['Документ закупки + Позиция'],spr_nb_,'Завод+Склад')==0\
          else insert_vpr(x['Документ закупки + Позиция'],spr_nb_,'Завод+Склад'),axis = 1  )


spr_znb_ = rev_to_dict(spr_znb['Завод+склад'],spr_znb['Комплекс'])
ZNB['Комплекс']=ZNB['Завод+Склад'].apply(lambda x:  insert_vpr(x,spr_znb_,'Комплекс'))
ZNB['Комплекс+Маттериал'] = ZNB['Комплекс'].astype('str')+' '+ZNB['Материал'].astype('str')
ZNB['Завод+НН'] = ZNB['Завод'].astype('str')+' #'+ZNB['Материал'].astype('str')
ZNB.fillna(0,inplace=True)



In [ ]:
# для прт пзк
ZNB1 = pd.read_excel(directory+'ZNB.xlsx',sheet_name='Sheet1')
UB05 = pd.read_excel(directory+'UB05.xlsx',sheet_name='Sheet1')
spr_znb = pd.read_excel(directory+'Справодник для ZNB.xlsx',sheet_name='ZNB')
ZNB1.fillna(0,inplace = True)
ZNB1['Материал'] = ZNB1['Материал'].apply(lambda x: str(x).replace('.0','')).astype('str')
ZNB1['Материал'] = ZNB1['Материал'].apply(lambda x: '0'+str(x) if  len(str(x))<9 else x)
ZNB1['Документ закупки'] = ZNB1['Документ закупки'].apply(lambda x: str(x).replace('.0','')).astype('str')
ZNB1['Позиция'] = ZNB1['Позиция'].apply(lambda x: str(x).replace('.0','')).astype('str')
ZNB1['Документ закупки + Позиция'] = ZNB1['Документ закупки'].astype('str')+' '+ZNB1['Позиция'].astype('str')
UB05['Документ закупки'] = UB05['Документ закупки'].apply(lambda x: str(x).replace('.0','')).astype('str')
UB05['Документ закупки + позиция'] = UB05['Документ закупки'].astype('str')+' '+UB05['Позиция'].astype('str')
UB05['Заявка + Позиция заявки'] = UB05['Заявка'].astype('str')+' '+UB05['Позиция заявки'].astype('str')
spr_UB05 = rev_to_dict(UB05['Документ закупки + позиция'],UB05['Заявка + Позиция заявки'])
ZNB1['Заявка + Позиция (для UB05 по ДП)'] = ZNB1['Документ закупки + Позиция'].apply(lambda x:  insert_vpr(x,spr_UB05,'Заявка + Позиция заявки'))
ZNB1['Номер элемента пополнения + Номер позиции пополнения + Наименование типа элемента ППМ пополн'] = ZNB1.apply(lambda x: x['Документ закупки + Позиция'] if  x['Заявка + Позиция (для UB05 по ДП)']==0 else x['Заявка + Позиция (для UB05 по ДП)'] ,axis = 1)+'ПртПЗк'
tab_avizot  = df_str_zap.groupby('Номер элемента пополнения + Номер позиции пополнения + Наименование типа элемента ППМ пополн').agg({'Количество пополнения':'sum'})
ZNB1= pd.merge(ZNB1,tab_avizot,how='left',on=['Номер элемента пополнения + Номер позиции пополнения + Наименование типа элемента ППМ пополн'])
ZNB1['Склад'] = ZNB1['Склад'].apply(lambda x: str(x).replace('.0','')).astype('str')
ZNB1['Склад'] = ZNB1['Склад'].apply(lambda x: '' if x==0 or x=='0' else x)
ZNB1['Завод'] = ZNB1['Завод'].apply(lambda x: str(x).replace('.0','')).astype('str')
ZNB1['Завод+Склад'] = ZNB1.apply(lambda x:x['Завод']+' '+x['Склад']  \
          if  insert_vpr(x['Документ закупки + Позиция'],spr_nb_,'Завод+Склад')==0\
          else insert_vpr(x['Документ закупки + Позиция'],spr_nb_,'Завод+Склад'),axis = 1  )
spr_znb_ = rev_to_dict(spr_znb['Завод+склад'],spr_znb['Комплекс'])
ZNB1['Комплекс']=ZNB1['Завод+Склад'].apply(lambda x:  insert_vpr(x,spr_znb_,'Комплекс'))
ZNB1['Комплекс+Маттериал'] = ZNB1['Комплекс'].astype('str')+' '+ZNB1['Материал'].astype('str')
ZNB1['Завод+НН'] = ZNB1['Завод'].astype('str')+' #'+ZNB1['Материал'].astype('str')
ZNB1.fillna(0,inplace=True)

In [ ]:
if  (0 in NB['Комплекс'].unique())==False and ('0' in NB['Комплекс'].unique())==False:
    print('В nb Комплекс подтянулся полностю')
else:
    print('В nb Комплекс НЕ ПОДТЯНУЛСЯ',str(NB['Комплекс'].value_counts()['0'])+' Позиции')
if (0 in ZNB1['Комплекс'].unique())==False and ('0' in ZNB1['Комплекс'].unique())==False:
    print('В ZNB1 Комплекс подтянулся полностю')
else:
    print('В ZNB1 Комплекс НЕ ПОДТЯНУЛСЯ',str(ZNB1['Комплекс'].value_counts()[0])+' Позиций')
if (0 in ZNB['Комплекс'].unique())==False and  ('0' in ZNB['Комплекс'].unique())==False:
    print('В ZNB Комплекс подтянулся полностю')
else:
    print('В ZNB Комплекс НЕ ПОДТЯНУЛСЯ',str(ZNB['Комплекс'].value_counts()[0])+' Позиций')

In [ ]:
NZ = df_str_zap[df_str_zap['Краткое наименование элемента ППМ']=='СтрЗап']
NZ_gr = ['Материал','Завод','Комплекс','Заказчик + HH','БЕ']
# перед группировкой заполнить none
NZ.fillna(0,inplace=True)
NZ_start = NZ.groupby(NZ_gr).agg({'Количество потребности':'sum'}).reset_index()
NZ_start['Завод+НН']  =  NZ_start['Завод'].astype('str')+' #'+NZ_start['Материал'].astype('str')
NZ_start.name = 'NZ'
NZ_start.fillna(0,inplace = True)

In [ ]:
# Запуск распределения остатков
# date_start_year = datetime.datetime(2019,1,1)
# current_date = datetime.datetime(2020,1,1)
df['index'] = [x for x in range(len(df))]
df['del'] = ''
df.reset_index(inplace = True,drop = True)

potr_copy = df[df['Страховой запас']>0].copy()
index___ = potr_copy[potr_copy['Страховой запас']>0].index
potr_copy.loc[index___,'Количество потребности'] = potr_copy.loc[index___,'Страховой запас']
potr_copy['Завод+НН'] = potr_copy['Завод'].astype('str')+' #'+potr_copy['Материал'].astype('str')
potr_copy['Завод+НН'] = potr_copy['Завод+НН'].str.strip()
ost_copy = ost_start_raspr_str_zap.copy()
ost_copy['осталось'] = ost_copy['страховой запас']
ost_copy['ЦС/ПЗ']='ПЗ'
potr_copy.name = 'potrebn_with_raspr_ost'
ost_copy.name =  'ost_with_potr'
# Запуск распределения остатков
potr_copy['Распределение_остатака'] = 0.000000
potr_copy['Сумма_распреденного_остатка'] = 0.000000


ostat(ost_st=ost_copy,potr_st=potr_copy,filters=filter_else,current_date = current_date,all_months=True,\
     col_name_0='осталось',col_name_1='Распределение_остатака',col_name_2 = 'Количество потребности',col_name_3 = 'Стоимость свободного остатка',col_name_4 = 'Cвободный запас',col_name_5 = 'Сумма_распреденного_остатка')    
potr_copy.loc[index___,'Количество потребности'] = 0

In [ ]:
start_time = time.time()
# Запуск распределения остатков
# date_start_year = datetime.datetime(2019,1,1)
# current_date = datetime.datetime(2019,10,1)
potr_copy1 = df[(df['Страховой запас']==0)&].copy()

ost_copy1 = ost_start_raspr.copy()
potr_copy1.name = 'potrebn_with_raspr_ost'
ost_copy1.name =  'ost_with_potr'
# Запуск распределения остатков
potr_copy1['Распределение_остатака'] = 0.000000
potr_copy1['Сумма_распреденного_остатка'] = 0.000000

ostat(ost_st=ost_copy1,potr_st=potr_copy1,filters=filter_else,current_date = current_date,all_months=True,\
     col_name_0='осталось',col_name_1='Распределение_остатака',col_name_2 = 'Количество потребности',col_name_3 = 'Стоимость свободного остатка',col_name_4 = 'Cвободный запас',col_name_5 = 'Сумма_распреденного_остатка')
print("--- %s seconds ---" % (time.time() - start_time))

In [ ]:
potr_copy_cons = pd.concat([potr_copy1,potr_copy])
potr_copy_cons.reset_index(inplace = True,drop = True)

In [ ]:
potr_copy_cons.name = '03_20_potr_copy'
save_backup('potr_copy_cons.h5',[potr_copy_cons])
ost_copy1.name = '03_20_ost_copy1'
save_backup('potr_copy_cons.h5',[ost_copy1])

In [ ]:
start_time = time.time()
potr_avizot = potr_copy_cons.copy()
avizot_copy = ZNB.copy()
potr_avizot.name = 'potr_avizot'
avizot_copy.name =  'avizot_copy'
potr_avizot['avizot'] = 0.000000
avizot_copy['осталось'] = avizot_copy['Количество пополнения']
avizot_raspr(ost_st=avizot_copy,potr_st=potr_avizot,filters=filter_else,current_date = current_date,all_months=True)
print("--- %s seconds ---" % (time.time() - start_time))

In [ ]:
start_time = time.time()
potr_prt_pzk = potr_avizot.copy()
prt_pzk_copy = ZNB1.copy()
potr_prt_pzk.name = 'potr_prt_pzk'
prt_pzk_copy.name =  'prt_pzk_copy'
potr_prt_pzk['законтрактовано'] = 0.000000
prt_pzk_copy['осталось'] = prt_pzk_copy['Количество пополнения']
prt_pzk(ost_st=prt_pzk_copy,potr_st=potr_prt_pzk,filters=filter_else,current_date = current_date,all_months=True)
print("--- %s seconds ---" % (time.time() - start_time))

In [ ]:
start_time = time.time()
potr_NB = potr_prt_pzk.copy()
NB_copy = NB[NB['Обозначение индикатора деблокирования']!='Тендер завершен'].copy()
potr_NB.name = 'potr_NB'
NB_copy.name =  'NB_copy'
potr_NB['NB'] = 0.000000
NB_copy['осталось'] = NB_copy['Количество пополнения']
NB_raspr(ost_st=NB_copy,potr_st=potr_NB,filters=filter_else,current_date = current_date,all_months=True)
print("--- %s seconds ---" % (time.time() - start_time))

In [ ]:
start_time = time.time()
potr_str_zapas = potr_NB.copy()
NZ_copy = NZ_start.copy()
potr_str_zapas.name = 'potr_str_zapas'
NZ_copy.name =  'NZ_copy'
potr_str_zapas['стрЗапас'] = 0.000000
NZ_copy['осталось'] = NZ_copy['Количество потребности']
str_zap_raspr(ost_st=NZ_copy,potr_st=potr_str_zapas,filters=filter_else,current_date = current_date,all_months=True)
vst = NZ_copy[(NZ_copy['осталось']>0)&(NZ_copy['Количество потребности']>0)]
vst['Завод+НН'] = vst['Завод'].astype('str')+' #'+vst['Материал'].astype('str')
print("--- %s seconds ---" % (time.time() - start_time))

In [ ]:
# распределение расхода
start_time = time.time()


# col_name_0 название столбца откуда рпспределяем
# col_name_1 навание с толбца куда распределяем
# col_name_2 название столбца с количеством потребности
# col_name_5 название столбца куда распределяем сумму
# Для цены:
# col_name_3 название столбца с суммой потребности
# col_name_4 название столбца с начальным кол-вом  потребности

potr_copy_for_rashod = potr_str_zapas.copy()
rashod_copy = rashod_start_raspr.copy()
potr_copy_for_rashod.name = 'potr_copy_for_rashod'
rashod_copy.name =  'rashod_copy'
# Запуск распределения остатков
potr_copy_for_rashod['Распределение_расхода'] = 0.000000
potr_copy_for_rashod['Сумма_распреденного_расхода'] = 0.000000
rashod_copy['осталось'] = rashod_copy['Кол-во в ЕИ ввода']

rashod_raspr(ost_st=rashod_copy,potr_st=potr_copy_for_rashod,filters=filter_else,current_date = current_date,all_months=True,col_name_0 ='осталось',col_name_1 ='Распределение_расхода', col_name_2 = 'Количество потребности',col_name_3 ='Сумма во ВВ',col_name_4 = 'Кол-во в ЕИ ввода',col_name_5 = 'Сумма_распреденного_расхода') 
print("--- %s seconds ---" % (time.time() - start_time))

# проверка:
raspr_rashod = rashod_copy[(rashod_copy['осталось']==0)&(rashod_copy['Кол-во в ЕИ ввода']>0)]['Кол-во в ЕИ ввода'].sum()
neraspr_rashod =  rashod_copy[(rashod_copy['осталось']>0)&(rashod_copy['Кол-во в ЕИ ввода']>0)]['Кол-во в ЕИ ввода'].sum()
Q_rashod =  rashod_start[rashod_start['Особый запас']=='Q']['Кол-во в ЕИ ввода'].sum()
raspr_rashod_money = rashod_copy[(rashod_copy['осталось']==0)&(rashod_copy['Кол-во в ЕИ ввода']>0)]['Сумма во ВВ'].sum()
neraspr_rashod_money =  rashod_copy[(rashod_copy['осталось']>0)&(rashod_copy['Кол-во в ЕИ ввода']>0)]['Сумма во ВВ'].sum()
Q_rashod_money =  rashod_start[rashod_start['Особый запас']=='Q']['Сумма во ВВ'].sum()

In [ ]:
start_time = time.time()
potr_copy_for_prihod = potr_copy_for_rashod.copy()
prihod_copy = prihod_start_raspr.copy()
potr_copy_for_prihod.name = 'potr_copy_for_prihod'
prihod_copy.name =  'prihod_copy'
# Запуск распределения остатков
potr_copy_for_prihod['Распределение_прихода'] = 0.000000
potr_copy_for_prihod['Сумма_распреденного_прихода'] = 0.000000
prihod_copy['осталось'] = prihod_copy['Количество']

prihod_raspr(ost_st=prihod_copy,potr_st=potr_copy_for_prihod,filters=filter_else,current_date = current_date,all_months=True,col_name_0 ='осталось',col_name_1 ='Распределение_прихода', col_name_2 = 'Количество потребности',col_name_3 ='Сумма во ВВ',col_name_4 = 'Количество',col_name_5 = 'Сумма_распреденного_прихода') 
print("--- %s seconds ---" % (time.time() - start_time))

raspr_prihod = prihod_copy[(prihod_copy['осталось']==0)&(prihod_copy['Количество']>0)]['Количество'].sum()
neraspr_prihod =  prihod_copy[(prihod_copy['осталось']>0)&(prihod_copy['Количество']>0)]['Количество'].sum()
Q_prihod =   prihod_start[prihod_start['Особый запас']=='Q']['Количество'].sum()
raspr_prihod_sum = prihod_copy[(prihod_copy['осталось']==0)&(prihod_copy['Количество']>0)]['Сумма во ВВ'].sum()
neraspr_prihod_sum =  prihod_copy[(prihod_copy['осталось']>0)&(prihod_copy['Количество']>0)]['Сумма во ВВ'].sum()
Q_prihod_sum =   prihod_start[prihod_start['Особый запас']=='Q']['Сумма во ВВ'].sum()

In [ ]:
potr_copy_for_prihod.name = '03_20'
save_backup('tto.h5',[potr_copy_for_prihod])

In [ ]:
potr_copy_for_prihod = get_in_backup(path_backup='tto.h5',name='02_20')

In [ ]:
potr_copy_for_prihod.name = '02_20_usd_raspr_ost'
save_backup('tto.h5',[potr_copy_for_prihod])

In [ ]:
# проверка распределения остатка и формирование сцепки с основным листом
# работа пок онсолидаци остатков

ost_Q = ost_start[(ost_start['Cвободный запас']>0)&(ost_start['Особый запас']=='Q')]
ost_Q['Признак Q остатка'] = True

ost_Q1 = ost_start[(ost_start['Cвободный запас']==0)&(ost_start['Особый запас']!='Q')]    

ost_Q  = pd.concat([ost_Q,ost_Q1])
ost_Q.fillna(0,inplace=True)

ff = ost_block[['Завод+склад+партия+особый запас+НН','Блокированный запас','Завод+НН','Комплекс','Партия']]
ost_with_block  =  pd.merge(ost_copy1,ff,how='outer')
ost_with_block.fillna(0,inplace = True)

ost_copy1['Cвободный запас'] = ost_copy1['осталось']
ost_copy1['Стоимость свободного остатка'] = (ost_copy1['Ст-ть запаса в конце периода']    / ost_copy1['Кол-во запаса в конце периода']) * ost_copy1['осталось']
# 1составляющая нераспределенный остаток
neraspr_ostat_tab = ost_copy1[(ost_copy1['осталось']>0)&(ost_copy1['Cвободный запас']>0)][['Комплекс','Завод+НН','Cвободный запас','Стоимость свободного остатка','Партия']].groupby(['Комплекс','Завод+НН','Партия']).sum().reset_index()
neraspr_ostat_tab['Признак нераспределнного остатка'] = True
# 2составляющая нераспределенный остаток
ost_Q_tab = ost_Q[['Признак Q остатка','Комплекс','Завод+НН','Cвободный запас','Стоимость свободного остатка','Базовая ЕИ','Материал','Партия']].groupby(['Комплекс','Завод+НН','Партия']).sum().reset_index()
# ost_Q_tab['Признак Q остатка'] = 0


ost1 = neraspr_ostat_tab[['Комплекс','Завод+НН','Cвободный запас','Стоимость свободного остатка','Партия']]



# 11111111111111111111111
ost2 = ost_Q_tab[['Комплекс','Завод+НН','Cвободный запас','Стоимость свободного остатка','Партия']]
ost2['Признак Q остатка'] = True




ost_ner_q_t = pd.concat([ost1]).groupby(['Комплекс','Завод+НН','Партия']).sum()

one = pd.merge(ost_ner_q_t,neraspr_ostat_tab[['Комплекс','Завод+НН','Признак нераспределнного остатка','Партия']],how='left',on=['Комплекс','Завод+НН','Партия'])
one = one.groupby(['Комплекс','Завод+НН','Партия']).sum().reset_index()

# # 
two  = pd.merge(one,ost_Q_tab[['Комплекс','Завод+НН','Партия']],how='outer',on=['Комплекс','Завод+НН','Партия'])
stoim_bl_zap =  ost_start[['Комплекс','Завод+НН','Ст-ть/блок. запаса','Партия']].groupby(['Комплекс','Завод+НН','Партия']).sum().reset_index()


aa = stoim_bl_zap.groupby(['Комплекс','Завод+НН','Партия']).sum().reset_index()
bb = ost_with_block[['Комплекс','Завод+НН','Блокированный запас','Ст-ть/блок. запаса','Партия']]
bb['Ст-ть/блок. запаса'] = 0

ost_dlya_scepki = pd.merge(aa,bb,how='outer').fillna(0).groupby(['Комплекс','Завод+НН','Партия']).sum().reset_index()

tottal_ost  = pd.merge(two,ost_dlya_scepki,how='outer').fillna(0)

a = ost_Q[['Комплекс','Завод+НН','Признак Q остатка','Партия']]
tottal_ost = pd.concat([tottal_ost,a[a['Признак Q остатка']==True].drop_duplicates()])
# tottal_ost = pd.merge(tottal_ost,a[a['Признак Q остатка']==True].drop_duplicates(),how='left',on=['Завод+НН','Комплекс'])
tottal_ost.rename(columns={"Cвободный запас": "Распределение_остатака",'Стоимость свободного остатка':'Сумма_распреденного_остатка'},inplace=True)
ost2.rename(columns={"Cвободный запас": "Распределение_остатака",'Стоимость свободного остатка':'Сумма_распреденного_остатка'},inplace=True)
tottal_ost = pd.concat([tottal_ost,ost2])
tottal_ost = tottal_ost.reset_index(drop=True)

tottal_ost = tottal_ost.loc[tottal_ost[['Распределение_остатака','Ст-ть/блок. запаса','Сумма_распреденного_остатка','Партия']].dropna(how = 'all').index]

f_tr = tottal_ost.apply(lambda x: True if x['Партия'][:2].isalpha() and x['Партия'][-2:].isalpha()==False else False,axis = 1)
for ind in tottal_ost[f_tr].index:
    st = tottal_ost.loc[ind,'Завод+НН'].split(' ')
    st[0] = tottal_ost.loc[ind,'Партия'][-4:]
    tottal_ost.loc[ind,'Завод+НН'] = ' '.join(st)
    
tottal_ost.drop('Партия',axis = 1,inplace = True)
tottal_ost.fillna(0,inplace = True)
ggttt = tottal_ost[f_tr].copy()
tottal_ost = tottal_ost.groupby(['Завод+НН','Комплекс','Признак Q остатка','Признак нераспределнного остатка']).sum().reset_index()


# raspr_prihod = prihod_copy[(prihod_copy['осталось']==0)&(prihod_copy['Количество']>0)]['Количество'].sum()
neraspr_prihod_tab =  prihod_copy[(prihod_copy['осталось']>0)&(prihod_copy['Количество']>0)]
Q_prihod_tab =   prihod_start[prihod_start['Особый запас']=='Q']

tab_ner_sc_prihod= neraspr_prihod_tab[['Комплекс','Завод+НН','Количество','Сумма во ВВ']].groupby(['Комплекс','Завод+НН']).sum().reset_index()
tab_ner_sc_prihod['Признак нераспределенного прихода'] = True
tab_Q_sc_prihod =   Q_prihod_tab[['Комплекс','Завод+НН','Количество','Сумма во ВВ']].groupby(['Комплекс','Завод+НН']).sum().reset_index()
tab_Q_sc_prihod['Признак Q прихода'] = True


tab_prihod = pd.concat([tab_ner_sc_prihod[['Комплекс','Завод+НН','Количество','Сумма во ВВ']]]).groupby(['Комплекс','Завод+НН']).sum().reset_index()
# ,tab_Q_sc_prihod[['Комплекс','Завод+НН','Количество','Сумма во ВВ']]

one_prihod = pd.merge(tab_prihod,tab_ner_sc_prihod[['Комплекс','Завод+НН','Признак нераспределенного прихода']],how='left')

# tab_prihod_with_cons = pd.merge(one_prihod,tab_Q_sc_prihod[['Комплекс','Завод+НН','Признак Q прихода']],how='left').fillna(0)

tab_prihod_with_cons = pd.concat([one_prihod,tab_Q_sc_prihod[['Комплекс','Завод+НН','Признак Q прихода']]])

tab_prihod_with_cons.rename(columns={"Количество": "Распределение_прихода",'Сумма во ВВ':'Сумма_распреденного_прихода'},inplace=True)

tab_Q_sc_prihod.rename(columns={"Количество": "Распределение_прихода",'Сумма во ВВ':'Сумма_распреденного_прихода'},inplace=True)
tab_prihod_with_cons = pd.concat([tab_Q_sc_prihod,tab_prihod_with_cons])
tab_prihod_with_cons = tab_prihod_with_cons.reset_index(drop=True)
tab_prihod_with_cons = tab_prihod_with_cons.loc[tab_prihod_with_cons[['Распределение_прихода','Сумма_распреденного_прихода']].dropna(how = 'all').index]



# работа по консолидации расхода

# raspr_rashod_tab = rashod_copy[(rashod_copy['осталось']==0)&(rashod_copy['Кол-во в ЕИ ввода']>0)]
neraspr_rashod_tab =  rashod_copy[(rashod_copy['осталось']>0)&(rashod_copy['Кол-во в ЕИ ввода']>0)]
Q_rashod_tab =  rashod_start[rashod_start['Особый запас']=='Q']

tab_ner_sc_rashod = neraspr_rashod_tab[['Комплекс','Завод+НН','Кол-во в ЕИ ввода','Сумма во ВВ']].groupby(['Комплекс','Завод+НН']).sum().reset_index()
tab_ner_sc_rashod['Признак нераспределенного расхода'] = True
tab_Q_sc_rashod =   Q_rashod_tab[['Комплекс','Завод+НН','Кол-во в ЕИ ввода','Сумма во ВВ']].groupby(['Комплекс','Завод+НН']).sum().reset_index()
tab_Q_sc_rashod['Признак Q расхода'] = True
tab_rashod = pd.concat([tab_ner_sc_rashod[['Комплекс','Завод+НН','Кол-во в ЕИ ввода','Сумма во ВВ']]]).groupby(['Комплекс','Завод+НН']).sum().reset_index()
one_rashod = pd.merge(tab_rashod,tab_ner_sc_rashod[['Комплекс','Завод+НН','Признак нераспределенного расхода']],how='left')

# tab_rashod_with_cons = pd.merge(one_rashod,tab_Q_sc_rashod[['Комплекс','Завод+НН','Признак Q расхода']],how='left').fillna(0)

tab_rashod_with_cons = pd.concat([one_rashod,tab_Q_sc_rashod[['Комплекс','Завод+НН','Признак Q расхода']]])


tab_rashod_with_cons.rename(columns={"Кол-во в ЕИ ввода": "Распределение_расхода",'Сумма во ВВ':'Сумма_распреденного_расхода'},inplace=True)


tab_Q_sc_rashod.rename(columns={"Кол-во в ЕИ ввода": "Распределение_расхода",'Сумма во ВВ':'Сумма_распреденного_расхода'},inplace=True)

tab_rashod_with_cons = pd.concat([tab_Q_sc_rashod,tab_rashod_with_cons])
# ,tab_Q_sc_rashod[['Комплекс','Завод+НН','Кол-во в ЕИ ввода','Сумма во ВВ']]

tab_rashod_with_cons = tab_rashod_with_cons.reset_index(drop=True)
tab_rashod_with_cons = tab_rashod_with_cons.loc[tab_rashod_with_cons[['Распределение_расхода','Сумма_распреденного_расхода']].dropna(how = 'all').index]


tab_rashod_with_cons['Q']  =tab_rashod_with_cons['Признак Q расхода']
tab_prihod_with_cons['Q'] = tab_prihod_with_cons['Признак Q прихода']
pre_low_part = pd.merge(tab_rashod_with_cons,tab_prihod_with_cons,how='outer',on=['Комплекс','Завод+НН','Q'])

tottal_ost['Q'] = tottal_ost['Признак Q остатка']

pre_low_part2 = pd.merge(pre_low_part,tottal_ost,how='outer',on=['Комплекс','Завод+НН','Q']).fillna(0)

arr_str_zap  = vst.groupby(['Завод+НН','Комплекс']).agg({'Количество потребности':'sum'}).reset_index()
arr_str_zap.rename(columns={'Количество потребности':'стрЗапас'},inplace=True)
arr_str_zap['Признак Q остатка'] = 0
pre_low_part3 = pd.merge(pre_low_part2,arr_str_zap,how='outer',on=['Комплекс','Завод+НН','Признак Q остатка']).fillna(0)

sprav_mat_cal['Завод+НН'] = sprav_mat_cal['Завод'].astype('str')+' #'+sprav_mat_cal['Материал'].astype('str')
sprav_mat_cal.drop_duplicates('Завод+НН',inplace=True)
sprav_mat_cal_soed = sprav_mat_cal[['Завод+НН','Имя плановика (завод)','Класс оценки','СтатусМтрл на заводе','Ведущий материал','Базовая ЕИ','Краткий текст материала']]
sprav_mat_cal_soed.rename(columns={'Имя плановика (завод)':'Плановик'},inplace=True)


pre_low_part3 = pd.merge(pre_low_part3,sprav_mat_cal_soed,how='left',on='Завод+НН')

potr_with_ostatok_total = pd.concat([potr_copy_for_prihod,pre_low_part3]).fillna(0)

# справочник для материала
mater = pd.concat([df[['Завод+НН','Материал']],NZ_start[['Завод+НН','Материал']],ZNB[['Завод+НН','Материал']],ostatok[['Завод+НН','Материал']],prihod[['Завод+НН','Материал']],rashod[['Завод+НН','Материал']]]).drop_duplicates()



# подключаем материал к общему фрейму
potr_with_ostatok_total = pd.merge(potr_with_ostatok_total.drop(['Материал'],axis=1),mater,how='left',on='Завод+НН').fillna(0)
# potr_with_ostatok_total = pd.merge(potr_with_ostatok_total.drop('БЕ',axis=1),BE,how='left',on='Завод+НН').fillna(0)

potr_with_ostatok_total.reset_index(inplace=True)


# красота

In [ ]:
# баланс количества остатка
int(potr_with_ostatok_total['Распределение_остатака'].sum()+potr_with_ostatok_total['Блокированный запас'].sum()-ostatok['Кол-во запаса в конце периода'].sum())

In [ ]:
# баланс суммы остатка
int(potr_with_ostatok_total['Ст-ть/блок. запаса'].sum() + potr_with_ostatok_total['Сумма_распреденного_остатка'].sum()-ostatok['Ст-ть запаса в конце периода'].sum())

In [ ]:
# баланс количества прихода
int(potr_with_ostatok_total['Распределение_прихода'].sum()-(raspr_prihod+neraspr_prihod+Q_prihod))

In [ ]:
#  баланс суммы прихода
int(potr_with_ostatok_total['Сумма_распреденного_прихода'].sum()-(raspr_prihod_sum+neraspr_prihod_sum+Q_prihod_sum))

In [ ]:
#  баланс количества расхода
int(potr_with_ostatok_total['Распределение_расхода'].sum()-(raspr_rashod+neraspr_rashod+Q_rashod))

In [ ]:
#  баланс суммы расхода
int(potr_with_ostatok_total['Сумма_распреденного_расхода'].sum()-(raspr_rashod_money+neraspr_rashod_money+Q_rashod_money))

In [ ]:
int(potr_with_ostatok_total['Распределение_расхода'].sum()-abs(rashod_start['Кол-во в ЕИ ввода'].sum()))

In [ ]:
int(potr_with_ostatok_total['Сумма_распреденного_расхода'].sum()-abs(rashod_start['Сумма во ВВ'].sum()))

In [ ]:
int(potr_with_ostatok_total['Распределение_прихода'].sum()-abs(prihod_start['Количество'].sum()))

In [ ]:
int(potr_with_ostatok_total['Сумма_распреденного_прихода'].sum()-abs(prihod_start['Сумма во ВВ'].sum()))

In [ ]:
int(potr_with_ostatok_total['стрЗапас'].sum()-NZ_start['Количество потребности'].sum())

In [ ]:
# цепляем остаток на начало периода
# potr_with_ostatok_total = pd.merge(potr_with_ostatok_total,ost_nach_perioda,on=['Категория ТМЦ','Завод+НН','Комплекс'],how='left').reset_index()

In [ ]:
# potr_with_ostatok_total = get_in_backup(path_backup='backup_for_powerBI_test_no.h5',name='f0_vertical_10')
# potr_copy_for_prihod = get_in_backup(path_backup='raspro.h5',name='prihod')
potr_with_ostatok_total = get_in_backup(path_backup='backup_for_powerBI_test_no.h5',name='f0_vertical_02_20_')

In [ ]:
# Сразу после распределения 
potr_copy_for_prihod.name = 'prihod_03_20_'
save_backup('raspro.h5',[potr_copy_for_prihod])

In [ ]:
# potr_with_ostatok_total.name = 'f0_vertical_08'
# save_backup('backup_for_powerBI.h5',[potr_with_ostatok_total])
potr_with_ostatok_total.name = 'f0_vertical_03_20_'
save_backup('backup_for_powerBI_test_no.h5',[potr_with_ostatok_total])

In [ ]:
# potr_with_ostatok_total.name = 'f0_vertical_08'
# save_backup('backup_for_powerBI.h5',[potr_with_ostatok_total])
# potr_with_ostatok_total.name = 'f0_vertical_03_20_usd_raspr'
# save_backup('backup_for_powerBI_test_no.h5',[potr_with_ostatok_total])

In [ ]:
potr_with_ostatok_total['Комплекс'].unique()

In [ ]:
# формирование f0

potr_with_ostatok_total['Завод']  = potr_with_ostatok_total['Завод+НН'].str.split(' #').apply(lambda x: x[0])
spr_zav_obl =  rev_to_dict(sprav_mdlv['Область ППМ'].astype('str').str.strip(),sprav_mdlv['Текст ОблПланирПотр'])


potr_with_ostatok_total['стрЗапас'] = potr_with_ostatok_total['Страховой запас']
fr_bad = potr_with_ostatok_total[(potr_with_ostatok_total['Блокированный запас']>0)&(potr_with_ostatok_total['Распределение_остатака']>0)]
potr_with_ostatok_total.loc[fr_bad.index,['Распределение_остатака','Сумма_распреденного_остатка','Распределение_прихода','Сумма_распреденного_прихода','Распределение_расхода','Сумма_распреденного_расхода','avizot','стрЗапас']]    = 0
fr_bad.loc[:,['Блокированный запас','Ст-ть/блок. запаса']]=0
potr_with_ostatok_total = pd.concat([potr_with_ostatok_total,fr_bad],axis=0,ignore_index=True)
potr_with_ostatok_total['Комплекс'] = potr_with_ostatok_total['Комплекс'].astype('str')
potr_with_ostatok_total['Категория ТМЦ'] = potr_with_ostatok_total.apply(lambda x: func_cat_tmc(x),axis = 1)
ind____  = potr_with_ostatok_total[potr_with_ostatok_total['стрЗапас']>0].index
potr_with_ostatok_total.loc[ind____,'Категория ТМЦ'] = 'страховой запас'



potr_with_ostatok_total['Заказ'] = potr_with_ostatok_total['Заказ'].apply(lambda x: str(x).replace('.0',''))
potr_with_ostatok_total['month'] = potr_with_ostatok_total['month'].apply( lambda x: datetime.datetime(1900,1,1) if x==0 else x)
potr_with_ostatok_total['Имя Завода + Материал + Заказ'] = potr_with_ostatok_total['Имя Завода'].astype('str')+' '+potr_with_ostatok_total['Материал'].astype('str')+' '+potr_with_ostatok_total['Заказ'].astype('str')
potr_with_ostatok_total['Код округа + Материал'] = potr_with_ostatok_total['Код округа'].astype('str')+' '+potr_with_ostatok_total['Материал'].astype('str')
potr_with_ostatok_total['Сумма потребности'] = potr_with_ostatok_total['Количество потребности']*potr_with_ostatok_total['Цена']
potr_with_ostatok_total['Сумма стрЗап'] = potr_with_ostatok_total['стрЗапас']*potr_with_ostatok_total['Цена']
potr_with_ostatok_total.reset_index(inplace=True,drop = True)

fil = potr_with_ostatok_total[potr_with_ostatok_total['Категория ТМЦ']=='ТНСЗ']

potr_with_ostatok_total.loc[fil.index,'Распределение_остатака'] = potr_with_ostatok_total.loc[fil.index,'Распределение_остатака']+ potr_with_ostatok_total.loc[fil.index,'Блокированный запас'] 
potr_with_ostatok_total.loc[fil.index,'Сумма_распреденного_остатка'] =potr_with_ostatok_total.loc[fil.index,'Сумма_распреденного_остатка']+ potr_with_ostatok_total.loc[fil.index,'Ст-ть/блок. запаса']  



# potr_with_ostatok_total.reset_index(inplace=True)

# # торо
potr_with_ostatok_total['ТОРО'] = 0
toro = potr_with_ostatok_total[ (potr_with_ostatok_total['Заказ'].str.startswith('1')) & (potr_with_ostatok_total['Вид документа потребности']=='AR')  ]
potr_with_ostatok_total.loc[toro.index,'ТОРО'] = 'ТОРО'
# # РР
potr_with_ostatok_total['РР'] = 0
pp = potr_with_ostatok_total[((potr_with_ostatok_total['Заказ'].str.startswith('2')) & (potr_with_ostatok_total['Вид документа потребности']=='AR'))  |  (potr_with_ostatok_total['Вид документа потребности']=='SB') ]
potr_with_ostatok_total.loc[pp.index,'РР'] = 'РР'
# # прочее
potr_with_ostatok_total['ПРОЧЕЕ'] = 0
prochee =  potr_with_ostatok_total[(potr_with_ostatok_total['Вид документа потребности']=='Z1') | (potr_with_ostatok_total['Вид документа потребности']=='UB02')  ]
potr_with_ostatok_total.loc[prochee.index,'ПРОЧЕЕ'] = 'ПРОЧЕЕ'


# для вставки модуля
rr = potr_with_ostatok_total.copy()
potr_with_ostatok_total['ТОРО'] = 0
potr_with_ostatok_total['РР'] = 0
potr_with_ostatok_total['ПРОЧЕЕ'] = 0


spr_kod_okruga = rev_to_dict(ZMM_E0078_MDLV['Область ППМ'].apply(lambda x: str(x).replace('.0',''))\
                             .astype('str').str.strip(),ZMM_E0078_MDLV['Код округа'])

spr_kod_okruga1 = rev_to_dict(T001W['Завод'].apply(lambda x: str(x).replace('.0','')).astype('str'),\
            T001W['Код округа'].apply(lambda x: str(x).replace('.0','')).astype('str'))

potr_with_ostatok_total['Код округа'] = potr_with_ostatok_total.apply(lambda x: insert_vpr(x['Завод'],spr_kod_okruga1,'Код округа') if insert_vpr(x['Область ППМ'],spr_kod_okruga,'Код округа')==0 else insert_vpr(x['Область ППМ'],spr_kod_okruga,'Код округа'),axis = 1).astype('str')

potr_with_ostatok_total = parallel_calc_fast(potr_with_ostatok_total,KONH,current_date,KONP,MARM,pl_kurs)



potr_with_ostatok_total.reset_index(inplace=True,drop = True)
dic = potr_with_ostatok_total.groupby('Завод+НН').agg({'Количество потребности':'sum'}).to_dict(orient= 'dict')['Количество потребности']
potr_with_ostatok_total['Категория_запаса'] = potr_with_ostatok_total.apply(lambda x: func_cat_tmc_new(x,current_date,dic),axis = 1)

potr_with_ostatok_total['Сумма стрЗап'] = potr_with_ostatok_total['стрЗапас']*potr_with_ostatok_total['Цена']




In [ ]:
potr_with_ostatok_total['Комплекс'].unique()

In [ ]:
potr_with_ostatok_total['Завод']  = potr_with_ostatok_total['Завод+НН'].str.split(' #').apply(lambda x: x[0])

spr_be = rev_to_dict(T001W['Завод'].apply(lambda x: str(x).replace('.0',''))\
                             .astype('str').str.strip(),T001W['Закуп. организация'].apply(lambda x: str(x).replace('.0','')).astype('str').str.strip())
potr_with_ostatok_total['БЕ'] = potr_with_ostatok_total['Завод'].apply(lambda x:  insert_vpr(x,spr_be,'Закуп. организация'))

potr_with_ostatok_total['Материал'] = potr_with_ostatok_total['Завод+НН'].str.split(' #').apply(lambda x: x[1])

potr_with_ostatok_total['Заказчик + HH'] = potr_with_ostatok_total['Комплекс'].astype('str')+' '+potr_with_ostatok_total['Материал'].astype('str')


spr_centr_zav_zav = rev_to_dict(central_zavod['Область ППМ'],central_zavod['Центральный Завод'])



for u in  potr_with_ostatok_total.index:
    if insert_vpr(potr_with_ostatok_total.at[u,'Область ППМ'],spr_centr_zav_zav,'Центральный Завод') ==0:
        potr_with_ostatok_total.at[u,'Центральный завод'] =  insert_vpr(potr_with_ostatok_total.at[u,'Завод'],spr_centr_zav_zav,'Центральный Завод')
    else:
        potr_with_ostatok_total.at[u,'Центральный завод'] =  insert_vpr(potr_with_ostatok_total.at[u,'Область ППМ'],spr_centr_zav_zav,'Центральный Завод')    
potr_with_ostatok_total['Центральный завод'] = potr_with_ostatok_total['Центральный завод'].apply(lambda x: str(x).replace('.0','')).astype('str')  

spr_cena = rev_to_dict(df[df['Цена']>0]['Материал'],df[df['Цена']>0]['Цена'])


potr_with_ostatok_total['Цена'] = potr_with_ostatok_total.apply(lambda x:x['Цена'] if x['Цена']!=0 else insert_vpr(x['Материал'],spr_cena,'Цена'),axis=1)

spr_group_mat_l1 = rev_to_dict(group_mat[group_mat['Группа материалов'].str.len()==2]['Группа материалов'],group_mat[group_mat['Группа материалов'].str.len()==2]['Обозначение 2 группы материалов'])
potr_with_ostatok_total['группа'] = potr_with_ostatok_total['Материал'].str[:2].apply(lambda x:  insert_vpr(x,spr_group_mat_l1,'Обозначение 2 группы материалов'))

spr_group_mat_l2 = rev_to_dict(group_mat[group_mat['Группа материалов'].str.len()==4]['Группа материалов'],group_mat[group_mat['Группа материалов'].str.len()==4]['Обозначение 2 группы материалов'])
potr_with_ostatok_total['подгруппа'] = potr_with_ostatok_total['Материал'].str[:4].apply(lambda x:  insert_vpr(x,spr_group_mat_l2,'Обозначение 2 группы материалов'))


spr_group_mat_l3 = rev_to_dict(group_mat[group_mat['Группа материалов'].str.len()==6]['Группа материалов'],group_mat[group_mat['Группа материалов'].str.len()==6]['Обозначение 2 группы материалов'])
potr_with_ostatok_total['подподгруппа'] = potr_with_ostatok_total['Материал'].str[:6].apply(lambda x:  insert_vpr(x,spr_group_mat_l3,'Обозначение 2 группы материалов'))


spr_obl_ppm_centr_zav = rev_to_dict(sprav_mat_cal_last_full['Материал'],sprav_mat_cal_last_full['Краткий текст материала'])
potr_with_ostatok_total['КрТекстМатериала'] = potr_with_ostatok_total['Материал'].apply(lambda x:  insert_vpr(x,spr_obl_ppm_centr_zav,'Краткий текст материала'))




spr_obl_ppm_baz_ei = rev_to_dict(sprav_mat_cal_last_full['Материал'],sprav_mat_cal_last_full['Базовая ЕИ'])
potr_with_ostatok_total['Базовая ЕИ'] = potr_with_ostatok_total['Материал'].apply(lambda x:  insert_vpr(x,spr_obl_ppm_baz_ei,'Базовая ЕИ'))

spr_obl_ppm_name_planov= rev_to_dict(sprav_mat_cal_last_full['Завод']+' #'+sprav_mat_cal_last_full['Материал'],sprav_mat_cal_last_full['Имя плановика (завод)'])
potr_with_ostatok_total['ЦЗ+НН'] = potr_with_ostatok_total['Центральный завод'].astype('str')+' #'+potr_with_ostatok_total['Материал'].astype('str')
potr_with_ostatok_total['Плановик'] = potr_with_ostatok_total['ЦЗ+НН'].apply(lambda x:  insert_vpr(x,spr_obl_ppm_name_planov,'Имя плановика (завод)'))
potr_with_ostatok_total.drop('ЦЗ+НН',inplace=True,axis = 1)


spr_obl_ppm_ocenka = rev_to_dict(sprav_mat_cal_last_full['Материал'],sprav_mat_cal_last_full['Класс оценки'])
potr_with_ostatok_total['Класс оценки'] = potr_with_ostatok_total['Материал'].apply(lambda x:  insert_vpr(x,spr_obl_ppm_ocenka,'Класс оценки'))
potr_with_ostatok_total['Класс оценки'] = potr_with_ostatok_total['Класс оценки'].apply(lambda x: str(x).replace('.0','')).astype('str')

spr_obl_ppm_stat_na_zav = rev_to_dict(sprav_mat_cal_last_full['Завод']+' #'+sprav_mat_cal_last_full['Материал'],sprav_mat_cal_last_full['СтатусМтрл на заводе'])
potr_with_ostatok_total['СтатусМтрл на заводе'] = potr_with_ostatok_total['Завод+НН'].apply(lambda x:  insert_vpr(x,spr_obl_ppm_stat_na_zav,'СтатусМтрл на заводе'))

spr_toro = rev_to_dict(rr[rr['ТОРО']!=0]['Завод+НН'],rr[rr['ТОРО']!=0]['ТОРО'])
spr_pp = rev_to_dict(rr[rr['РР']!=0]['Завод+НН'],rr[rr['РР']!=0]['РР'])
spr_prochee = rev_to_dict(rr[rr['ПРОЧЕЕ']!=0]['Завод+НН'],rr[rr['ПРОЧЕЕ']!=0]['ПРОЧЕЕ'])
potr_with_ostatok_total['ПРОЧЕЕ'] = potr_with_ostatok_total['Завод+НН'].apply(lambda x:  insert_vpr(x,spr_prochee,'ПРОЧЕЕ'))
potr_with_ostatok_total['ТОРО'] = potr_with_ostatok_total['Завод+НН'].apply(lambda x:  insert_vpr(x,spr_toro,'ТОРО'))
potr_with_ostatok_total['РР'] = potr_with_ostatok_total['Завод+НН'].apply(lambda x:  insert_vpr(x,spr_pp,'РР'))

potr_with_ostatok_total.reset_index(drop=True,inplace=True)

spr_kod_okruga = rev_to_dict(ZMM_E0078_MDLV['Область ППМ'].apply(lambda x: str(x).replace('.0',''))\
                             .astype('str').str.strip(),ZMM_E0078_MDLV['Код округа'])

spr_kod_okruga1 = rev_to_dict(T001W['Завод'].apply(lambda x: str(x).replace('.0','')).astype('str'),\
            T001W['Код округа'].apply(lambda x: str(x).replace('.0','')).astype('str'))

potr_with_ostatok_total['Код округа'] = potr_with_ostatok_total.apply(lambda x: insert_vpr(x['Завод'],spr_kod_okruga1,'Код округа') if insert_vpr(x['Область ППМ'],spr_kod_okruga,'Код округа')==0 else insert_vpr(x['Область ППМ'],spr_kod_okruga,'Код округа'),axis = 1).astype('str')


potr_with_ostatok_total = parallel_calc_fast(potr_with_ostatok_total,KONH,current_date,KONP,MARM,pl_kurs)


potr_with_ostatok_total.fillna(0,inplace = True)

In [ ]:
potr_with_ostatok_total['Комплекс'].unique()

In [ ]:
# potr_with_ostatok_total.name = 'f0_vertical_08'
# save_backup('backup_for_powerBI.h5',[potr_with_ostatok_total])
potr_with_ostatok_total.name = 'f0_vertical_03_20_usd_raspr'
save_backup('backup_for_before_transp.h5',[potr_with_ostatok_total])

In [ ]:
ind = ['Категория_запаса','Модуль','Заказ','Категория ТМЦ','Материал','БЕ','Заказчик + HH','Имя Завода + Материал + Заказ','Завод+НН','Завод + Склад','Завод + Область ППМ','Код округа + Материал','Комплекс','Завод','Склад потребности','Область ППМ','Центральный завод','Код округа','Имя Завода','КрТекстМатериала','Базовая ЕИ','Признак аналога','СтатусМтрл на заводе','Класс оценки','Плановик','группа','подгруппа','подподгруппа','Цена','Недропользование','ПлановСрокПоставки']
col = ['Категория_запаса','Модуль','NB','законтрактовано','Заказ','Категория ТМЦ','Блокированный запас','стрЗапас','Сумма стрЗап','avizot','Сумма_распреденного_расхода','Распределение_расхода','Сумма_распреденного_прихода','Распределение_прихода','Сумма_распреденного_остатка','Распределение_остатака','Количество потребности','month','Материал','БЕ','Заказчик + HH','Имя Завода + Материал + Заказ','Завод+НН','Завод + Склад','Завод + Область ППМ','Код округа + Материал','Комплекс','Завод','Склад потребности','Область ППМ','Центральный завод','Код округа','Имя Завода','КрТекстМатериала','Базовая ЕИ','Признак аналога','СтатусМтрл на заводе','Класс оценки','Плановик','группа','подгруппа','подподгруппа','Цена','Недропользование','ПлановСрокПоставки','Сумма потребности']
tab_f0 =potr_with_ostatok_total[col] 
f0 =  pd.pivot_table(tab_f0,columns=['month'],index=ind,aggfunc='sum',fill_value=0).reset_index()

In [ ]:
# priznak_raschet_nachalo_goda это рассчет на 0101
def f0_consolidation(df,current_date,ind,col,first_month_of_kvartal,priznak_raschet_nachalo_goda,sp):
    def yy(x,sp):
        return x[sp].sum()
    
    
    
    f0 =   df
#     f0 =  pd.pivot_table(tab_f0,columns=['month'],index=ind,aggfunc='sum',fill_value=0).reset_index()  
    cols__lev_f0 = f0[ind]
    cols__lev_f0.columns = ind
    bl_potr = f0['Количество потребности']
    bl_potr.columns = ['ПП кол-во '+str(x.month)+'/'+ str(x.year) for x in bl_potr.columns] 
    pp_sum_col = pd.DataFrame(bl_potr.loc[:,'ПП кол-во '+str(current_date.month+1)+'/'+ str(current_date.year):'ПП кол-во '+str(12)+'/'+ str(current_date.year)].sum(axis = 1),columns=['Текущее кол-во ПП'])


    bl_potr_sum  = f0['Сумма потребности']
    bl_potr_sum.columns = ['ПП сумма '+str(x.month)+'/'+ str(x.year) for x in bl_potr_sum.columns]
    pp_sum_sum = pd.DataFrame(bl_potr_sum.loc[:,'ПП сумма '+str(current_date.month+1)+'/'+ str(current_date.year):'ПП сумма '+str(12)+'/'+ str(current_date.year)].sum(axis = 1),columns=['Текущая сумма ПП'])
    
    

    col_vo_prix = pd.DataFrame(f0['Распределение_прихода'].sum(axis = 1),columns=[str(current_date.month)+'/'+str(current_date.year)+' ФП Кол-во'])
    sum_prix = pd.DataFrame(f0['Сумма_распреденного_прихода'].sum(axis = 1),columns=[str(current_date.month)+'/'+str(current_date.year)+' ФП Сумма'])
    

    col_vo_rash = pd.DataFrame(f0['Распределение_расхода'].sum(axis = 1),columns=[str(current_date.month)+'/'+str(current_date.year)+' ФC Кол-во'])
    summ_rash = pd.DataFrame(f0['Сумма_распреденного_расхода'].sum(axis = 1),columns=[str(current_date.month)+'/'+str(current_date.year)+' ФC Сумма'])

 

    col_vo_ost = pd.DataFrame(f0['Распределение_остатака'].sum(axis = 1),columns=[str(current_date.month)+'/'+str(current_date.year)+' Остаток Кол-во'])
    sum_ost = pd.DataFrame(f0['Сумма_распреденного_остатка'].sum(axis = 1),columns=[str(current_date.month)+'/'+str(current_date.year)+' Остаток Сумма'])

    avizot =  pd.DataFrame(f0['avizot'].sum(axis = 1),columns=['Товар в пути (ZNB5, ZNB2/3-АвизОт)'])

    str_zap_col_vo = pd.DataFrame(f0['стрЗапас'].sum(axis = 1),columns=['SS (норма запаса на складе)'])
    str_zap_summ = pd.DataFrame(f0['Сумма стрЗап'].sum(axis = 1),columns=['SS сумма'])
    
    NB = pd.DataFrame(f0['NB'].sum(axis = 1),columns=['Заявка на закупку(NB)'])
    ZAK = pd.DataFrame(f0['законтрактовано'].sum(axis = 1),columns=['Законтрактовано(ZNB-ПртПЗк)'])

    f0_cons = pd.concat([cols__lev_f0,bl_potr,pp_sum_col,bl_potr_sum,pp_sum_sum,col_vo_prix,sum_prix,col_vo_rash,summ_rash,col_vo_ost,sum_ost,avizot,str_zap_col_vo,str_zap_summ,NB,ZAK],axis=1)

    #     при рассчете 12 месяца для остатка на начало периода закомментировать
#     \\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\
#     f0_cons = pd.merge(f0_cons,ost_nach_perioda,on=['Склад потребности','Заказ','Категория ТМЦ','Комплекс','Завод+НН','Модуль'],how='outer')
# \\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\    
    f0_cons.fillna(0,inplace=True)
    
    if current_date.month == 1 and priznak_raschet_nachalo_goda==True:
        f0_cons['Остаток на начало года Кол-во'] =  f0_cons[str(current_date.month)+'/'+str(current_date.year)+' Остаток Кол-во']  
        f0_cons['Остаток на начало года Сумма'] =  f0_cons[str(current_date.month)+'/'+str(current_date.year)+' Остаток Сумма']    
        f0_cons.drop([str(current_date.month)+'/'+str(current_date.year)+' Остаток Кол-во',\
                     str(current_date.month)+'/'+str(current_date.year)+' Остаток Сумма'],inplace = True,axis = 1)
        
    n = 12
    if current_date.month==10 and priznak_raschet_nachalo_goda ==False:
        n+=1
#         ПП сумма 5/2019
#         ПП кол-во 1/current_date.year+1
#         ПП кол-во n/current_date.year+1
        f0_cons.rename(columns={'ПП кол-во 1/'+ str(current_date.year+1): 'ПП кол-во ' + str(n) + '/' + str(current_date.year)},inplace=True)
        f0_cons.rename(columns={'ПП сумма 1/'+str(current_date.year+1): 'ПП сумма '+str(n)+'/'+ str(current_date.year)},inplace=True)
    elif current_date.month ==11 and priznak_raschet_nachalo_goda ==False:
        n+=2
        f0_cons.rename(columns={'ПП кол-во 1/'+str(current_date.year+1): 'ПП кол-во '+str(n-1)+'/'+ str(current_date.year)},inplace=True)
        f0_cons.rename(columns={'ПП сумма 1/'+str(current_date.year+1): 'ПП сумма '+str(n-1)+'/'+ str(current_date.year)},inplace=True)
        f0_cons.rename(columns={'ПП кол-во 2/'+str(current_date.year+1): 'ПП кол-во '+str(n)+'/'+ str(current_date.year)},inplace=True)
        f0_cons.rename(columns={'ПП сумма 2/'+str(current_date.year+1): 'ПП сумма '+str(n)+'/'+ str(current_date.year)},inplace=True)
    elif current_date.month ==12 and priznak_raschet_nachalo_goda ==False:    
        n+=3
        f0_cons.rename(columns={'ПП кол-во 1/'+str(current_date.year+1): 'ПП кол-во '+str(n-2)+'/'+ str(current_date.year)},inplace=True)
        f0_cons.rename(columns={'ПП сумма 1/'+str(current_date.year+1): 'ПП сумма '+str(n-2)+'/'+ str(current_date.year)},inplace=True)
        f0_cons.rename(columns={'ПП кол-во 2/'+str(current_date.year+1): 'ПП кол-во '+str(n-1)+'/'+ str(current_date.year)},inplace=True)
        f0_cons.rename(columns={'ПП сумма 2/'+str(current_date.year+1): 'ПП сумма '+str(n-1)+'/'+ str(current_date.year)},inplace=True)
        f0_cons.rename(columns={'ПП кол-во 3/'+str(current_date.year+1): 'ПП кол-во '+str(n)+'/'+ str(current_date.year)},inplace=True)
        f0_cons.rename(columns={'ПП сумма 3/'+str(current_date.year+1): 'ПП сумма '+str(n)+'/'+ str(current_date.year)},inplace=True)
    
   
    cols_bz_col_vo = ['БЗ кол-во '+str(x)+'/'+str(current_date.year) for x in range(1,n+1)]
    
    cols_bz_summ = ['БЗ сумма '+str(x)+'/'+str(current_date.year) for x in range(1,n+1)]
    

    for i in cols_bz_col_vo:f0_cons[i] = 0
    f0_cons['ИТОГО БЗ Кол-во'] = 0
    for i in cols_bz_summ:f0_cons[i] = 0
    f0_cons['ИТОГО БЗ Сумма'] = 0


    cols_ost_prognoz = ['Остаток прогноз Кол-во '+str(x)+'/'+str(current_date.year) for x in range(1,n+1)]
    cols_ost_prognoz_summa = ['Остаток прогноз Сумма '+str(x)+'/'+str(current_date.year) for x in range(1,n+1)]
    for i in cols_ost_prognoz:f0_cons[i] = 0
    for i in cols_ost_prognoz_summa:f0_cons[i] = 0
    

        
    # если сейчас 1 месяц тогда текущий опираем на остаток на начало года
    if current_date.month == 1 and priznak_raschet_nachalo_goda==True:
        

        f0_cons.loc[:,cols_bz_col_vo[current_date.month-1]] = f0_cons.apply(lambda x: yy(x,sp)+x['SS (норма запаса на складе)']-x['Остаток на начало года Кол-во']-x['Товар в пути (ZNB5, ZNB2/3-АвизОт)']  \
                                                                            if (x['Остаток на начало года Кол-во']+x['Товар в пути (ZNB5, ZNB2/3-АвизОт)'])-yy(x,sp)-x['SS (норма запаса на складе)']<0 else 0,axis = 1)
            
        f0_cons.loc[:,cols_ost_prognoz[current_date.month-1]] =f0_cons['Остаток на начало года Кол-во'].values+f0_cons['Товар в пути (ZNB5, ZNB2/3-АвизОт)'].values+f0_cons.loc[:,cols_bz_col_vo[current_date.month-1]].values-f0_cons[sp].sum(axis = 1).values

        
        ff = current_date.month-1
        for i in cols_bz_col_vo[current_date.month:]:

            f0_cons[i] = f0_cons.apply(lambda x: x['ПП кол-во'+str(i[9:])]+x['SS (норма запаса на складе)']-x['Остаток прогноз Кол-во'+str(cols_bz_col_vo[ff][9:])]  if x['Остаток прогноз Кол-во'+str(cols_bz_col_vo[ff][9:])]-x['ПП кол-во'+str(i[9:])]-x['SS (норма запаса на складе)']<0 else 0,axis = 1)
    #         формкла для остатка
            f0_cons['Остаток прогноз Кол-во'+str(i[9:])] = f0_cons['Остаток прогноз Кол-во'+str(cols_bz_col_vo[ff][9:])].values+f0_cons[i].values-f0_cons['ПП кол-во'+str(i[9:])].values
#             print('Остаток прогноз Кол-во'+str(cols_bz_col_vo[ff][9:]))

            ff+=1
        f0_cons[cols_ost_prognoz_summa[0]] = f0_cons.apply(lambda x: x[cols_bz_col_vo[0]]* (x['Остаток на начало года Сумма']/  x['Остаток на начало года Кол-во']) if x['Остаток на начало года Кол-во']>0 else x['Остаток прогноз Кол-во '+str(current_date.month+1)+'/'+str(current_date.year)]*x['Цена'] ,axis = 1)
        
        for i in cols_ost_prognoz_summa[current_date.month-1:]:
             f0_cons[i] =    f0_cons.apply(lambda x: x['Остаток прогноз Кол-во'+str(i[21:])]*(x['Остаток на начало года Сумма']/  x['Остаток на начало года Кол-во']) if x['Остаток на начало года Сумма']>0 and  x['Остаток на начало года Кол-во']>0 else x['Остаток прогноз Кол-во'+str(i[21:])]*x['Цена'] ,axis = 1)

        
    else:
        #количество bz     
        f0_cons.loc[:,cols_bz_col_vo[:current_date.month]] = 0
        f0_cons.loc[:,cols_ost_prognoz[:current_date.month]] = 0
        f0_cons.loc[:,cols_ost_prognoz_summa[:current_date.month]] = 0
        f0_cons.loc[:,cols_bz_col_vo[current_date.month]] = f0_cons.apply(lambda x: yy(x,sp)+x['SS (норма запаса на складе)']-x[str(current_date.month)+'/'+str(current_date.year)+' Остаток Кол-во']-x['Товар в пути (ZNB5, ZNB2/3-АвизОт)']  \
                                                                          if (x[str(current_date.month)+'/'+str(current_date.year)+' Остаток Кол-во']+x['Товар в пути (ZNB5, ZNB2/3-АвизОт)'])- yy(x,sp)-x['SS (норма запаса на складе)']<0 else 0,axis = 1)
        f0_cons.loc[:,cols_ost_prognoz[current_date.month]] = f0_cons[str(current_date.month)+'/'+str(current_date.year)+' Остаток Кол-во'].values+f0_cons['Товар в пути (ZNB5, ZNB2/3-АвизОт)'].values+f0_cons.loc[:,cols_bz_col_vo[current_date.month]].values-f0_cons[sp].sum(axis = 1).values


        ff = current_date.month
        for i in cols_bz_col_vo[current_date.month+1:]:

            f0_cons[i] = f0_cons.apply(lambda x: x['ПП кол-во'+str(i[9:])]+x['SS (норма запаса на складе)']-x['Остаток прогноз Кол-во'+str(cols_bz_col_vo[ff][9:])]  if x['Остаток прогноз Кол-во'+str(cols_bz_col_vo[ff][9:])]-x['ПП кол-во'+str(i[9:])]-x['SS (норма запаса на складе)']<0 else 0,axis = 1)
    #         формкла для остатка
            f0_cons['Остаток прогноз Кол-во'+str(i[9:])] = f0_cons['Остаток прогноз Кол-во'+str(cols_bz_col_vo[ff][9:])].values+f0_cons[i].values-f0_cons['ПП кол-во'+str(i[9:])].values
        
        

            ff+=1
            
        for i in cols_ost_prognoz_summa[current_date.month:]:
             f0_cons[i] =    f0_cons.apply(lambda x: x['Остаток прогноз Кол-во'+str(i[21:])]*(x[str(current_date.month)+'/'+str(current_date.year)+' Остаток Сумма']/  x[str(current_date.month)+'/'+str(current_date.year)+' Остаток Кол-во']) if x[str(current_date.month)+'/'+str(current_date.year)+' Остаток Кол-во']>0 else x['Остаток прогноз Кол-во'+str(i[21:])]*x['Цена'] ,axis = 1)

    # сумма бз    
    f0_cons.loc[:,cols_bz_summ]=f0_cons.loc[:,cols_bz_col_vo].mul(f0_cons['Цена'],axis = 0).values  
    f0_cons['ИТОГО БЗ Сумма'] = f0_cons.loc[:,cols_bz_summ].sum(axis=1)
    f0_cons['ИТОГО БЗ Кол-во'] = f0_cons.loc[:,cols_bz_col_vo].sum(axis=1)
    

    
    f0_cons['Категория ТМЦ'] = f0_cons['Категория ТМЦ'] .astype('category')
    f0_cons['Категория ТМЦ'].cat.set_categories(['Текущая потребность', 'Без ПП', 'Проект', 'ТНСЗ', 'ГО','Неликвиды', 'Спецодежда','страховой запас'],inplace = True)

    f0_cons = f0_cons.sort_values('Категория ТМЦ')
    f0_cons['Категория ТМЦ'] = f0_cons['Категория ТМЦ'].astype('str')
    f0_cons.fillna(0,inplace=True)
    f0_cons['Материал'] = f0_cons['Материал'].astype('str')
    
    return f0_cons

        

In [ ]:
# current_date = datetime.datetime(2020,1,1)
# first_month_of_kvartal = 1
# для всех месяцев
c_f0 =  f0_consolidation(current_date=current_date_f0,df=f0,col=col,ind=ind,first_month_of_kvartal = first_month_of_kvartal,priznak_raschet_nachalo_goda = False,sp = sp)
# первый месяц когда рассчет основывается на оатсках на начало года
# c_f0 =  f0_consolidation(current_date=current_date_f0,df=f0,col=col,ind=ind,first_month_of_kvartal = first_month_of_kvartal,priznak_raschet_nachalo_goda = True,sp = sp)


In [ ]:
# c_f0['Завод']  = c_f0['Завод+НН'].str.split(' #').apply(lambda x: x[0])

# pr_be = rev_to_dict(T001W['Завод'].apply(lambda x: str(x).replace('.0',''))\
#                              .astype('str').str.strip(),T001W['Закуп. организация'].apply(lambda x: str(x).replace('.0','')).astype('str').str.strip())
# c_f0['БЕ'] = c_f0['Завод'].apply(lambda x:  insert_vpr(x,spr_be,'БЕ'))

# c_f0['Материал'] = c_f0['Завод+НН'].str.split(' #').apply(lambda x: x[1])

# c_f0['Заказчик + HH'] = c_f0['Комплекс'].astype('str')+' '+c_f0['Материал'].astype('str')


# spr_centr_zav_zav = rev_to_dict(central_zavod['Область ППМ'],central_zavod['Центральный Завод'])



# for u in  c_f0.index:
#     if insert_vpr(c_f0.at[u,'Область ППМ'],spr_centr_zav_zav,'Центральный Завод') ==0:
#         c_f0.at[u,'Центральный завод'] =  insert_vpr(c_f0.at[u,'Завод'],spr_centr_zav_zav,'Центральный Завод')
#     else:
#         c_f0.at[u,'Центральный завод'] =  insert_vpr(c_f0.at[u,'Область ППМ'],spr_centr_zav_zav,'Центральный Завод')    
# c_f0['Центральный завод'] = c_f0['Центральный завод'].apply(lambda x: str(x).replace('.0','')).astype('str')  

# spr_cena = rev_to_dict(df[df['Цена']>0]['Материал'],df[df['Цена']>0]['Цена'])


# c_f0['Цена'] = c_f0.apply(lambda x:x['Цена'] if x['Цена']!=0 else insert_vpr(x['Материал'],spr_cena,'Цена'),axis=1)

# spr_group_mat_l1 = rev_to_dict(group_mat[group_mat['Группа материалов'].str.len()==2]['Группа материалов'],group_mat[group_mat['Группа материалов'].str.len()==2]['Обозначение 2 группы материалов'])
# c_f0['группа'] = c_f0['Материал'].str[:2].apply(lambda x:  insert_vpr(x,spr_group_mat_l1,'Обозначение 2 группы материалов'))

# spr_group_mat_l2 = rev_to_dict(group_mat[group_mat['Группа материалов'].str.len()==4]['Группа материалов'],group_mat[group_mat['Группа материалов'].str.len()==4]['Обозначение 2 группы материалов'])
# c_f0['подгруппа'] = c_f0['Материал'].str[:4].apply(lambda x:  insert_vpr(x,spr_group_mat_l2,'Обозначение 2 группы материалов'))


# spr_group_mat_l3 = rev_to_dict(group_mat[group_mat['Группа материалов'].str.len()==6]['Группа материалов'],group_mat[group_mat['Группа материалов'].str.len()==6]['Обозначение 2 группы материалов'])
# c_f0['подподгруппа'] = c_f0['Материал'].str[:6].apply(lambda x:  insert_vpr(x,spr_group_mat_l3,'Обозначение 2 группы материалов'))


# spr_obl_ppm_centr_zav = rev_to_dict(sprav_mat_cal_last_full['Материал'],sprav_mat_cal_last_full['Краткий текст материала'])
# c_f0['КрТекстМатериала'] = c_f0['Материал'].apply(lambda x:  insert_vpr(x,spr_obl_ppm_centr_zav,'Краткий текст материала'))




# spr_obl_ppm_baz_ei = rev_to_dict(sprav_mat_cal_last_full['Материал'],sprav_mat_cal_last_full['Базовая ЕИ'])
# c_f0['Базовая ЕИ'] = c_f0['Материал'].apply(lambda x:  insert_vpr(x,spr_obl_ppm_baz_ei,'Базовая ЕИ'))

# spr_obl_ppm_name_planov= rev_to_dict(sprav_mat_cal_last_full['Завод']+' #'+sprav_mat_cal_last_full['Материал'],sprav_mat_cal_last_full['Имя плановика (завод)'])
# c_f0['ЦЗ+НН'] = c_f0['Центральный завод'].astype('str')+' #'+c_f0['Материал'].astype('str')
# c_f0['Плановик'] = c_f0['ЦЗ+НН'].apply(lambda x:  insert_vpr(x,spr_obl_ppm_name_planov,'Имя плановика (завод)'))
# c_f0.drop('ЦЗ+НН',inplace=True,axis = 1)


# spr_obl_ppm_ocenka = rev_to_dict(sprav_mat_cal_last_full['Материал'],sprav_mat_cal_last_full['Класс оценки'])
# c_f0['Класс оценки'] = c_f0['Материал'].apply(lambda x:  insert_vpr(x,spr_obl_ppm_ocenka,'Класс оценки'))
# c_f0['Класс оценки'] = c_f0['Класс оценки'].apply(lambda x: str(x).replace('.0','')).astype('str')

# spr_obl_ppm_stat_na_zav = rev_to_dict(sprav_mat_cal_last_full['Завод']+' #'+sprav_mat_cal_last_full['Материал'],sprav_mat_cal_last_full['СтатусМтрл на заводе'])
# c_f0['СтатусМтрл на заводе'] = c_f0['Завод+НН'].apply(lambda x:  insert_vpr(x,spr_obl_ppm_stat_na_zav,'СтатусМтрл на заводе'))

spr_toro = rev_to_dict(rr[rr['ТОРО']!=0]['Завод+НН'],rr[rr['ТОРО']!=0]['ТОРО'])
spr_pp = rev_to_dict(rr[rr['РР']!=0]['Завод+НН'],rr[rr['РР']!=0]['РР'])
spr_prochee = rev_to_dict(rr[rr['ПРОЧЕЕ']!=0]['Завод+НН'],rr[rr['ПРОЧЕЕ']!=0]['ПРОЧЕЕ'])
c_f0['ПРОЧЕЕ'] = c_f0['Завод+НН'].apply(lambda x:  insert_vpr(x,spr_prochee,'ПРОЧЕЕ'))
c_f0['ТОРО'] = c_f0['Завод+НН'].apply(lambda x:  insert_vpr(x,spr_toro,'ТОРО'))
c_f0['РР'] = c_f0['Завод+НН'].apply(lambda x:  insert_vpr(x,spr_pp,'РР'))

c_f0.reset_index(drop=True,inplace=True)
ind_nan =  c_f0.loc[:,'ПП кол-во 1/1900':'Остаток прогноз Сумма 12/'+str(current_date.year)].replace({0:np.nan}).dropna(how = 'all')
c_f0 = c_f0.loc[ind_nan.index]

# spr_kod_okruga = rev_to_dict(ZMM_E0078_MDLV['Область ППМ'].apply(lambda x: str(x).replace('.0',''))\
#                              .astype('str').str.strip(),ZMM_E0078_MDLV['Код округа'])

# spr_kod_okruga1 = rev_to_dict(T001W['Завод'].apply(lambda x: str(x).replace('.0','')).astype('str'),\
#             T001W['Код округа'].apply(lambda x: str(x).replace('.0','')).astype('str'))

# c_f0['Код округа'] = c_f0.apply(lambda x: insert_vpr(x['Завод'],spr_kod_okruga1,'Код округа') if insert_vpr(x['Область ППМ'],spr_kod_okruga,'Код округа')==0 else insert_vpr(x['Область ППМ'],spr_kod_okruga,'Код округа'),axis = 1).astype('str')





# c_f0 = price_main__(c_f0,A902,A900,KONP,current_date,pl_kurs)

c_f0.fillna(0,inplace = True)

In [ ]:

c_f0.drop([x for x in  c_f0.columns if (x.find('ПП кол-во')!=-1 or x.find('ПП сумма')!=-1) and int(x[-4:])!=current_date_f0.year],axis = 1,inplace = True)
c_f0['Текущее кол-во ПП'] = c_f0.loc[:,[x for x in  c_f0.columns if x.find('ПП кол-во')!=-1 ]].sum(axis = 1)
c_f0['Текущая сумма ПП'] = c_f0.loc[:,[x for x in  c_f0.columns if x.find('ПП сумма')!=-1 ]].sum(axis = 1)

In [ ]:
c_f0['Комплекс'].value_counts()

In [ ]:
# проверка f0

In [ ]:
int(c_f0[str(current_date.month)+'/'+str(current_date.year)+ ' Остаток Кол-во'].sum()-ostatok['Кол-во запаса в конце периода'].sum())

In [ ]:
int(c_f0[str(current_date.month)+'/'+str(current_date.year)+ ' Остаток Сумма'].sum()-ostatok['Ст-ть запаса в конце периода'].sum())

In [ ]:
int(c_f0[str(current_date.month)+'/'+str(current_date.year)+ ' ФП Кол-во'].sum()-abs(prihod_start['Количество'].sum()))

In [ ]:
int(c_f0[str(current_date.month)+'/'+str(current_date.year)+ ' ФП Сумма'].sum()-abs(prihod_start['Сумма во ВВ'].sum()))

In [ ]:
int(c_f0[str(current_date.month)+'/'+str(current_date.year)+ ' ФC Кол-во'].sum()-abs(rashod_start['Кол-во в ЕИ ввода'].sum()))

In [ ]:
int(c_f0[str(current_date.month)+'/'+str(current_date.year)+ ' ФC Сумма'].sum()-abs(rashod_start['Сумма во ВВ'].sum()))

In [ ]:
int(c_f0['Остаток на начало года Кол-во'].sum()-abs(ost_nach_perioda['Остаток на начало года Кол-во'].sum()))

In [ ]:
int(c_f0['Остаток на начало года Сумма'].sum()-abs(ost_nach_perioda['Остаток на начало года Сумма'].sum()))

In [ ]:
c_f0.shape

In [ ]:
# int(c_f0['SS (норма запаса на складе)'].sum()-NZ_start['Количество потребности'].sum())

In [ ]:
# save_pyex(df=c_f0,path='C:\\Users\\Илья\\Desktop\\kazzinc\\Uploads1\\extr_xlsx\\f0_01_test.xlsx')

c_f0.name = 'c_f0_02_20_'
save_backup('backup_f0.h5',[c_f0])

In [ ]:
c_f0.name = 'c_f0_02_20_ost_usd'
save_backup('backup_f0.h5',[c_f0])

In [ ]:
c_f0 =  get_in_backup(path_backup='backup_f0.h5',name='c_f0_02_20_')

In [ ]:
for ind, i in enumerate(np.array_split(c_f0,2)) :
    save_pyex(df=i,path='C:\\Users\\Илья\\Desktop\\kazzinc\\Uploads1\\'+folder+'\\extr_xlsx\\'+str(ind)+'.xlsx')

In [ ]:
cons_f0 =  get_in_backup(path_backup='backup_f0.h5',name='c_f0_01_ost')


In [ ]:
# формирование выгрузки общий + текущий месяц!!!!!!!!!!!!!!!!
# !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!

ind = ['Модуль','Заказ','Категория ТМЦ','Материал','БЕ','Заказчик + HH','Имя Завода + Материал + Заказ','Завод+НН','Завод + Склад','Завод + Область ППМ','Код округа + Материал','Комплекс','Завод','Склад потребности','Область ППМ','Центральный завод','Код округа','Имя Завода','КрТекстМатериала','Базовая ЕИ','Признак аналога','СтатусМтрл на заводе','Класс оценки','Плановик','группа','подгруппа','подподгруппа','Цена','Недропользование','ПлановСрокПоставки']
col = ['NB','законтрактовано','Заказ','Категория ТМЦ','Блокированный запас','стрЗапас','Сумма стрЗап','avizot','Сумма_распреденного_расхода','Распределение_расхода','Сумма_распреденного_прихода','Распределение_прихода','Сумма_распреденного_остатка','Распределение_остатака','Количество потребности','month','Материал','БЕ','Заказчик + HH','Имя Завода + Материал + Заказ','Завод+НН','Завод + Склад','Завод + Область ППМ','Код округа + Материал','Комплекс','Завод','Склад потребности','Область ППМ','Центральный завод','Код округа','Имя Завода','КрТекстМатериала','Базовая ЕИ','Признак аналога','СтатусМтрл на заводе','Класс оценки','Плановик','группа','подгруппа','подподгруппа','Цена','Недропользование','ПлановСрокПоставки','Сумма потребности']

# cons_f0 =  get_in_backup(path_backup='f0_cons_10.h5',name='f0_cons_10')
# # cons_f0 = get_in_backup(path_backup='backup_f0_cons.h5',name='c_f0_cons_09')
# current_f0 = get_in_backup(path_backup='backup_f0.h5',name='c_f0_11')




        
       
        
cons_f0['Завод']  = cons_f0['Завод+НН'].str.split(' #').apply(lambda x: x[0])
spr_be = rev_to_dict(T001W['Завод'].apply(lambda x: str(x).replace('.0',''))\
                             .astype('str').str.strip(),T001W['Закуп. организация'].apply(lambda x: str(x).replace('.0','')).astype('str').str.strip())
cons_f0['БЕ'] = cons_f0['Завод'].apply(lambda x:  insert_vpr(x,spr_be,'Закуп. организация'))

cons_f0['Материал'] = cons_f0['Завод+НН'].str.split(' #').apply(lambda x: x[1])

cons_f0['Заказчик + HH'] = cons_f0['Комплекс'].astype('str')+' '+cons_f0['Материал'].astype('str')


spr_centr_zav_zav = rev_to_dict(central_zavod['Область ППМ'],central_zavod['Центральный Завод'])

cons_f0.reset_index(inplace=True,drop=True)

for u in  cons_f0.index:
    if insert_vpr(cons_f0.at[u,'Область ППМ'],spr_centr_zav_zav,'Центральный Завод') ==0:
        cons_f0.at[u,'Центральный завод'] =  insert_vpr(cons_f0.at[u,'Завод'],spr_centr_zav_zav,'Центральный Завод')
    else:
        cons_f0.at[u,'Центральный завод'] =  insert_vpr(cons_f0.at[u,'Область ППМ'],spr_centr_zav_zav,'Центральный Завод')    
cons_f0['Центральный завод'] = cons_f0['Центральный завод'].apply(lambda x: str(x).replace('.0','')).astype('str')  



spr_group_mat_l1 = rev_to_dict(group_mat[group_mat['Группа материалов'].str.len()==2]['Группа материалов'],group_mat[group_mat['Группа материалов'].str.len()==2]['Обозначение 2 группы материалов'])
cons_f0['группа'] = cons_f0['Материал'].str[:2].apply(lambda x:  insert_vpr(x,spr_group_mat_l1,'Обозначение 2 группы материалов'))

spr_group_mat_l2 = rev_to_dict(group_mat[group_mat['Группа материалов'].str.len()==4]['Группа материалов'],group_mat[group_mat['Группа материалов'].str.len()==4]['Обозначение 2 группы материалов'])
cons_f0['подгруппа'] = cons_f0['Материал'].str[:4].apply(lambda x:  insert_vpr(x,spr_group_mat_l2,'Обозначение 2 группы материалов'))


spr_group_mat_l3 = rev_to_dict(group_mat[group_mat['Группа материалов'].str.len()==6]['Группа материалов'],group_mat[group_mat['Группа материалов'].str.len()==6]['Обозначение 2 группы материалов'])
cons_f0['подподгруппа'] = cons_f0['Материал'].str[:6].apply(lambda x:  insert_vpr(x,spr_group_mat_l3,'Обозначение 2 группы материалов'))


spr_obl_ppm_centr_zav = rev_to_dict(sprav_mat_cal_last_full['Материал'],sprav_mat_cal_last_full['Краткий текст материала'])
cons_f0['КрТекстМатериала'] = cons_f0['Материал'].apply(lambda x:  insert_vpr(x,spr_obl_ppm_centr_zav,'Краткий текст материала'))




spr_obl_ppm_baz_ei = rev_to_dict(sprav_mat_cal_last_full['Материал'],sprav_mat_cal_last_full['Базовая ЕИ'])
cons_f0['Базовая ЕИ'] = cons_f0['Материал'].apply(lambda x:  insert_vpr(x,spr_obl_ppm_baz_ei,'Базовая ЕИ'))

spr_obl_ppm_name_planov= rev_to_dict(sprav_mat_cal_last_full['Завод']+' #'+sprav_mat_cal_last_full['Материал'],sprav_mat_cal_last_full['Имя плановика (завод)'])
cons_f0['ЦЗ+НН'] = cons_f0['Центральный завод'].astype('str')+' #'+cons_f0['Материал'].astype('str')
cons_f0['Плановик'] = cons_f0['ЦЗ+НН'].apply(lambda x:  insert_vpr(x,spr_obl_ppm_name_planov,'Имя плановика (завод)'))
cons_f0.drop('ЦЗ+НН',inplace=True,axis = 1)


spr_obl_ppm_ocenka = rev_to_dict(sprav_mat_cal_last_full['Материал'],sprav_mat_cal_last_full['Класс оценки'])
cons_f0['Класс оценки'] = cons_f0['Материал'].apply(lambda x:  insert_vpr(x,spr_obl_ppm_ocenka,'Класс оценки'))
cons_f0['Класс оценки'] = cons_f0['Класс оценки'].apply(lambda x: str(x).replace('.0','')).astype('str')

spr_obl_ppm_stat_na_zav = rev_to_dict(sprav_mat_cal_last_full['Завод']+' #'+sprav_mat_cal_last_full['Материал'],sprav_mat_cal_last_full['СтатусМтрл на заводе'])
cons_f0['СтатусМтрл на заводе'] = cons_f0['Завод+НН'].apply(lambda x:  insert_vpr(x,spr_obl_ppm_stat_na_zav,'СтатусМтрл на заводе'))
   

spr_obl_ppm = rev_to_dict(spr_for_obl_ppm['Завод'].apply(lambda x: str(x).replace('.0','')).astype('str')+spr_for_obl_ppm['Склад'].\
            apply(lambda x: str(x).replace('.0','')).astype('str'),spr_for_obl_ppm['Область ППМ'])
cons_f0['Склад потребности'] = cons_f0['Склад потребности'].apply(lambda x:'' if x ==0 or x=='0' else x)
cons_f0['Область ППМ']  = cons_f0\
                                    .apply(lambda x: x['Завод'] if    insert_vpr(x['Завод']+x['Склад потребности'],spr_obl_ppm,'Область ППМ')==0 else  \
                                     insert_vpr(x['Завод']+x['Склад потребности'],spr_obl_ppm,'Область ППМ'),axis = 1).astype('str')

spr_kod_okruga = rev_to_dict(ZMM_E0078_MDLV['Область ППМ'].apply(lambda x: str(x).replace('.0',''))\
                             .astype('str').str.strip(),ZMM_E0078_MDLV['Код округа'])

spr_kod_okruga1 = rev_to_dict(T001W['Завод'].apply(lambda x: str(x).replace('.0','')).astype('str'),\
            T001W['Код округа'].apply(lambda x: str(x).replace('.0','')).astype('str'))

cons_f0['Код округа'] = cons_f0.apply(lambda x: insert_vpr(x['Завод'],spr_kod_okruga1,'Код округа') if insert_vpr(x['Область ППМ'],spr_kod_okruga,'Код округа')==0 else insert_vpr(x['Область ППМ'],spr_kod_okruga,'Код округа'),axis = 1).astype('str')
    




# c_f = pd.concat([current_f0.set_index(ind),
#                  cons_f0.set_index(ind).loc[:,str(current_date.month-1)+'/2019 Остаток Кол-во':str(current_date.month-2)+'/2019 ФC Сумма']],axis=1).reset_index()





# arr5 = pd.concat([c_f[ind],  
#         c_f.loc[:,[x for x in c_f.columns if x.find('ПП кол-во')!=-1]],   c_f['Текущее кол-во ПП'],
#         c_f.loc[:,[x for x in c_f.columns if x.find('ПП сумма')!=-1]],    c_f['Текущая сумма ПП'],
#         c_f.loc[:,[x for x in c_f.columns if x.find('Остаток Кол-во')!=-1]],
#         c_f.loc[:,[x for x in c_f.columns if x.find('Остаток Сумма')!=-1]],
#         c_f.loc[:,[x for x in c_f.columns if x.find('ФП Кол-во')!=-1]],
#         c_f.loc[:,[x for x in c_f.columns if x.find('ФП Сумма')!=-1]],
#         c_f.loc[:,[x for x in c_f.columns if x.find('ФC Кол-во')!=-1]],
#         c_f.loc[:,[x for x in c_f.columns if x.find('ФC Сумма')!=-1]],
#         c_f.loc[:,'Товар в пути (ZNB5, ZNB2/3-АвизОт)':'РР'] ],axis = 1,)





In [ ]:
c_f0.fillna(0,inplace=True)
cons_f0.fillna(0,inplace=True)
c_f0.reset_index(drop=True,inplace = True)
cons_f0.reset_index(drop=True,inplace = True)

istochniki = [cons_f0,c_f0]

for ist in istochniki:
    for col in ind:
        ist[col] = ist[col].astype('str')

In [ ]:


for ist in istochniki:
    for col in ind:
        ist[col] = ist[col].astype('str')
        ist['Цена'] = '0'
        
cons_f0['Завод']  = cons_f0['Завод+НН'].str.split(' #').apply(lambda x: x[0])
spr_zav_obl =  rev_to_dict(sprav_mdlv['Область ППМ'].astype('str').str.strip(),sprav_mdlv['Текст ОблПланирПотр'])
cons_f0['Имя Завода'] = cons_f0.apply(lambda x: insert_vpr(x['Завод'],spr_zav_obl,'Текст ОблПланирПотр')  if x['Область ППМ']=='' or x['Область ППМ']==0 else insert_vpr(x['Область ППМ'],spr_zav_obl,'Текст ОблПланирПотр'),axis = 1 )
cons_f0['Имя Завода + Материал + Заказ'] = cons_f0['Имя Завода'].astype('str')+' '+cons_f0['Материал'].astype('str')+' '+cons_f0['Заказ'].astype('str')


spr_obl_ppm = rev_to_dict(spr_for_obl_ppm['Завод'].apply(lambda x: str(x).replace('.0','')).astype('str')+spr_for_obl_ppm['Склад'].\
            apply(lambda x: str(x).replace('.0','')).astype('str'),spr_for_obl_ppm['Область ППМ'])

cons_f0['Область ППМ']  = cons_f0\
                                    .apply(lambda x: x['Завод'] if    insert_vpr(x['Завод']+x['Склад потребности'],spr_obl_ppm,'Область ППМ')==0 else  \
                                     insert_vpr(x['Завод']+x['Склад потребности'],spr_obl_ppm,'Область ППМ'),axis = 1).astype('str')

c_f0['Область ППМ']  = c_f0\
                                    .apply(lambda x: x['Завод'] if    insert_vpr(x['Завод']+x['Склад потребности'],spr_obl_ppm,'Область ППМ')==0 else  \
                                     insert_vpr(x['Завод']+x['Склад потребности'],spr_obl_ppm,'Область ППМ'),axis = 1).astype('str')


c_f0['Склад потребности'] = c_f0['Склад потребности'].apply(lambda x:'' if x ==0 or x=='0' else x)
c_f0['Область ППМ']  = c_f0\
                                    .apply(lambda x: x['Завод'] if    insert_vpr(x['Завод']+x['Склад потребности'],spr_obl_ppm,'Область ППМ')==0 else  \
                                     insert_vpr(x['Завод']+x['Склад потребности'],spr_obl_ppm,'Область ППМ'),axis = 1).astype('str')

c_f0['Завод + Область ППМ'] = c_f0['Завод'].astype('str')+' '+c_f0['Область ППМ'].astype('str')
cons_f0['Завод + Область ППМ'] = cons_f0['Завод'].astype('str')+' '+cons_f0['Область ППМ'].astype('str')

spr_zav_kompl = rev_to_dict(sprav_po_kompl['Завод + Область ППМ'].astype('str').str.strip(),sprav_po_kompl['Комплекс'])
c_f0['Комплекс'] = c_f0['Завод + Область ППМ'].apply(lambda x: insert_vpr(x,spr_zav_kompl,'Комплекс') )
cons_f0['Комплекс'] = cons_f0['Завод + Область ППМ'].apply(lambda x: insert_vpr(x,spr_zav_kompl,'Комплекс') )

cons_f0['Код округа + Материал'] = cons_f0['Код округа'].astype('str')+' '+cons_f0['Материал'].astype('str')

c_f0['Код округа + Материал'] = c_f0['Код округа'].astype('str')+' '+c_f0['Материал'].astype('str')

c_f0['Заказчик + HH'] = c_f0['Комплекс'].astype('str')+' '+c_f0['Материал'].astype('str')
cons_f0['Заказчик + HH'] = cons_f0['Комплекс'].astype('str')+' '+cons_f0['Материал'].astype('str')

cons_f0['Склад потребности'] = cons_f0['Склад потребности'].apply(lambda x:'' if x ==0 or x=='0' else x)

spr_zav_mat = rev_to_dict(sprav_mat_cal['Завод']+' '+sprav_mat_cal['Материал'],sprav_mat_cal['Плановый срок поставки (завод)'])
cons_f0['Центральный завод'] = cons_f0['Центральный завод'].astype('str')
for i in cons_f0.index:
        try:
            if cons_f0.at[i,'Недропользование']==1:cons_f0.at[i,'ПлановСрокПоставки'] =  spr_zav_mat[cons_f0.at[i,'Завод']+' '+cons_f0.at[i,'Материал']]['Плановый срок поставки (завод)']
            else:cons_f0.at[i,'ПлановСрокПоставки'] = spr_zav_mat[cons_f0.at[i,'Центральный завод']+' '+cons_f0.at[i,'Материал']]['Плановый срок поставки (завод)']
        except KeyError:
            pass
        
c_f0['Центральный завод'] = c_f0['Центральный завод'].astype('str')
for i in c_f0.index:
        try:
            if c_f0.at[i,'Недропользование']==1:c_f0.at[i,'ПлановСрокПоставки'] =  spr_zav_mat[c_f0.at[i,'Завод']+' '+c_f0.at[i,'Материал']]['Плановый срок поставки (завод)']
            else:c_f0.at[i,'ПлановСрокПоставки'] = spr_zav_mat[c_f0.at[i,'Центральный завод']+' '+c_f0.at[i,'Материал']]['Плановый срок поставки (завод)']
        except KeyError:
            pass
        
cons_f0['ПлановСрокПоставки'] = cons_f0['ПлановСрокПоставки'].apply(lambda x:int(x.split('.')[0]) if  type(x)==str else x)
c_f0['ПлановСрокПоставки'] = c_f0['ПлановСрокПоставки'].apply(lambda x:int(x.split('.')[0]) if  type(x)==str else x)

for ist in istochniki:
    for col in ind:
        ist[col] = ist[col].astype('str')


In [ ]:
cons_f0['ПлановСрокПоставки'].unique()

In [ ]:
c_f0['ПлановСрокПоставки'].unique()

In [ ]:
ind = ['Категория_запаса','Модуль','Заказ','Категория ТМЦ','Материал','БЕ','Заказчик + HH','Имя Завода + Материал + Заказ','Завод+НН','Завод + Склад','Завод + Область ППМ','Код округа + Материал','Комплекс','Завод','Склад потребности','Область ППМ','Центральный завод','Код округа','Имя Завода','КрТекстМатериала','Базовая ЕИ','Признак аналога','СтатусМтрл на заводе','Класс оценки','Плановик','группа','подгруппа','подподгруппа','Цена','Недропользование','ПлановСрокПоставки']
c_f = pd.concat([c_f0.groupby(ind).sum(),
                cons_f0.groupby(ind).agg({'Остаток на начало года Кол-во':'sum','Остаток на начало года Сумма':'sum'})],axis=1).reset_index()

In [ ]:
c_f.shape

In [ ]:
arr5 = c_f
arr5 = price_main__(arr5,A902,A900,KONP,current_date,pl_kurs)


# обозначаем категории
arr5['Категория ТМЦ'] = arr5['Категория ТМЦ'] .astype('category')
arr5['Категория ТМЦ'].cat.set_categories(['Текущая потребность', 'Без ПП', 'Проект', 'ТНСЗ', 'ГО','Неликвиды', 'Спецодежда','страховой запас'],inplace = True)

arr5 = arr5.sort_values('Категория ТМЦ')
arr5['Категория ТМЦ'] = arr5['Категория ТМЦ'].astype('str')
arr5.fillna(0,inplace=True)
# arr5.drop('ПП кол-во 1/1900',inplace=True,axis = 1)
# arr5.drop('ПП сумма 1/1900',inplace=True,axis = 1)


In [ ]:
rr = potr_with_ostatok_total.copy()
spr_toro = rev_to_dict(rr[rr['ТОРО']!=0]['Завод+НН'],rr[rr['ТОРО']!=0]['ТОРО'])
spr_pp = rev_to_dict(rr[rr['РР']!=0]['Завод+НН'],rr[rr['РР']!=0]['РР'])
spr_prochee = rev_to_dict(rr[rr['ПРОЧЕЕ']!=0]['Завод+НН'],rr[rr['ПРОЧЕЕ']!=0]['ПРОЧЕЕ'])
arr5['ПРОЧЕЕ'] = arr5['Завод+НН'].apply(lambda x:  insert_vpr(x,spr_prochee,'ПРОЧЕЕ'))
arr5['ТОРО'] = arr5['Завод+НН'].apply(lambda x:  insert_vpr(x,spr_toro,'ТОРО'))
arr5['РР'] = arr5['Завод+НН'].apply(lambda x:  insert_vpr(x,spr_pp,'РР'))

In [ ]:
arr5

In [ ]:
arr5 = arr5.reset_index(drop = True)

In [ ]:
arr5.name = 'c_f0_cons_01'
save_backup('backup_f0_cons.h5',[arr5])

In [ ]:
arr5 =  get_in_backup(path_backup='backup_f0_cons.h5',name='c_f0_cons_02')

In [ ]:
for ind, i in enumerate(np.array_split(arr5,2)) :
    save_pyex(df=i,path='C:\\Users\\Илья\\Desktop\\kazzinc\\Uploads1\\'+folder+'\\extr_xlsx\\'+str(ind)+'.xlsx')
    

In [ ]:
tttt1 = c_f0.groupby(ind).sum().reset_index()

In [ ]:
tttt2 = cons_f0.groupby(ind).agg({'Остаток на начало года Кол-во':'sum','Остаток на начало года Сумма':'sum'}).reset_index()

In [ ]:
tttt1 = get_in_backup(path_backup='backup_f0_cons.h5',name='c_f0_cons_02')

In [ ]:
arr5.drop('level_0',axis = 1,inplace = True)

In [ ]:
arr5

In [ ]:
tttt1[(tttt1['Материал']=='040104016')&(tttt1['Завод']=='1201')]

In [ ]:
tttt2[(tttt2['Материал']=='040104016')&(tttt2['Завод']=='1201')]

In [ ]:
tt = 

In [ ]:
c_f['Категория ТМЦ'].unique()

In [ ]:
potr_with_ostatok_total['БЕ'].unique()

In [ ]:
save_backup('backup_f0.h5',[c_f0_01,c_f0_02,c_f0_03,c_f0_04,c_f0_05,c_f0_06])

In [ ]:
# сохранение остатков на начало периода
col_vo_name = 'Остаток на начало года Кол-во'
summ_name = 'Остаток на начало года Сумма'

ar_ost_n = c_f0[['Склад потребности','Заказ','Категория ТМЦ','Комплекс','Завод+НН','Модуль',col_vo_name,summ_name]].groupby(['Склад потребности','Заказ','Категория ТМЦ','Комплекс','Завод+НН','Модуль']).sum().reset_index().drop_duplicates()

ost_nach_perioda = ar_ost_n[ar_ost_n[col_vo_name]>0]
ost_nach_perioda.rename(columns={col_vo_name: "Остаток на начало года Кол-во",summ_name:'Остаток на начало года Сумма'},inplace=True)
ost_nach_perioda.name= 'ost_nach_perioda'
save_backup('backup_fine.h5',[ost_nach_perioda])
save_backup('backup.h5',[ost_nach_perioda])
save_pyex(df=ost_nach_perioda,path='C:\\Users\\Илья\\Desktop\\kazzinc\\Uploads1\\ost_nach_perioda.xlsx')

In [ ]:
# Формирование Общего f0 для одной выгрузки экстракта
ind = ['Модуль','Заказ','Категория ТМЦ','Материал','БЕ','Заказчик + HH','Имя Завода + Материал + Заказ','Завод+НН','Завод + Склад','Завод + Область ППМ','Код округа + Материал','Комплекс','Завод','Склад потребности','Область ППМ','Центральный завод','Код округа','Имя Завода','КрТекстМатериала','Базовая ЕИ','Признак аналога','СтатусМтрл на заводе','Класс оценки','Плановик','группа','подгруппа','подподгруппа','Цена','Недропользование','ПлановСрокПоставки']
col = ['NB','законтрактовано','Заказ','Категория ТМЦ','Блокированный запас','стрЗапас','Сумма стрЗап','avizot','Сумма_распреденного_расхода','Распределение_расхода','Сумма_распреденного_прихода','Распределение_прихода','Сумма_распреденного_остатка','Распределение_остатака','Количество потребности','month','Материал','БЕ','Заказчик + HH','Имя Завода + Материал + Заказ','Завод+НН','Завод + Склад','Завод + Область ППМ','Код округа + Материал','Комплекс','Завод','Склад потребности','Область ППМ','Центральный завод','Код округа','Имя Завода','КрТекстМатериала','Базовая ЕИ','Признак аналога','СтатусМтрл на заводе','Класс оценки','Плановик','группа','подгруппа','подподгруппа','Цена','Недропользование','ПлановСрокПоставки','Сумма потребности']


c_f0_01 = get_in_backup(path_backup='backup_f0.h5',name='c_f0_01')
c_f0_02 = get_in_backup(path_backup='backup_f0.h5',name='c_f0_02')
c_f0_03 = get_in_backup(path_backup='backup_f0.h5',name='c_f0_03')
c_f0_04 = get_in_backup(path_backup='backup_f0.h5',name='c_f0_04')
c_f0_05 = get_in_backup(path_backup='backup_f0.h5',name='c_f0_05')
c_f0_06 = get_in_backup(path_backup='backup_f0.h5',name='c_f0_06')
c_f0_07 = get_in_backup(path_backup='backup_f0.h5',name='c_f0_07')
c_f0_08 = get_in_backup(path_backup='backup_f0.h5',name='c_f0_08')


istochniki = [c_f0_01,c_f0_02,c_f0_03,c_f0_04,c_f0_05,c_f0_06,c_f0_07,c_f0_08]

for ist in istochniki:
    for col in ind:
        ist[col] = ist[col].astype('str')
        ist['Цена'] = '0'
        
        
        

c_f = pd.concat([c_f0_01.set_index(ind).loc[:,'1/2019 ФП Кол-во':'1/2019 Остаток Сумма'],
                 c_f0_02.set_index(ind).loc[:,'2/2019 ФП Кол-во':'2/2019 Остаток Сумма'],
                 c_f0_03.set_index(ind).loc[:,'3/2019 ФП Кол-во':'3/2019 Остаток Сумма'],
                 c_f0_04.set_index(ind).loc[:,'4/2019 ФП Кол-во':'4/2019 Остаток Сумма'],
                 c_f0_05.set_index(ind).loc[:,'5/2019 ФП Кол-во':'5/2019 Остаток Сумма'],
                 c_f0_06.set_index(ind).loc[:,'6/2019 ФП Кол-во':'6/2019 Остаток Сумма'],
                 c_f0_07.set_index(ind).loc[:,'7/2019 ФП Кол-во':'7/2019 Остаток Сумма'],
                 c_f0_08.set_index(ind)],axis=1).reset_index()




arr5 = pd.concat([c_f[ind],  
        c_f.loc[:,[x for x in c_f.columns if x.find('ПП кол-во')!=-1]],   c_f['Текущее кол-во ПП'],
        c_f.loc[:,[x for x in c_f.columns if x.find('ПП сумма')!=-1]],    c_f['Текущая сумма ПП'],
        c_f.loc[:,[x for x in c_f.columns if x.find('Остаток Кол-во')!=-1]],
        c_f.loc[:,[x for x in c_f.columns if x.find('Остаток Сумма')!=-1]],
        c_f.loc[:,[x for x in c_f.columns if x.find('ФП Кол-во')!=-1]],
        c_f.loc[:,[x for x in c_f.columns if x.find('ФП Сумма')!=-1]],
        c_f.loc[:,[x for x in c_f.columns if x.find('ФC Кол-во')!=-1]],
        c_f.loc[:,[x for x in c_f.columns if x.find('ФC Сумма')!=-1]],
        c_f.loc[:,'Товар в пути (ZNB5, ZNB2/3-АвизОт)':]  ],axis = 1,)


# обозначаем категории
arr5['Категория ТМЦ'] = arr5['Категория ТМЦ'] .astype('category')
arr5['Категория ТМЦ'].cat.set_categories(['Текущая потребность', 'Без ПП', 'Проект', 'ТНСЗ', 'ГО','Неликвиды', 'Спецодежда'],inplace = True)

arr5 = arr5.sort_values('Категория ТМЦ')
arr5['Категория ТМЦ'] = arr5['Категория ТМЦ'].astype('str')
arr5.fillna(0,inplace=True)
arr5.drop('ПП кол-во 1/1900',inplace=True,axis = 1)
arr5.drop('ПП сумма 1/1900',inplace=True,axis = 1)



In [ ]:


arr5['Цена'] = arr5['Цена'].astype('float64')

A902['Код округа'] = A902['Код округа'].astype('str')
A902['Материал'] = A902['Материал'].astype('str')
A902['Материал'] = A902['Материал'].apply(lambda x: '0'+str(x) if  len(str(x))<9 else x)
A900['Материал'] = A900['Материал'].astype('str')
A900['Материал'] = A900['Материал'].apply(lambda x: '0'+str(x) if  len(str(x))<9 else x)
arr5['Материал'] = arr5['Материал'].astype('str')
A902.fillna(0,inplace=True)
A900.fillna(0,inplace=True)
KONP.fillna(0,inplace=True)
potr_gr = dict(tuple(arr5[arr5['Цена']==0].groupby(['Код округа','Материал'])))
potr_gr_mat = dict(tuple(arr5[arr5['Цена']==0].groupby(['Материал'])))
konp_gr = dict(tuple(KONP.groupby(['Номер записи условий'])))
# c_f0['Цена'] = 0.0000
A900 = A900[A900['Конец действия'].apply(lambda x: x.year)==current_date.year]
A902 = A902[A902['Конец действия'].apply(lambda x: x.year)==current_date.year]
main_price(potr_gr,potr_gr_mat,A900,A902,konp_gr,arr5)

In [ ]:
save_pyex(df=arr5,path='C:\\Users\\Илья\\Desktop\\kazzinc\\Uploads1\\extr_xlsx\\f0_cons.xlsx')

In [ ]:
for ind, i in enumerate(np.array_split(arr5,3)) :
    save_pyex(df=i,path='C:\\Users\\User\\Desktop\\kazzinc\\Uploads1\\extr_xlsx\\'+str(ind)+'.xlsx')

In [ ]:
# для bi
c_f0_01 = get_in_backup(path_backup='backup_for_powerBI.h5',name='f0_vertical_01')
c_f0_02 = get_in_backup(path_backup='backup_for_powerBI.h5',name='f0_vertical_02')
c_f0_03 = get_in_backup(path_backup='backup_for_powerBI.h5',name='f0_vertical_03')
c_f0_04 = get_in_backup(path_backup='backup_for_powerBI.h5',name='f0_vertical_04')
c_f0_05 = get_in_backup(path_backup='backup_for_powerBI.h5',name='f0_vertical_05')
c_f0_06 = get_in_backup(path_backup='backup_for_powerBI.h5',name='f0_vertical_06')
c_f0_07 = get_in_backup(path_backup='backup_for_powerBI.h5',name='f0_vertical_07')
c_f0_08 = get_in_backup(path_backup='backup_for_powerBI.h5',name='f0_vertical_08')
c_f0_09 = get_in_backup(path_backup='backup_for_powerBI.h5',name='f0_vertical_09')

months = {1:'Январь',2:'Февраль',3:'Март',4:'Апрель',5:'Май',6:'Июнь',7:'Июль',8:'Август',9:'Сентябрь',10:'Октябрь',11:'Ноябрь',12:'Декабрь'}
for ind,df_for_bi in enumerate([c_f0_01,c_f0_02,c_f0_03,c_f0_04,c_f0_05,c_f0_06,c_f0_07,c_f0_08,c_f0_09],start=1) :
    df_for_bi['Месяц'] = str(datetime.datetime(2019,ind,1))

df_for_bi = pd.DataFrame()
df_for_bi = pd.concat([c_f0_01,c_f0_02,c_f0_03,c_f0_04,c_f0_05,c_f0_06,c_f0_07,c_f0_08,c_f0_09])






df_for_bi['Категория ТМЦ'] = df_for_bi.apply(lambda x: func_cat_tmc(x),axis = 1)

df_for_bi['Завод']  = df_for_bi['Завод+НН'].str.split(' #').apply(lambda x: x[0])
BE = pd.DataFrame()
for i in range(1,10):
    BE = BE.append(get_in_backup(path_backup='backup_f0.h5',name='c_f0_0'+str(i))[['Завод','БЕ']])
BE = BE.drop_duplicates()   
spr_be = rev_to_dict(BE['Завод'],BE['БЕ'])
df_for_bi['БЕ'] = df_for_bi['Завод'].apply(lambda x:  insert_vpr(x,spr_be,'БЕ'))
df_for_bi['Материал'] = df_for_bi['Завод+НН'].str.split(' #').apply(lambda x: x[1])

df_for_bi['Заказчик + HH'] = df_for_bi['Комплекс'].astype('str')+' '+df_for_bi['Материал'].astype('str')


spr_centr_zav_zav = rev_to_dict(central_zavod['Область ППМ'],central_zavod['Центральный Завод'])

df_for_bi.reset_index(inplace=True,drop=True)

for u in  df_for_bi.index:
    if insert_vpr(df_for_bi.at[u,'Область ППМ'],spr_centr_zav_zav,'Центральный Завод') ==0:
        df_for_bi.at[u,'Центральный завод'] =  insert_vpr(df_for_bi.at[u,'Завод'],spr_centr_zav_zav,'Центральный Завод')
    else:
        df_for_bi.at[u,'Центральный завод'] =  insert_vpr(df_for_bi.at[u,'Область ППМ'],spr_centr_zav_zav,'Центральный Завод')    
df_for_bi['Центральный завод'] = df_for_bi['Центральный завод'].apply(lambda x: str(x).replace('.0','')).astype('str')  



spr_group_mat_l1 = rev_to_dict(group_mat[group_mat['Группа материалов'].str.len()==2]['Группа материалов'],group_mat[group_mat['Группа материалов'].str.len()==2]['Обозначение 2 группы материалов'])
df_for_bi['группа'] = df_for_bi['Материал'].str[:2].apply(lambda x:  insert_vpr(x,spr_group_mat_l1,'Обозначение 2 группы материалов'))

spr_group_mat_l2 = rev_to_dict(group_mat[group_mat['Группа материалов'].str.len()==4]['Группа материалов'],group_mat[group_mat['Группа материалов'].str.len()==4]['Обозначение 2 группы материалов'])
df_for_bi['подгруппа'] = df_for_bi['Материал'].str[:4].apply(lambda x:  insert_vpr(x,spr_group_mat_l2,'Обозначение 2 группы материалов'))


spr_group_mat_l3 = rev_to_dict(group_mat[group_mat['Группа материалов'].str.len()==6]['Группа материалов'],group_mat[group_mat['Группа материалов'].str.len()==6]['Обозначение 2 группы материалов'])
df_for_bi['подподгруппа'] = df_for_bi['Материал'].str[:6].apply(lambda x:  insert_vpr(x,spr_group_mat_l3,'Обозначение 2 группы материалов'))


spr_obl_ppm_centr_zav = rev_to_dict(sprav_mat_cal_last_full['Материал'],sprav_mat_cal_last_full['Краткий текст материала'])
df_for_bi['КрТекстМатериала'] = df_for_bi['Материал'].apply(lambda x:  insert_vpr(x,spr_obl_ppm_centr_zav,'Краткий текст материала'))




spr_obl_ppm_baz_ei = rev_to_dict(sprav_mat_cal_last_full['Материал'],sprav_mat_cal_last_full['Базовая ЕИ'])
df_for_bi['Базовая ЕИ'] = df_for_bi['Материал'].apply(lambda x:  insert_vpr(x,spr_obl_ppm_baz_ei,'Базовая ЕИ'))

spr_obl_ppm_name_planov= rev_to_dict(sprav_mat_cal_last_full['Завод']+' #'+sprav_mat_cal_last_full['Материал'],sprav_mat_cal_last_full['Имя плановика (завод)'])
df_for_bi['ЦЗ+НН'] = df_for_bi['Центральный завод'].astype('str')+' #'+df_for_bi['Материал'].astype('str')
df_for_bi['Плановик'] = df_for_bi['ЦЗ+НН'].apply(lambda x:  insert_vpr(x,spr_obl_ppm_name_planov,'Имя плановика (завод)'))
df_for_bi.drop('ЦЗ+НН',inplace=True,axis = 1)


spr_obl_ppm_ocenka = rev_to_dict(sprav_mat_cal_last_full['Материал'],sprav_mat_cal_last_full['Класс оценки'])
df_for_bi['Класс оценки'] = df_for_bi['Материал'].apply(lambda x:  insert_vpr(x,spr_obl_ppm_ocenka,'Класс оценки'))
df_for_bi['Класс оценки'] = df_for_bi['Класс оценки'].apply(lambda x: str(x).replace('.0','')).astype('str')

spr_obl_ppm_stat_na_zav = rev_to_dict(sprav_mat_cal_last_full['Завод']+' #'+sprav_mat_cal_last_full['Материал'],sprav_mat_cal_last_full['СтатусМтрл на заводе'])
df_for_bi['СтатусМтрл на заводе'] = df_for_bi['Завод+НН'].apply(lambda x:  insert_vpr(x,spr_obl_ppm_stat_na_zav,'СтатусМтрл на заводе'))


In [ ]:
A902['Код округа'] = A902['Код округа'].astype('str')
A902['Материал'] = A902['Материал'].astype('str')
A902['Материал'] = A902['Материал'].apply(lambda x: '0'+str(x) if  len(str(x))<9 else x)
A900['Материал'] = A900['Материал'].astype('str')
A900['Материал'] = A900['Материал'].apply(lambda x: '0'+str(x) if  len(str(x))<9 else x)
df_for_bi['Материал'] = df_for_bi['Материал'].astype('str')
A902.fillna(0,inplace=True)
A900.fillna(0,inplace=True)
KONP.fillna(0,inplace=True)
potr_gr = dict(tuple(df_for_bi[df_for_bi['Цена']==0].groupby(['Код округа','Материал'])))
potr_gr_mat = dict(tuple(df_for_bi[df_for_bi['Цена']==0].groupby(['Материал'])))
konp_gr = dict(tuple(KONP.groupby(['Номер записи условий'])))
# c_f0['Цена'] = 0.0000
A900 = A900[A900['Конец действия'].apply(lambda x: x.year)==current_date.year]
A902 = A902[A902['Конец действия'].apply(lambda x: x.year)==current_date.year]
main_price(potr_gr,potr_gr_mat,A900,A902,konp_gr,df_for_bi)

In [ ]:
# for ind,df_for_bi in enumerate([c_f0_01,c_f0_02,c_f0_03,c_f0_04,c_f0_05,c_f0_06,c_f0_07,c_f0_08]):
#     print(type(df_for_bi))

In [ ]:
df_for_bi.to_csv('C:\\Users\\Илья\\Desktop\\kazzinc\\bi1.csv',encoding='utf-8-sig',sep = ';',decimal=',')

In [ ]:
df_for_bi.name = 'f0_vertical_cons'
save_backup('df_for_bi_cons.h5',[df_for_bi])

In [ ]:
c_f[(c_f['Материал']=='210701004')&(c_f['БЕ']=='3120')&(c_f['Завод+НН']=='3520 #210701004')]

In [ ]:
c_f0_01['1/2019 ФП Кол-во'].sum()


In [ ]:
cons_f0['1/2019 ФП '].sum()

In [ ]:
c_f['1/2019 ФП Кол-во'].sum()

In [ ]:
arr5['1/2019 ФП Кол-во'].sum()

In [ ]:
c_f0_01['1/2019 ФП Кол-во'].sum()

In [ ]:
arr5['2/2019 ФC Сумма'].sum()

In [ ]:
c_f0_02['2/2019 ФC Сумма'].sum()

In [ ]:
current_f0['9/2019 Остаток Сумма'].sum()

In [ ]:
arr5['9/2019 Остаток Сумма'].sum()

In [ ]:
c_f0_01 = get_in_backup(path_backup='backup_f0.h5',name='c_f0_01')

In [ ]:
ost_nach_perioda['Остаток на начало года Кол-во'].sum()

In [ ]:
arr5['Остаток на начало года Кол-во'].sum()

In [ ]:
arr5 = get_in_backup(path_backup='f0_cons_09.h5',name='f0_cons_09')

In [ ]:
arr5

In [ ]:
arr5.name = 'f0_cons_11'
save_backup('f0_cons_11.h5',[arr5])

In [ ]:
arr5

In [ ]:
arr5.shape

In [ ]:
for ind, i in enumerate(np.array_split(arr5,2)) :
    save_pyex(df=i,path='C:\\Users\\Илья\\Desktop\\kazzinc\\Uploads1\\extr_xlsx\\'+str(ind)+'t.xlsx')

In [ ]:
save_pyex(df=arr5,path='C:\\Users\\Илья\\Desktop\\kazzinc\\Uploads1\\extr_xlsx\\f0_cons_11_test.xlsx')

In [ ]:
# формирование bi отчетности new 15.10.2019
# df_for_bi = pd.concat([arr1,arr2])
# df_for_bi = get_in_backup(path_backup='backup_for_powerBI.h5',name='f0_vertical_09')
# df_for_bi.drop('Unnamed: 109',axis = 1,inplace = True)

c_f0_01 = get_in_backup(path_backup='backup_for_powerBI.h5',name='f0_vertical_01')
c_f0_02 = get_in_backup(path_backup='backup_for_powerBI.h5',name='f0_vertical_02')
c_f0_03 = get_in_backup(path_backup='backup_for_powerBI.h5',name='f0_vertical_03')
c_f0_04 = get_in_backup(path_backup='backup_for_powerBI.h5',name='f0_vertical_04')
c_f0_05 = get_in_backup(path_backup='backup_for_powerBI.h5',name='f0_vertical_05')
c_f0_06 = get_in_backup(path_backup='backup_for_powerBI.h5',name='f0_vertical_06')
c_f0_07 = get_in_backup(path_backup='backup_for_powerBI.h5',name='f0_vertical_07')
c_f0_08 = get_in_backup(path_backup='backup_for_powerBI.h5',name='f0_vertical_08')
c_f0_09 = get_in_backup(path_backup='backup_for_powerBI.h5',name='f0_vertical_09')

for ind,df_for_bi in enumerate([c_f0_01,c_f0_02,c_f0_03,c_f0_04,c_f0_05,c_f0_06,c_f0_07,c_f0_08,c_f0_09],start=1) :
    df_for_bi['Месяц'] = datetime.datetime(2019,ind,1)

df_for_bi = pd.DataFrame()
df_for_bi = pd.concat([c_f0_01,c_f0_02,c_f0_03,c_f0_04,c_f0_05,c_f0_06,c_f0_07,c_f0_08,c_f0_09])

df_for_bi['Категория ТМЦ'] = df_for_bi.apply(lambda x: func_cat_tmc(x),axis = 1)
# df_for_bi = df_for_bi[df_for_bi['Категория ТМЦ']=='Текущая потребность']
# df_for_bi.to_csv('C:\\Users\\Илья\\Desktop\\kazzinc\\bi.csv',encoding='utf-8-sig',sep = ';',decimal=',')

df_for_bi['Завод']  = df_for_bi['Завод+НН'].str.split(' #').apply(lambda x: x[0])
BE = pd.DataFrame()
for i in range(1,10):
    BE = BE.append(get_in_backup(path_backup='backup_f0.h5',name='c_f0_0'+str(i))[['Завод','БЕ']])
BE = BE.drop_duplicates()   
spr_be = rev_to_dict(BE['Завод'],BE['БЕ'])
df_for_bi['БЕ'] = df_for_bi['Завод'].apply(lambda x:  insert_vpr(x,spr_be,'БЕ'))
df_for_bi['Материал'] = df_for_bi['Завод+НН'].str.split(' #').apply(lambda x: x[1])

df_for_bi['Заказчик + HH'] = df_for_bi['Комплекс'].astype('str')+' '+df_for_bi['Материал'].astype('str')


spr_centr_zav_zav = rev_to_dict(central_zavod['Область ППМ'],central_zavod['Центральный Завод'])

df_for_bi.reset_index(inplace=True,drop=True)

for u in  df_for_bi.index:
    if insert_vpr(df_for_bi.at[u,'Область ППМ'],spr_centr_zav_zav,'Центральный Завод') ==0:
        df_for_bi.at[u,'Центральный завод'] =  insert_vpr(df_for_bi.at[u,'Завод'],spr_centr_zav_zav,'Центральный Завод')
    else:
        df_for_bi.at[u,'Центральный завод'] =  insert_vpr(df_for_bi.at[u,'Область ППМ'],spr_centr_zav_zav,'Центральный Завод')    
df_for_bi['Центральный завод'] = df_for_bi['Центральный завод'].apply(lambda x: str(x).replace('.0','')).astype('str')  



spr_group_mat_l1 = rev_to_dict(group_mat[group_mat['Группа материалов'].str.len()==2]['Группа материалов'],group_mat[group_mat['Группа материалов'].str.len()==2]['Обозначение 2 группы материалов'])
df_for_bi['группа'] = df_for_bi['Материал'].str[:2].apply(lambda x:  insert_vpr(x,spr_group_mat_l1,'Обозначение 2 группы материалов'))

spr_group_mat_l2 = rev_to_dict(group_mat[group_mat['Группа материалов'].str.len()==4]['Группа материалов'],group_mat[group_mat['Группа материалов'].str.len()==4]['Обозначение 2 группы материалов'])
df_for_bi['подгруппа'] = df_for_bi['Материал'].str[:4].apply(lambda x:  insert_vpr(x,spr_group_mat_l2,'Обозначение 2 группы материалов'))


spr_group_mat_l3 = rev_to_dict(group_mat[group_mat['Группа материалов'].str.len()==6]['Группа материалов'],group_mat[group_mat['Группа материалов'].str.len()==6]['Обозначение 2 группы материалов'])
df_for_bi['подподгруппа'] = df_for_bi['Материал'].str[:6].apply(lambda x:  insert_vpr(x,spr_group_mat_l3,'Обозначение 2 группы материалов'))


spr_obl_ppm_centr_zav = rev_to_dict(sprav_mat_cal_last_full['Материал'],sprav_mat_cal_last_full['Краткий текст материала'])
df_for_bi['КрТекстМатериала'] = df_for_bi['Материал'].apply(lambda x:  insert_vpr(x,spr_obl_ppm_centr_zav,'Краткий текст материала'))




spr_obl_ppm_baz_ei = rev_to_dict(sprav_mat_cal_last_full['Материал'],sprav_mat_cal_last_full['Базовая ЕИ'])
df_for_bi['Базовая ЕИ'] = df_for_bi['Материал'].apply(lambda x:  insert_vpr(x,spr_obl_ppm_baz_ei,'Базовая ЕИ'))

spr_obl_ppm_name_planov= rev_to_dict(sprav_mat_cal_last_full['Завод']+' #'+sprav_mat_cal_last_full['Материал'],sprav_mat_cal_last_full['Имя плановика (завод)'])
df_for_bi['ЦЗ+НН'] = df_for_bi['Центральный завод'].astype('str')+' #'+df_for_bi['Материал'].astype('str')
df_for_bi['Плановик'] = df_for_bi['ЦЗ+НН'].apply(lambda x:  insert_vpr(x,spr_obl_ppm_name_planov,'Имя плановика (завод)'))
df_for_bi.drop('ЦЗ+НН',inplace=True,axis = 1)


spr_obl_ppm_ocenka = rev_to_dict(sprav_mat_cal_last_full['Материал'],sprav_mat_cal_last_full['Класс оценки'])
df_for_bi['Класс оценки'] = df_for_bi['Материал'].apply(lambda x:  insert_vpr(x,spr_obl_ppm_ocenka,'Класс оценки'))
df_for_bi['Класс оценки'] = df_for_bi['Класс оценки'].apply(lambda x: str(x).replace('.0','')).astype('str')

spr_obl_ppm_stat_na_zav = rev_to_dict(sprav_mat_cal_last_full['Завод']+' #'+sprav_mat_cal_last_full['Материал'],sprav_mat_cal_last_full['СтатусМтрл на заводе'])
df_for_bi['СтатусМтрл на заводе'] = df_for_bi['Завод+НН'].apply(lambda x:  insert_vpr(x,spr_obl_ppm_stat_na_zav,'СтатусМтрл на заводе'))


# df_for_bi['Заказ'] = df_for_bi['Заказ'].apply(lambda x: str(x).replace('.0','')).astype('str')
# df_for_bi.loc[df_for_bi[df_for_bi['Заказ'].str.startswith('2')].index,'Заказ'] = '0'


# ind = ['month','Модуль','Заказ','Категория ТМЦ','Материал','БЕ','Заказчик + HH','Завод+НН','Завод + Склад','Завод + Область ППМ','Комплекс','Завод','Склад потребности','Область ППМ','Центральный завод','Код округа','Имя Завода','КрТекстМатериала','Базовая ЕИ','Признак аналога','СтатусМтрл на заводе','Класс оценки','Плановик','группа','подгруппа','подподгруппа','Цена','Недропользование','ПлановСрокПоставки']
# df_for_bi['Цена'] = 0
# df_for_bi['Цена'] = df_for_bi['Цена'].astype('str')
# for st in ind:
#     df_for_bi[st] = df_for_bi[st].astype('str')
    

    
# aggregate= dict((x,'sum') for x in df_for_bi.columns if df_for_bi[x].dtype=='float64' and x!='Цена')
# aggregate['Цена'] = ['max']



# ind.remove('Цена')
# df_for_bi = df_for_bi.groupby(ind).agg(aggregate).reset_index()
# df_for_bi['Цена'] = df_for_bi['Цена'].astype('float64')
# df_for_bi.columns =   [x[0] for x in df_for_bi.columns]

A902['Код округа'] = A902['Код округа'].astype('str')
A902['Материал'] = A902['Материал'].astype('str')
A902['Материал'] = A902['Материал'].apply(lambda x: '0'+str(x) if  len(str(x))<9 else x)
A900['Материал'] = A900['Материал'].astype('str')
A900['Материал'] = A900['Материал'].apply(lambda x: '0'+str(x) if  len(str(x))<9 else x)
df_for_bi['Материал'] = df_for_bi['Материал'].astype('str')
A902.fillna(0,inplace=True)
A900.fillna(0,inplace=True)
KONP.fillna(0,inplace=True)
potr_gr = dict(tuple(df_for_bi[df_for_bi['Цена']==0].groupby(['Код округа','Материал'])))
potr_gr_mat = dict(tuple(df_for_bi[df_for_bi['Цена']==0].groupby(['Материал'])))
konp_gr = dict(tuple(KONP.groupby(['Номер записи условий'])))
# c_f0['Цена'] = 0.0000
A900 = A900[A900['Конец действия'].apply(lambda x: x.year)==current_date.year]
A902 = A902[A902['Конец действия'].apply(lambda x: x.year)==current_date.year]
main_price(potr_gr,potr_gr_mat,A900,A902,konp_gr,df_for_bi)

In [ ]:
save_pyex(df=df_for_bi,path='C:\\Users\\Илья\\Desktop\\kazzinc\\bi1.xlsx')

In [ ]:
df_for_bi.to_csv('C:\\Users\\Илья\\Desktop\\kazzinc\\bi1.csv',encoding='utf-8-sig',sep = ';',decimal=',')

In [ ]:
df_for_bi['Распределение_остатака'].sum()

In [ ]:
df_for_bi.drop(['Unnamed: 104','Unnamed: 109'],axis = 1,inplace=True)

In [ ]:
df_for_bi.name = 'f0_vertical_cons'
save_backup('df_for_bi_cons.h5',[df_for_bi])

In [ ]:
c_f0_08 =  get_in_backup(path_backup='backup_for_powerBI.h5',name='f0_vertical_08')
c_f0_09 =  get_in_backup(path_backup='backup_for_powerBI.h5',name='f0_vertical_09')

c_f0_10 =  get_in_backup(path_backup='backup_for_powerBI_test_no.h5',name='f0_vertical_10')
c_f0_11 =  get_in_backup(path_backup='backup_for_powerBI_test_no.h5',name='f0_vertical_11')

In [ ]:
# df_for_bi = c_f0_08.copy()
# df_for_bi = c_f0_09.copy()
# df_for_bi = c_f0_10.copy()
df_for_bi = c_f0_11.copy()

In [ ]:
# POWRER BI По месячно 11112019

fr_bad = df_for_bi[(df_for_bi['Блокированный запас']>0)&(df_for_bi['Распределение_остатака']>0)]
df_for_bi.loc[fr_bad.index,['Распределение_остатака','Сумма_распреденного_остатка','Распределение_прихода','Сумма_распреденного_прихода','Распределение_расхода','Сумма_распреденного_расхода','avizot','стрЗапас']]    = 0
fr_bad.loc[:,['Блокированный запас','Ст-ть/блок. запаса']]=0
df_for_bi = pd.concat([df_for_bi,fr_bad],axis=0,ignore_index=True)
df_for_bi['Категория ТМЦ'] = df_for_bi.apply(lambda x: func_cat_tmc(x),axis = 1)

df_for_bi['month'] = df_for_bi['month'].apply( lambda x: datetime.datetime(1900,1,1) if x==0 else x)

fil = df_for_bi[df_for_bi['Категория ТМЦ']=='ТНСЗ']

df_for_bi.loc[fil.index,'Распределение_остатака'] = df_for_bi.loc[fil.index,'Распределение_остатака']+ df_for_bi.loc[fil.index,'Блокированный запас'] 
df_for_bi.loc[fil.index,'Сумма_распреденного_остатка'] =df_for_bi.loc[fil.index,'Сумма_распреденного_остатка']+ df_for_bi.loc[fil.index,'Ст-ть/блок. запаса']  




# менять согласно месяца
df_for_bi['Месяц'] = datetime.datetime(2019,11,1)




df_for_bi['Завод']  = df_for_bi['Завод+НН'].str.split(' #').apply(lambda x: x[0])
BE = pd.DataFrame()
for i in range(1,11):
    if i <10:
        BE = BE.append(get_in_backup(path_backup='backup_f0.h5',name='c_f0_0'+str(i))[['Завод','БЕ']])
    else:
        BE = BE.append(get_in_backup(path_backup='backup_f0.h5',name='c_f0_'+str(i))[['Завод','БЕ']])
BE = BE.drop_duplicates()   
spr_be = rev_to_dict(BE['Завод'],BE['БЕ'])
df_for_bi['БЕ'] = df_for_bi['Завод'].apply(lambda x:  insert_vpr(x,spr_be,'БЕ'))
df_for_bi['Материал'] = df_for_bi['Завод+НН'].str.split(' #').apply(lambda x: x[1])

df_for_bi['Заказчик + HH'] = df_for_bi['Комплекс'].astype('str')+' '+df_for_bi['Материал'].astype('str')


spr_centr_zav_zav = rev_to_dict(central_zavod['Область ППМ'],central_zavod['Центральный Завод'])

df_for_bi.reset_index(inplace=True,drop=True)

for u in  df_for_bi.index:
    if insert_vpr(df_for_bi.at[u,'Область ППМ'],spr_centr_zav_zav,'Центральный Завод') ==0:
        df_for_bi.at[u,'Центральный завод'] =  insert_vpr(df_for_bi.at[u,'Завод'],spr_centr_zav_zav,'Центральный Завод')
    else:
        df_for_bi.at[u,'Центральный завод'] =  insert_vpr(df_for_bi.at[u,'Область ППМ'],spr_centr_zav_zav,'Центральный Завод')    
df_for_bi['Центральный завод'] = df_for_bi['Центральный завод'].apply(lambda x: str(x).replace('.0','')).astype('str')  



spr_group_mat_l1 = rev_to_dict(group_mat[group_mat['Группа материалов'].str.len()==2]['Группа материалов'],group_mat[group_mat['Группа материалов'].str.len()==2]['Обозначение 2 группы материалов'])
df_for_bi['группа'] = df_for_bi['Материал'].str[:2].apply(lambda x:  insert_vpr(x,spr_group_mat_l1,'Обозначение 2 группы материалов'))

spr_group_mat_l2 = rev_to_dict(group_mat[group_mat['Группа материалов'].str.len()==4]['Группа материалов'],group_mat[group_mat['Группа материалов'].str.len()==4]['Обозначение 2 группы материалов'])
df_for_bi['подгруппа'] = df_for_bi['Материал'].str[:4].apply(lambda x:  insert_vpr(x,spr_group_mat_l2,'Обозначение 2 группы материалов'))


spr_group_mat_l3 = rev_to_dict(group_mat[group_mat['Группа материалов'].str.len()==6]['Группа материалов'],group_mat[group_mat['Группа материалов'].str.len()==6]['Обозначение 2 группы материалов'])
df_for_bi['подподгруппа'] = df_for_bi['Материал'].str[:6].apply(lambda x:  insert_vpr(x,spr_group_mat_l3,'Обозначение 2 группы материалов'))


spr_obl_ppm_centr_zav = rev_to_dict(sprav_mat_cal_last_full['Материал'],sprav_mat_cal_last_full['Краткий текст материала'])
df_for_bi['КрТекстМатериала'] = df_for_bi['Материал'].apply(lambda x:  insert_vpr(x,spr_obl_ppm_centr_zav,'Краткий текст материала'))




spr_obl_ppm_baz_ei = rev_to_dict(sprav_mat_cal_last_full['Материал'],sprav_mat_cal_last_full['Базовая ЕИ'])
df_for_bi['Базовая ЕИ'] = df_for_bi['Материал'].apply(lambda x:  insert_vpr(x,spr_obl_ppm_baz_ei,'Базовая ЕИ'))

spr_obl_ppm_name_planov= rev_to_dict(sprav_mat_cal_last_full['Завод']+' #'+sprav_mat_cal_last_full['Материал'],sprav_mat_cal_last_full['Имя плановика (завод)'])
df_for_bi['ЦЗ+НН'] = df_for_bi['Центральный завод'].astype('str')+' #'+df_for_bi['Материал'].astype('str')
df_for_bi['Плановик'] = df_for_bi['ЦЗ+НН'].apply(lambda x:  insert_vpr(x,spr_obl_ppm_name_planov,'Имя плановика (завод)'))
df_for_bi.drop('ЦЗ+НН',inplace=True,axis = 1)


spr_obl_ppm_ocenka = rev_to_dict(sprav_mat_cal_last_full['Материал'],sprav_mat_cal_last_full['Класс оценки'])
df_for_bi['Класс оценки'] = df_for_bi['Материал'].apply(lambda x:  insert_vpr(x,spr_obl_ppm_ocenka,'Класс оценки'))
df_for_bi['Класс оценки'] = df_for_bi['Класс оценки'].apply(lambda x: str(x).replace('.0','')).astype('str')

spr_obl_ppm_stat_na_zav = rev_to_dict(sprav_mat_cal_last_full['Завод']+' #'+sprav_mat_cal_last_full['Материал'],sprav_mat_cal_last_full['СтатусМтрл на заводе'])
df_for_bi['СтатусМтрл на заводе'] = df_for_bi['Завод+НН'].apply(lambda x:  insert_vpr(x,spr_obl_ppm_stat_na_zav,'СтатусМтрл на заводе'))




A902['Код округа'] = A902['Код округа'].astype('str')
A902['Материал'] = A902['Материал'].astype('str')
A902['Материал'] = A902['Материал'].apply(lambda x: '0'+str(x) if  len(str(x))<9 else x)
A900['Материал'] = A900['Материал'].astype('str')
A900['Материал'] = A900['Материал'].apply(lambda x: '0'+str(x) if  len(str(x))<9 else x)
df_for_bi['Материал'] = df_for_bi['Материал'].astype('str')
A902.fillna(0,inplace=True)
A900.fillna(0,inplace=True)
KONP.fillna(0,inplace=True)
potr_gr = dict(tuple(df_for_bi[df_for_bi['Цена']==0].groupby(['Код округа','Материал'])))
potr_gr_mat = dict(tuple(df_for_bi[df_for_bi['Цена']==0].groupby(['Материал'])))
konp_gr = dict(tuple(KONP.groupby(['Номер записи условий'])))
# c_f0['Цена'] = 0.0000
A900 = A900[A900['Конец действия'].apply(lambda x: x.year)==current_date.year]
A902 = A902[A902['Конец действия'].apply(lambda x: x.year)==current_date.year]
main_price(potr_gr,potr_gr_mat,A900,A902,konp_gr,df_for_bi)


df_for_bi.drop([x for x in df_for_bi.columns if x.find('Unnamed')!=-1],axis = 1,inplace=True)

In [ ]:
# df_for_bi.drop([x for x in df_for_bi.columns if x.find('Unnamed')!=-1],axis = 1,inplace=True)

In [ ]:
df_for_bi['Дата потребности'].apply(lambda x: x.year)

In [ ]:
c_f0_08 = df_for_bi.copy()

In [ ]:
c_f0_09 = df_for_bi.copy()

In [ ]:
c_f0_10 = df_for_bi.copy()

In [ ]:
c_f0_11 = df_for_bi.copy()

In [ ]:
cons_power_bi = pd.concat([c_f0_08,c_f0_09,c_f0_10,c_f0_11])
cons_power_bi.name = 'cons_power_bi'

In [ ]:
save_backup('cons_power_bi.h5',[cons_power_bi])


In [ ]:
ar = get_in_backup(path_backup='cons_power_bi.h5',name='cons_power_bi1')

In [ ]:
# подмена -1 месяца для power bi
ar.reset_index(drop=True,inplace = True)
ar1 = ar[ar['Месяц']==datetime.datetime(2019,11,1)]
ar2 = ar1[ar1['month'].dt.month==10].index
ar.loc[ar2,'month'] = datetime.datetime(2019,12,1)

In [ ]:
# cons_power_bi  = ar.copy()
# cons_power_bi.name = 'cons_power_bi'
# save_backup('cons_power_bi.h5',[cons_power_bi])

cons_power_bi  = ar.copy()
cons_power_bi.name = 'cons_power_bi_with_rcm'
save_backup('cons_power_bi.h5',[cons_power_bi])


# cons_power_bi  = ar.copy()
# cons_power_bi.name = 'cons_power_bi1'
# save_backup('cons_power_bi.h5',[cons_power_bi])


In [ ]:
ar1 = ar[ar['Месяц']!=datetime.datetime(2019,11,1)]

In [ ]:
ar = pd.concat([ar1,c_f0_11])

In [ ]:
ar['Месяц'].unique()

In [ ]:
cons_power_bi = cons_power_bi[cons_power_bi['month'].dt.month==11]

In [ ]:
ar[400000:600000].to_csv('C:\\Users\\Илья\\Desktop\\kazzinc\\bi1.csv',encoding='utf-8-sig',sep = ';',decimal=',')

In [ ]:
cons_power_bi.to_csv('C:\\Users\\Илья\\Desktop\\kazzinc\\bi1.csv',encoding='utf-8-sig',sep = ';',decimal=',')

In [ ]:
cons_power_bi  = ar.copy()

In [ ]:
ar[:100000]

In [ ]:
# Конвертация целевого остатка остатка для bi целевой
sprav_mdlv = pd.read_excel('C:\\Users\\Илья\\Desktop\\kazzinc\\files for power bi\\Целевой.xlsx',sheet_name='sheet1')
sprav_mdlv.columns = [x.replace('\n','') for x in [x.replace('\r','') for x in sprav_mdlv.columns]]  
# save_pyex(df=sprav_mdlv,path='C:\\Users\\Илья\\Desktop\\kazzinc\\Целевой.xlsx')
aa = sprav_mdlv.set_index(list(sprav_mdlv.loc[:,'Укрупненный Заказчик':'Категория ТМЦ (Операционный бюджет, Проекты, ТНСЗ и др.)'].columns)).stack().reset_index()
ind = aa['level_9'].apply(lambda x:True if x.find('Сумма')!=-1 else False)
bb = aa.loc[~ind]
cc = aa.loc[ind]
bb.rename(columns={0: "Кол-во"},inplace=True)
cc.rename(columns={0: "Сумма"},inplace=True)
tot = pd.concat([cc,bb])
months = {'Январь':1,'Февраль':2,'Март':3,'Апрель':4,'Май':5,'Июнь':6,'Июль':7,'Август':8,'Сентябрь':9,'Октябрь':10,'Ноябрь':11,'Декабрь':12}
tot['Date'] = tot.apply(lambda x: datetime.datetime(2019,months[x['level_9'][4:x['level_9'].find(',')].strip()],1)   ,axis = 1)
tot.fillna(0,inplace=True)

In [ ]:
tot.to_csv('C:\\Users\\Илья\\Desktop\\kazzinc\\files for power bi\\целевой.csv',encoding='utf-8-sig',sep = ';',decimal=',')

In [ ]:
cons_power_bi

In [ ]:
c_f0_01.name = 'c_f0_01'
c_f0_02.name = 'c_f0_02'
c_f0_03.name = 'c_f0_03'
c_f0_04.name = 'c_f0_04'
c_f0_05.name = 'c_f0_05'
c_f0_06.name = 'c_f0_06'
save_backup('backup_f0.h5',[c_f0_01,c_f0_02,c_f0_03,c_f0_04,c_f0_05,c_f0_06])

In [ ]:
save_pyex(df=arr5,path='C:\\Users\\User\\Desktop\\kazzinc\\Uploads1\\extr_xlsx\\f0.xlsx')

In [ ]:
ar

In [ ]:
arr_8 = ar[ar['Месяц'].dt.month==8]
arr_9 = ar[ar['Месяц'].dt.month==9]
arr_10 = ar[ar['Месяц'].dt.month==10]
arr_11 = ar[ar['Месяц'].dt.month==11]

In [ ]:
rcm = pd.read_excel('E:\\RCM\\RCM 11.xlsx',sheet_name='Sheet1')

In [ ]:
# Подтягивает RCM для PBI
def RCM(f0,ZNB,rcm):
    spr_kompl_mat_komp_znb_full = rev_to_dict(ZNB['Комплекс+Маттериал'],ZNB ['Карточка RCM'])
    spr_mat_znb = rev_to_dict(ZNB['Материал'],ZNB ['Карточка RCM'])

    for ind in f0.index:
        if (f0.at[ind,'avizot']   + f0.at[ind,'законтрактовано'])>0:
            if insert_vpr(f0.at[ind,'Заказчик + HH'],spr_kompl_mat_komp_znb_full,'Карточка RCM')==0 and f0.at[ind,'Комплекс'] in  filter_else['one'].values:
                f0.at[ind,'RCM']  = insert_vpr(f0.at[ind,'Материал'],spr_mat_znb,'Карточка RCM')
            else:
                f0.at[ind,'RCM'] = insert_vpr(f0.at[ind,'Заказчик + HH'],spr_kompl_mat_komp_znb_full,'Карточка RCM')
    f0['RCM'] = f0['RCM'].apply(lambda x: str(x).replace('.0','')).astype('str')  
    rcm['Ид. случая'] = rcm['Ид. случая'].astype('str')
    spr_dat_zak = rev_to_dict(rcm['Ид. случая'], rcm['Дата заключения'])
    f0['Дата заключения RCM'] = f0['RCM'].apply(lambda x: datetime.datetime(1900,1,1)  if insert_vpr(x,spr_dat_zak,'Дата заключения')==0 else insert_vpr(x,spr_dat_zak,'Дата заключения') )
    f0['Дата заключения RCM'] = f0['Дата заключения RCM'].apply(lambda x: datetime.datetime(1900,1,1) if x==0 else x)
    f0.fillna(0,inplace=True)
    return f0

In [ ]:
a = []
for i in [ar]:  
    a.append(RCM(i,ZNB,rcm))
    

In [ ]:
ar = pd.concat(a)

In [ ]:
rcm['Ид. случая']

In [ ]:
potr_with_ostatok_total[(potr_with_ostatok_total['Сумма_распреденного_прихода']>0)&(potr_with_ostatok_total['Распределение_прихода']==0)]

In [ ]:
c_f0['6/2019 ФП Кол-во'].sum()

In [ ]:
potr_with_ostatok_total['Распределение_прихода'].sum()

In [ ]:
ost_Q_tab

In [ ]:
rcm.fillna(0,inplace=True)

In [ ]:
potr_with_ostatok_total['Категория ТМЦ'].unique()

In [ ]:
f0 =  pd.pivot_table(tab_f0,columns=['month'],index=ind,aggfunc='sum',fill_value=0).reset_index()

In [ ]:
bl_potr = f0['Количество потребности']
bl_potr.columns = ['ПП кол-во '+str(x.month)+'/'+ str(x.year) for x in bl_potr.columns] 
pp_sum_col = pd.DataFrame(bl_potr.sum(axis = 1),columns=['Текущее кол-во ПП'])


bl_potr_sum  = f0['Сумма потребности']
bl_potr_sum.columns = ['ПП сумма '+str(x.month)+'/'+ str(x.year) for x in bl_potr_sum.columns]
pp_sum_sum = pd.DataFrame(bl_potr_sum.sum(axis = 1),columns=['Текущая сумма ПП'])

col_vo_prix = pd.DataFrame(f0['Распределение_прихода'].sum(axis = 1),columns=[str(current_date.month)+'/'+str(current_date.year)+' ФП Кол-во'])
sum_prix = pd.DataFrame(f0['Сумма_распреденного_прихода'].sum(axis = 1),columns=[str(current_date.month)+'/'+str(current_date.year)+' ФП Сумма'])


col_vo_rash = pd.DataFrame(f0['Распределение_расхода'].sum(axis = 1),columns=[str(current_date.month)+'/'+str(current_date.year)+' ФC Кол-во'])
summ_rash = pd.DataFrame(f0['Сумма_распреденного_расхода'].sum(axis = 1),columns=[str(current_date.month)+'/'+str(current_date.year)+' ФC Сумма'])

# ost_nach_per = pd.DataFrame(f0['Остаток на начало года Кол-во'].sum(axis = 1),columns=[str(current_date.month)+'/'+str(current_date.year)+' ФC Сумма'])



col_vo_ost = pd.DataFrame(f0['Распределение_остатака'].sum(axis = 1),columns=[str(current_date.month)+'/'+str(current_date.year)+' Остаток Кол-во'])
sum_ost = pd.DataFrame(f0['Сумма_распреденного_остатка'].sum(axis = 1),columns=[str(current_date.month)+'/'+str(current_date.year)+' Остаток Сумма'])

avizot =  pd.DataFrame(f0['avizot'].sum(axis = 1),columns=['Товар в пути (ZNB5, ZNB2/3-АвизОт)'])

str_zap_col_vo = pd.DataFrame(f0['стрЗапас'].sum(axis = 1),columns=['SS (норма запаса на складе)'])
str_zap_summ = pd.DataFrame(f0['Сумма стрЗап'].sum(axis = 1),columns=['SS сумма'])

f0_cons = pd.concat([cols__lev_f0,bl_potr,pp_sum_col,bl_potr_sum,pp_sum_sum,col_vo_prix,sum_prix,col_vo_rash,summ_rash,col_vo_ost,sum_ost,avizot,str_zap_col_vo,str_zap_summ],axis=1)

cols_bz_col_vo = ['БЗ кол-во '+str(x)+'/'+str(current_date.year) for x in range(1,13)]

cols_bz_summ = ['БЗ сумма '+str(x)+'/'+str(current_date.year) for x in range(1,13)]

for i in cols_bz_col_vo:f0_cons[i] = 0
f0_cons['ИТОГО БЗ Кол-во'] = 0
for i in cols_bz_summ:f0_cons[i] = 0
f0_cons['ИТОГО БЗ Сумма'] = 0


cols_ost_prognoz = ['Остаток прогноз Кол-во '+str(x)+'/'+str(current_date.year) for x in range(1,13)]
cols_ost_prognoz_summa = ['Остаток прогноз Сумма '+str(x)+'/'+str(current_date.year) for x in range(1,13)]
for i in cols_ost_prognoz:f0_cons[i] = 0
for i in cols_ost_prognoz_summa:f0_cons[i] = 0

    
    


In [ ]:
# если сейчас 1 месяц тогда текущий опираем на остаток на начало года
if current_date.month == 1:
#     
    pass
else:
    #количество bz     
    f0_cons.loc[:,cols_bz_col_vo[:current_date.month]] = 0
    f0_cons.loc[:,cols_ost_prognoz[:current_date.month]] = 0
    f0_cons.loc[:,cols_ost_prognoz_summa[:current_date.month]] = 0
    f0_cons.loc[:,cols_bz_col_vo[current_date.month]] = f0_cons.apply(lambda x: x['ПП кол-во '+str(current_date.month+1)+'/'+str(current_date.year)]+x['SS (норма запаса на складе)']-x[str(current_date.month)+'/'+str(current_date.year)+' Остаток Кол-во']-x['Товар в пути (ZNB5, ZNB2/3-АвизОт)']  if (x[str(current_date.month)+'/'+str(current_date.year)+' Остаток Кол-во']+x['Товар в пути (ZNB5, ZNB2/3-АвизОт)'])-x['ПП кол-во '+str(current_date.month+1)+'/'+str(current_date.year)]-x['SS (норма запаса на складе)']<0 else 0,axis = 1)
    f0_cons.loc[:,cols_ost_prognoz[current_date.month]] = f0_cons[str(current_date.month)+'/'+str(current_date.year)+' Остаток Кол-во'].values+f0_cons['Товар в пути (ZNB5, ZNB2/3-АвизОт)'].values+f0_cons.loc[:,cols_bz_col_vo[current_date.month]].values-f0_cons['ПП кол-во '+str(current_date.month+1)+'/'+str(current_date.year)].values

    #     print('ПП кол-во '+str(current_date.month)+'/'+str(current_date.year))
#     print(str(current_date.month)+'/'+str(current_date.year)+' Остаток Кол-во')
#     print('ПП кол-во '+str(current_date.month+1)+'/'+str(current_date.year))
    ff = current_date.month
    for i in cols_bz_col_vo[current_date.month+1:]:
        
        f0_cons[i] = f0_cons.apply(lambda x: x['ПП кол-во'+str(i[9:])]+x['Остаток прогноз Кол-во'+str(i[9:])]  if x['Остаток прогноз Кол-во'+str(i[9:])]-x['ПП кол-во'+str(i[9:])]-x['SS (норма запаса на складе)']<0 else 0,axis = 1)
#         формкла для остатка
        f0_cons['Остаток прогноз Кол-во'+str(i[9:])] = f0_cons['Остаток прогноз Кол-во'+str(cols_bz_col_vo[ff][9:])].values+f0_cons[i].values-f0_cons['ПП кол-во'+str(i[9:])].values
#         print('Остаток прогноз Кол-во'+str(cols_bz_col_vo[ff][9:]))    
#         print(i)
#         print('ПП кол-во'+str(i[9:]))
        
        ff+=1
# сумма бз    
f0_cons.loc[:,cols_bz_summ]=f0_cons.loc[:,cols_bz_col_vo].mul(f0_cons['Цена'],axis = 0).values  
f0_cons['ИТОГО БЗ Сумма'] = f0_cons.loc[:,cols_bz_summ].sum(axis=1)
# сумма остатка
for i in cols_ost_prognoz_summa[current_date.month:]:
     f0_cons[i] =    f0_cons.apply(lambda x: x['Остаток прогноз Кол-во'+str(i[21:])]*(x[str(current_date.month)+'/'+str(current_date.year)+' Остаток Сумма']/  x[str(current_date.month)+'/'+str(current_date.year)+' Остаток Кол-во']) if x[str(current_date.month)+'/'+str(current_date.year)+' Остаток Кол-во']>0 else x['Остаток прогноз Кол-во'+str(i[21:])]*x['Цена'] ,axis = 1)


In [ ]:
fil = potr_with_ostatok_total[potr_with_ostatok_total['Категория ТМЦ']=='ТНСЗ']
potr_with_ostatok_total.loc[fil.index,'Распределение_остатака'] = potr_with_ostatok_total.loc[fil.index,'Блокированный запас'] 
potr_with_ostatok_total.loc[fil.index,'Сумма_распреденного_остатка'] = potr_with_ostatok_total.loc[fil.index,'Ст-ть/блок. запаса']  

In [ ]:
potr_with_ostatok_total['Распределение_остатака'].sum()

In [ ]:
# dict_for_f0['ТНСЗ']['Распределение_остатака'] = dict_for_f0['ТНСЗ']['Блокированный запас']
# dict_for_f0['ТНСЗ']['Сумма_распреденного_остатка'] = dict_for_f0['ТНСЗ']['Ст-ть/блок. запаса']

In [ ]:
f0[f0['Категория ТМЦ']=='ТНСЗ']

In [ ]:
tottal_ost['Блокированный запас'].sum()
tottal_ost[tottal_ost['Блокированный запас']>0]['Распределение_остатака'].sum()

In [ ]:
tottal_ost[tottal_ost['']]

In [ ]:
fr_bad = potr_with_ostatok_total[(potr_with_ostatok_total['Блокированный запас']>0)&(potr_with_ostatok_total['Распределение_остатака']>0)]
potr_with_ostatok_total.loc[fr_bad.index,['Распределение_остатака','Сумма_распреденного_остатка']]    = 0
fr_bad.loc[:,['Блокированный запас','Ст-ть/блок. запаса']]=0
potr_with_ostatok_total = pd.concat([potr_with_ostatok_total,fr_bad],axis=0,ignore_index=True)
potr_with_ostatok_total['Категория ТМЦ'] = potr_with_ostatok_total.apply(lambda x: func_cat_tmc(x),axis = 1)
dict_for_f0 = dict((x,potr_with_ostatok_total[potr_with_ostatok_total['Категория ТМЦ']==x] )for x in  potr_with_ostatok_total['Категория ТМЦ'].unique())

In [ ]:
potr_with_ostatok_total['Распределение_остатака'].sum()  + potr_with_ostatok_total['Блокированный запас'].sum() 

In [ ]:
fr_bad1['Блокированный запас'].sum()

In [ ]:
fr_bad['Блокированный запас'].sum()

In [ ]:
potr_with_ostatok_total['Блокированный запас'].sum() 

In [ ]:
potr_with_ostatok_total['Распределение_остатака'].sum()

In [ ]:
potr_with_ostatok_total.shape

In [ ]:
fr_bad.shape

In [ ]:
potr_with_ostatok_total.shape

In [ ]:
486806-486527

In [ ]:
pd.concat([potr_with_ostatok_total,fr_bad],axis=0,ignore_index=True)['Распределение_остатака'].sum()

In [ ]:
potr_with_ostatok_total['Категория ТМЦ']

In [ ]:
fr_bad.shape

In [ ]:
len(fr_bad.index)

In [ ]:
fr_bad['Распределение_остатака'].sum()

In [ ]:
potr_with_ostatok_total.loc[fr_bad.index,['Сумма_распреденного_остатка','Сумма_распреденного_остатка']]

In [ ]:
dict_for_f0['ТНСЗ']['Распределение_остатака'] = dict_for_f0['ТНСЗ']['Блокированный запас']
dict_for_f0['ТНСЗ']['Сумма_распреденного_остатка'] = dict_for_f0['ТНСЗ']['Ст-ть/блок. запаса']

In [ ]:
dict_for_f0['ТНСЗ']['Распределение_остатака'] .sum()

In [ ]:
type(potr_with_ostatok_total['month'].unique()[-1])

In [ ]:
result_ostatok.to_csv('C:\\Users\\User\\Desktop\\kazzinc\\Uploads\\raspr_no_block.csv',encoding='utf-8-sig',sep = ';',decimal=',')

In [ ]:
result_ostatok_.to_csv('C:\\Users\\User\\Desktop\\kazzinc\\Uploads\\raspr_with_block.csv',encoding='utf-8-sig',sep = ';',decimal=',')

In [ ]:
ost_block.to_csv('C:\\Users\\User\\Desktop\\kazzinc\\Uploads\\block.csv',encoding='utf-8-sig',sep = ';')

In [ ]:
for ind, i in enumerate(np.array_split(a,2)) :
    save_pyex(df=i,path='C:\\Users\\User\\Desktop\\kazzinc\\Uploads1\\extr_xlsx\\'+str(ind)+'.xlsx')

In [ ]:
for ind, i in enumerate(np.array_split(df,2)) :
    save_pyex(df=i,path='C:\\Users\\Илья\\Desktop\\kazzinc\\Uploads1\\extr_xlsx\\'+str(ind)+'.xlsx')

In [ ]:
save_pyex(df=pd.merge(c_f0,ffff,on=['Заказ','Категория ТМЦ','Область ППМ','Материал'],how='outer'),path='C:\\Users\\User\\Desktop\\kazzinc\\Uploads1\\extr_xlsx\\f0.xlsx')

In [ ]:
save_pyex(df=arr5,path='C:\\Users\\User\\Desktop\\kazzinc\\Uploads1\\extr_xlsx\\f0.xlsx')


In [ ]:
directory = 'C:\\Users\\User\\Desktop\\kazzinc\\Uploads1\\'

In [ ]:
spr_ost.info()

In [ ]:
spr_ost

In [ ]:
sprav_po_kompl = pd.read_excel(directory+'Справочник по комплексам.xlsx',sheet_name='Комплекс',dtype={'Область ППМ':'str','Завод':'str'})
spr_komplex_cs_pz = rev_to_dict(sprav_po_kompl['Завод'].astype('str'),sprav_po_kompl['ЦС/ПЗ'])

ostatok['ЦС/ПЗ'] = ostatok['Завод'].apply(lambda x:  insert_vpr(x,spr_komplex_cs_pz,'ЦС/ПЗ'))



In [ ]:
sprav_po_kompl = pd.read_excel(directory+'Справочник по комплексам.xlsx',sheet_name='Комплекс',dtype={'Область ППМ':'str','Завод':'str'})

In [ ]:
ostatok['Комплекс'] = ostatok['Завод+Склад+Партия'].apply(lambda x: insert_vpr(x,spr_komplex_ost,'Комплекс') )
ostatok['Категория запаса'] = ostatok.apply(lambda x:category_of_zapas(x),axis=1)

In [ ]:
pd.merge(main_t,test,on=['БЕ','Материал'])

In [ ]:
main_t.columns

In [ ]:
test

In [ ]:
rashod['Вид движения'] = rashod['Вид движения'].astype('str')
rashod[rashod.apply(lambda x: list(x['Материал'])[-1]!='R'and x['Вид движения']!='262' ,axis = 1)] 

In [ ]:
ost_start['Ст-ть запаса в конце периода'].sum()

In [ ]:
ost_start['Стоимость свободного остатка'].sum()

In [ ]:
ost_start['Cвободный запас'].sum()

In [ ]:
raspr_summa+ner_summa

In [ ]:
ost_start['Ст-ть запаса в конце периода'].sum()

In [ ]:
ost_start['Ст-ть/блок. запаса'].sum()

In [ ]:
raspr_summa = potr_copy_for_prihod['Сумма_распреденного_остатка'].sum()

In [ ]:
ner = ost_copy[(ost_copy['осталось']>0)&(ost_copy['Cвободный запас']>0)]['Стоимость свободного остатка'].sum()

In [ ]:
rasp = ost_copy[(ost_copy['осталось']==0)&(ost_copy['Cвободный запас']>0)]['Стоимость свободного остатка'].sum()

In [ ]:
ner+rasp+q

In [ ]:
q =  ost_start[(ost_start['Cвободный запас']>0)&(ost_start['Особый запас']=='Q')]['Стоимость свободного остатка'].sum()

In [ ]:
ost_copy[(ost_copy['Материал']=='910220502')&(ost_copy['Комплекс']=='ГОК Алтай')].to_csv('C:\\Users\\User\\Desktop\\kazzinc\\Uploads\\ost.csv',sep = ';',encoding='utf-8-sig')

In [ ]:
potr_copy[(potr_copy['Материал']=='910220502')&(potr_copy['Комплекс']=='ГОК Алтай')].to_csv('C:\\Users\\User\\Desktop\\kazzinc\\Uploads\\potr.csv',sep = ';',encoding='utf-8-sig')

In [ ]:
avizot_copy.to_csv('C:\\Users\\User\\Desktop\\kazzinc\\Uploads\\ost.csv',sep = ';',encoding='utf-8-sig')

In [ ]:
potr_avizot.to_csv('C:\\Users\\User\\Desktop\\kazzinc\\Uploads\\potr.csv',sep = ';',encoding='utf-8-sig')

In [ ]:
save_pyex(df=pd.concat([cols__lev_f0,bl_potr,pp_sum_col,bl_potr_sum,pp_sum_sum],axis=1),path='C:\\Users\\User\\Desktop\\kazzinc\\Uploads1\\extr_xlsx\\f0.xlsx')

In [ ]:
save_pyex(df=arr5,path='C:\\Users\\User\\Desktop\\kazzinc\\Uploads1\\extr_xlsx\\f0+1.xlsx')

In [ ]:
save_pyex(df=rashod_start,path='C:\\Users\\User\\Desktop\\kazzinc\\Uploads1\\extr_xlsx\\rash_start.xlsx')

In [ ]:
save_pyex(df=rashod_start_raspr,path='C:\\Users\\User\\Desktop\\kazzinc\\Uploads1\\extr_xlsx\\rash_start_raspr.xlsx')

In [ ]:
save_pyex(df=ostatok,path='C:\\Users\\User\\Desktop\\kazzinc\\Uploads1\\extr_xlsx\\ostat.xlsx')

In [ ]:
save_pyex(df=ost_start_raspr,path='C:\\Users\\User\\Desktop\\kazzinc\\Uploads1\\extr_xlsx\\ostat_raspr_t.xlsx')

In [ ]:
save_pyex(df=ost_block,path='C:\\Users\\User\\Desktop\\kazzinc\\Uploads1\\extr_xlsx\\ostat_block.xlsx')

In [ ]:
save_pyex(df=rashod,path='C:\\Users\\User\\Desktop\\kazzinc\\Uploads1\\extr_xlsx\\rashod.xlsx')

In [ ]:
spr_obl_ppm_centr_zav = rev_to_dict(sprav_mat_cal_last_full['Материал'],sprav_mat_cal_last_full['Краткий текст материала'])
cons_f0['КрТекстМатериала'] = cons_f0['Материал'].apply(lambda x:  insert_vpr(x,spr_obl_ppm_centr_zav,'Краткий текст материала'))


In [ ]:
spr_obl_ppm_centr_zav = rev_to_dict(sprav_mat_cal_last_full['Материал'],sprav_mat_cal_last_full['Краткий текст материала'])

In [ ]:
spr_obl_ppm_centr_zav['150116055']

In [ ]:
sprav_mat_cal_last_full[sprav_mat_cal_last_full['Материал']=='150116055']

In [ ]:
sprav_mat_cal_last_full.shape

In [ ]:
[x for x in  sprav_mat_cal_last_full['Материал'].unique() if  x.find('100602050')!=-1]

In [ ]:
sprav_mat_cal.shape

In [ ]:
sprav_mat_cal = exlsx_f(directory_mc)

In [ ]:
directory_mc = 'C:\\Users\\Илья\\Desktop\\kazzinc\\Uploads1\\для ZMM_MATCATAL\\'

In [ ]:
[x for x in  sprav_mat_cal['Материал'].unique() if  x.find('100602050')!=-1]

In [ ]:
sprav_mat_cal_last_full = sprav_mat_cal.copy()

In [ ]:
[x for x in  sprav_mat_cal_last_full['Материал'].unique() if  x.find('010602019')!=-1]

In [ ]:
c_f0 = get_in_backup(path_backup='backup.h5',name='sprav_po_kompl')

In [ ]:
# формирование выгрузки общий + текущий месяц!!!!!!!!!!!!!!!!
# !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!

ind = ['Модуль','Заказ','Категория ТМЦ','Материал','БЕ','Заказчик + HH','Имя Завода + Материал + Заказ','Завод+НН','Завод + Склад','Завод + Область ППМ','Код округа + Материал','Комплекс','Завод','Склад потребности','Область ППМ','Центральный завод','Код округа','Имя Завода','КрТекстМатериала','Базовая ЕИ','Признак аналога','СтатусМтрл на заводе','Класс оценки','Плановик','группа','подгруппа','подподгруппа','Цена','Недропользование','ПлановСрокПоставки']
col = ['NB','законтрактовано','Заказ','Категория ТМЦ','Блокированный запас','стрЗапас','Сумма стрЗап','avizot','Сумма_распреденного_расхода','Распределение_расхода','Сумма_распреденного_прихода','Распределение_прихода','Сумма_распреденного_остатка','Распределение_остатака','Количество потребности','month','Материал','БЕ','Заказчик + HH','Имя Завода + Материал + Заказ','Завод+НН','Завод + Склад','Завод + Область ППМ','Код округа + Материал','Комплекс','Завод','Склад потребности','Область ППМ','Центральный завод','Код округа','Имя Завода','КрТекстМатериала','Базовая ЕИ','Признак аналога','СтатусМтрл на заводе','Класс оценки','Плановик','группа','подгруппа','подподгруппа','Цена','Недропользование','ПлановСрокПоставки','Сумма потребности']


cons_f0 = get_in_backup(path_backup='backup_f0_cons.h5',name='c_f0_cons_08')
current_f0 = get_in_backup(path_backup='backup_f0.h5',name='c_f0_09')


istochniki = [cons_f0,current_f0]

for ist in istochniki:
    for col in ind:
        ist[col] = ist[col].astype('str')
        ist['Цена'] = '0'

        
        
        
cons_f0['Завод']  = cons_f0['Завод+НН'].str.split(' #').apply(lambda x: x[0])
BE = pd.DataFrame()
# Менять здесь
for i in range(1,10):
    BE = BE.append(get_in_backup(path_backup='backup_f0.h5',name='c_f0_0'+str(i))[['Завод','БЕ']])
BE = BE.drop_duplicates()   
spr_be = rev_to_dict(BE['Завод'],BE['БЕ'])
cons_f0['БЕ'] = cons_f0['Завод'].apply(lambda x:  insert_vpr(x,spr_be,'БЕ'))
cons_f0['Материал'] = cons_f0['Завод+НН'].str.split(' #').apply(lambda x: x[1])

cons_f0['Заказчик + HH'] = cons_f0['Комплекс'].astype('str')+' '+cons_f0['Материал'].astype('str')


spr_centr_zav_zav = rev_to_dict(central_zavod['Область ППМ'],central_zavod['Центральный Завод'])

cons_f0.reset_index(inplace=True,drop=True)

for u in  cons_f0.index:
    if insert_vpr(cons_f0.at[u,'Область ППМ'],spr_centr_zav_zav,'Центральный Завод') ==0:
        cons_f0.at[u,'Центральный завод'] =  insert_vpr(cons_f0.at[u,'Завод'],spr_centr_zav_zav,'Центральный Завод')
    else:
        cons_f0.at[u,'Центральный завод'] =  insert_vpr(cons_f0.at[u,'Область ППМ'],spr_centr_zav_zav,'Центральный Завод')    
cons_f0['Центральный завод'] = cons_f0['Центральный завод'].apply(lambda x: str(x).replace('.0','')).astype('str')  



spr_group_mat_l1 = rev_to_dict(group_mat[group_mat['Группа материалов'].str.len()==2]['Группа материалов'],group_mat[group_mat['Группа материалов'].str.len()==2]['Обозначение 2 группы материалов'])
cons_f0['группа'] = cons_f0['Материал'].str[:2].apply(lambda x:  insert_vpr(x,spr_group_mat_l1,'Обозначение 2 группы материалов'))

spr_group_mat_l2 = rev_to_dict(group_mat[group_mat['Группа материалов'].str.len()==4]['Группа материалов'],group_mat[group_mat['Группа материалов'].str.len()==4]['Обозначение 2 группы материалов'])
cons_f0['подгруппа'] = cons_f0['Материал'].str[:4].apply(lambda x:  insert_vpr(x,spr_group_mat_l2,'Обозначение 2 группы материалов'))


spr_group_mat_l3 = rev_to_dict(group_mat[group_mat['Группа материалов'].str.len()==6]['Группа материалов'],group_mat[group_mat['Группа материалов'].str.len()==6]['Обозначение 2 группы материалов'])
cons_f0['подподгруппа'] = cons_f0['Материал'].str[:6].apply(lambda x:  insert_vpr(x,spr_group_mat_l3,'Обозначение 2 группы материалов'))


spr_obl_ppm_centr_zav = rev_to_dict(sprav_mat_cal_last_full['Материал'],sprav_mat_cal_last_full['Краткий текст материала'])
cons_f0['КрТекстМатериала'] = cons_f0['Материал'].apply(lambda x:  insert_vpr(x,spr_obl_ppm_centr_zav,'Краткий текст материала'))




spr_obl_ppm_baz_ei = rev_to_dict(sprav_mat_cal_last_full['Материал'],sprav_mat_cal_last_full['Базовая ЕИ'])
cons_f0['Базовая ЕИ'] = cons_f0['Материал'].apply(lambda x:  insert_vpr(x,spr_obl_ppm_baz_ei,'Базовая ЕИ'))

spr_obl_ppm_name_planov= rev_to_dict(sprav_mat_cal_last_full['Завод']+' #'+sprav_mat_cal_last_full['Материал'],sprav_mat_cal_last_full['Имя плановика (завод)'])
cons_f0['ЦЗ+НН'] = cons_f0['Центральный завод'].astype('str')+' #'+cons_f0['Материал'].astype('str')
cons_f0['Плановик'] = cons_f0['ЦЗ+НН'].apply(lambda x:  insert_vpr(x,spr_obl_ppm_name_planov,'Имя плановика (завод)'))
cons_f0.drop('ЦЗ+НН',inplace=True,axis = 1)


spr_obl_ppm_ocenka = rev_to_dict(sprav_mat_cal_last_full['Материал'],sprav_mat_cal_last_full['Класс оценки'])
cons_f0['Класс оценки'] = cons_f0['Материал'].apply(lambda x:  insert_vpr(x,spr_obl_ppm_ocenka,'Класс оценки'))
cons_f0['Класс оценки'] = cons_f0['Класс оценки'].apply(lambda x: str(x).replace('.0','')).astype('str')

spr_obl_ppm_stat_na_zav = rev_to_dict(sprav_mat_cal_last_full['Завод']+' #'+sprav_mat_cal_last_full['Материал'],sprav_mat_cal_last_full['СтатусМтрл на заводе'])
cons_f0['СтатусМтрл на заводе'] = cons_f0['Завод+НН'].apply(lambda x:  insert_vpr(x,spr_obl_ppm_stat_na_zav,'СтатусМтрл на заводе'))
   



current_f0.fillna(0,inplace=True)
cons_f0.fillna(0,inplace=True)
current_f0.reset_index(drop=True,inplace = True)
cons_f0.reset_index(drop=True,inplace = True)
c_f = pd.concat([current_f0.set_index(ind),
                 cons_f0.set_index(ind).loc[:,'1/2019 Остаток Кол-во':'8/2019 ФC Сумма']],axis=1).reset_index()



In [ ]:
cons_f0[cons_f0['Материал']=='010602019']

In [ ]:
current_f0[current_f0['Материал']=='010602019']

In [ ]:
arr5[arr5['Материал']=='010602019']

In [ ]:
group_mat = get_in_backup(path_backup='backup.h5',name='group_mat')

In [ ]:
central_zavod = get_in_backup(path_backup='backup.h5',name='central_zavod')

In [ ]:
c_f.shape

In [ ]:
c_f[c_f['Материал']=='100602050']

In [ ]:
arr5 = pd.concat([c_f[ind],  
        c_f.loc[:,[x for x in c_f.columns if x.find('ПП кол-во')!=-1]],   c_f['Текущее кол-во ПП'],
        c_f.loc[:,[x for x in c_f.columns if x.find('ПП сумма')!=-1]],    c_f['Текущая сумма ПП'],
        c_f.loc[:,[x for x in c_f.columns if x.find('Остаток Кол-во')!=-1]],
        c_f.loc[:,[x for x in c_f.columns if x.find('Остаток Сумма')!=-1]],
        c_f.loc[:,[x for x in c_f.columns if x.find('ФП Кол-во')!=-1]],
        c_f.loc[:,[x for x in c_f.columns if x.find('ФП Сумма')!=-1]],
        c_f.loc[:,[x for x in c_f.columns if x.find('ФC Кол-во')!=-1]],
        c_f.loc[:,[x for x in c_f.columns if x.find('ФC Сумма')!=-1]],
        c_f.loc[:,'Товар в пути (ZNB5, ZNB2/3-АвизОт)':'РР'] ],axis = 1,)

In [ ]:
arr5.shape

In [ ]:
arr5[arr5['Материал']=='010602019']['СтатусМтрл на заводе'].values

In [ ]:
arr5['Базовая ЕИ'].value_counts()

In [ ]:
pd.DataFrame(arr5[arr5['Базовая ЕИ']=='0']['Материал'].unique())

In [ ]:
save_pyex(df=pd.DataFrame(arr5[arr5['Базовая ЕИ']=='0']['Материал'].unique()),path='C:\\Users\\Илья\\Desktop\\kazzinc\\Uploads1\\extr_xlsx\\001.xlsx')

In [ ]:
arr5[arr5['Базовая ЕИ']=='0']['Материал'].unique()

In [ ]:
spr_obl_ppm_baz_ei['010602019']

In [ ]:
sprav_mat_cal_last_full.name = 'sprav_mat_cal_last_full'

In [ ]:
save_backup('backup.h5',[sprav_mat_cal_last_full])

In [ ]:
cons_f0 = get_in_backup(path_backup='backup_f0_cons.h5',name='c_f0_cons_08')
current_f0 = get_in_backup(path_backup='backup_f0.h5',name='c_f0_09')

In [ ]:
arr5.shape

In [ ]:
cons_f0[cons_f0['Материал']=='010602019']['СтатусМтрл на заводе'].values

In [ ]:
current_f0[current_f0['Материал']=='010602019']['СтатусМтрл на заводе'].values

In [ ]:
cons_f0['СтатусМтрл на заводе'] = cons_f0['СтатусМтрл на заводе'].astype('str')

In [ ]:
c_f = pd.concat([current_f0.set_index(ind),
                 cons_f0.set_index(ind).loc[:,'1/2019 Остаток Кол-во':'8/2019 ФC Сумма']],axis=1).reset_index()

In [ ]:
arr5.shape

In [ ]:
arr5 = pd.concat([c_f[ind],  
        c_f.loc[:,[x for x in c_f.columns if x.find('ПП кол-во')!=-1]],   c_f['Текущее кол-во ПП'],
        c_f.loc[:,[x for x in c_f.columns if x.find('ПП сумма')!=-1]],    c_f['Текущая сумма ПП'],
        c_f.loc[:,[x for x in c_f.columns if x.find('Остаток Кол-во')!=-1]],
        c_f.loc[:,[x for x in c_f.columns if x.find('Остаток Сумма')!=-1]],
        c_f.loc[:,[x for x in c_f.columns if x.find('ФП Кол-во')!=-1]],
        c_f.loc[:,[x for x in c_f.columns if x.find('ФП Сумма')!=-1]],
        c_f.loc[:,[x for x in c_f.columns if x.find('ФC Кол-во')!=-1]],
        c_f.loc[:,[x for x in c_f.columns if x.find('ФC Сумма')!=-1]],
        c_f.loc[:,'Товар в пути (ZNB5, ZNB2/3-АвизОт)':'РР'] ],axis = 1,)

In [ ]:
c_f0_01 = get_in_backup(path_backup='backup_f0.h5',name='c_f0_01')
c_f0_02 = get_in_backup(path_backup='backup_f0.h5',name='c_f0_02')
c_f0_03 = get_in_backup(path_backup='backup_f0.h5',name='c_f0_03')
c_f0_04 = get_in_backup(path_backup='backup_f0.h5',name='c_f0_04')
c_f0_05 = get_in_backup(path_backup='backup_f0.h5',name='c_f0_05')
c_f0_06 = get_in_backup(path_backup='backup_f0.h5',name='c_f0_06')
c_f0_07 = get_in_backup(path_backup='backup_f0.h5',name='c_f0_07')
c_f0_08 = get_in_backup(path_backup='backup_f0.h5',name='c_f0_08')

In [ ]:
# формирование bi отчетности new 15.10.2019

df_for_bi = get_in_backup(path_backup='backup_for_powerBI.h5',name='f0_vertical_09')
df_for_bi.drop('Unnamed: 109',axis = 1,inplace = True)
df_for_bi['Категория ТМЦ'] = df_for_bi.apply(lambda x: func_cat_tmc(x),axis = 1)
df_for_bi = df_for_bi[df_for_bi['Категория ТМЦ']=='Текущая потребность']
# df_for_bi.to_csv('C:\\Users\\Илья\\Desktop\\kazzinc\\bi.csv',encoding='utf-8-sig',sep = ';',decimal=',')

df_for_bi['Завод']  = df_for_bi['Завод+НН'].str.split(' #').apply(lambda x: x[0])
BE = pd.DataFrame()
for i in range(1,10):
    BE = BE.append(get_in_backup(path_backup='backup_f0.h5',name='c_f0_0'+str(i))[['Завод','БЕ']])
BE = BE.drop_duplicates()   
spr_be = rev_to_dict(BE['Завод'],BE['БЕ'])
df_for_bi['БЕ'] = df_for_bi['Завод'].apply(lambda x:  insert_vpr(x,spr_be,'БЕ'))
df_for_bi['Материал'] = df_for_bi['Завод+НН'].str.split(' #').apply(lambda x: x[1])

df_for_bi['Заказчик + HH'] = df_for_bi['Комплекс'].astype('str')+' '+df_for_bi['Материал'].astype('str')


spr_centr_zav_zav = rev_to_dict(central_zavod['Область ППМ'],central_zavod['Центральный Завод'])

df_for_bi.reset_index(inplace=True,drop=True)

for u in  df_for_bi.index:
    if insert_vpr(df_for_bi.at[u,'Область ППМ'],spr_centr_zav_zav,'Центральный Завод') ==0:
        df_for_bi.at[u,'Центральный завод'] =  insert_vpr(df_for_bi.at[u,'Завод'],spr_centr_zav_zav,'Центральный Завод')
    else:
        df_for_bi.at[u,'Центральный завод'] =  insert_vpr(df_for_bi.at[u,'Область ППМ'],spr_centr_zav_zav,'Центральный Завод')    
df_for_bi['Центральный завод'] = df_for_bi['Центральный завод'].apply(lambda x: str(x).replace('.0','')).astype('str')  



spr_group_mat_l1 = rev_to_dict(group_mat[group_mat['Группа материалов'].str.len()==2]['Группа материалов'],group_mat[group_mat['Группа материалов'].str.len()==2]['Обозначение 2 группы материалов'])
df_for_bi['группа'] = df_for_bi['Материал'].str[:2].apply(lambda x:  insert_vpr(x,spr_group_mat_l1,'Обозначение 2 группы материалов'))

spr_group_mat_l2 = rev_to_dict(group_mat[group_mat['Группа материалов'].str.len()==4]['Группа материалов'],group_mat[group_mat['Группа материалов'].str.len()==4]['Обозначение 2 группы материалов'])
df_for_bi['подгруппа'] = df_for_bi['Материал'].str[:4].apply(lambda x:  insert_vpr(x,spr_group_mat_l2,'Обозначение 2 группы материалов'))


spr_group_mat_l3 = rev_to_dict(group_mat[group_mat['Группа материалов'].str.len()==6]['Группа материалов'],group_mat[group_mat['Группа материалов'].str.len()==6]['Обозначение 2 группы материалов'])
df_for_bi['подподгруппа'] = df_for_bi['Материал'].str[:6].apply(lambda x:  insert_vpr(x,spr_group_mat_l3,'Обозначение 2 группы материалов'))


spr_obl_ppm_centr_zav = rev_to_dict(sprav_mat_cal_last_full['Материал'],sprav_mat_cal_last_full['Краткий текст материала'])
df_for_bi['КрТекстМатериала'] = df_for_bi['Материал'].apply(lambda x:  insert_vpr(x,spr_obl_ppm_centr_zav,'Краткий текст материала'))




spr_obl_ppm_baz_ei = rev_to_dict(sprav_mat_cal_last_full['Материал'],sprav_mat_cal_last_full['Базовая ЕИ'])
df_for_bi['Базовая ЕИ'] = df_for_bi['Материал'].apply(lambda x:  insert_vpr(x,spr_obl_ppm_baz_ei,'Базовая ЕИ'))

spr_obl_ppm_name_planov= rev_to_dict(sprav_mat_cal_last_full['Завод']+' #'+sprav_mat_cal_last_full['Материал'],sprav_mat_cal_last_full['Имя плановика (завод)'])
df_for_bi['ЦЗ+НН'] = df_for_bi['Центральный завод'].astype('str')+' #'+df_for_bi['Материал'].astype('str')
df_for_bi['Плановик'] = df_for_bi['ЦЗ+НН'].apply(lambda x:  insert_vpr(x,spr_obl_ppm_name_planov,'Имя плановика (завод)'))
df_for_bi.drop('ЦЗ+НН',inplace=True,axis = 1)


spr_obl_ppm_ocenka = rev_to_dict(sprav_mat_cal_last_full['Материал'],sprav_mat_cal_last_full['Класс оценки'])
df_for_bi['Класс оценки'] = df_for_bi['Материал'].apply(lambda x:  insert_vpr(x,spr_obl_ppm_ocenka,'Класс оценки'))
df_for_bi['Класс оценки'] = df_for_bi['Класс оценки'].apply(lambda x: str(x).replace('.0','')).astype('str')

spr_obl_ppm_stat_na_zav = rev_to_dict(sprav_mat_cal_last_full['Завод']+' #'+sprav_mat_cal_last_full['Материал'],sprav_mat_cal_last_full['СтатусМтрл на заводе'])
df_for_bi['СтатусМтрл на заводе'] = df_for_bi['Завод+НН'].apply(lambda x:  insert_vpr(x,spr_obl_ppm_stat_na_zav,'СтатусМтрл на заводе'))


df_for_bi['Заказ'] = df_for_bi['Заказ'].apply(lambda x: str(x).replace('.0','')).astype('str')
df_for_bi.loc[df_for_bi[df_for_bi['Заказ'].str.startswith('2')].index,'Заказ'] = '0'


ind = ['month','Модуль','Заказ','Категория ТМЦ','Материал','БЕ','Заказчик + HH','Завод+НН','Завод + Склад','Завод + Область ППМ','Комплекс','Завод','Склад потребности','Область ППМ','Центральный завод','Код округа','Имя Завода','КрТекстМатериала','Базовая ЕИ','Признак аналога','СтатусМтрл на заводе','Класс оценки','Плановик','группа','подгруппа','подподгруппа','Цена','Недропользование','ПлановСрокПоставки']
df_for_bi['Цена'] = 0
df_for_bi['Цена'] = df_for_bi['Цена'].astype('str')
for st in ind:
    df_for_bi[st] = df_for_bi[st].astype('str')
    

    
aggregate= dict((x,'sum') for x in df_for_bi.columns if df_for_bi[x].dtype=='float64' and x!='Цена')
aggregate['Цена'] = ['max']



ind.remove('Цена')
df_for_bi = df_for_bi.groupby(ind).agg(aggregate).reset_index()
df_for_bi['Цена'] = df_for_bi['Цена'].astype('float64')
df_for_bi.columns =   [x[0] for x in df_for_bi.columns]

A902['Код округа'] = A902['Код округа'].astype('str')
A902['Материал'] = A902['Материал'].astype('str')
A902['Материал'] = A902['Материал'].apply(lambda x: '0'+str(x) if  len(str(x))<9 else x)
A900['Материал'] = A900['Материал'].astype('str')
A900['Материал'] = A900['Материал'].apply(lambda x: '0'+str(x) if  len(str(x))<9 else x)
df_for_bi['Материал'] = df_for_bi['Материал'].astype('str')
A902.fillna(0,inplace=True)
A900.fillna(0,inplace=True)
KONP.fillna(0,inplace=True)
potr_gr = dict(tuple(df_for_bi[df_for_bi['Цена']==0].groupby(['Код округа','Материал'])))
potr_gr_mat = dict(tuple(df_for_bi[df_for_bi['Цена']==0].groupby(['Материал'])))
konp_gr = dict(tuple(KONP.groupby(['Номер записи условий'])))
# c_f0['Цена'] = 0.0000
A900 = A900[A900['Конец действия'].apply(lambda x: x.year)==current_date.year]
A902 = A902[A902['Конец действия'].apply(lambda x: x.year)==current_date.year]
main_price(potr_gr,potr_gr_mat,A900,A902,konp_gr,df_for_bi)

In [ ]:
c_f.shape

In [ ]:
c_f['ПлановСрокПоставки'].unique()

In [ ]:
c_f['БЕ'].unique()

In [ ]:
for ind, i in enumerate(np.array_split(arr5,2)) :
    save_pyex(df=i,path='C:\\Users\\Илья\\Desktop\\kazzinc\\Uploads1\\extr_xlsx\\'+str(ind)+'.xlsx')

In [ ]:
df_for_bi['СрочностьПотребности'].unique()


In [ ]:
df['СрочностьПотребности'].unique()

In [ ]:
arr5 = get_in_backup(path_backup='backup_for_powerBI.h5',name='f0_vertical_09')

In [ ]:
arr5['СрочностьПотребности'].unique()

In [ ]:
datetime.datetime(1988,1,1)

In [ ]:
arr1 = get_in_backup(path_backup='backup_for_powerBI.h5',name='f0_vertical_09')

In [ ]:
arr2 = get_in_backup(path_backup='backup_for_powerBI.h5',name='f0_vertical_08')

In [ ]:
arr2.drop('Unnamed: 104',axis = 1,inplace = True)
arr1.drop('Unnamed: 109',axis = 1,inplace = True)
arr1['Месяц'] = datetime.datetime(2019,9,1)
arr2['Месяц'] = datetime.datetime(2019,8,1)

In [ ]:
arr1.shape

In [ ]:
arr2.shape

In [ ]:
pd.concat([arr1,arr2])['Месяц']

In [ ]:
# для bi
c_f0_01 = get_in_backup(path_backup='backup_for_powerBI.h5',name='f0_vertical_01')
c_f0_02 = get_in_backup(path_backup='backup_for_powerBI.h5',name='f0_vertical_02')
c_f0_03 = get_in_backup(path_backup='backup_for_powerBI.h5',name='f0_vertical_03')
c_f0_04 = get_in_backup(path_backup='backup_for_powerBI.h5',name='f0_vertical_04')
c_f0_05 = get_in_backup(path_backup='backup_for_powerBI.h5',name='f0_vertical_05')
c_f0_06 = get_in_backup(path_backup='backup_for_powerBI.h5',name='f0_vertical_06')
c_f0_07 = get_in_backup(path_backup='backup_for_powerBI.h5',name='f0_vertical_07')
c_f0_08 = get_in_backup(path_backup='backup_for_powerBI.h5',name='f0_vertical_08')
c_f0_09 = get_in_backup(path_backup='backup_for_powerBI.h5',name='f0_vertical_09')

In [ ]:
for ind,df_for_bi in enumerate([c_f0_01,c_f0_02,c_f0_03,c_f0_04,c_f0_05,c_f0_06,c_f0_07,c_f0_08,c_f0_09],start=1) :
    df_for_bi['Месяц'] = str(datetime.datetime(2019,ind,1))

df_for_bi = pd.DataFrame()
df_for_bi = df_for_bi.concat([c_f0_01,c_f0_02,c_f0_03,c_f0_04,c_f0_05,c_f0_06,c_f0_07,c_f0_08,c_f0_09])

In [ ]:
df_for_bi.drop(['Unnamed: 104','Unnamed: 109'],axis = 1,inplace = True)

In [ ]:
df_for_bi['Месяц'].unique()

In [ ]:
df_for_bi.to_csv('C:\\Users\\Илья\\Desktop\\kazzinc\\bi1.csv',encoding='utf-8-sig',sep = ';',decimal=',',)

In [ ]:
df_for_bi['month'][0]

In [ ]:
df_for_bi['Месяц'][0]

In [ ]:
df_for_bi

In [ ]:
df_for_bi['Месяц'][0].month

In [ ]:
arr = df_for_bi['Месяц'].apply(lambda x: True if x.month==8 else False)

In [ ]:
arr1 = df_for_bi[arr]

In [ ]:
arr2 = arr1['month'].apply(lambda x: True if x.month>=10 and   x.month<=12  else False)

In [ ]:
arr2

In [ ]:
arr1[arr2]['Количество потребности'].sum()

In [ ]:
arr1[arr2]['month']

In [ ]:
arr2

In [ ]:
arr1['month'][2352077].month

In [ ]:
df_for_bi['Комплекс'].unique()

In [ ]:
df_for_bi['Комплекс'].unique()

In [ ]:
df_for_bi.info(verboose = True)

In [ ]:
df_for_bi.name = 'f0_vertical_cons'
save_backup('df_for_bi_cons.h5',[df_for_bi])

In [ ]:
df_for_bi = get_in_backup(path_backup='df_for_bi_cons.h5',name='f0_vertical_cons')

In [ ]:
df_for_bi['Месяц'][0]

In [ ]:
df_for_bi[df_for_bi['Материал']=='893203151']

In [ ]:
df = pd.read_csv('C:\\Users\\Илья\\Desktop\\kazzinc\\Uploads1\\extr_xlsx\\bi1.csv',sep = ';')

In [ ]:
df

In [ ]:
a = df_for_bi.to_json()

In [ ]:
sprav_mat_cal_last_full

In [ ]:
c_f0_09 = get_in_backup(path_backup='backup_for_powerBI.h5',name='f0_vertical_09')

In [ ]:
c_f0_09

In [ ]:
df_for_bi.drop(['Unnamed: 104','Unnamed: 109'],axis = 1,inplace=True)

In [ ]:
c_f0_09

In [ ]:
df[] 

In [ ]:
df[df['Дата потребности']<=datetime.datetime(2020,1,31)]['Дата потребности'].unique()

In [ ]:
df['Дата потребности'].apply(lambda x: x.year).unique()

In [ ]:
rr = df[df['Дата потребности']<=datetime.datetime(2020,1,31)]

In [ ]:
rr[rr['Дата потребности'].apply(lambda x: x.year)==2020]

In [ ]:
df = pd.read_csv(directory+'CSV.csv',sep = ';',parse_dates=['Дата потребности'],dtype={'Материал':'str','Завод':'str'})

In [ ]:
df['Дата потребности'].apply(lambda x: x.year).unique()

In [ ]:
'ПП кол-во 1/'+str(current_date.year+1)

In [ ]:
c_f0

In [ ]:
c_f0

In [ ]:
spr_obl_ppm_baz_ei = rev_to_dict(sprav_mat_cal_last_full['Материал'],sprav_mat_cal_last_full['Базовая ЕИ'])
c_f0['Базовая ЕИ'] = c_f0['Материал'].apply(lambda x:  insert_vpr(x,spr_obl_ppm_baz_ei,'Базовая ЕИ'))

In [ ]:
c_f0['Базовая ЕИ'].value_counts()

In [ ]:
sprav_mat_cal_last_full.shape

In [ ]:
c_f0[c_f0['Базовая ЕИ']==0]

In [ ]:
n = 13
cols_ost_prognoz_summa = ['Остаток прогноз Сумма '+str(x)+'/'+str(current_date.year) for x in range(1,n+1)]

In [ ]:
for i in cols_ost_prognoz_summa[current_date.month:]:
    c_f0[i] =    c_f0.apply(lambda x: x['Остаток прогноз Кол-во'+str(i[21:])]*(x[str(current_date.month)+'/'+str(current_date.year)+' Остаток Сумма']/  x[str(current_date.month)+'/'+str(current_date.year)+' Остаток Кол-во']) if x[str(current_date.month)+'/'+str(current_date.year)+' Остаток Кол-во']>0 else x['Остаток прогноз Кол-во'+str(i[21:])] *x['Цена'],axis = 1)

In [ ]:
current_date.month

In [ ]:
current_date

In [ ]:
current_date = datetime.datetime(2019,10,1)

In [ ]:
f0_cons[i] =    f0_cons.apply(lambda x: x['Остаток прогноз Кол-во'+str(i[21:])]*(x[str(current_date.month)+'/'+str(current_date.year)+' Остаток Сумма']/  x[str(current_date.month)+'/'+str(current_date.year)+' Остаток Кол-во']) if x[str(current_date.month)+'/'+str(current_date.year)+' Остаток Кол-во']>0 else x['Остаток прогноз Кол-во'+str(i[21:])].multiply(x['Цена']),axis = 1)


In [ ]:
c_f0['Цена']

In [ ]:
c_f0['Остаток прогноз Сумма 13/2019']

In [ ]:
c_f0['Остаток прогноз Сумма 13/2019'].mul(c_f0['Цена'])

In [ ]:
x['Цена']

In [ ]:
c_f0[c_f0['Материал']=='195010029']

In [ ]:
cons_f0 = get_in_backup(path_backup='backup_f0_cons.h5',name='c_f0_cons_09')

In [ ]:
cons_f0

In [ ]:
f0 =  get_in_backup(path_backup='f0_cons_09.h5',name='f0_cons_09')

In [ ]:
current_f0 = get_in_backup(path_backup='backup_f0.h5',name='c_f0_10')

In [ ]:
current_f0

In [ ]:
current_date.month+1

In [ ]:
cons_f0

In [ ]:
arr5

In [ ]:
cons_f0

In [ ]:
arr5

In [ ]:
df = pd.read_csv(directory+'CSV.csv',sep = ';',parse_dates=['Дата потребности'],dtype={'Материал':'str','Завод':'str'})

In [ ]:
directory = 'C:\\Users\\Илья\\Desktop\\kazzinc\\Uploads1\\'

In [ ]:
df.name = 'extract'
save_backup('backup_test_del.h5',[df])

In [ ]:
df = pd.read_csv('E:\csv.csv',sep = ';',parse_dates=['Дата потребности'],dtype={'Материал':'str','Завод':'str'})

In [ ]:
pd.DataFrame(df['Завод'].unique()) 

In [ ]:
save_pyex(df=pd.DataFrame(df['Завод'].unique()) ,path='E:\csv.xlsx')

In [ ]:
Завод склад, обл ппм, кол-во потребности,дата потребности, бе  по поду дате потребности

In [ ]:
df['Дата потребности'][0]

In [ ]:
df['Дата потребности'] = df['Дата потребности'].apply(lambda x: '01.01.1900' if x =='00.00.0000' else  x)
df['Дата потребности'] = [datetime.datetime(int(x.split('.')[2]),int(x.split('.')[1]),int(x.split('.')[0])) for x in df['Дата потребности'].values]
# собираю дату с группировкой по месяцу
month = df['Дата потребности'].apply(lambda x:  str(x.year)+'-'+str(x.month)+'-01'+ ' 00:00:00')
month =pd.to_datetime(month) 

In [ ]:
df.fillna(0,inplace=True)

In [ ]:
df[['Завод','Область ППМ','Дата потребности','БЕ']]

In [ ]:
df['Дата потребности'].apply(lambda x:x.year).unique()

In [ ]:
# надо Наташе каждый месяц
for year in df_str_zap['Дата потребности'].apply(lambda x:x.year).unique():
    ind = df_str_zap['Дата потребности'].apply(lambda x:True if x.year==year else False)
    d =  df_str_zap[['Завод','Область ППМ','Дата потребности','БЕ','Количество потребности','Материал','Вид документа потребности','Комплекс','Базовая ЕИ']] 
    
    save_pyex(df=d[ind] ,path='E:\\' + str(year) +  '.xlsx')

In [ ]:
df['Количество потребности'] = df['Количество потребности'].str.replace('-','')

In [ ]:
cons_f0 =  get_in_backup(path_backup='f0_cons_09.h5',name='f0_cons_09')

In [ ]:
c_f0['Базовая ЕИ'].value_counts()

In [ ]:
df

In [ ]:
c_f0

In [ ]:
arr5.shape

In [ ]:
cons_f0.shape

In [ ]:
directory = 'C:\\Users\\Илья\\Desktop\\kazzinc\\Uploads1\\'
df = pd.read_csv(directory+'CSV.csv',sep = ';',parse_dates=['Дата потребности'],dtype={'Материал':'str','Завод':'str'})

In [ ]:
df = df[df['Дата потребности']<=datetime.datetime(2020,1,31)]

spr_zav_dict = rev_to_dict(sprav_po_kompl['Завод'],sprav_po_kompl['ЦС/ПЗ'])
df['ЦС/ПЗ'] = df['Завод'].apply(lambda x:  insert_vpr(x,spr_zav_dict,'ЦС/ПЗ'))
df['Номер потребности'] = df['Номер потребности'].apply(lambda x: str(x).replace('.0','')).astype('str')
df['МВЗ'] = df['МВЗ'].apply(lambda x: str(x).replace('.0','')).astype('str')


df['Тип материала'] = df['Тип материала'].apply(lambda x: str(x).replace('.0','')).astype('str')
df['БЕ'] = df['БЕ'].apply(lambda x: str(x).replace('.0','')).astype('str')



# недропользование неполнятно
spr_nedra = rev_to_dict(sprav_nedra['Завод + Склад'],sprav_nedra['Завод + Склад'])
# df['Недропользование']= df['Завод'].apply(lambda x: insert_vpr(x,spr_nedra,'Завод'))   
df['Склад потребности'] = df['Склад потребности'].apply(lambda x: str(x).replace('.0','')).astype('str')
df['Завод + Склад'] = df['Завод']+' '+df['Склад потребности']
df['Недропользование'] = df['Завод + Склад'].apply(lambda x: 0 if   insert_vpr(x,spr_nedra,'Завод + Склад')==0 else 1)

# Вывод по докум-ам потребности
df['Вывод по документам потребности'] = df.apply(lambda x: formula_vivod_po_doc_potrebn(x),axis=1 )


df['Вывод по документам пополнения'] = df.apply(lambda x: formula_vivod_po_doc_popolneniya(x),axis=1 )

In [ ]:
df[(df['Вывод по документам потребности']!='OK')|(df['Вывод по документам пополнения']!='OK')]

In [ ]:
save_pyex(df=df[(df['Вывод по документам потребности']!='OK')|(df['Вывод по документам пополнения']!='OK')] ,path='E:\11112019EXTR.xlsx')

In [ ]:
f0 =  get_in_backup(path_backup='backup_f0.h5',name='c_f0_10')

In [ ]:
f0.loc[:,'ПП кол-во 1/2019':'ПП кол-во 13/2019']

In [ ]:
f0.reset_index(drop = True,inplace = True)

In [ ]:
pd.concat([f0.loc[:,'ПП кол-во 1/2019':'ПП кол-во 13/2019'],f0['Комплекс'],f0['Материал']],axis = 1)

In [ ]:
save_pyex(df=pd.concat([f0.loc[:,'ПП кол-во 1/2019':'ПП кол-во 13/2019'],f0['Комплекс'],f0['Материал']],axis = 1) ,path='E:\11112019EXTR_with_potr.xlsx')

In [ ]:
f0

In [ ]:
c_f0_08

In [ ]:
f0.loc[f0[f0['Материал'] == '610902015'].index]['Цена'] = 0

In [ ]:
A902[A902['Материал']=='610902015']

In [ ]:
potr_gr = dict(tuple(df_for_bi[df_for_bi['Цена']==0].groupby(['Код округа','Материал'])))

In [ ]:
potr_gr

In [ ]:
f0[f0['Материал'] == '610902015']

In [ ]:
rr =  get_in_backup(path_backup='backup_for_powerBI_test_no.h5',name='f0_vertical_10')

In [ ]:
aa = rr[rr['Материал'] == '610902015'].copy()

In [ ]:
aa['Цена'] = 0

In [ ]:
A902['Код округа'] = A902['Код округа'].astype('str')
A902['Материал'] = A902['Материал'].astype('str')
A902['Материал'] = A902['Материал'].apply(lambda x: '0'+str(x) if  len(str(x))<9 else x)
A900['Материал'] = A900['Материал'].astype('str')
A900['Материал'] = A900['Материал'].apply(lambda x: '0'+str(x) if  len(str(x))<9 else x)
aa['Материал'] = aa['Материал'].astype('str')
A902.fillna(0,inplace=True)
A900.fillna(0,inplace=True)
KONP.fillna(0,inplace=True)
potr_gr = dict(tuple(aa[aa['Цена']==0].groupby(['Код округа','Материал'])))
potr_gr_mat = dict(tuple(aa[aa['Цена']==0].groupby(['Материал'])))
konp_gr = dict(tuple(KONP.groupby(['Номер записи условий'])))
# c_f0['Цена'] = 0.0000
A900 = A900[A900['Конец действия'].apply(lambda x: x.year)==current_date.year]
A902 = A902[A902['Конец действия'].apply(lambda x: x.year)==current_date.year]
main_price(potr_gr,potr_gr_mat,A900,A902,konp_gr,aa)

In [ ]:
aa['Код округа']

In [ ]:
# Формирование цены
def get_df_f(dfm,cols,filtrs):
    try:
        dd = dfm[(dfm[cols[0]] == filtrs[0]) & (dfm[cols[1]] == filtrs[1])]
        return dd
    except KeyError:
        return pd.DataFrame()

def get_df_f1(dfm,cols,filtrs):
    try:
        dd = dfm[(dfm[cols[0]] == filtrs[0])]
        return dd
    except KeyError:
        return pd.DataFrame()


def insert_price_into_potr(price,into_df,df):
    for ind in into_df.index:
        if df.at[ind,'Цена']==0:
            df.at[ind,'Цена'] = price

def mex_price(dat,konp_gr,df,potr_gr,i):
    if len(dat)>0:
        # print(dat['Номер записи условий'].values[0])
        konp_ = konp_gr[dat['Номер записи условий'].values[0]]
        # если есть  в конп
        if len(konp_)>0:
            # print(konp_)
            if konp_['Базовая ЕИ'].values[0]==0 or konp_['Базовая ЕИ'].values[0]==konp_['Единица измерения'].values[0]:
                price = (konp_['Сумма условия'].values[0]  / konp_['Единица цены'].values[0])*345
                insert_price_into_potr(price,potr_gr[i],df)
            elif konp_['Базовая ЕИ'].values[0]!=konp_['Единица измерения'].values[0]:

                if konp_['Числитель пересчета'].values[0]==0:
                    price = konp_['Сумма условия'].values[0] *345
                    insert_price_into_potr(price, potr_gr[i],df)
                else:
                    price = ((konp_['Сумма условия'].values[0]*konp_['Числитель пересчета'].values[0])/konp_['Единица цены'].values[0])*345
                    insert_price_into_potr(price, potr_gr[i],df)


def main_price(potr_gr,potr_gr1,A900,A902,konp_gr,df):
    for i in potr_gr.keys():
        dat = get_df_f(A902, ['Код округа', 'Материал'], [i[0], i[1]])
        mex_price(dat,konp_gr,df,potr_gr,i)
    for i in potr_gr1.keys():
        dat = get_df_f1(A900, ['Материал'], [i])
        mex_price(dat, konp_gr, df, potr_gr1, i)

In [ ]:
dat = get_df_f(A902, ['Код округа', 'Материал'], ['ZH', '610902015'])

In [ ]:
A902[A902['Материал']=='610902015']['Код округа'].unique()

In [ ]:
aa['Код округа'].unique()

In [ ]:
dat

In [ ]:
A902[(A902['Код округа']=='ZH')&(A902['Материал']=='610902015')]

In [ ]:
dat

In [ ]:
aa

In [ ]:
konp_gr['0000296822']

In [ ]:
potr_gr = dict(tuple(aa[aa['Цена']==0].groupby(['Код округа','Материал'])))

In [ ]:
potr_gr.keys()

In [ ]:
for i in potr_gr.keys():
    print(get_df_f(A902, ['Код округа', 'Материал'], [i[0], i[1]]))

In [ ]:
konp_ = konp_gr[dat['Номер записи условий'].values[0]]

In [ ]:
dat = get_df_f(A902, ['Код округа', 'Материал'], ['ZH', '610902015'])

In [ ]:
len(dat)

In [ ]:
konp_ = konp_gr[dat['Номер записи условий'].values[0]]

In [ ]:
konp_

In [ ]:
konp_['Базовая ЕИ'].values[0]

In [ ]:
konp_['Базовая ЕИ'].values[0]==konp_['Единица измерения'].values[0]

In [ ]:
price = (konp_['Сумма условия'].values[0]  / konp_['Единица цены'].values[0])*345

In [ ]:
price

In [ ]:
main_price(potr_gr,potr_gr_mat,A900,A902,konp_gr,aa)

In [ ]:
aa

In [ ]:
potr_with_ostatok_total = get_in_backup(path_backup='backup_for_powerBI_test_no.h5',name='f0_vertical_10')

In [ ]:
directory = 'C:\\Users\\Илья\\Desktop\\kazzinc\\Uploads1\\'

In [ ]:
c_f0[c_f0['Материал']=='610902015']

In [ ]:
potr_with_ostatok_total[potr_with_ostatok_total['Материал']=='610902015']

In [ ]:
arr5.shape

In [ ]:
cons_f0.shape

In [ ]:
arr5.columns

In [ ]:
list(arr5.columns)

In [ ]:
list_a = list(arr5.columns)

In [ ]:
[i for i in y if y[i]>1]

In [ ]:
[x for x in list_a if list_a.count(x) > 1]

In [ ]:
ZNB[(ZNB['Материал']=='060211279')&(ZNB1['Завод']=='1599')]['Заявка + Позиция (для UB05 по ДП)'].unique()

In [ ]:
df99 = pd.read_csv(directory+'CSV.csv',sep = ';',parse_dates=['Дата потребности'],dtype={'Материал':'str','Завод':'str'})

In [ ]:
cons_f0.set_index(ind).loc[:,str(current_date.month-1)+'/2019 Остаток Кол-во':str(current_date.month-2)+'/2019 ФC Сумма']

In [ ]:
df = get_in_backup(path_backup='backup.h5',name='extract')

In [ ]:
df[(df['Материал']=='060211279')]['Номер элемента пополнения + Номер позиции пополнения'].unique()

In [ ]:
arr5 = pd.concat([c_f[ind],  
        c_f.loc[:,[x for x in c_f.columns if x.find('ПП кол-во')!=-1]],   c_f['Текущее кол-во ПП'],
        c_f.loc[:,[x for x in c_f.columns if x.find('ПП сумма')!=-1]],    c_f['Текущая сумма ПП'],
        c_f.loc[:,[x for x in c_f.columns if x.find('Остаток Кол-во')!=-1]],
        c_f.loc[:,[x for x in c_f.columns if x.find('Остаток Сумма')!=-1]],
        c_f.loc[:,[x for x in c_f.columns if x.find('ФП Кол-во')!=-1]],
        c_f.loc[:,[x for x in c_f.columns if x.find('ФП Сумма')!=-1]],
        c_f.loc[:,[x for x in c_f.columns if x.find('ФC Кол-во')!=-1]],
        c_f.loc[:,[x for x in c_f.columns if x.find('ФC Сумма')!=-1]],
        c_f.loc[:,'Товар в пути (ZNB5, ZNB2/3-АвизОт)':'РР'] ],axis = 1,)

In [ ]:
df[(df['Материал']=='060211279')&(df['Номер элемента пополнения + Номер позиции пополнения']=='55064944 10')]

In [ ]:
df99[df99['Материал']=='060211279']['Номер элемента пополнения'].unique()

In [ ]:
df99['Материал'].unique()

In [ ]:
df99['Номер элемента пополнения'] = df99['Номер элемента пополнения'].astype('str')

In [ ]:
55064944.0

In [ ]:
df99[(df99['Материал']=='060211279')&(df99['Номер элемента пополнения']=='55064944.0')]

In [ ]:
df99['Номер элемента пополнения'].apply(lambda x: x[:2]=='55')

In [ ]:
ss = df99[df99['Номер элемента пополнения'].apply(lambda x: x[:2]=='55')]

In [ ]:
ss[(ss['Материал']=='060211279')]['Дата потребности'].unique()

In [ ]:
df[(df['Материал']=='060211279')&(df['Вид документа пополнения']=='UB05')]

In [ ]:
ZNB[(ZNB['Материал']=='060211279')]

In [ ]:
ZNB

In [ ]:
UB05

In [ ]:
df_str_zap[(df_str_zap['Материал']=='060211279')&(df_str_zap['Вид документа пополнения']=='UB05')]

In [ ]:
ffff = tab_avizot.reset_index()

In [ ]:
ffff[ffff['Номер элемента пополнения + Номер позиции пополнения + Наименование типа элемента ППМ пополн']=='55064943 10']

In [ ]:
rr = ffff['Номер элемента пополнения + Номер позиции пополнения + Наименование типа элемента ППМ пополн'].apply(lambda x: x.find('55064943 10')!=-1)

In [ ]:
ffff[rr]

In [ ]:
ZNB['Номер элемента пополнения + Номер позиции пополнения + Наименование типа элемента ППМ пополн'].apply(lambda x: x.find('55064943 10')!=-1)

In [ ]:
ZNB

In [ ]:
UB05

In [ ]:
save_pyex(df=ZNB ,path='C:\\Users\\Илья\\Desktop\\kazzinc\\Uploads1\\extr_xlsx\\11111.xlsx')

In [ ]:
save_pyex(df=tab_avizot.reset_index() ,path='C:\\Users\\Илья\\Desktop\\kazzinc\\Uploads1\\extr_xlsx\\22222.xlsx')

In [ ]:
tab_avizot.reset_index()

In [ ]:
df99 = pd.read_excel('C:\\Users\\Илья\\Desktop\\kazzinc\\files for power bi\\j\\9.xlsx')
df99['Месяц']  = datetime.datetime(2019,9,1)
df99['one'] = 1

In [ ]:
df99 = df99[['Модуль','группа','Комплекс','Ответственность','Описание отклонения по ответственности','one','Месяц']].\
groupby(['Модуль','группа','Комплекс','Ответственность','Описание отклонения по ответственности','Месяц']).agg({'one':'count'}).reset_index()

In [ ]:
save_pyex(df=a.append([df09,df10,df11]) ,path='C:\\Users\\Илья\\Desktop\\kazzinc\\files for power bi\\two_diagrams.xlsx')

In [ ]:
df09 = df99.copy()

In [ ]:
df10 = df99.copy()

In [ ]:
df11 = df99.copy()

In [ ]:
a = pd.DataFrame()

In [ ]:
a.append([df09,df10,df11])

In [ ]:
df09.shape

In [ ]:
5.650360e+04+1.214061e+05+6.668837e+06+1.223524e+05+7.945835e+05

In [ ]:
122352.41+6725340.68+121406.09+794583.46

In [ ]:
# Запуск распределения остатков
# date_start_year = datetime.datetime(2019,1,1)
current_date = datetime.datetime(2019,10,1)
potr_copy = df[df['Материал']=='121106205'].copy()
ost_copy = ost_start_raspr.copy()
potr_copy.name = 'potrebn_with_raspr_ost'
ost_copy.name =  'ost_with_potr'
# Запуск распределения остатков
potr_copy['Распределение_остатака'] = 0.000000
potr_copy['Сумма_распреденного_остатка'] = 0.000000


ostat(ost_st=ost_copy,potr_st=potr_copy,filters=filter_else,current_date = current_date,all_months=False)    
# save_backup('backup.h5',[ost_start,ostatok,block_zapas,ost_start_raspr])

In [ ]:
potr_NB

In [ ]:
NB_copy[NB_copy['Материал']=='121106205']

In [ ]:
master_zav_nn = dict(tuple(potr_NB[potr_NB['month'] > current_date].groupby('Заказчик + HH')))

In [ ]:
potr_NB

In [ ]:
potr_NB = potr_prt_pzk.copy()
NB_copy = NB.copy()
potr_NB.name = 'potr_NB'
NB_copy.name =  'NB_copy'
potr_NB['NB'] = 0.000000
NB_copy['осталось'] = NB_copy['Количество пополнения']
NB_raspr(ost_st=NB_copy,potr_st=potr_NB,filters=filter_else,current_date = current_date,all_months=True)

In [ ]:
# управление механикой остатков(вызывать ее)
def NB_raspr(ost_st, potr_st, current_date, filters,all_months):
    #     группировка остатков по
    if all_months ==True:
        master_zav_nn = dict(tuple(potr_st[potr_st['month'] >= current_date].groupby('Заказчик + HH')))
    else:
        master_zav_nn = dict(tuple(potr_st[potr_st['month'] > current_date].groupby('Заказчик + HH')))
    pz_ost = ost_st
    # пз
#     print(master_zav_nn.keys())
    for pz_slave_row in pz_ost.index:
        df_master = ret_data_dict(pz_ost.at[pz_slave_row, 'Комплекс+Маттериал'], master_zav_nn)
        if df_master is None:
            df_master = pd.DataFrame()
        # если есть потребность у сцепки зав нн
        # pz_ost.at[pz_slave_row, 'осталось'] = 1600
        if len(df_master) > 0 and pz_ost.at[pz_slave_row, 'осталось'] > 0:
            df_master = df_master.sort_values('Дата потребности')
            # функция
            df_master = mex_NB(df_master, pz_ost, pz_slave_row, ost_st, potr_st)
            master_zav_nn[pz_ost.at[pz_slave_row, 'Комплекс+Маттериал']] = df_master
    if all_months ==True:
        master_zav_nn = dict(tuple(potr_st[potr_st['month'] >= current_date].groupby('Материал')))
    else:
        master_zav_nn = dict(tuple(potr_st[potr_st['month'] > current_date].groupby('Материал')))

    for pz_slave_row in pz_ost.index:
        if pz_ost.at[pz_slave_row, 'Комплекс'] == 'ALL1000':
            df_master = ret_data_dict(pz_ost.at[pz_slave_row, 'Материал'], master_zav_nn)
            if df_master is None:
                df_master = pd.DataFrame()
            # если есть потребность у сцепки зав нн
            # pz_ost.at[pz_slave_row, 'осталось'] = 1600
            if len(df_master) > 0 and pz_ost.at[pz_slave_row, 'осталось'] > 0:
                df_master = df_master[df_master['Комплекс'].isin(filters['one'])].sort_values('Дата потребности')
                # функция
                df_master = mex_NB(df_master, pz_ost, pz_slave_row, ost_st, potr_st)
                master_zav_nn[pz_ost.at[pz_slave_row, 'Материал']] = df_master

In [ ]:
master_zav_nn = dict(tuple(potr_NB[potr_NB['month'] >= current_date].groupby('Заказчик + HH')))

In [ ]:
ret_data_dict(NB_copy.at[1121775, 'Комплекс+Маттериал'], master_zav_nn)

In [ ]:
potr_NB.at[1121775, 'Комплекс+Маттериал']

In [ ]:
NB_copy[NB_copy['Комплекс+Маттериал']=='АТК 201100093']

In [ ]:
def main_price_f(df,A902,A900):
    konp_gr = KONP.set_index(['Номер записи условий'])
    potr_gr = df.set_index(['Код округа','Материал']).copy()
    A902 = A902.set_index(['Код округа','Материал'])
    A900 = A900.set_index(['Материал'])
    for i in potr_gr.index:
        dat = get_df_f(A902, [i[0], i[1]])
        mex_price(dat,konp_gr,df,potr_gr,i)
    potr_gr1 = df.set_index(['Материал'])
    for i in potr_gr1.index:
        dat = get_df_f1(A900, i)
        
        mex_price(dat, konp_gr, df, potr_gr1, i)
    return df

In [ ]:
def insert_price_into_potr(price,into_df,df):
    print(into_df)
    for ind in into_df:
        
        if df.loc[df['index']==ind]['Цена'].values[0]==0:
            df.loc[df['index']==ind]['Цена'].values[0] = price

def mex_price(dat,konp_gr,df,potr_gr,i):
    
    if len(dat)>0:
        # print(dat['Номер записи условий'].values[0])
        konp_ = konp_gr.xs(dat['Номер записи условий'].values[0]).to_frame().T

        # если есть  в конп
        if len(konp_)>0:
            print(i)
            print(potr_gr.xs(i).shape[0])
            if  potr_gr.xs(i).shape[0]==1:
                r = potr_gr.xs(i).to_frame().T['index']
                print(r)
            elif potr_gr.xs(i).shape[0]>1:
                r = potr_gr.xs(i)['index']
                print(r)
                    
            if konp_['Базовая ЕИ'].values[0]==0 or konp_['Базовая ЕИ'].values[0]==konp_['Единица измерения'].values[0]:
                price = (konp_['Сумма условия'].values[0]  / konp_['Единица цены'].values[0])*345
                
                insert_price_into_potr(price,r,df)
            elif konp_['Базовая ЕИ'].values[0]!=konp_['Единица измерения'].values[0]:

                if konp_['Числитель пересчета'].values[0]==0:
                    price = konp_['Сумма условия'].values[0] *345
#                     print(potr_gr)
                    insert_price_into_potr(price, r,df)
                else:
                    price = ((konp_['Сумма условия'].values[0]*konp_['Числитель пересчета'].values[0])/konp_['Единица цены'].values[0])*345
#                     print(potr_gr)
                    insert_price_into_potr(price, r,df)

In [ ]:
def get_df_f1(dfm,filtrs):
    try:
        dd = dfm.xs(filtrs).to_frame().T
        return dd
    except KeyError:
        return pd.DataFrame()

In [ ]:
def get_df_f(dfm,filtrs):
    try:
        dd = dfm.xs([filtrs[0],filtrs[1]]).to_frame().T
        return dd
    except KeyError:
        return pd.DataFrame()

In [ ]:
def price_main2020(df,A902,A900):
    df['index'] = [x for x in range(len(df.index))]
    df = main_price_f(df,A902,A900)
    return df

In [ ]:
# potr_gr = df.set_index(['Код округа','Материал'])
# potr_gr1 = df.set_index(['Материал'])
# konp_gr = KONP.set_index(['Номер записи условий'])

In [ ]:

c_f011111 = c_f0[:100].copy()
oo = price_main2020(c_f011111,A902,A900)

In [ ]:
df.set_index(['Материал']).xs('561613019')

In [ ]:
oo

In [ ]:
A900[A900['Материал']=='200700003']

In [ ]:
potr_gr.index[:10][0]

In [ ]:
df.xs(['ZY','200700003'])

In [ ]:
c_f0.xs(312747).to_frame().T

In [ ]:
import numpy as np
import time
from multiprocessing import Pool

def parallel_calc_fast_(df,A902,A900,KONP,current_date,plan_kurs):
    start_time = time.time()
    pool = Pool(processes=3)
    
    
    A902['Код округа'] = A902['Код округа'].astype('str')
    A902['Материал'] = A902['Материал'].astype('str')
    A902['Материал'] = A902['Материал'].apply(lambda x: '0'+str(x) if  len(str(x))<9 else x)
    A900['Материал'] = A900['Материал'].astype('str')
    A900['Материал'] = A900['Материал'].apply(lambda x: '0'+str(x) if  len(str(x))<9 else x)
    df['Материал'] = df['Материал'].astype('str')
    A902.fillna(0,inplace=True)
    A900.fillna(0,inplace=True)
    KONP.fillna(0,inplace=True)
#     potr_gr = dict(tuple(df.groupby(['Код округа','Материал'])))
#     potr_gr_mat = dict(tuple(df.groupby(['Материал'])))
#     konp_gr = dict(tuple(KONP.groupby(['Номер записи условий'])))
    df['Цена'] = 0.0000
    A900 = A900[A900['Конец действия'].apply(lambda x: x.year)==current_date.year]
    A902 = A902[A902['Конец действия'].apply(lambda x: x.year)==current_date.year]
    print('1')
    
    arrs =  np.array_split(df,3)
    arrs =  [(potr_gr,potr_gr_mat,A900,A902,konp_gr,arr,plan_kurs) for arr in arrs]
    print('2')
    df_list = pool.map(func_read_ex.main_price_f, arrs)
    df = pd.concat(df_list)
    print("--- %s seconds ---" % (time.time() - start_time))
    pool.close()
    return df 


In [ ]:
aa  = parallel_calc_fast_(potr_with_ostatok_total,A902,A900,KONP,current_date,345)

In [ ]:
func_read_ex.main_price_f

In [ ]:
for ind, i in enumerate(np.array_split(df,2)) :
    save_pyex(df=i,path='C:\\Users\\Илья\\Desktop\\kazzinc\\Uploads1\\extr_xlsx\\df'+str(ind)+'.xlsx')

In [ ]:

sprav_mat_cal_last_full['Завод'] = sprav_mat_cal_last_full['Завод'].apply(lambda x: str(x).replace('.0','')).astype('str')
sprav_mat_cal_last_full['Материал'] = sprav_mat_cal_last_full['Материал'].apply(lambda x: str(x).replace('.0','')).astype('str')
sprav_mat_cal_last_full['Материал'] = sprav_mat_cal_last_full['Материал'].apply(lambda x: '0'+str(x) if  len(str(x))<9 else x)

sprav_mat_cal['Завод'] = sprav_mat_cal['Завод'].apply(lambda x: str(x).replace('.0','')).astype('str')
sprav_mat_cal['Материал'] = sprav_mat_cal['Материал'].apply(lambda x: str(x).replace('.0','')).astype('str')
sprav_mat_cal['Материал'] = sprav_mat_cal['Материал'].apply(lambda x: '0'+str(x) if  len(str(x))<9 else x)

In [ ]:
spr_str_zap__= rev_to_dict(sprav_mat_cal_last_full['Завод']+' #'+sprav_mat_cal_last_full['Материал'],sprav_mat_cal_last_full['Страховой запас'])

In [ ]:
spr_str_zap__

In [ ]:

ost_start[ost_start['страховой запас']>0]

In [ ]:
save_pyex(df=ost_start,path='C:\\Users\\Илья\\Desktop\\kazzinc\\Uploads1\\extr_xlsx\\ostat.xlsx')

In [ ]:
ost_start

In [ ]:
ost_block = ost_start[(ost_start['Блокированный запас']>0)]
ost_block.name = 'ost_block'

ost_start_raspr = ost_start[(ost_start['Cвободный запас']>0)&(ost_start['Особый запас']!='Q')]
ost_start_raspr.name = 'ost_start_raspr'

In [ ]:
df

In [ ]:
df

In [ ]:
ost_start

In [ ]:
ost_start['страховой запас_п'] = ost_start['страховой запас']
ind__ = ost_start[ost_start.apply(lambda x: True if (x['осталось']-x['страховой запас'])<0  else False,axis = 1 )].index
ost_start.loc[ind__,'страховой запас'] = 0
ost_start['temp_razn_ost_str_zap'] = ost_start.apply(lambda x: x['осталось']-x['страховой запас'] if  (x['осталось']-x['страховой запас'])>0 and x['страховой запас']>0 and len(x['Партия'])==2  else 0,axis = 1)
ost_start['занулить'] = ost_start.apply(lambda x:'да' if  (x['осталось']-x['страховой запас']==0) and x['страховой запас']>0 and len(x['Партия'])==2  else 0,axis = 1)
ind_str_zap = ost_start[ost_start['temp_razn_ost_str_zap']>0].index
ind_str_zanul = ost_start[ost_start['занулить']=='да'].index
ost_start.loc[ind_str_zap,'осталось'] = ost_start.loc[ind_str_zap,'temp_razn_ost_str_zap']
ost_start.loc[ind_str_zanul,'осталось'] = 0
ind_ = ost_start[ost_start.apply(lambda x: True if x['страховой запас']>0 and  len(x['Партия'])>2 else False,axis = 1 )].index
ost_start.loc[ind_,'страховой запас'] = 0


In [ ]:
ost_start_raspr1

In [ ]:
ost_start[ost_start['temp_razn_ost_str_zap']>0]

In [ ]:
ost_start[ost_start['занулить']=='да']

In [ ]:
ost_start['Кол-во запаса в конце периода'].sum()

In [ ]:
ost_start['осталось'].sum()+ost_start['страховой запас'].sum()

In [ ]:
ost_block = ost_start[(ost_start['Блокированный запас']>0)]

In [ ]:
ost_start_raspr = ost_start[(ost_start['осталось']>0)&(ost_start['Особый запас']!='Q')]
ost_start_raspr_str_zap = ost_start[(ost_start['страховой запас']>0)&(ost_start['Особый запас']!='Q')]

In [ ]:
ost_block['Блокированный запас'].sum()

In [ ]:
ost_start_raspr['осталось'].sum()+ost_start_raspr_str_zap['страховой запас'].sum()+ost_block['Блокированный запас'].sum()+ost_start[ost_start['Особый запас']=='Q']['осталось'].sum()

In [ ]:
65508275.03899999-65767275

In [ ]:
ost_start[ost_start['Особый запас']=='Q']['осталось'].sum()

In [ ]:
df

In [ ]:
df

In [ ]:
df['month'].unique()

In [ ]:
df['month'] =    df.apply(lambda x: datetime.datetime(2199,1,1)  if x['Страховой запас']>0 else x['month'],axis = 1)
df['Дата потребности'] =    df.apply(lambda x: datetime.datetime(2199,1,1)  if x['Страховой запас']>0 else x['Дата потребности'],axis = 1)
df['ЦС/ПЗ'] =  df.apply(lambda x: 'ПЗ'  if x['Страховой запас']>0 else x['ЦС/ПЗ'],axis = 1)

In [ ]:
df

In [ ]:
df = get_in_backup(path_backup='backup.h5',name='extract')

In [ ]:
potr_copy[potr_copy['Распределение_остатака']>0]

In [ ]:
potr_copy['Распределение_остатака'].sum()

In [ ]:
ost_start_raspr_str_zap['страховой запас'].sum()

In [ ]:
ost_copy1[ost_copy1['осталось']==0]['Кол-во запаса в конце периода'].sum()

In [ ]:
potr_copy1['Распределение_остатака'].sum()

In [ ]:
ost_copy

In [ ]:
ost_copy

In [ ]:
potr_copy[potr_copy['Распределение_остатака']>0]

In [ ]:
potr_copy

In [ ]:
potr_copy_cons['Распределение_остатака'].sum()

In [ ]:
potr_copy

In [ ]:
[dd.count(x) for x in list(dd) if dd.count(x) >1 ]

In [ ]:
dd = list(ost_start_raspr_str_zap['Завод+Склад']+ost_start_raspr_str_zap['Партия']+ost_start_raspr_str_zap['Материал'])

In [ ]:
dd.count('1502 1000UK')

In [ ]:
dd

In [ ]:
ost_copy[(ost_copy['осталось']>0)&(ost_copy['Cвободный запас']>0)]['Cвободный запас'].sum()

In [ ]:
ost_copy[(ost_copy['осталось']>0)&(ost_copy['Cвободный запас']>0)][['Комплекс','Завод+НН','Cвободный запас','Стоимость свободного остатка']].groupby(['Комплекс','Завод+НН']).sum().reset_index()

In [ ]:
prov_ost

In [ ]:
df_str_zap

In [ ]:
sprav_mat_cal_last_full[(sprav_mat_cal_last_full['Материал']=='010401026')&(sprav_mat_cal_last_full['Страховой запас']>0)]

In [ ]:
spr_str_zap__= rev_to_dict(sprav_mat_cal_last_full['Завод']+sprav_mat_cal_last_full['Материал']+sprav_mat_cal_last_full['Код округа'],sprav_mat_cal_last_full['Страховой запас'])
ost_start['страховой запас'] = (ost_start['Завод']+ost_start['Материал']+ost_start['Партия']).apply(lambda x:insert_vpr(x,spr_str_zap__,'Страховой запас') )

In [ ]:
ost_start['страховой запас'].unique()

In [ ]:
ost_start['Партия'].unique()

In [ ]:
ost_start['Завод'].unique()

In [ ]:
neraspr_ostat_tab[neraspr_ostat_tab['Завод+НН']=='3520 #442302040']

In [ ]:
ost_copy1[(ost_copy1['Материал']=='442302040')&()]

In [ ]:
sprav_spp = pd.read_excel(directory+'СПП.xlsx',sheet_name='Sheet1')

to_str_spp = ['СПП-элемент','Место возникновения прибыли']

for i in to_str_spp:
    sprav_spp[i] = sprav_spp[i].apply(lambda x: str(x).replace('.0','')).astype('str')
    sprav_spp[i] = sprav_spp[i].apply(lambda x: '' if x=='0' else x)

In [ ]:
sprav_spp

In [ ]:
ostatok['СПП-элемент'].unique()


In [ ]:
spr__spp = rev_to_dict(sprav_spp['СПП-элемент'],sprav_spp['Место возникновения прибыли'])


In [ ]:
spr_str_zap__= rev_to_dict(sprav_mat_cal_last_full['Завод']+sprav_mat_cal_last_full['Материал']+sprav_mat_cal_last_full['Код округа'],sprav_mat_cal_last_full['Страховой запас'])
ost_start['страховой запас'] = (ost_start['Завод']+ost_start['Материал']+ost_start['Партия']).apply(lambda x:insert_vpr(x,spr_str_zap__,'Страховой запас') )


In [ ]:
sprav_mat_cal_last_full

In [ ]:
to_str_spr_mat = ['СтатусМтрл на заводе','Завод','Материал']

for i in to_str_spr_mat:
    sprav_mat_cal_last_full[i].fillna(0,inplace = True)
    sprav_mat_cal_last_full[i] = sprav_mat_cal_last_full[i].apply(lambda x: str(x).replace('.0','')).astype('str')
    sprav_mat_cal_last_full[i] = sprav_mat_cal_last_full[i].apply(lambda x: '' if x=='0' or x==0 else x)

In [ ]:
spr__st_mrlt = rev_to_dict(sprav_mat_cal_last_full['Завод']+' #'+sprav_mat_cal_last_full['Материал'],sprav_mat_cal_last_full['СтатусМтрл на заводе'])
ostatok['СтатусМтрл на заводе'] = ostatok.apply(lambda x:insert_vpr(x['Завод+НН'],spr__st_mrlt,'СтатусМтрл на заводе'),axis = 1 )

In [ ]:
ostatok['Завод+НН']

In [ ]:
ostatok['СтатусМтрл на заводе'] = ostatok.apply(lambda x:insert_vpr(x['Завод+НН'],spr__st_mrlt,'СтатусМтрл на заводе'),axis = 1 ).unique()

In [ ]:
spr__st_mrlt

In [ ]:
sprav_mat_cal_last_full = get_in_backup(path_backup='backup.h5',name='sprav_mat_cal_last_full')

In [ ]:
sprav_mat_cal_last_full

In [ ]:
potr_with_ostatok_total = potr_with_ostatok_total.set_index('Завод+НН')

In [ ]:
potr_with_ostatok_total.reset_index(inplace=True,drop = True)

In [ ]:
potr_with_ostatok_total

In [ ]:
sprav_spp = pd.read_excel(directory+'СПП.xlsx',sheet_name='Sheet1')

In [ ]:
def func_cat_tmc_new(x,current_date,dic):
    if (x['Количество потребности']>0 and current_date.year == x['Дата потребности'].year) or \
    (x['Категория ТМЦ'] in['ТНСЗ','страховой запас','Проект','ГО','Спецодежда']):
        return 'Востребованный'
    elif x['Количество потребности']>0 and x['Дата потребности'].year>current_date.year and x['Распределение_остатака']>0:
        return 'Среднесрочный'
    elif x['Комплекс'] =='НЕЛИКВИДЫ':
        return 'Неликвиды'
    elif dic[x['Завод+НН']]==0:
        return 'Невостребованный'
    else:
        return 0
  

In [ ]:
potr_with_ostatok_total.reset_index(inplace=True,drop = True)
dic = potr_with_ostatok_total.groupby('Завод+НН').agg({'Количество потребности':'sum'}).to_dict(orient= 'dict')['Количество потребности']

potr_with_ostatok_total['Категория_запаса'] = potr_with_ostatok_total.apply(lambda x: func_cat_tmc_new(x,current_date,dic),axis = 1)

In [ ]:
potr_with_ostatok_total[potr_with_ostatok_total['Категория_запаса']==0]['Дата потребности'].unique()

In [ ]:
potr_with_ostatok_total.xs('1206 #030101004')

In [ ]:
current_date = datetime.datetime(2020,1,1)

In [ ]:
potr_with_ostatok_total[potr_with_ostatok_total['Материал']=='200700003']

In [ ]:
potr_with_ostatok_total.xs(aa['Завод+НН'][1])

In [ ]:
aa = potr_with_ostatok_total[:1000].copy()

In [ ]:
aa['Завод+НН'][1]

In [ ]:
potr_with_ostatok_total.set_index('Завод+НН')

In [ ]:
start_time = time.time()
aa.apply(lambda x: func_cat_tmc_new(x,current_date,potr_with_ostatok_total),axis = 1)
print("--- %s seconds ---" % (time.time() - start_time))

In [ ]:
start_time = time.time()
aa.apply(lambda x: func_cat_tmc_new(x,current_date,potr_with_ostatok_total),axis = 1)
print("--- %s seconds ---" % (time.time() - start_time))

In [ ]:
potr_with_ostatok_total.loc['1206 #030101004']

In [ ]:
yyyy = potr_with_ostatok_total.groupby('Завод+НН').agg({'Количество потребности':'sum'})

In [ ]:
potr_with_ostatok_total = potr_with_ostatok_total.set_index('Завод+НН')

In [ ]:
yyyy.to_dict(orient= 'dict')['Количество потребности']

In [ ]:
potr_with_ostatok_total.groupby('Завод+НН').agg({'Количество потребности':'sum'}).to_dict(orient= 'dict')['Количество потребности']

In [ ]:
ind_ost = ostatok[(ostatok['СтатусМтрл на заводе']=='2')&(ostatok['ЦС/ПЗ']=='ЦС')].index
ostatok.loc[ind_ost,'Комплекс'] = 'НЕЛИКВИДЫ'

In [ ]:
ostatok[(ostatok['СтатусМтрл на заводе']=='2')&(ostatok['ЦС/ПЗ']=='ЦС')]

In [ ]:
# priznak_raschet_nachalo_goda это рассчет на 0101
def f0_consolidation(df,current_date,ind,col,first_month_of_kvartal,ost_nach_perioda,priznak_raschet_nachalo_goda,sp):
    def yy(x,sp):
        return x[sp].sum()
    
    
    
    f0 =   df
#     f0 =  pd.pivot_table(tab_f0,columns=['month'],index=ind,aggfunc='sum',fill_value=0).reset_index()  
    cols__lev_f0 = f0[ind]
    cols__lev_f0.columns = ind
    bl_potr = f0['Количество потребности']
    bl_potr.columns = ['ПП кол-во '+str(x.month)+'/'+ str(x.year) for x in bl_potr.columns] 
    pp_sum_col = pd.DataFrame(bl_potr.loc[:,'ПП кол-во '+str(current_date.month+1)+'/'+ str(current_date.year):'ПП кол-во '+str(12)+'/'+ str(current_date.year)].sum(axis = 1),columns=['Текущее кол-во ПП'])


    bl_potr_sum  = f0['Сумма потребности']
    bl_potr_sum.columns = ['ПП сумма '+str(x.month)+'/'+ str(x.year) for x in bl_potr_sum.columns]
    pp_sum_sum = pd.DataFrame(bl_potr_sum.loc[:,'ПП сумма '+str(current_date.month+1)+'/'+ str(current_date.year):'ПП сумма '+str(12)+'/'+ str(current_date.year)].sum(axis = 1),columns=['Текущая сумма ПП'])
    
    

    col_vo_prix = pd.DataFrame(f0['Распределение_прихода'].sum(axis = 1),columns=[str(current_date.month)+'/'+str(current_date.year)+' ФП Кол-во'])
    sum_prix = pd.DataFrame(f0['Сумма_распреденного_прихода'].sum(axis = 1),columns=[str(current_date.month)+'/'+str(current_date.year)+' ФП Сумма'])
    

    col_vo_rash = pd.DataFrame(f0['Распределение_расхода'].sum(axis = 1),columns=[str(current_date.month)+'/'+str(current_date.year)+' ФC Кол-во'])
    summ_rash = pd.DataFrame(f0['Сумма_распреденного_расхода'].sum(axis = 1),columns=[str(current_date.month)+'/'+str(current_date.year)+' ФC Сумма'])

 

    col_vo_ost = pd.DataFrame(f0['Распределение_остатака'].sum(axis = 1),columns=[str(current_date.month)+'/'+str(current_date.year)+' Остаток Кол-во'])
    sum_ost = pd.DataFrame(f0['Сумма_распреденного_остатка'].sum(axis = 1),columns=[str(current_date.month)+'/'+str(current_date.year)+' Остаток Сумма'])

    avizot =  pd.DataFrame(f0['avizot'].sum(axis = 1),columns=['Товар в пути (ZNB5, ZNB2/3-АвизОт)'])

    str_zap_col_vo = pd.DataFrame(f0['стрЗапас'].sum(axis = 1),columns=['SS (норма запаса на складе)'])
    str_zap_summ = pd.DataFrame(f0['Сумма стрЗап'].sum(axis = 1),columns=['SS сумма'])
    
    NB = pd.DataFrame(f0['NB'].sum(axis = 1),columns=['Заявка на закупку(NB)'])
    ZAK = pd.DataFrame(f0['законтрактовано'].sum(axis = 1),columns=['Законтрактовано(ZNB-ПртПЗк)'])

    f0_cons = pd.concat([cols__lev_f0,bl_potr,pp_sum_col,bl_potr_sum,pp_sum_sum,col_vo_prix,sum_prix,col_vo_rash,summ_rash,col_vo_ost,sum_ost,avizot,str_zap_col_vo,str_zap_summ,NB,ZAK],axis=1)

    #     при рассчете 12 месяца для остатка на начало периода закомментировать
#     \\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\
    f0_cons = pd.merge(f0_cons,ost_nach_perioda,on=['Склад потребности','Заказ','Категория ТМЦ','Комплекс','Завод+НН','Модуль'],how='outer')
# \\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\    
    f0_cons.fillna(0,inplace=True)
    
    if current_date.month == 1 and priznak_raschet_nachalo_goda==True:
        f0_cons['Остаток на начало года Кол-во'] =  f0_cons[str(current_date.month)+'/'+str(current_date.year)+' Остаток Кол-во']  
        f0_cons['Остаток на начало года Сумма'] =  f0_cons[str(current_date.month)+'/'+str(current_date.year)+' Остаток Сумма']    
        f0_cons.drop([str(current_date.month)+'/'+str(current_date.year)+' Остаток Кол-во',\
                     str(current_date.month)+'/'+str(current_date.year)+' Остаток Сумма'],inplace = True,axis = 1)
        
    n = 12
    if current_date.month==10 and priznak_raschet_nachalo_goda ==False:
        n+=1
#         ПП сумма 5/2019
#         ПП кол-во 1/current_date.year+1
#         ПП кол-во n/current_date.year+1
        f0_cons.rename(columns={'ПП кол-во 1/'+ str(current_date.year+1): 'ПП кол-во ' + str(n) + '/' + str(current_date.year)},inplace=True)
        f0_cons.rename(columns={'ПП сумма 1/'+str(current_date.year+1): 'ПП сумма '+str(n)+'/'+ str(current_date.year)},inplace=True)
    elif current_date.month ==11 and priznak_raschet_nachalo_goda ==False:
        n+=2
        f0_cons.rename(columns={'ПП кол-во 1/'+str(current_date.year+1): 'ПП кол-во '+str(n-1)+'/'+ str(current_date.year)},inplace=True)
        f0_cons.rename(columns={'ПП сумма 1/'+str(current_date.year+1): 'ПП сумма '+str(n-1)+'/'+ str(current_date.year)},inplace=True)
        f0_cons.rename(columns={'ПП кол-во 2/'+str(current_date.year+1): 'ПП кол-во '+str(n)+'/'+ str(current_date.year)},inplace=True)
        f0_cons.rename(columns={'ПП сумма 2/'+str(current_date.year+1): 'ПП сумма '+str(n)+'/'+ str(current_date.year)},inplace=True)
    elif current_date.month ==12 and priznak_raschet_nachalo_goda ==False:    
        n+=3
        f0_cons.rename(columns={'ПП кол-во 1/'+str(current_date.year+1): 'ПП кол-во '+str(n-2)+'/'+ str(current_date.year)},inplace=True)
        f0_cons.rename(columns={'ПП сумма 1/'+str(current_date.year+1): 'ПП сумма '+str(n-2)+'/'+ str(current_date.year)},inplace=True)
        f0_cons.rename(columns={'ПП кол-во 2/'+str(current_date.year+1): 'ПП кол-во '+str(n-1)+'/'+ str(current_date.year)},inplace=True)
        f0_cons.rename(columns={'ПП сумма 2/'+str(current_date.year+1): 'ПП сумма '+str(n-1)+'/'+ str(current_date.year)},inplace=True)
        f0_cons.rename(columns={'ПП кол-во 3/'+str(current_date.year+1): 'ПП кол-во '+str(n)+'/'+ str(current_date.year)},inplace=True)
        f0_cons.rename(columns={'ПП сумма 3/'+str(current_date.year+1): 'ПП сумма '+str(n)+'/'+ str(current_date.year)},inplace=True)
    
   
    cols_bz_col_vo = ['БЗ кол-во '+str(x)+'/'+str(current_date.year) for x in range(1,n+1)]
    
    cols_bz_summ = ['БЗ сумма '+str(x)+'/'+str(current_date.year) for x in range(1,n+1)]
    

    for i in cols_bz_col_vo:f0_cons[i] = 0
    f0_cons['ИТОГО БЗ Кол-во'] = 0
    for i in cols_bz_summ:f0_cons[i] = 0
    f0_cons['ИТОГО БЗ Сумма'] = 0


    cols_ost_prognoz = ['Остаток прогноз Кол-во '+str(x)+'/'+str(current_date.year) for x in range(1,n+1)]
    cols_ost_prognoz_summa = ['Остаток прогноз Сумма '+str(x)+'/'+str(current_date.year) for x in range(1,n+1)]
    for i in cols_ost_prognoz:f0_cons[i] = 0
    for i in cols_ost_prognoz_summa:f0_cons[i] = 0
    

        
    # если сейчас 1 месяц тогда текущий опираем на остаток на начало года
    if current_date.month == 1 and priznak_raschet_nachalo_goda==True:
        

        f0_cons.loc[:,cols_bz_col_vo[current_date.month-1]] = f0_cons.apply(lambda x: yy(x,sp)+x['SS (норма запаса на складе)']-x['Остаток на начало года Кол-во']-x['Товар в пути (ZNB5, ZNB2/3-АвизОт)']  \
                                                                            if (x['Остаток на начало года Кол-во']+x['Товар в пути (ZNB5, ZNB2/3-АвизОт)'])-yy(x,sp)-x['SS (норма запаса на складе)']<0 else 0,axis = 1)
            
        f0_cons.loc[:,cols_ost_prognoz[current_date.month-1]] =f0_cons['Остаток на начало года Кол-во'].values+f0_cons['Товар в пути (ZNB5, ZNB2/3-АвизОт)'].values+f0_cons.loc[:,cols_bz_col_vo[current_date.month-1]].values-f0_cons[sp].sum(axis = 1).values

        
        ff = current_date.month-1
        for i in cols_bz_col_vo[current_date.month:]:

            f0_cons[i] = f0_cons.apply(lambda x: x['ПП кол-во'+str(i[9:])]+x['SS (норма запаса на складе)']-x['Остаток прогноз Кол-во'+str(cols_bz_col_vo[ff][9:])]  if x['Остаток прогноз Кол-во'+str(cols_bz_col_vo[ff][9:])]-x['ПП кол-во'+str(i[9:])]-x['SS (норма запаса на складе)']<0 else 0,axis = 1)
    #         формкла для остатка
            f0_cons['Остаток прогноз Кол-во'+str(i[9:])] = f0_cons['Остаток прогноз Кол-во'+str(cols_bz_col_vo[ff][9:])].values+f0_cons[i].values-f0_cons['ПП кол-во'+str(i[9:])].values
#             print('Остаток прогноз Кол-во'+str(cols_bz_col_vo[ff][9:]))

            ff+=1
        f0_cons[cols_ost_prognoz_summa[0]] = f0_cons.apply(lambda x: x[cols_bz_col_vo[0]]* (x['Остаток на начало года Сумма']/  x['Остаток на начало года Кол-во']) if x['Остаток на начало года Кол-во']>0 else x['Остаток прогноз Кол-во '+str(current_date.month+1)+'/'+str(current_date.year)]*x['Цена'] ,axis = 1)
        
        for i in cols_ost_prognoz_summa[current_date.month-1:]:
             f0_cons[i] =    f0_cons.apply(lambda x: x['Остаток прогноз Кол-во'+str(i[21:])]*(x['Остаток на начало года Сумма']/  x['Остаток на начало года Кол-во']) if x['Остаток на начало года Сумма']>0 and  x['Остаток на начало года Кол-во']>0 else x['Остаток прогноз Кол-во'+str(i[21:])]*x['Цена'] ,axis = 1)

        
    else:
        #количество bz     
        f0_cons.loc[:,cols_bz_col_vo[:current_date.month]] = 0
        f0_cons.loc[:,cols_ost_prognoz[:current_date.month]] = 0
        f0_cons.loc[:,cols_ost_prognoz_summa[:current_date.month]] = 0
        f0_cons.loc[:,cols_bz_col_vo[current_date.month]] = f0_cons.apply(lambda x: yy(x,sp)+x['SS (норма запаса на складе)']-x[str(current_date.month)+'/'+str(current_date.year)+' Остаток Кол-во']-x['Товар в пути (ZNB5, ZNB2/3-АвизОт)']  \
                                                                          if (x[str(current_date.month)+'/'+str(current_date.year)+' Остаток Кол-во']+x['Товар в пути (ZNB5, ZNB2/3-АвизОт)'])- yy(x,sp)-x['SS (норма запаса на складе)']<0 else 0,axis = 1)
        f0_cons.loc[:,cols_ost_prognoz[current_date.month]] = f0_cons[str(current_date.month)+'/'+str(current_date.year)+' Остаток Кол-во'].values+f0_cons['Товар в пути (ZNB5, ZNB2/3-АвизОт)'].values+f0_cons.loc[:,cols_bz_col_vo[current_date.month]].values-f0_cons[sp].sum(axis = 1).values


        ff = current_date.month
        for i in cols_bz_col_vo[current_date.month+1:]:

            f0_cons[i] = f0_cons.apply(lambda x: x['ПП кол-во'+str(i[9:])]+x['SS (норма запаса на складе)']-x['Остаток прогноз Кол-во'+str(cols_bz_col_vo[ff][9:])]  if x['Остаток прогноз Кол-во'+str(cols_bz_col_vo[ff][9:])]-x['ПП кол-во'+str(i[9:])]-x['SS (норма запаса на складе)']<0 else 0,axis = 1)
    #         формкла для остатка
            f0_cons['Остаток прогноз Кол-во'+str(i[9:])] = f0_cons['Остаток прогноз Кол-во'+str(cols_bz_col_vo[ff][9:])].values+f0_cons[i].values-f0_cons['ПП кол-во'+str(i[9:])].values
        
        

            ff+=1
            
        for i in cols_ost_prognoz_summa[current_date.month:]:
             f0_cons[i] =    f0_cons.apply(lambda x: x['Остаток прогноз Кол-во'+str(i[21:])]*(x[str(current_date.month)+'/'+str(current_date.year)+' Остаток Сумма']/  x[str(current_date.month)+'/'+str(current_date.year)+' Остаток Кол-во']) if x[str(current_date.month)+'/'+str(current_date.year)+' Остаток Кол-во']>0 else x['Остаток прогноз Кол-во'+str(i[21:])]*x['Цена'] ,axis = 1)

    # сумма бз    
    f0_cons.loc[:,cols_bz_summ]=f0_cons.loc[:,cols_bz_col_vo].mul(f0_cons['Цена'],axis = 0).values  
    f0_cons['ИТОГО БЗ Сумма'] = f0_cons.loc[:,cols_bz_summ].sum(axis=1)
    f0_cons['ИТОГО БЗ Кол-во'] = f0_cons.loc[:,cols_bz_col_vo].sum(axis=1)
    

    
    f0_cons['Категория ТМЦ'] = f0_cons['Категория ТМЦ'] .astype('category')
    f0_cons['Категория ТМЦ'].cat.set_categories(['Текущая потребность', 'Без ПП', 'Проект', 'ТНСЗ', 'ГО','Неликвиды', 'Спецодежда','страховой запас'],inplace = True)

    f0_cons = f0_cons.sort_values('Категория ТМЦ')
    f0_cons['Категория ТМЦ'] = f0_cons['Категория ТМЦ'].astype('str')
    f0_cons.fillna(0,inplace=True)
    f0_cons = f0_cons[f0_cons['Материал'].apply(lambda x:len(x)<=10)]
    return f0_cons

In [ ]:
# current_date = datetime.datetime(2020,1,1)
# first_month_of_kvartal = 1
# для всех месяцев
# c_f0 =  f0_consolidation(current_date=current_date_f0,df=f0,col=col,ind=ind,first_month_of_kvartal = first_month_of_kvartal,ost_nach_perioda=ost_nach_perioda.drop_duplicates(),priznak_raschet_nachalo_goda = False,sp = sp)
# первый месяц когда рассчет основывается на оатсках на начало года
c_f0 =  f0_consolidation(current_date=current_date_f0,df=f0,col=col,ind=ind,first_month_of_kvartal = first_month_of_kvartal,ost_nach_perioda=ost_nach_perioda.drop_duplicates(),priznak_raschet_nachalo_goda = True,sp = sp)


In [ ]:
ZNB1

In [ ]:
yy = ZNB1[(ZNB1['Материал']=='501001006')]

In [ ]:
yy['Склад']

In [ ]:
df_str_zap[df_str_zap['Материал']=='501001006']['Номер элемента пополнения'].unique()

In [ ]:
ZNB1['Комплекс']=ZNB1['Завод+Склад'].apply(lambda x:  insert_vpr(x,spr_znb_,'Комплекс'))

In [ ]:
list(spr_znb_.keys())

In [ ]:
tttt = pd.read_excel(directory+'ZNB.xlsx',sheet_name='Sheet1')

In [ ]:
ZNB1 = pd.read_excel(directory+'ZNB.xlsx',sheet_name='Sheet1')
UB05 = pd.read_excel(directory+'UB05.xlsx',sheet_name='Sheet1')
spr_znb = pd.read_excel(directory+'Справодник для ZNB.xlsx',sheet_name='ZNB')
ZNB1.fillna(0,inplace = True)
ZNB1['Материал'] = ZNB1['Материал'].apply(lambda x: str(x).replace('.0','')).astype('str')
ZNB1['Материал'] = ZNB1['Материал'].apply(lambda x: '0'+str(x) if  len(str(x))<9 else x)
ZNB1['Документ закупки'] = ZNB1['Документ закупки'].apply(lambda x: str(x).replace('.0','')).astype('str')
ZNB1['Позиция'] = ZNB1['Позиция'].apply(lambda x: str(x).replace('.0','')).astype('str')

In [ ]:
ZNB1[(ZNB1['Материал']=='501001006')]['Склад']

In [ ]:
ZNB1[(ZNB1['Материал']=='501001006')]['Завод+Склад'][99038]

In [ ]:
spr_nb_

In [ ]:
NB['Комплекс'].unique() in 

In [ ]:
0 in NB['Комплекс'].unique()
'0' in NB['Комплекс'].unique()
0 in ZNB1['Комплекс'].unique()
'0' in ZNB1['Комплекс'].unique()
0 in ZNB['Комплекс'].unique()
'0' in ZNB['Комплекс'].unique()

In [ ]:
ZNB1[(ZNB1['Комплекс']==0)]

In [ ]:
save_pyex(df=ZNB1[(ZNB1['Комплекс']==0)&(ZNB1['Количество пополнения']>0)],path='C:\\Users\\Илья\\Desktop\\kazzinc\\Uploads1\\extr_xlsx\\111.xlsx')

In [ ]:
df = get_in_backup(path_backup='tto.h5',name='01')

In [ ]:
potr_with_ostatok_total

In [ ]:
df

In [ ]:
save_pyex(df=df_str_zap[df_str_zap['ff']!=0],path='C:\\Users\\Илья\\Desktop\\kazzinc\\Uploads1\\extr_xlsx\\111.xlsx')

In [ ]:
[x for x in ZNB1['Номер элемента пополнения + Номер позиции пополнения + Наименование типа элемента ППМ пополн'].unique() if str(x).find('ПртПЗ')!=-1]

In [ ]:
df_str_zap[(df_str_zap['Вывод по документам потребности']!='OK')|(df_str_zap['Вывод по документам пополнения']!='OK')]

In [ ]:
Rxport = pd.read_excel('C:\\Users\\Илья\\Desktop\\kazzinc\\EXPORT.xlsx',sheet_name='Sheet1')

In [ ]:
Rxport.fillna(0,inplace = True)

In [ ]:
for col in Rxport.loc[:,'Склад потребности':].columns:
    Rxport[col] = Rxport[col].apply(lambda x: str(x).replace('.0','')).astype('str')

In [ ]:
Rxport['Завод'] = Rxport['Завод'].apply(lambda x: str(x).replace('.0','')).astype('str')

In [ ]:
dictionary = {}
for ind in Rxport.index:
    
    for col in Rxport.loc[:,'Склад потребности':].columns:
        if Rxport.loc[ind,col]!='0':
            dictionary[Rxport.loc[ind,'Завод']+' '+Rxport.loc[ind,col]] = Rxport.loc[ind,1]
df_str_zap['ff'] = df_str_zap['Завод + Склад'].apply(lambda x:  insert_vpr_(x,dictionary))

In [ ]:
Rxport['Склад потребности.1'][1]

In [ ]:
dictionary = {}
for ind in Rxport.index:
    
    for col in Rxport.loc[:,'Склад потребности':].columns:
        dictionary[Rxport.loc[ind,'Завод']+' '+Rxport.loc[ind,col]] = Rxport.loc[ind,1]
df_str_zap['ff'] = df_str_zap['Завод + Склад'].apply(lambda x:  insert_vpr_(x,dictionary))

In [ ]:
dictionary

In [ ]:
df_str_zap['ff'] = df_str_zap['Завод + Склад'].apply(lambda x:  insert_vpr_(x,dictionary))

In [ ]:
df_str_zap.shape

In [ ]:
def insert_vpr_(x,dictt):
    try:
        return dictt[x]
    except KeyError:
        return 0

In [ ]:
df_str_zap[df_str_zap['ff']!=0]

In [ ]:
a = get_in_backup(path_backup='backup_f0.h5',name='c_f0_01_ost')

In [ ]:
a[['Модуль','Материал','Комплекс','Завод','КрТекстМатериала','Базовая ЕИ','Цена','пп','Остаток на начало года Кол-во','Остаток на начало года Сумма','Товар в пути (ZNB5, ZNB2/3-АвизОт)','SS (норма запаса на складе)','БЗ','прогнозн ост]]

In [ ]:
20 год
>20 года

In [ ]:
a.reset_index(inplace=True,drop = True)

In [ ]:
a.shape

In [ ]:
a[['Модуль','Материал','Комплекс','Завод','КрТекстМатериала','Базовая ЕИ','Цена','Остаток на начало года Кол-во','Остаток на начало года Сумма','Товар в пути (ZNB5, ZNB2/3-АвизОт)','SS (норма запаса на складе)']]

In [ ]:
a.loc[:,'ПП кол-во 1/2020':'ПП сумма 12/2020']

In [ ]:
a.loc[:,'БЗ кол-во 1/2020':'Остаток прогноз Сумма 12/2020']

In [ ]:
df_for_bi = pd.concat([a[['Модуль','Материал','Комплекс','Завод','КрТекстМатериала','Базовая ЕИ','Цена','Остаток на начало года Кол-во','Остаток на начало года Сумма','Товар в пути (ZNB5, ZNB2/3-АвизОт)','SS (норма запаса на складе)']],\
          a.loc[:,'ПП кол-во 1/2020':'ПП сумма 12/2020'],a.loc[:,'БЗ кол-во 1/2020':'Остаток прогноз Сумма 12/2020']],axis = 1)

In [ ]:
aggregate= dict((x,'sum') for x in df_for_bi.columns if df_for_bi[x].dtype=='float64')

In [ ]:
aggregate['Цена'] = 'max'

In [ ]:
[x for x in df_for_bi.columns if  df_for_bi[x].dtype!='float64']

In [ ]:
df_for_bi.groupby([x for x in df_for_bi.columns if  df_for_bi[x].dtype!='float64']).agg(aggregate).reset_index()

In [ ]:
yyyyy = df_for_bi.groupby([x for x in df_for_bi.columns if  df_for_bi[x].dtype!='float64']).agg(aggregate).reset_index()

In [ ]:
yyyyy[yyyyy['Материал']=='035001034']

In [ ]:
save_pyex(df=df_for_bi.groupby([x for x in df_for_bi.columns if  df_for_bi[x].dtype!='float64']).agg(aggregate).reset_index(),path='C:\\Users\\Илья\\Desktop\\kazzinc\\Uploads1\\extr_xlsx\\ff.xlsx')

In [ ]:
df = get_in_backup(path_backup='backup.h5',name='extract')

In [ ]:
df1 = df[df['Дата потребности'].dt.year>2020]

In [ ]:
df1['Дата потребности'] = df1['Дата потребности'].apply(lambda x :datetime.datetime(x.year,x.month,1))

In [ ]:
datetime.datetime(2020,1,1)

In [ ]:
di = {}

In [ ]:
['Модуль', 'Материал', 'Комплекс', 'Завод', 'КрТекстМатериала', 'Базовая ЕИ']

In [ ]:
df1[['Модуль', 'Материал', 'Комплекс', 'Завод', 'КрТекстМатериала', 'Базовая ЕИ','Дата потребности','Количество потребности','Цена']].groupby(['Модуль', 'Материал', 'Комплекс', 'Завод', 'КрТекстМатериала', 'Базовая ЕИ','Дата потребности']).\
agg({'Количество потребности':'sum','Цена':'max'}).reset_index()

In [ ]:
save_pyex(df=df1[['Модуль', 'Материал', 'Комплекс', 'Завод', 'КрТекстМатериала', 'Базовая ЕИ','Дата потребности','Количество потребности','Цена']].groupby(['Модуль', 'Материал', 'Комплекс', 'Завод', 'КрТекстМатериала', 'Базовая ЕИ','Дата потребности']).\
agg({'Количество потребности':'sum','Цена':'max'}).reset_index(),path='C:\\Users\\Илья\\Desktop\\kazzinc\\Uploads1\\extr_xlsx\\ff1.xlsx')

In [ ]:
directory_mc = 'C:\\Users\\Илья\\Desktop\\gg\\Расход\\'
rash = exlsx_f(directory_mc)

In [ ]:
for ind, i in enumerate(np.array_split(potr_with_ostatok_total,3)) :
    save_pyex(df=i,path='C:\\Users\\Илья\\Desktop\\kazzinc\\Uploads1\\extr_xlsx\\'+str(ind)+'t.xlsx')

In [ ]:
potr_with_ostatok_total = get_in_backup(path_backup='backup_for_powerBI_test_no.h5',name='f0_vertical_01')

In [ ]:
rash.fillna('',inplace=True)

In [ ]:
rash

In [ ]:
r = pd.read_excel('E:\\Расход 2019.xlsx',sheet_name='sheet1')

In [ ]:
fffff = r[['Комплекс', 'Модуль', 'Завод', 'Материал', 'Кол-во в ЕИ ввода','Сумма во ВВ','name_file']]

In [ ]:
yyy = pd.pivot_table(fffff,columns=['name_file'],index=['Комплекс', 'Модуль', 'Завод', 'Материал'],aggfunc='sum',fill_value=0).reset_index()

In [ ]:
[x[0]+' '+str(x[1]) for x in yyy.columns]

In [ ]:
yyy.columns = [x[0]+' '+str(x[1]) for x in yyy.columns]

In [ ]:
yyy

In [ ]:
save_pyex(df=ost_copy1,path='C:\\Users\\Илья\\Desktop\\kazzinc\\Uploads1\\extr_xlsx\\ost.xlsx')

In [ ]:
aa = get_in_backup(path_backup='backup_for_powerBI_test_no.h5',name='f0_vertical_01')

In [ ]:
aa[:150000]

In [ ]:
df_str_zap = get_in_backup(path_backup='backup.h5',name='df_str_zap')

In [ ]:
df = get_in_backup(path_backup='backup.h5',name='extract')

In [ ]:
df_str_zap = df.copy()

In [ ]:
directory = 'C:\\Users\\Илья\\Desktop\\kazzinc\\Uploads1\\'
f = pd.read_csv(directory+'CSV.csv',sep = ';',parse_dates=['Дата потребности'],dtype={'Материал':'str','Завод':'str'})

In [ ]:
f.shape

In [ ]:
df_str_zap.shape

In [ ]:
ind = df['Дата потребности'].apply(lambda x:True if x.year==2020 else False)

In [ ]:
df_str_zap.loc[ind].shape

In [ ]:
df.loc[ind].shape

In [ ]:
df = df_str_zap[(df_str_zap['ЦС/ПЗ']=='ПЗ')&(df_str_zap['Вывод по документам потребности']=='OK')&(df_str_zap['Баланс (закрытие потребности)']==0)&(df_str_zap['Тип ППМ']=='PD')]
df = df[df['Материал'].apply(lambda x:len(x)<=10)]

In [ ]:
ost_copy1

In [ ]:
df = get_in_backup(path_backup='backup.h5',name='extract')

In [ ]:
df.info(verbose  = True)

In [ ]:
df['Дата потребности'].astype('str')

In [ ]:
df['Завод получатель'].unique()

In [ ]:
df.columns

In [ ]:
l = [x for x in df.columns if df[x].dtype=='float64' or df[x].dtype=='int64']
to_str_df = [x for x in l if x not in['Количество пополнения','Количество потребности','Страховой запас','Цена']]
for i in to_str_df:
    df[i] = df[i].apply(lambda x: str(x).replace('.0','')).astype('str')
    df[i] = df[i].apply(lambda x: '' if x=='0' else x)

In [ ]:

# df['Дата потребности'] = [datetime.datetime(int(x.split('.')[2]),int(x.split('.')[1]),int(x.split('.')[0])) for x in df['Дата потребности'].values]

for i in [x for x in df.columns if x.find('Дата')!=-1]:
    df[i] = df[i].apply(lambda x: '01.01.1900' if x =='00.00.0000' or x ==0 or x=='0' else  x)
    if df[i].dtype=='object':
        df[i] = [datetime.datetime(int(x.split('.')[2]),int(x.split('.')[1]),int(x.split('.')[0])) for x in df[i].values]
    df[i] = df[i].apply(lambda x :datetime.datetime(x.year,x.month,1))

In [ ]:
df.drop(['Дата потребности'],axis = 1,inplace = True)
df.rename(columns={"month": "Дата потребности"},inplace=True)
dt = [x for x in df.columns if df[x].dtype=='datetime64[ns]']

for i in dt:
    df[i] = df[i].astype('str')

In [ ]:
aggregate= dict((x,'sum') for x in df.columns if df[x].dtype=='float64' and x!='Цена')
aggregate['Цена'] = ['max']
df.drop([x for x in df.columns if x.find('Unnamed')!=-1],axis = 1,inplace=True)

In [ ]:
aggregate

In [ ]:
aa = df.groupby([x for x in df.columns if df[x].dtype=='object']).agg(aggregate).reset_index()

In [ ]:
aa = df.groupby([x for x in df.columns if df[x].dtype=='object']).agg(aggregate).reset_index()

In [ ]:
aa.shape

In [ ]:
df.shape

In [ ]:
df[df['Материал']=='230301041'].shape

In [ ]:
aa[aa['Материал']=='230301041'].shape

In [ ]:
aa[aa['Материал']=='060102003'].shape

In [ ]:
df[df['Материал']=='060102003'].shape

In [ ]:
aa[aa['Материал']=='060102003']['Unnamed: 109']

In [ ]:
aa

In [ ]:
for ind, i in enumerate(np.array_split(df,3)) :
    save_pyex(df=i,path='C:\\Users\\Илья\\Desktop\\kazzinc\\Uploads1\\extr_xlsx\\'+str(ind)+'.xlsx')

In [ ]:
for ind, i in enumerate(np.array_split(aa,3)) :
    save_pyex(df=i,path='C:\\Users\\Илья\\Desktop\\kazzinc\\Uploads1\\extr_xlsx\\'+str(ind+10)+'.xlsx')

In [ ]:
aa.columns = [x[0] for x in aa.columns]

In [ ]:
test1 = df[['Материал','Количество потребности']].copy()

In [ ]:
sp_mat = pd.DataFrame(test1['Материал'].unique())

In [ ]:
sp_mat['ind'] = [x for x in range(sp_mat.shape[0])]

In [ ]:
sp_mat = sp_mat.set_index(0)

In [ ]:
dd = sp_mat.to_dict()['ind']

In [ ]:
test1['Материал'] = test1['Материал'].apply(lambda x:dd[x] )


In [ ]:
test1['cdfsc'] = 0

In [ ]:
matr = test1.values

In [ ]:
sp_mat.to_dict()[0]

In [ ]:
matr[:,3] = 0

In [ ]:
start_time = time.time()
for i in range(len(matr)):
    matr[matr[:,0]==5]
print("--- %s seconds ---" % (time.time() - start_time))

In [ ]:
len(matr)

In [ ]:
start_time = time.time()
for i in range(len(test1)):
    test1[test1['Материал']=='260101004']
print("--- %s seconds ---" % (time.time() - start_time))

In [ ]:
test1

In [ ]:
pd.DataFrame(matr) 

In [ ]:
a = np.array([[1, 2, 3], [4, 5, 6]]) 

In [ ]:
b = np.where(a<4) 
print(b)

In [ ]:
b

In [ ]:
a[a[:,0]==1]

In [ ]:
dd

In [ ]:
matr

In [ ]:
test1

In [ ]:
start_time = time.time()
matr[matr[:,0]==5]
print("--- %s seconds ---" % (time.time() - start_time))

In [ ]:
start_time = time.time()
test1[test1['Материал']==5]
print("--- %s seconds ---" % (time.time() - start_time))

In [ ]:
from multiprocessing import Array

In [ ]:
Array('df',df,lock=True)

In [ ]:
# управление механикой остатков(вызывать ее)
def ostat(ost_st, potr_st, current_date, filters,all_months):
    # master_cs_pz = dict(tuple(potr_st[potr_st['month'] > current_date].groupby('ЦС/ПЗ')))
    slave_cs_pz = dict(tuple(ost_st.groupby('ЦС/ПЗ')))
    
  
    
    # пз
    pz_ost = ost_st[ost_st['ЦС/ПЗ']=='ПЗ']
    
    for pz_slave_row in pz_ost.index:
        
        # df_master = ret_data_dict(pz_ost.at[pz_slave_row, 'Завод+НН'], master_zav_nn)
        if all_months ==True:
            df_master = potr_st[(potr_st['Завод+НН'] == pz_ost.at[pz_slave_row, 'Завод+НН'])&(potr_st['month'] >= current_date)]
            
        else:   
            df_master = potr_st[(potr_st['Завод+НН'] == pz_ost.at[pz_slave_row, 'Завод+НН'])&(potr_st['month'] > current_date)]

        if len(df_master) > 0 and pz_ost.at[pz_slave_row, 'осталось'] > 0:
            df_master = df_master.sort_values('Дата потребности')
            # функция
            # print(df_master[['Количество потребности', 'Дата потребности', 'Распределение_остатака']])
            mex_ost(df_master, pz_ost, pz_slave_row, ost_st, potr_st)
            # print(df_master[['Количество потребности', 'Дата потребности', 'Распределение_остатака']])

    # вторая часть

    slave_cs_pz = dict(tuple(ost_st.groupby('ЦС/ПЗ')))

    # master_zak_nn = dict(tuple(potr_st[potr_st['month'] > current_date].groupby('Заказчик + HH')))
    #
    # master_material = dict(tuple(potr_st[potr_st['month'] > current_date].groupby('Материал')))

    pz_ost = ost_st[ost_st['ЦС/ПЗ']=='ЦС']
    for pz_slave_row in pz_ost.index:
        # по номенклатуре ищем только заказчика all 1000

        if ost_st.at[pz_slave_row, 'Комплекс'] == 'ALL1000':
            # df_master = ret_data_dict(pz_ost.at[pz_slave_row, 'Материал'], master_material)
            
            if all_months==True:
                 df_master = potr_st[(potr_st['Материал']==pz_ost.at[pz_slave_row, 'Материал'])&(potr_st['month'] >= current_date)]
            else:
                
                df_master = potr_st[(potr_st['Материал']==pz_ost.at[pz_slave_row, 'Материал'])&(potr_st['month'] > current_date)]
            if len(df_master) > 0 and pz_ost.at[pz_slave_row, 'осталось'] > 0:
                # отсекаем только all1000 и сортируем

                df_master = df_master[df_master['Комплекс'].isin(filters['one'])].sort_values('Дата потребности')
                # print(df_master[['Количество потребности', 'Дата потребности', 'Распределение_остатака']])
                mex_ost(df_master, pz_ost, pz_slave_row, ost_st, potr_st)
                # print(df_master[['Количество потребности', 'Дата потребности', 'Распределение_остатака']])


                a = 0
        # по заказчик +нн
        else:
            
            if all_months==True:
                df_master = potr_st[(potr_st['Заказчик + HH'] == pz_ost.at[pz_slave_row, 'Заказчик + HH'])&(potr_st['month'] >= current_date)]
            else:
                df_master = potr_st[(potr_st['Заказчик + HH'] == pz_ost.at[pz_slave_row, 'Заказчик + HH'])&(potr_st['month'] > current_date)]
            if len(df_master) > 0 and pz_ost.at[pz_slave_row, 'осталось'] > 0:
                df_master = df_master.sort_values('Дата потребности')
                # print(df_master[['Количество потребности', 'Дата потребности', 'Распределение_остатака']])
                mex_ost(df_master, pz_ost, pz_slave_row, ost_st, potr_st)
                # print(df_master[['Количество потребности', 'Дата потребности', 'Распределение_остатака']])

#     potr_st.to_csv('C:\\Users\\User\\Desktop\\kazzinc\\Uploads\\test\\dfsl.csv', encoding='utf-8-sig',sep=';')
#     ost_st.to_csv('C:\\Users\\User\\Desktop\\kazzinc\\Uploads\\test\\ost.csv', encoding='utf-8-sig', sep=';')


# мехника распределения остатков
def mex_ost(df_master, pz_ost, pz_slave_row, ost_st, potr_st):
    ostatok = pz_ost.at[pz_slave_row, 'осталось']
    
    len_it = len(df_master) - 1
    for ind, row_master_index in enumerate(df_master.index):
        # print(df_master[['Количество потребности', 'Дата потребности', 'Распределение_остатака']])
        # если покрыта вся потребность то ничего не делаем
        #         if df_master.at[row_master_index, 'Количество потребности'] == df_master.at[row_master_index, 'Распределение_остатака']:
        #             pass
        # если это последний элемент нужно записать то что осталось в остатке
        #  1 прибавляем в столбец Распределение_остатака гласного фрейма все то что осталось
        #  2 вставляем в лист остатков в столбец забрали
        #  3 вставлем в маленькие выборки аналогично
        #  4 рассчитываем сумму по цене и прибавляем к имеющейся в ячейке
        if len_it == ind:
            price = ost_st.at[pz_slave_row, 'Стоимость свободного остатка'] / ost_st.at[pz_slave_row, 'Cвободный запас']
            summa = ostatok * price

            potr_st.at[row_master_index, 'Сумма_распреденного_остатка'] = potr_st.at[
                                                                              row_master_index, 'Сумма_распреденного_остатка'] + summa

            
            potr_st.at[row_master_index, 'Распределение_остатака'] = potr_st.at[
                                                                         row_master_index, 'Распределение_остатака'] + ostatok
            
            pz_ost.at[pz_slave_row, 'осталось'] = 0
            ost_st.at[pz_slave_row, 'осталось'] = 0
            # print(df_master[['Количество потребности','Дата потребности','Распределение_остатака']])
            ostatok = 0
            return
        # если покарыта не вся потребность  и не последняя строка тогда и не хватает чтобы закрыть потребность:
        #  1 прибавляем в столбец  Распределение_остатака гласного фрейма потребностей только остаток
        #  2 зануляем остаток
        #  3 зануляем в листе остатков в столбец осталось
        #  4 вставлем в маленькие выборки аналогично
        #  5 рассчитываем сумму по цене и прибавляем к имеющейся в ячейке
        elif (potr_st.at[row_master_index, 'Количество потребности'] > potr_st.at[row_master_index, 'Распределение_остатака']) and (
                potr_st.at[row_master_index, 'Количество потребности'] >= ostatok):
            if ostatok+potr_st.at[row_master_index, 'Распределение_остатака']>potr_st.at[row_master_index, 'Количество потребности']:
                a=0
                price = ost_st.at[pz_slave_row, 'Стоимость свободного остатка'] / ost_st.at[pz_slave_row, 'Cвободный запас']
                vzyali = potr_st.at[row_master_index, 'Количество потребности']-potr_st.at[row_master_index, 'Распределение_остатака']
                summa = vzyali * price

                potr_st.at[row_master_index, 'Сумма_распреденного_остатка'] = potr_st.at[
                                                                                  row_master_index, 'Сумма_распреденного_остатка'] + summa

                
                potr_st.at[row_master_index, 'Распределение_остатака'] = potr_st.at[
                                                                             row_master_index, 'Распределение_остатака'] + vzyali

                pz_ost.at[pz_slave_row, 'осталось'] = pz_ost.at[pz_slave_row, 'осталось']-vzyali
                ost_st.at[pz_slave_row, 'осталось'] = ost_st.at[pz_slave_row, 'осталось']-vzyali
                ostatok =  ostatok -vzyali

            else:
                price = ost_st.at[pz_slave_row, 'Стоимость свободного остатка'] / ost_st.at[pz_slave_row, 'Cвободный запас']
                summa = ostatok * price
                potr_st.at[row_master_index, 'Сумма_распреденного_остатка'] = potr_st.at[
                                                                                  row_master_index, 'Сумма_распреденного_остатка'] + summa
                
                potr_st.at[row_master_index, 'Распределение_остатака'] = potr_st.at[row_master_index, 'Распределение_остатака'] + ostatok
                
                pz_ost.at[pz_slave_row, 'осталось'] = 0
                ost_st.at[pz_slave_row, 'осталось'] = 0
                ostatok = 0
                # print(df_master[['Количество потребности', 'Дата потребности', 'Распределение_остатака']])
                return

            # если покарыта не вся потребность  и не последняя строка тогда и хватает закрыть потребность :
            #  1 прибавляем в столбец  Распределение_остатака гласного фрейма потребностей только потребность
            #  2 отнимаем от остатка потребность
            #  3 отнимаем в листе остатков в столбец осталось
            #  4 вставлем в маленькие выборки аналогично
            #  5 рассчитываем сумму по цене и прибавляем к имеющейся в ячейке
        elif (potr_st.at[row_master_index, 'Количество потребности'] > potr_st.at[row_master_index, 'Распределение_остатака']) and (potr_st.at[row_master_index, 'Количество потребности'] < ostatok):
            price = ost_st.at[pz_slave_row, 'Стоимость свободного остатка'] / ost_st.at[pz_slave_row, 'Cвободный запас']
            vzyali = potr_st.at[row_master_index, 'Количество потребности'] - potr_st.at[row_master_index, 'Распределение_остатака']
            summa = vzyali * price

            
            
            potr_st.at[row_master_index, 'Сумма_распреденного_остатка'] = potr_st.at[
                                                                              row_master_index, 'Сумма_распреденного_остатка'] + summa


           
            
            potr_st.at[row_master_index, 'Распределение_остатака'] = potr_st.at[row_master_index, 'Распределение_остатака'] + (potr_st.at[row_master_index, 'Количество потребности'] -potr_st.at[row_master_index, 'Распределение_остатака'])
            
            pz_ost.at[pz_slave_row, 'осталось'] = pz_ost.at[pz_slave_row, 'осталось'] - vzyali
            ost_st.at[pz_slave_row, 'осталось'] = ost_st.at[pz_slave_row, 'осталось'] - vzyali
            ostatok = ostatok - vzyali
            # print(df_master[['Количество потребности', 'Дата потребности', 'Распределение_остатака']])

In [ ]:
# управление механикой остатков(вызывать ее)
def ostat(ost_st, potr_st, current_date, filters,all_months,col_name_0,col_name_1,col_name_2,col_name_3,col_name_4,col_name_5):
    
       
    
#     cons = potr_st.sort_values(['Область ППМ','сортировка по модулю'])
    pz_ost = ost_st[ost_st['ЦС/ПЗ']=='ПЗ']
    cons = potr_st.set_index(['Завод+НН','del'])
    
    if all_months ==True:
        dat = cons[cons['month']>=current_date].sort_values('Дата потребности')
                
    else:   
        dat = cons[cons['month']>current_date].sort_values('Дата потребности')
    
    # 15,2,1
    for pz_slave_row in pz_ost[pz_ost[col_name_0]!=0].index:
#         print(pz_ost.loc[pz_slave_row,'Материал'])
        try: 
            df_master = dat.xs(pz_ost.loc[pz_slave_row,'Завод+НН'])
        except KeyError:
            df_master = pd.DataFrame()
        
        if len(df_master)>0:
            mex_ost(df_master, pz_ost, pz_slave_row, ost_st, potr_st,col_name_0,col_name_1,col_name_2,col_name_3,col_name_4,col_name_5)

            
            
    pz_ost = ost_st[ost_st['ЦС/ПЗ']=='ЦС']
    cons = potr_st.reset_index().set_index(['Материал','Заказчик + HH','del'])
    
    if all_months ==True:
        dat = cons[cons['month']>=current_date].sort_values('Дата потребности')
                
    else:   
        dat = cons[cons['month']>current_date].sort_values('Дата потребности')
    
    for pz_slave_row in pz_ost[pz_ost[col_name_0]!=0].index:
        if ost_st.at[pz_slave_row, 'Комплекс'] == 'ALL1000':
            
            try:
                df_master = dat.xs(pz_ost.at[pz_slave_row, 'Материал'])
            except KeyError:
                df_master = pd.DataFrame()    
               
            if len(df_master) > 0:
                
                # отсекаем только all1000 и сортируем
                df_master = df_master[df_master['Комплекс'].isin(filters['one'])].sort_values('Дата потребности')
                mex_ost(df_master, pz_ost, pz_slave_row, ost_st, potr_st,col_name_0,col_name_1,col_name_2,col_name_3,col_name_4,col_name_5)
                
        else:
            
            try:
                df_master = dat.xs(pz_ost.at[pz_slave_row, 'Заказчик + HH'],level = 1)
            except KeyError:
                df_master = pd.DataFrame()    
            
            
            if len(df_master) > 0:
                
                df_master = df_master.sort_values('Дата потребности')
                
                mex_ost(df_master, pz_ost, pz_slave_row, ost_st, potr_st,col_name_0,col_name_1,col_name_2,col_name_3,col_name_4,col_name_5)
                      
                 
            #             распределяем
             
        
    
   
    

# мехника распределения остатков 
# col_name_0 название столбца откуда рпспределяем
# col_name_1 навание с толбца куда распределяем
# col_name_2 название столбца с количеством потребности
# col_name_5 название столбца куда распределяем сумму
# Для цены:
# col_name_3 название столбца с суммой потребности
# col_name_4 название столбца с начальным кол-вом  потребности
def mex_ost(df_master, pz_ost, pz_slave_row, ost_st, potr_st,col_name_0,col_name_1,col_name_2,col_name_3,col_name_4,col_name_5):
    df_master = df_master.reset_index().set_index(['index'])
#     print(df_master.index)
    ostatok = pz_ost.at[pz_slave_row, col_name_0]
    len_it = len(df_master) - 1
    for ind, row_master_index in enumerate(df_master.index):
#         print(df_master)
        if len_it == ind:
            if pz_ost.at[pz_slave_row, col_name_3]==0 or pz_ost.at[pz_slave_row, col_name_4]==0:
                price = 0
            else:
                price = pz_ost.at[pz_slave_row, col_name_3]/pz_ost.at[pz_slave_row, col_name_4]
            summa = ostatok*price
            
            potr_st.at[row_master_index, col_name_5] = potr_st.at[row_master_index, col_name_5] + summa
            potr_st.at[row_master_index, col_name_1] = potr_st.at[row_master_index, col_name_1] + ostatok

            pz_ost.at[pz_slave_row, col_name_0] = 0
            ost_st.at[pz_slave_row, col_name_0] = 0
            # print(df_master[['Количество потребности','Дата потребности','Распределение_остатака']])
            ostatok = 0
            return
        
        elif (potr_st.at[row_master_index, col_name_2] > potr_st.at[row_master_index, col_name_1]) and (
                potr_st.at[row_master_index, col_name_2] >= ostatok):
            if ostatok+potr_st.at[row_master_index, col_name_1]>potr_st.at[row_master_index, col_name_2]:
                a=0
                if pz_ost.at[pz_slave_row, col_name_3]==0 or pz_ost.at[pz_slave_row, col_name_4]==0:
                    price = 0
                else:    
                    price = pz_ost.at[pz_slave_row, col_name_3]/pz_ost.at[pz_slave_row, col_name_4]
                
                vzyali = potr_st.at[row_master_index, col_name_2]-potr_st.at[row_master_index, col_name_1]
                
                summa = vzyali*price
                
                potr_st.at[row_master_index, col_name_5] = potr_st.at[row_master_index, col_name_5] + summa
                potr_st.at[row_master_index, col_name_1] = potr_st.at[row_master_index, col_name_1] + vzyali

                pz_ost.at[pz_slave_row, col_name_0] = pz_ost.at[pz_slave_row, col_name_0]-vzyali
                ost_st.at[pz_slave_row, col_name_0] = ost_st.at[pz_slave_row, col_name_0]-vzyali
                ostatok =  ostatok -vzyali

            else:
                if pz_ost.at[pz_slave_row, col_name_3]==0 or pz_ost.at[pz_slave_row, col_name_4]==0:
                    price = 0
                else:
                    price = pz_ost.at[pz_slave_row, col_name_3]/pz_ost.at[pz_slave_row, col_name_4]
                summa = ostatok*price
                potr_st.at[row_master_index, col_name_5] = potr_st.at[row_master_index, col_name_5] + summa
                potr_st.at[row_master_index, col_name_1] = potr_st.at[row_master_index, col_name_1] + ostatok

                pz_ost.at[pz_slave_row, col_name_0] = 0
                ost_st.at[pz_slave_row, col_name_0] = 0
                ostatok = 0
                # print(df_master[['Количество потребности', 'Дата потребности', 'Распределение_остатака']])
                return

        elif (potr_st.at[row_master_index, col_name_2] > potr_st.at[row_master_index, col_name_1]) and (potr_st.at[row_master_index, col_name_2] < ostatok):
            if pz_ost.at[pz_slave_row, col_name_3]==0 or pz_ost.at[pz_slave_row, col_name_4]==0:
                price = 0
            else:
                price = pz_ost.at[pz_slave_row, col_name_3]/pz_ost.at[pz_slave_row, col_name_4]
            
            vzyali = potr_st.at[row_master_index, col_name_2]-potr_st.at[row_master_index, col_name_1]
            
            summa = vzyali*price
            potr_st.at[row_master_index, col_name_5] = potr_st.at[row_master_index, col_name_5] + summa
            potr_st.at[row_master_index, col_name_1] = potr_st.at[row_master_index, col_name_1] + (potr_st.at[row_master_index, col_name_2] -potr_st.at[row_master_index, col_name_1])

            pz_ost.at[pz_slave_row, col_name_0] = pz_ost.at[pz_slave_row, col_name_0] - vzyali
            ost_st.at[pz_slave_row, col_name_0] = ost_st.at[pz_slave_row, col_name_0] - vzyali
            ostatok = ostatok - vzyali
            # print(df_master[['Количество потребности', 'Дата потребности', 'Распределение_остатака']])

In [ ]:
df

In [ ]:
df['del'] = ''
cons = df.set_index(['Материал','Комплекс','Заказчик + HH','del'])
dat =  cons[cons['month']>current_date]

In [ ]:
cons.xs(('УПРАВЛЕНИЕ КАЗЦИНК 200700003'),level=2)

In [ ]:
cons

In [ ]:
dat.xs(['1101 #230301042'])

In [ ]:
df = get_in_backup(path_backup='backup.h5',name='extract')

In [ ]:
cons

In [ ]:
start_time = time.time()
ostatok_with_block.rename(columns={"материал с учетом аналогов": 'Материал','измененная партия':'Партия'},inplace=True)
potr_copy = cons.copy()
ost_copy = ostatok_with_block.copy()
# Запуск распределения остатков
potr_copy['Распределение_остатака  на начало'] = 0.000000
potr_copy['Сумма Распределение_остатака  на начало'] = 0.000000
potr_copy['index'] = [x for x in range(len(potr_copy))]

ost_copy['осталось'] = ost_copy['Кол-во запаса в начале периода с учетом исключения блокированного запаси и особого запаса']

ostat(ost_st=ost_copy,potr_st=potr_copy,col_name_0='осталось',col_name_1='Распределение_остатака  на начало',col_name_2 = 'Количество потребности',col_name_3 = 'Стоимость в начале периода с учетом исключения блокированного запаси и особого запаса',col_name_4 = 'Кол-во запаса в начале периода с учетом исключения блокированного запаси и особого запаса',col_name_5 = 'Сумма Распределение_остатака  на начало')
# откуда куда что
print("--- %s seconds ---" % (time.time() - start_time))

In [ ]:
df['index'] = [x for x in range(len(df))]
df['del'] = ''
df.reset_index(inplace = True,drop = True)

potr_copy = df[df['Страховой запас']>0].copy()
index___ = potr_copy[potr_copy['Страховой запас']>0].index
potr_copy.loc[index___,'Количество потребности'] = potr_copy.loc[index___,'Страховой запас']
potr_copy['Завод+НН'] = potr_copy['Завод'].astype('str')+' #'+potr_copy['Материал'].astype('str')
potr_copy['Завод+НН'] = potr_copy['Завод+НН'].str.strip()
ost_copy = ost_start_raspr_str_zap.copy()
ost_copy['осталось'] = ost_copy['страховой запас']
ost_copy['ЦС/ПЗ']='ПЗ'
potr_copy.name = 'potrebn_with_raspr_ost'
ost_copy.name =  'ost_with_potr'
# Запуск распределения остатков
potr_copy['Распределение_остатака'] = 0.000000
potr_copy['Сумма_распреденного_остатка'] = 0.000000


ostat(ost_st=ost_copy,potr_st=potr_copy,filters=filter_else,current_date = current_date,all_months=True,\
     col_name_0='осталось',col_name_1='Распределение_остатака',col_name_2 = 'Количество потребности',col_name_3 = 'Стоимость свободного остатка',col_name_4 = 'Cвободный запас',col_name_5 = 'Сумма_распреденного_остатка')    
potr_copy.loc[index___,'Количество потребности'] = 0

In [ ]:
start_time = time.time()
# Запуск распределения остатков
# date_start_year = datetime.datetime(2019,1,1)
# current_date = datetime.datetime(2019,10,1)
potr_copy1 = df[df['Страховой запас']==0].copy()
ost_copy1 = ost_start_raspr.copy()
potr_copy1.name = 'potrebn_with_raspr_ost'
ost_copy1.name =  'ost_with_potr'
# Запуск распределения остатков
potr_copy1['Распределение_остатака'] = 0.000000
potr_copy1['Сумма_распреденного_остатка'] = 0.000000

ostat(ost_st=ost_copy1,potr_st=potr_copy1,filters=filter_else,current_date = current_date,all_months=True,\
     col_name_0='осталось',col_name_1='Распределение_остатака',col_name_2 = 'Количество потребности',col_name_3 = 'Стоимость свободного остатка',col_name_4 = 'Cвободный запас',col_name_5 = 'Сумма_распреденного_остатка')
print("--- %s seconds ---" % (time.time() - start_time))

In [ ]:
potr_copy['Распределение_остатака'].sum()

In [ ]:
potr_copy1['Распределение_остатака'].sum()

In [ ]:
ost_copy1

In [ ]:
potr_copy1[potr_copy1['Распределение_остатака']>0]

In [ ]:
ost_start_raspr_str_zap

In [ ]:
pz_ost = ost_copy1[ost_copy1['ЦС/ПЗ']=='ЦС']
cons = potr_copy1.reset_index().set_index(['Материал','Заказчик + HH','del'])

In [ ]:
pz_ost

In [ ]:
cons

In [ ]:
dat.xs(['010101015'])

In [ ]:
dat.xs(['УКМК 010101015'],level = 2)

In [ ]:
dat

In [ ]:
cons.xs('УПРАВЛЕНИЕ КАЗЦИНК 200700003',level = 1)

In [ ]:
ost_copy1 = ost_start_raspr.copy()[:1000]

In [ ]:
for row in ost_copy1[:1].itertuples():
    print(row)

In [ ]:
ost_copy1

In [ ]:
# управление механикой остатков(вызывать ее)
def ostat(ost_st, potr_st, current_date, filters,all_months,col_name_0,col_name_1,col_name_2,col_name_3,col_name_4,col_name_5):
    
       
    
#     cons = potr_st.sort_values(['Область ППМ','сортировка по модулю'])
    pz_ost = ost_st[ost_st['ЦС/ПЗ']=='ПЗ']
    cons = potr_st.set_index(['Завод+НН','del'])
    
    if all_months ==True:
        dat = cons[cons['month']>=current_date].sort_values('Дата потребности')
                
    else:   
        dat = cons[cons['month']>current_date].sort_values('Дата потребности')
    
    # 15,2,1
    for pz_slave_row in pz_ost[pz_ost[col_name_0]!=0].itertuples():
#         print(pz_ost.loc[pz_slave_row,'Материал'])
        try: 
            df_master = cons.xs(pz_slave_row.Завод_НН)
        except KeyError:
            df_master = pd.DataFrame()

        if len(df_master)>0:
            pass
            
#             mex_ost(df_master, pz_ost, pz_slave_row, ost_st, potr_st,col_name_0,col_name_1,col_name_2,col_name_3,col_name_4,col_name_5)

            
            
    pz_ost = ost_st[ost_st['ЦС/ПЗ']=='ЦС']
    cons = potr_st.reset_index().set_index(['Материал','Заказчик + HH','del'])
    
    for pz_slave_row in pz_ost[pz_ost[col_name_0]!=0].itertuples():
        if pz_slave_row.Комплекс == 'ALL1000':
            
            try:
                df_master = cons.xs(pz_slave_row.Материал)
            except KeyError:
                df_master = pd.DataFrame()    
               
            if len(df_master) > 0:
                # отсекаем только all1000 и сортируем
                df_master = df_master[df_master['Комплекс'].isin(filters['one'])].sort_values('Дата потребности')
#                 mex_ost(df_master, pz_ost, pz_slave_row, ost_st, potr_st,col_name_0,col_name_1,col_name_2,col_name_3,col_name_4,col_name_5)
                
        else:
            
            try:
                df_master = cons.xs(pz_slave_row.Заказчик_HH,level = 1)
            except KeyError:
                df_master = pd.DataFrame()    
            
            
            if len(df_master) > 0:
                df_master = df_master.sort_values('Дата потребности')
                
#                 mex_ost(df_master, pz_ost, pz_slave_row, ost_st, potr_st,col_name_0,col_name_1,col_name_2,col_name_3,col_name_4,col_name_5)
                      
                 
            #             распределяем
             
        
    
   
    

# мехника распределения остатков 
# col_name_0 название столбца откуда рпспределяем
# col_name_1 навание с толбца куда распределяем
# col_name_2 название столбца с количеством потребности
# col_name_5 название столбца куда распределяем сумму
# Для цены:
# col_name_3 название столбца с суммой потребности
# col_name_4 название столбца с начальным кол-вом  потребности
def mex_ost(df_master, pz_ost, pz_slave_row, ost_st, potr_st,col_name_0,col_name_1,col_name_2,col_name_3,col_name_4,col_name_5):
    df_master = df_master.reset_index().set_index(['index'])
#     print(df_master.index)
    ostatok = pz_ost.at[pz_slave_row, col_name_0]
    len_it = len(df_master) - 1
    for ind, row_master_index in enumerate(df_master.index):
#         print(df_master)
        if len_it == ind:
            if pz_ost.at[pz_slave_row, col_name_3]==0 or pz_ost.at[pz_slave_row, col_name_4]==0:
                price = 0
            else:
                price = pz_ost.at[pz_slave_row, col_name_3]/pz_ost.at[pz_slave_row, col_name_4]
            summa = ostatok*price
            
            potr_st.at[row_master_index, col_name_5] = potr_st.at[row_master_index, col_name_5] + summa
            potr_st.at[row_master_index, col_name_1] = potr_st.at[row_master_index, col_name_1] + ostatok

            pz_ost.at[pz_slave_row, col_name_0] = 0
            ost_st.at[pz_slave_row, col_name_0] = 0
            # print(df_master[['Количество потребности','Дата потребности','Распределение_остатака']])
            ostatok = 0
            return
        
        elif (potr_st.at[row_master_index, col_name_2] > potr_st.at[row_master_index, col_name_1]) and (
                potr_st.at[row_master_index, col_name_2] >= ostatok):
            if ostatok+potr_st.at[row_master_index, col_name_1]>potr_st.at[row_master_index, col_name_2]:
                a=0
                if pz_ost.at[pz_slave_row, col_name_3]==0 or pz_ost.at[pz_slave_row, col_name_4]==0:
                    price = 0
                else:    
                    price = pz_ost.at[pz_slave_row, col_name_3]/pz_ost.at[pz_slave_row, col_name_4]
                
                vzyali = potr_st.at[row_master_index, col_name_2]-potr_st.at[row_master_index, col_name_1]
                
                summa = vzyali*price
                
                potr_st.at[row_master_index, col_name_5] = potr_st.at[row_master_index, col_name_5] + summa
                potr_st.at[row_master_index, col_name_1] = potr_st.at[row_master_index, col_name_1] + vzyali

                pz_ost.at[pz_slave_row, col_name_0] = pz_ost.at[pz_slave_row, col_name_0]-vzyali
                ost_st.at[pz_slave_row, col_name_0] = ost_st.at[pz_slave_row, col_name_0]-vzyali
                ostatok =  ostatok -vzyali

            else:
                if pz_ost.at[pz_slave_row, col_name_3]==0 or pz_ost.at[pz_slave_row, col_name_4]==0:
                    price = 0
                else:
                    price = pz_ost.at[pz_slave_row, col_name_3]/pz_ost.at[pz_slave_row, col_name_4]
                summa = ostatok*price
                potr_st.at[row_master_index, col_name_5] = potr_st.at[row_master_index, col_name_5] + summa
                potr_st.at[row_master_index, col_name_1] = potr_st.at[row_master_index, col_name_1] + ostatok

                pz_ost.at[pz_slave_row, col_name_0] = 0
                ost_st.at[pz_slave_row, col_name_0] = 0
                ostatok = 0
                # print(df_master[['Количество потребности', 'Дата потребности', 'Распределение_остатака']])
                return

        elif (potr_st.at[row_master_index, col_name_2] > potr_st.at[row_master_index, col_name_1]) and (potr_st.at[row_master_index, col_name_2] < ostatok):
            if pz_ost.at[pz_slave_row, col_name_3]==0 or pz_ost.at[pz_slave_row, col_name_4]==0:
                price = 0
            else:
                price = pz_ost.at[pz_slave_row, col_name_3]/pz_ost.at[pz_slave_row, col_name_4]
            
            vzyali = potr_st.at[row_master_index, col_name_2]-potr_st.at[row_master_index, col_name_1]
            
            summa = vzyali*price
            potr_st.at[row_master_index, col_name_5] = potr_st.at[row_master_index, col_name_5] + summa
            potr_st.at[row_master_index, col_name_1] = potr_st.at[row_master_index, col_name_1] + (potr_st.at[row_master_index, col_name_2] -potr_st.at[row_master_index, col_name_1])

            pz_ost.at[pz_slave_row, col_name_0] = pz_ost.at[pz_slave_row, col_name_0] - vzyali
            ost_st.at[pz_slave_row, col_name_0] = ost_st.at[pz_slave_row, col_name_0] - vzyali
            ostatok = ostatok - vzyali
            # print(df_master[['Количество потребности', 'Дата потребности', 'Распределение_остатака']])

In [ ]:
ost_start_raspr.shape

In [ ]:
start_time = time.time()


# col_name_0 название столбца откуда рпспределяем
# col_name_1 навание с толбца куда распределяем
# col_name_2 название столбца с количеством потребности
# col_name_5 название столбца куда распределяем сумму
# Для цены:
# col_name_3 название столбца с суммой потребности
# col_name_4 название столбца с начальным кол-вом  потребности

potr_copy_for_rashod = potr_copy_cons.copy()
rashod_copy = rashod_start_raspr.copy()
potr_copy_for_rashod.name = 'potr_copy_for_rashod'
rashod_copy.name =  'rashod_copy'
# Запуск распределения остатков
potr_copy_for_rashod['Распределение_расхода'] = 0.000000
potr_copy_for_rashod['Сумма_распреденного_расхода'] = 0.000000
rashod_copy['осталось'] = rashod_copy['Кол-во в ЕИ ввода']

rashod_raspr(ost_st=rashod_copy,potr_st=potr_copy_for_rashod,filters=filter_else,current_date = current_date,all_months=True,col_name_0 ='осталось',col_name_1 ='Распределение_расхода', col_name_2 = 'Количество потребности',col_name_3 ='Сумма во ВВ',col_name_4 = 'Кол-во в ЕИ ввода',col_name_5 = 'Сумма_распреденного_расхода') 
print("--- %s seconds ---" % (time.time() - start_time))

In [ ]:
prihod_start_rasp.shape

In [ ]:
ttt = get_in_backup(path_backup='tto.h5',name='01')

In [ ]:
potr_copy_cons['законтрактовано'].sum()

In [ ]:
ttt['законтрактовано'].sum()

In [ ]:
potr_copy_for_prihod['NB'].sum()

In [ ]:
ttt['NB'].sum()

In [ ]:
start_time = time.time()
potr_copy_for_prihod = potr_copy_for_rashod.copy()
prihod_copy = prihod_start_raspr.copy()
potr_copy_for_prihod.name = 'potr_copy_for_prihod'
prihod_copy.name =  'prihod_copy'
# Запуск распределения остатков
potr_copy_for_prihod['Распределение_прихода'] = 0.000000
potr_copy_for_prihod['Сумма_распреденного_прихода'] = 0.000000
prihod_copy['осталось'] = prihod_copy['Количество']

prihod_raspr(ost_st=prihod_copy,potr_st=potr_copy_for_prihod,filters=filter_else,current_date = current_date,all_months=True,col_name_0 ='осталось',col_name_1 ='Распределение_прихода', col_name_2 = 'Количество потребности',col_name_3 ='Сумма во ВВ',col_name_4 = 'Количество',col_name_5 = 'Сумма_распреденного_прихода') 
print("--- %s seconds ---" % (time.time() - start_time))

In [ ]:
prihod_start_raspr

In [ ]:
ttt.shape

In [ ]:
potr_copy_for_prihod.shape

In [ ]:
potr_copy_for_prihod

In [ ]:
ttt

In [ ]:
cons = potr_copy_for_prihod.set_index(['Заказчик + HH','del']).sort_values('Дата потребности')

In [ ]:
cons

In [ ]:
start_time = time.time()
potr_prt_pzk = potr_avizot.copy()
prt_pzk_copy = ZNB1.copy()
potr_prt_pzk.name = 'potr_prt_pzk'
prt_pzk_copy.name =  'prt_pzk_copy'
potr_prt_pzk['законтрактовано'] = 0.000000
prt_pzk_copy['осталось'] = prt_pzk_copy['Количество пополнения']
prt_pzk(ost_st=prt_pzk_copy,potr_st=potr_prt_pzk,filters=filter_else,current_date = current_date,all_months=True,col_name_0 ='осталось')
print("--- %s seconds ---" % (time.time() - start_time))

In [ ]:
prt_pzk_copy

In [ ]:
def main_price(potr_gr,potr_gr1,A900,A902,konp_gr,df):
    
    def price(x):
        if x['Цена']==0:
            return 1
        elif x['Цена']==345:
            return 1
        else:
            return x['Цена']
    
    for i in potr_gr.keys():
        dat = get_df_f(A902, ['Код округа', 'Материал'], [i[0], i[1]])
#         mex_price(dat,konp_gr,df,potr_gr,i)
    for i in potr_gr1.keys():
        dat = get_df_f1(A900, ['Материал'], [i])
#         mex_price(dat, konp_gr, df, potr_gr1, i)
        
    df['Цена'] = df.apply(lambda x: price(x),axis = 1)

In [ ]:
start_time = time.time()
A902['Код округа'] = A902['Код округа'].astype('str')
A902['Материал'] = A902['Материал'].astype('str')
A902['Материал'] = A902['Материал'].apply(lambda x: '0'+str(x) if  len(str(x))<9 else x)
A900['Материал'] = A900['Материал'].astype('str')
A900['Материал'] = A900['Материал'].apply(lambda x: '0'+str(x) if  len(str(x))<9 else x)
df['Материал'] = df['Материал'].astype('str')
KONP['Номер записи условий'] = KONP['Номер записи условий'].astype('str')
A902.fillna(0,inplace=True)
A900.fillna(0,inplace=True)
KONP.fillna(0,inplace=True)
potr_gr = dict(tuple(df.groupby(['Код округа','Материал'])))
potr_gr_mat = dict(tuple(df.groupby(['Материал'])))
konp_gr = dict(tuple(KONP.groupby(['Номер записи условий'])))
df['Цена'] = 0.0000
A900 = A900[A900['Конец действия'].apply(lambda x: x.year)==current_date.year]
A902 = A902[A902['Конец действия'].apply(lambda x: x.year)==current_date.year]
main_price(potr_gr,potr_gr_mat,A900,A902,konp_gr,df)
print("--- %s seconds ---" % (time.time() - start_time))

In [ ]:
start_time = time.time()
df['index'] = [x for x in range(len(df))]
konp_gr = KONP.set_index('Номер записи условий')
df['Цена'] = 0.0000
A900_ = A900[A900['Конец действия'].apply(lambda x: x.year)==current_date.year]
A902_ = A902[A902['Конец действия'].apply(lambda x: x.year)==current_date.year]
main_price_f(A900_,A902_,konp_gr,df)
print("--- %s seconds ---" % (time.time() - start_time))

In [ ]:
def main_price_f(A900,A902,konp_gr,df):
    
    def price(x):
        if x['Цена']==0:
            return 1
        elif x['Цена']==345:
            return 1
        else:
            return x['Цена']
    potr_gr = df.set_index(['Код округа','Материал'])
    for ind in A902.index:
        
        try: 
            dat = potr_gr.xs([A902.at[ind,'Код округа'],A902.at[ind,'Материал']],level = [0,1])
        except KeyError:
            dat = pd.DataFrame()
        if len(dat)>0:
            pass
        
        
    potr_gr1 = df.set_index(['Материал'])
    for ind in A900.index:
        
        try: 
            dat = potr_gr1.xs(A900.at[ind,'Материал'])
        except KeyError:
            dat = pd.DataFrame()
        if len(dat)>0:
            pass
        

        
    df['Цена'] = df.apply(lambda x: price(x),axis = 1)

In [ ]:
def price_main__(df,A902,A900,KONP,current_date,pl_kurs):
    start_time = time.time()
    pool = Pool(processes=6)
    df['index'] = [x for x in range(len(df))]
    A902['Код округа'] = A902['Код округа'].astype('str')
    A902['Материал'] = A902['Материал'].astype('str')
    A902['Материал'] = A902['Материал'].apply(lambda x: '0'+str(x) if  len(str(x))<9 else x)
    A900['Материал'] = A900['Материал'].astype('str')
    A900['Материал'] = A900['Материал'].apply(lambda x: '0'+str(x) if  len(str(x))<9 else x)
    df['Материал'] = df['Материал'].astype('str')
    KONP['Номер записи условий'] = KONP['Номер записи условий'].astype('str')
    KONP['dd'] = ''
    df['dd'] = ''
    A902.fillna(0,inplace=True)
    A900.fillna(0,inplace=True)
    KONP.fillna(0,inplace=True)
    convert_factor(df,A902,'Материал','Материал_902')
    convert_factor(df,A902,'Код округа','Код округа_902')
    convert_factor(df,A900,'Материал','Материал_900')
    convert_factor(KONP,A902,'Номер записи условий','Номер записи условий_902')
    convert_factor(KONP,A900,'Номер записи условий','Номер записи условий_900')
    konp_gr = KONP.set_index(['Номер записи условий_902','dd'])
    konp_gr2 = KONP.set_index(['Номер записи условий_900','dd'])
    df['Цена'] = 0.0000
    A900_ = A900[A900['Конец действия'].apply(lambda x: x.year)==current_date.year]
    A902_ = A902[A902['Конец действия'].apply(lambda x: x.year)==current_date.year]
    df.reset_index(inplace = True,drop = True)
    arrs =  np.array_split(df,12)
    arrs =  [(A900_,A902_,konp_gr,arr,pl_kurs,konp_gr2) for arr in arrs]
    df_list = pool.map(func_read_ex.main_price_f, arrs)
    df = pd.concat(df_list)
    print("--- %s seconds ---" % (time.time() - start_time))
    pool.close()
    df.drop('dd',axis = 1,inplace = True)
    return df 

In [ ]:
aa = price_main__(df,A902,A900,KONP,current_date,345)


In [ ]:
def convert_factor(df1,df2,col_from,col_to):
    dic = dict(zip(pd.concat([df1[col_from].astype('str'),df2[col_from].astype('str')]).drop_duplicates(),pd.concat([df1[col_from].astype('str'),df2[col_from].astype('str')]).drop_duplicates().rank(method='dense')))
    df1[col_to] =  df1[col_from].map(dic)
    df2[col_to] =  df2[col_from].map(dic)
    return 

In [ ]:
convert_factor(df,A902,'Материал','Материал_902')
convert_factor(df,A902,'Код округа','Код округа_902')
convert_factor(df,A900,'Материал','Материал_900')
convert_factor(KONP,A902,'Номер записи условий','Номер записи условий_902')
convert_factor(KONP,A900,'Номер записи условий','Номер записи условий_900')

In [ ]:
df['Материал_'] = df['Материал'].map(dict(zip()))

In [ ]:
def convert_factor(df1,df2,col_from,col_to):
    dic = dict(zip(pd.concat([df1[col_from],df2[col_from]]).drop_duplicates(),pd.concat([df1[col_from],df2[col_from]]).drop_duplicates().rank(method='dense')))
    df1[col_to] =  df1[col_from].map(dic)
    df2[col_to] =  df2[col_from].map(dic)
    return

In [ ]:
def price_main__(df,A902,A900,KONP,current_date,pl_kurs):
    start_time = time.time()
    pool = Pool(processes=6)
    df['index'] = [x for x in range(len(df))]
    A902['Код округа'] = A902['Код округа'].astype('str')
    A902['Материал'] = A902['Материал'].astype('str')
    A902['Материал'] = A902['Материал'].apply(lambda x: '0'+str(x) if  len(str(x))<9 else x)
    A900['Материал'] = A900['Материал'].astype('str')
    A900['Материал'] = A900['Материал'].apply(lambda x: '0'+str(x) if  len(str(x))<9 else x)
    df['Материал'] = df['Материал'].astype('str')
    KONP['Номер записи условий'] = KONP['Номер записи условий'].astype('str')
    KONP['dd'] = ''
    df['dd'] = ''
    A902.fillna(0,inplace=True)
    A900.fillna(0,inplace=True)
    KONP.fillna(0,inplace=True)
    konp_gr = KONP.set_index(['Номер записи условий','dd'])
    df['Цена'] = 0.0000
    A900_ = A900[A900['Конец действия'].apply(lambda x: x.year)==current_date.year]
    A902_ = A902[A902['Конец действия'].apply(lambda x: x.year)==current_date.year]
    df.reset_index(inplace = True,drop = True)
    arrs =  np.array_split(df,12)
    arrs =  [(A900_,A902_,konp_gr,arr,pl_kurs) for arr in arrs]
    df_list = pool.map(func_read_ex.main_price_f, arrs)
    df = pd.concat(df_list)
    print("--- %s seconds ---" % (time.time() - start_time))
    pool.close()
    df.drop('dd',axis = 1,inplace = True)
    return df 

In [ ]:
iw13 = pd.read_excel(directory+'IW13.xlsx',sheet_name='Sheet1')

In [ ]:
rcm = pd.read_excel(directory+'RCM.xlsx',sheet_name='Sheet1')

In [ ]:
potr_with_ostatok_total['NB']

In [ ]:
potr_with_ostatok_total['Плановик']

In [ ]:
datetime.timedelta(days=calendar.monthrange(2019,1)[1])

In [ ]:
import calendar

In [ ]:
calendar.monthrange(2019,1)[1]

In [ ]:
potr_with_ostatok_total['month']

In [ ]:
iw13 = pd.read_excel(directory+'IW13.xlsx',sheet_name='Sheet1')
rcm = pd.read_excel(directory+'RCM.xlsx',sheet_name='Sheet1')

In [ ]:
del sprav_mat_cal_last_full

In [ ]:
datetime.datetime.today() 

In [ ]:
datetime.datetime(datetime.datetime(2020, 2, 4, 13, 35, 49, 2853).year,datetime.datetime(2020, 2, 4, 13, 35, 49, 2853).month,1)

In [ ]:
datetime.timedelta(50)

In [ ]:
def ob_(args):
    potr_with_ostatok_total = args[0]
    potr_with_ostatok_total['БЗ'] = potr_with_ostatok_total.apply(lambda x:0 \
                              if x['Количество потребности']+x['Страховой запас']-x['Распределение_остатака']<=0 else \
                             x['Количество потребности']+x['Страховой запас']-x['Распределение_остатака'],axis = 1)
    potr_with_ostatok_total['ПО'] = potr_with_ostatok_total.apply(lambda x:0 \
                              if x['Количество потребности']+x['Страховой запас']-x['Распределение_остатака']>=0 else \
                             x['Распределение_остатака']-x['Количество потребности']-x['Страховой запас']-x['БЗ'],axis = 1)
    return df

In [ ]:
def xxxx(x):
    if x['Количество потребности']+x['Страховой запас']-x['Распределение_остатака']<=0:
        return 0
    else:
        return x['Количество потребности']+x['Страховой запас']-x['Распределение_остатака']
    
    

In [ ]:
def ob_big(df):
    start_time = time.time()
    pool = Pool(processes=4)

    arrs =  np.array_split(df,12)
    arrs =  [(arr) for arr in arrs]
    df_list = pool.map(func_read_ex.ob_, arrs)
    df = pd.concat(df_list)
    print("--- %s seconds ---" % (time.time() - start_time))
    pool.close()
    return df

In [ ]:

aa = ob_big(potr_with_ostatok_total)

In [ ]:
potr_with_ostatok_total

In [ ]:
potr_with_ostatok_total.loc[0]

In [ ]:
for i in potr_with_ostatok_total.index:
    row = potr_with_ostatok_total.loc[i]
    potr_with_ostatok_total.loc[i,'БЗ'] = xxxx(row)
    

In [ ]:
iw13 = pd.read_excel(directory+'IW13.xlsx',sheet_name='Sheet1')
rcm = pd.read_excel(directory+'RCM.xlsx',sheet_name='Sheet1')

In [ ]:
potr_with_ostatok_total['Распределение_остатака'].unique()

In [ ]:
for col in potr_with_ostatok_total.columns:
    if isinstance(potr_with_ostatok_total.loc[0,col], Decimal):
        potr_with_ostatok_total[col] = potr_with_ostatok_total[col].astype('float64')
    

In [ ]:
for i in potr_with_ostatok_total.dtypes:
    print (i)

In [ ]:
def save_csv(df,path):
    data = df.copy()
#     for col in data.columns:
#         if isinstance(data.loc[0,col], Decimal):
# #             print(col)
#             data[col] = data[col].astype('float64')
            
    
    data.to_csv(path,decimal=',',float_format='%.6f',sep = ';')
    del data

In [ ]:
save_csv(potr_with_ostatok_total,directory+'total.csv')

In [ ]:
potr_with_ostatok_total['Обеспеченность, кол-во, %'].unique()

In [ ]:
potr_with_ostatok_total['Обеспеченность, кол-во, %'] = potr_with_ostatok_total.apply(lambda x:  obesp__kolvo_proc(x),axis = 1)

In [ ]:
potr_with_ostatok_total['Распределение_остатака'].sum()

In [ ]:
potr_with_ostatok_total['стрЗапас'].sum()

In [ ]:
tt = pd.read_csv(directory+'total.csv',sep = ';',parse_dates=['Дата потребности'],dtype={'Материал':'str','Завод':'str'})

In [ ]:
tt['Распределение_остатака'].values

In [ ]:
tt['Распределение_остатака'].sum()

In [ ]:
directory = 'C:\\Users\\Илья\\Desktop\\kazzinc\\Uploads1\\'

In [ ]:
potr_with_ostatok_total['Распределение_остатака']

In [ ]:
tt = potr_with_ostatok_total['Распределение_остатака']

In [ ]:
potr_with_ostatok_total.to_json(directory+'total.json')

In [ ]:
potr_with_ostatok_total['Распределение_остатака'].sum()

In [ ]:
iw13 = pd.read_excel(directory+'IW13.xlsx',sheet_name='Sheet1')
rcm = pd.read_excel(directory+'RCM.xlsx',sheet_name='Sheet1')

In [ ]:
potr_with_ostatok_total.name = 'f0_vertical_01'
save_backup('backup_for_sql.h5',[potr_with_ostatok_total])

In [ ]:
potr_with_ostatok_total = get_in_backup(path_backup='backup_for_sql.h5',name='f0_vertical_01')

In [ ]:
ost_Q1[f_tr]['Партия'].unique()

In [ ]:
f_tr = ost_Q1.apply(lambda x: True if x['Партия'][:2].isalpha() and x['Партия'][-2:].isalpha()==False else False,axis = 1)
for ind in ost_Q1[f_tr].index:
    st = ost_Q1.loc[ind,'Завод+НН'].split(' ')
    st[0] = ost_Q1.loc[ind,'Партия'][-4:]
    ost_Q1.loc[ind,'Завод+НН'] = ' '.join(st)
    
    ost_Q1.loc[ind,'Завод'] = ost_Q1.loc[ind,'Партия'][-4:]
    
    st = ost_Q1.loc[ind,'Завод+Склад'].split(' ')
    st[0] = ost_Q1.loc[ind,'Партия'][-4:]
    ost_Q1.loc[ind,'Завод+Склад'] = ' '.join(st)
    
    st = ost_Q1.loc[ind,'Завод+склад+партия+особый запас+НН'].split(' ')
    st[0] = ost_Q1.loc[ind,'Партия'][-4:]
    ost_Q1.loc[ind,'Завод+склад+партия+особый запас+НН'] = ' '.join(st)
    
    

In [ ]:
# проверка распределения остатка и формирование сцепки с основным листом
# работа пок онсолидаци остатков
neraspr_ostat = ost_copy1[(ost_copy1['осталось']>0)&(ost_copy1['Cвободный запас']>0)]['Cвободный запас'].sum()
ost_Q = ost_start[(ost_start['Cвободный запас']>0)&(ost_start['Особый запас']=='Q')]
ost_Q['Признак Q остатка'] = True

ost_Q1 = ost_start[(ost_start['Cвободный запас']==0)&(ost_start['Особый запас']!='Q')]
ost_Q  = pd.concat([ost_Q,ost_Q1])
ost_Q.fillna(0,inplace=True)
ost_q_sum = ost_Q['Кол-во запаса в конце периода'].sum()
ff = ost_block[['Завод+склад+партия+особый запас+НН','Блокированный запас','Завод+НН','Комплекс']]
ost_with_block  =  pd.merge(ost_copy1,ff,how='outer')
block_zap = ost_with_block['Блокированный запас'].sum()
ost_with_block.fillna(0,inplace = True)
prov_ost = potr_copy_for_prihod['Распределение_остатака'].sum()+neraspr_ostat+ost_q_sum+block_zap
ost_copy1['Cвободный запас'] = ost_copy1['осталось']
ost_copy1['Стоимость свободного остатка'] = (ost_copy1['Ст-ть запаса в конце периода']    / ost_copy1['Кол-во запаса в конце периода']) * ost_copy1['осталось']
# 1составляющая нераспределенный остаток
neraspr_ostat_tab = ost_copy1[(ost_copy1['осталось']>0)&(ost_copy1['Cвободный запас']>0)][['Комплекс','Завод+НН','Cвободный запас','Стоимость свободного остатка']].groupby(['Комплекс','Завод+НН']).sum().reset_index()
neraspr_ostat_tab['Признак нераспределнного остатка'] = True
# 2составляющая нераспределенный остаток
ost_Q_tab = ost_Q[['Признак Q остатка','Комплекс','Завод+НН','Cвободный запас','Стоимость свободного остатка','Базовая ЕИ','Материал']].groupby(['Комплекс','Завод+НН']).sum().reset_index()
# ost_Q_tab['Признак Q остатка'] = 0


ost1 = neraspr_ostat_tab[['Комплекс','Завод+НН','Cвободный запас','Стоимость свободного остатка']]

In [ ]:
ost_Q1[f_tr]

In [ ]:
ost_Q1[ost_Q1['Партия']=='KO']

In [ ]:
potr_with_ostatok_total['Завод']  = potr_with_ostatok_total['Завод+НН'].str.split(' #').apply(lambda x: x[0])
spr_zav_obl =  rev_to_dict(sprav_mdlv['Область ППМ'].astype('str').str.strip(),sprav_mdlv['Текст ОблПланирПотр'])
potr_with_ostatok_total['Имя Завода'] = potr_with_ostatok_total.apply(lambda x: insert_vpr(x['Завод'],spr_zav_obl,'Текст ОблПланирПотр')  if x['Область ППМ']=='' or x['Область ППМ']==0 else insert_vpr(x['Область ППМ'],spr_zav_obl,'Текст ОблПланирПотр'),axis = 1 )

In [ ]:
potr_with_ostatok_total['Имя Завода'].value_counts()

In [ ]:
potr_with_ostatok_total[potr_with_ostatok_total['Имя Завода']==0]['Завод'].unique()

In [ ]:
for ind, i in enumerate(np.array_split(c_f0,2)) :
    save_pyex(df=i,path='C:\\Users\\Илья\\Desktop\\kazzinc\\Uploads1\\extr_xlsx\\'+str(ind)+'.xlsx')

In [ ]:
c_f0

In [ ]:
save_pyex(df=potr_with_ostatok_total[potr_with_ostatok_total['Категория_запаса']=='Востребованный'][:100000],path='C:\\Users\\Илья\\Desktop\\kazzinc\\Uploads1\\extr_xlsx\\111.xlsx')

In [ ]:
potr_with_ostatok_total['Распределение_остатака'].sum()

In [ ]:
potr_with_ostatok_total

In [ ]:
# заполняем дополнительные столбцы в экстракте

# df = df[df['Дата потребности']<=datetime.datetime(2020,2,29)]
df['Завод'] = df['Завод'].apply(lambda x: str(x).replace('.0','')).astype('str')
spr_zav_dict = rev_to_dict(sprav_po_kompl['Завод'],sprav_po_kompl['ЦС/ПЗ'])
df['ЦС/ПЗ'] = df['Завод'].apply(lambda x:  insert_vpr(x,spr_zav_dict,'ЦС/ПЗ'))
df['Номер потребности'] = df['Номер потребности'].apply(lambda x: str(x).replace('.0','')).astype('str')
df['МВЗ'] = df['МВЗ'].apply(lambda x: str(x).replace('.0','')).astype('str')
df['Материал'] = df['Материал'].apply(lambda x: str(x).replace('.0','')).astype('str')

df['Тип материала'] = df['Тип материала'].apply(lambda x: str(x).replace('.0','')).astype('str')
df['БЕ'] = df['БЕ'].apply(lambda x: str(x).replace('.0','')).astype('str')



# недропользование неполнятно
spr_nedra = rev_to_dict(sprav_nedra['Завод + Склад'],sprav_nedra['Завод + Склад'])
# df['Недропользование']= df['Завод'].apply(lambda x: insert_vpr(x,spr_nedra,'Завод'))   
df['Склад потребности'] = df['Склад потребности'].apply(lambda x: str(x).replace('.0','')).astype('str')
df['Завод + Склад'] = df['Завод']+' '+df['Склад потребности']
df['Недропользование'] = df['Завод + Склад'].apply(lambda x: 0 if   insert_vpr(x,spr_nedra,'Завод + Склад')==0 else 1)

# Вывод по докум-ам потребности
df['Вывод по документам потребности'] = df.apply(lambda x: formula_vivod_po_doc_potrebn(x),axis=1 )


df['Вывод по документам пополнения'] = df.apply(lambda x: formula_vivod_po_doc_popolneniya(x),axis=1 )

In [ ]:
df['ЦС/ПЗ'].value_counts()

In [ ]:
df['Модуль'] = df.apply(lambda x: modul(x),axis=1 )

In [ ]:
df['Завод+НН'] = df['Завод'].astype('str')+' #'+df['Материал'].astype('str')
df['Завод+НН'] = df['Завод+НН'].str.strip()

df['Номер элемента пополнения'] = df['Номер элемента пополнения'].apply(lambda x: str(x).replace('.0',''))

df['Номер элемента пополнения + Номер позиции пополнения'] = df['Номер элемента пополнения'].astype('str')+' '+df['Номер позиции пополнения'].astype('str')
df['Материал'] = df['Материал'].apply(lambda x: '0'+str(x) if  len(str(x))<9 else x)


sprav_mat_cal_last_full = sprav_mat_cal.copy()

to_str_spr_mat = ['СтатусМтрл на заводе','Завод','Материал']

for i in to_str_spr_mat:
    sprav_mat_cal_last_full[i].fillna(0,inplace = True)
    sprav_mat_cal_last_full[i] = sprav_mat_cal_last_full[i].apply(lambda x: str(x).replace('.0','')).astype('str')
    sprav_mat_cal_last_full[i] = sprav_mat_cal_last_full[i].apply(lambda x: '' if x=='0' or x==0 else x)
    sprav_mat_cal[i].fillna(0,inplace = True)
    sprav_mat_cal[i] = sprav_mat_cal[i].apply(lambda x: str(x).replace('.0','')).astype('str')
    sprav_mat_cal[i] = sprav_mat_cal[i].apply(lambda x: '' if x=='0' or x==0 else x)

sprav_mat_cal_last_full['Материал'] = sprav_mat_cal_last_full['Материал'].apply(lambda x: '0'+str(x) if  len(str(x))<9 else x)

sprav_mat_cal['Завод'] = sprav_mat_cal['Завод'].apply(lambda x: str(x).replace('.0','')).astype('str')
sprav_mat_cal['Материал'] = sprav_mat_cal['Материал'].apply(lambda x: str(x).replace('.0','')).astype('str')
sprav_mat_cal['Материал'] = sprav_mat_cal['Материал'].apply(lambda x: '0'+str(x) if  len(str(x))<9 else x)

sprav_mat_cal = sprav_mat_cal[sprav_mat_cal['Материал'].isin(df['Материал'].unique())]



df['Номер элемента пополнения + Номер позиции пополнения + Наименование типа элемента ППМ пополн'] = df['Номер элемента пополнения + Номер позиции пополнения'].astype('str')+ df.apply(lambda x: 'ПртПЗк' if x['Вид документа пополнения']=='UB05'else x['Наименование типа элемента ППМ пополн'] ,axis=1).astype('str')



df['Номер элемента пополнения + Номер позиции пополнения'] = df['Номер элемента пополнения'].astype('str')+' '+df['Номер позиции пополнения'].astype('str')

df['Область ППМ'] = df['Область ППМ'].apply(lambda x: str(x).replace('.0','')).astype('str')

df['Область ППМ'] = df['Область ППМ'].apply(lambda x:"" if x=='0' else x)


df['Завод + Область ППМ'] = df['Завод'].astype('str')+' '+df['Область ППМ'].astype('str')

# чистим
df['Завод + Область ППМ'] =  df['Завод + Область ППМ'].apply(lambda x: str(x).replace('0.0','').replace('.0','')).str.strip()



In [ ]:
spr_zav_obl =  rev_to_dict(sprav_mdlv['Область ППМ'].astype('str').str.strip(),sprav_mdlv['Текст ОблПланирПотр'])
df['Имя Завода'] = df.apply(lambda x: insert_vpr(x['Завод'],spr_zav_obl,'Текст ОблПланирПотр')  if x['Область ППМ']=='' else insert_vpr(x['Область ППМ'],spr_zav_obl,'Текст ОблПланирПотр'),axis = 1 )

In [ ]:
df['Область ППМ'].unique()

In [ ]:
spr_zav_kompl = rev_to_dict(sprav_po_kompl['Завод + Область ППМ'].astype('str').str.strip(),sprav_po_kompl['Комплекс'])
df['Комплекс'] = df['Завод + Область ППМ'].apply(lambda x: insert_vpr(x,spr_zav_kompl,'Комплекс') )

In [ ]:
df['Комплекс'].value_counts()

In [ ]:
df[df['Комплекс']==0]['Завод + Область ППМ'].unique()

In [ ]:
spr_zav_kompl

In [ ]:
df['Заказчик + HH'] =df['Комплекс'].astype('str')+' '+df['Материал'].astype('str')


df['month'] = df['Дата потребности'].apply(lambda x: str(x.year)+'-'+str(x.month)+ '-01')
df['month'] = pd.to_datetime(df['month']) 











In [ ]:
spr_for_obl_ppm = pd.read_excel(directory+'MDLG.xlsx',sheet_name='Sheet1')

In [ ]:
spr_obl_ppm = rev_to_dict(spr_for_obl_ppm['Завод'].apply(lambda x: str(x).replace('.0','')).astype('str')+spr_for_obl_ppm['Склад'].\
            apply(lambda x: str(x).replace('.0','')).astype('str'),spr_for_obl_ppm['Область ППМ'])
df['Область ППМ'] =df['Область ППМ'] = df\
                                    .apply(lambda x: x['Завод'] if    insert_vpr(x['Завод']+x['Склад потребности'],spr_obl_ppm,'Область ППМ')==0 else  \
                                     insert_vpr(x['Завод']+x['Склад потребности'],spr_obl_ppm,'Область ППМ'),axis = 1).astype('str')
spr_kod_okruga= rev_to_dict(ZMM_E0078_MDLV['Область ППМ'].astype('str').str.strip(),ZMM_E0078_MDLV['Код округа'])
df['Код округа'] = df['Область ППМ'].apply(lambda x:  insert_vpr(x,spr_kod_okruga,'Код округа'))

In [ ]:
df.apply(lambda x: insert_vpr(x['Завод']+x['Склад потребности'],spr_obl_ppm,'Область ППМ'),axis = 1).astype('str').unique()

In [ ]:
df.apply(lambda x: insert_vpr(x['Завод']+x['Склад потребности'],spr_obl_ppm,'Область ППМ'),axis = 1).astype('str').value_counts()

In [ ]:
df['Код округа'].value_counts()

In [ ]:
df['Код округа'].value_counts()

In [ ]:
df[df['Код округа']==0]['Область ППМ'].unique()

In [ ]:
central_zavod['Область ППМ'] = central_zavod['Область ППМ'].astype('str')
spr_obl_ppm_centr_zav = rev_to_dict(central_zavod['Область ППМ'],central_zavod['Центральный Завод'])
df['Центральный завод'] = df['Область ППМ'].apply(lambda x:  insert_vpr(x,spr_obl_ppm_centr_zav,'Центральный Завод'))

spr_obl_ppm_centr_zav = rev_to_dict(sprav_mat_cal['Завод']+' #'+sprav_mat_cal['Материал'],sprav_mat_cal['Краткий текст материала'])
df['КрТекстМатериала'] = df['Завод+НН'].apply(lambda x:  insert_vpr(x,spr_obl_ppm_centr_zav,'Краткий текст материала'))

spr_obl_ppm_baz_ei = rev_to_dict(sprav_mat_cal['Завод']+' #'+sprav_mat_cal['Материал'],sprav_mat_cal['Базовая ЕИ'])
df['Базовая ЕИ'] = df['Завод+НН'].apply(lambda x:  insert_vpr(x,spr_obl_ppm_baz_ei,'Базовая ЕИ'))

spr_obl_ppm_ved_mat = rev_to_dict(sprav_mat_cal['Завод']+' #'+sprav_mat_cal['Материал'],sprav_mat_cal['Ведущий материал'])
df['Признак аналога'] = df['Завод+НН'].apply(lambda x:  insert_vpr(x,spr_obl_ppm_ved_mat,'Ведущий материал'))

spr_obl_ppm_stat_na_zav = rev_to_dict(sprav_mat_cal['Завод']+' #'+sprav_mat_cal['Материал'],sprav_mat_cal['СтатусМтрл на заводе'])
df['СтатусМтрл на заводе'] = df['Завод+НН'].apply(lambda x:  insert_vpr(x,spr_obl_ppm_stat_na_zav,'СтатусМтрл на заводе'))


spr_obl_ppm_ocenka = rev_to_dict(sprav_mat_cal['Материал'],sprav_mat_cal['Класс оценки'])
df['Класс оценки'] = df['Материал'].apply(lambda x:  insert_vpr(x,spr_obl_ppm_ocenka,'Класс оценки'))
df['Класс оценки'] = df['Класс оценки'].apply(lambda x: str(x).replace('.0','')).astype('str')




spr_obl_ppm_name_planov= rev_to_dict(sprav_mat_cal['Завод']+' #'+sprav_mat_cal['Материал'],sprav_mat_cal['Имя плановика (завод)'])
df['ЦЗ+НН'] = df['Центральный завод'].astype('str')+' #'+df['Материал'].astype('str')
df['Плановик'] = df['ЦЗ+НН'].apply(lambda x:  insert_vpr(x,spr_obl_ppm_name_planov,'Имя плановика (завод)'))


spr_group_mat_l1 = rev_to_dict(group_mat[group_mat['Группа материалов'].str.len()==2]['Группа материалов'],group_mat[group_mat['Группа материалов'].str.len()==2]['Обозначение 2 группы материалов'])
df['группа'] = df['Материал'].str[:2].apply(lambda x:  insert_vpr(x,spr_group_mat_l1,'Обозначение 2 группы материалов'))

spr_group_mat_l2 = rev_to_dict(group_mat[group_mat['Группа материалов'].str.len()==4]['Группа материалов'],group_mat[group_mat['Группа материалов'].str.len()==4]['Обозначение 2 группы материалов'])
df['подгруппа'] = df['Материал'].str[:4].apply(lambda x:  insert_vpr(x,spr_group_mat_l2,'Обозначение 2 группы материалов'))


spr_group_mat_l3 = rev_to_dict(group_mat[group_mat['Группа материалов'].str.len()==6]['Группа материалов'],group_mat[group_mat['Группа материалов'].str.len()==6]['Обозначение 2 группы материалов'])
df['подподгруппа'] = df['Материал'].str[:6].apply(lambda x:  insert_vpr(x,spr_group_mat_l3,'Обозначение 2 группы материалов'))

df['ТИП номенклатуры'] = 0.0


In [ ]:
KONP['Индикатор удаления'].unique()

In [ ]:
KONP = KONP[KONP['Индикатор удаления']!='X']

In [ ]:
df = price_main__(df,A902,A900,KONP,current_date,pl_kurs)

In [ ]:
df[(df['Материал']=='060102003')&(df['Цена']==1)]['Код округа'].unique()

In [ ]:
df[df['Материал']=='060102003']

In [ ]:
A902[A902['Материал']=='060102003']

In [ ]:
A900[A900['Материал']=='060102003']

In [ ]:
KONP[KONP['Номер записи условий']=='0000391305']

In [ ]:
spr_zav_mat = rev_to_dict(sprav_mat_cal['Завод']+' '+sprav_mat_cal['Материал'],sprav_mat_cal['Плановый срок поставки (завод)'])
df['Центральный завод'] = df['Центральный завод'].astype('str')
for i in df.index:
        try:
            if df.at[i,'Недропользование']==1:df.at[i,'ПлановСрокПоставки'] =  spr_zav_mat[df.at[i,'Завод']+' '+df.at[i,'Материал']]['Плановый срок поставки (завод)']
            else:df.at[i,'ПлановСрокПоставки'] = spr_zav_mat[df.at[i,'Центральный завод']+' '+df.at[i,'Материал']]['Плановый срок поставки (завод)']
        except KeyError:
            pass


df.fillna(0,inplace=True)

# копия для формирования страхового запаса
df_str_zap = df.copy()
# df_str_zap = df_str_zap[df_str_zap['Дата потребности']<=datetime.datetime(2020,2,29)]
df_str_zap.name =  'df_str_zap'
# df = df[df['Дата потребности']<=datetime.datetime(2020,2,29)]

df = df[(df['ЦС/ПЗ']=='ПЗ')&(df['Вывод по документам потребности']=='OK')&(df['Баланс (закрытие потребности)']==0)&(df['Тип ППМ']=='PD')]
df = df[df['Материал'].apply(lambda x:len(x)<=10)]
mat_cat_str_zap = sprav_mat_cal_last_full[sprav_mat_cal_last_full['Страховой запас']>0]
df = pd.concat([df,mat_cat_str_zap[['Материал','Завод','Код округа','Страховой запас']]])

df.fillna(0,inplace = True)

df['month'] =    df.apply(lambda x: datetime.datetime(2199,1,1)  if x['Страховой запас']>0 else x['month'],axis = 1)
df['Дата потребности'] =    df.apply(lambda x: datetime.datetime(2199,1,1)  if x['Страховой запас']>0 else x['Дата потребности'],axis = 1)
df['ЦС/ПЗ'] =  df.apply(lambda x: 'ПЗ'  if x['Страховой запас']>0 else x['ЦС/ПЗ'],axis = 1)
df['Завод+НН'] = df['Завод'].astype('str')+' #'+df['Материал'].astype('str')
df['Завод+НН'] = df['Завод+НН'].str.strip()
df.reset_index(inplace = True,drop = True)

In [ ]:
# spr_zav_kompl = rev_to_dict(sprav_po_kompl['Завод + Область ППМ'].astype('str').str.strip(),sprav_po_kompl['Комплекс'])
df['Комплекс'] = df.apply(lambda x:  insert_vpr(x['Завод'],spr_zav_kompl,'Комплекс') if x['Комплекс']==0 else x['Комплекс'],axis = 1)

In [ ]:
spr_zav_kompl

In [ ]:
df['Комплекс'].value_counts()

In [ ]:
potr_with_ostatok_total['Сумма стрЗап'].sum()

In [ ]:
potr_with_ostatok_total[potr_with_ostatok_total['стрЗапас']>0]

In [ ]:
df['Особый запас'].unique()

In [ ]:
df = df[df['Особый запас']!='Q']

In [ ]:
df.shape

In [ ]:
spr_kod_okruga = rev_to_dict(ZMM_E0078_MDLV['Область ППМ'].apply(lambda x: str(x).replace('.0',''))\
                             .astype('str').str.strip(),ZMM_E0078_MDLV['Код округа'])

spr_kod_okruga1 = rev_to_dict(T001W['Завод'].apply(lambda x: str(x).replace('.0','')).astype('str'),\
            T001W['Код округа'].apply(lambda x: str(x).replace('.0','')).astype('str'))

df['Код округа'] = df.apply(lambda x: insert_vpr(x['Завод'],spr_kod_okruga1,'Код округа') if insert_vpr(x['Область ППМ'],spr_kod_okruga,'Код округа')==0 else insert_vpr(x['Область ППМ'],spr_kod_okruga,'Код округа'),axis = 1).astype('str')

In [ ]:
T001W = pd.read_excel(directory+'T001W.xlsx',sheet_name='Sheet1')
T001W.fillna(0,inplace=True)




In [ ]:
df['Код округа'] = df.apply(lambda x: insert_vpr(x['Завод'],spr_kod_okruga1,'Код округа') if insert_vpr(x['Область ППМ'],spr_kod_okruga,'Код округа')==0 else insert_vpr(x['Область ППМ'],spr_kod_okruga,'Код округа'),axis = 1).astype('str')

In [ ]:
df = get_in_backup(path_backup='backup.h5',name='extract')

In [ ]:
df['Код округа'].value_counts()

In [ ]:
ZMM_E0078_MDLV = pd.read_excel(directory+'ZMM_E0078_MDLV.xlsx',sheet_name='Sheet1',dtype={'Область ППМ':'str','Код округа':'str'})

In [ ]:
df['Код округа'].value_counts()

In [ ]:



pr_be = rev_to_dict(T001W['Завод'].apply(lambda x: str(x).replace('.0',''))\
                             .astype('str').str.strip(),T001W['Закуп. организация'].apply(lambda x: str(x).replace('.0','')).astype('str').str.strip())
c_f0['БЕ'] = c_f0['Завод'].apply(lambda x:  insert_vpr(x,spr_be,'БЕ'))

In [ ]:
c_f0['БЕ'].value_counts()

In [ ]:
# c_f = pd.concat([current_f0.set_index(ind),cons_f0.set_index(ind)[[x for x in cons_f0.columns if x.find('ФП')!=-1 or x.find('ФC')!=-1 or x.find('Остаток на начало ')!=-1]]],axis=1).reset_index()
c_f = pd.concat([current_f0.set_index(ind),cons_f0.set_index(ind)[[x for x in cons_f0.columns if  x.find('Остаток на начало ')!=-1]]],axis=1).reset_index()

In [ ]:
current_f0.set_index(ind)

In [ ]:
cons_f0.set_index(ind)

In [ ]:
c_f.shape

In [ ]:
[x for x in cons_f0.columns if x.find('ФП')!=-1 or x.find('ФC')!=-1 or x.find('Остаток на начало ')!=-1]

In [ ]:
cons_f0.set_index(ind)[[x for x in cons_f0.columns if x.find('ФП')!=-1 or x.find('ФC')!=-1 or x.find('Остаток на начало ')!=-1]]

In [ ]:
tttttyy = current_f0.set_index(ind).index.duplicated()

In [ ]:
tttttyy = list(tttttyy)

In [ ]:
df.index.duplicated()

In [ ]:
[tttttyy.count(x) for x in tttttyy ]

In [ ]:
colsss  = list(cons_f0.columns)

In [ ]:
[x for x in colsss1 if colsss1.count(x)>1]

In [ ]:
colsss1  = list(current_f0.columns)

In [ ]:
ind

In [ ]:
c_f0[c_f0.set_index(ind).index.duplicated()]

In [ ]:
c_f0.shape

In [ ]:
c_f0.shape

In [ ]:
c_f0

In [ ]:
df = pd.read_csv(directory+'CSV.csv',sep = ';',parse_dates=['Дата потребности'],dtype={'Материал':'str','Завод':'str'})
sprav_po_kompl = pd.read_excel(directory+'Справочник по комплексам.xlsx',sheet_name='Комплекс',dtype={'Область ППМ':'str','Завод':'str'})
sprav_nedra = pd.read_excel(directory+'Справочник недрапользование.xlsx',sheet_name='Недра-ненедра')

In [ ]:
# |||||||||||||||||||||блок обработка данных листа  extract
# заполняю даты нулевыми если 0
df['Дата потребности'] = df['Дата потребности'].apply(lambda x: '01.01.1900' if x =='00.00.0000' else  x)
df['Дата потребности'] = [datetime.datetime(int(x.split('.')[2]),int(x.split('.')[1]),int(x.split('.')[0])) for x in df['Дата потребности'].values]
# собираю дату с группировкой по месяцу
month = df['Дата потребности'].apply(lambda x:  str(x.year)+'-'+str(x.month)+'-01'+ ' 00:00:00')
month =pd.to_datetime(month) 
# заменяю символ - 
df['Количество потребности'] = df['Количество потребности'].str.replace('-','')
df['Баланс (закрытие потребности)'] = df['Баланс (закрытие потребности)'].str.replace('-','')
# Работаю с числами
df['Вид документа потребности'] = df['Вид документа потребности'].astype('str')
df['Вид документа пополнения'] = df['Вид документа пополнения'].astype('str')
# меняем цифры на нормальные флоат
replace_float(df,['Количество потребности','Количество пополнения','Баланс (закрытие потребности)'])

df.fillna(0,inplace=True)
sprav_po_kompl.fillna(0,inplace=True)
sprav_nedra.fillna(0,inplace=True)
# T001W.fillna(0,inplace=True)

In [ ]:
# заполняем дополнительные столбцы в экстракте

# df = df[df['Дата потребности']<=datetime.datetime(2020,2,29)]
df['Завод'] = df['Завод'].apply(lambda x: str(x).replace('.0','')).astype('str')
spr_zav_dict = rev_to_dict(sprav_po_kompl['Завод'],sprav_po_kompl['ЦС/ПЗ'])
df['ЦС/ПЗ'] = df['Завод'].apply(lambda x:  insert_vpr(x,spr_zav_dict,'ЦС/ПЗ'))
df['Номер потребности'] = df['Номер потребности'].apply(lambda x: str(x).replace('.0','')).astype('str')
df['МВЗ'] = df['МВЗ'].apply(lambda x: str(x).replace('.0','')).astype('str')
df['Материал'] = df['Материал'].apply(lambda x: str(x).replace('.0','')).astype('str')

df['Тип материала'] = df['Тип материала'].apply(lambda x: str(x).replace('.0','')).astype('str')
df['БЕ'] = df['БЕ'].apply(lambda x: str(x).replace('.0','')).astype('str')



# недропользование неполнятно
spr_nedra = rev_to_dict(sprav_nedra['Завод + Склад'],sprav_nedra['Завод + Склад'])
# df['Недропользование']= df['Завод'].apply(lambda x: insert_vpr(x,spr_nedra,'Завод'))   
df['Склад потребности'] = df['Склад потребности'].apply(lambda x: str(x).replace('.0','')).astype('str')
df['Завод + Склад'] = df['Завод']+' '+df['Склад потребности']
df['Недропользование'] = df['Завод + Склад'].apply(lambda x: 0 if   insert_vpr(x,spr_nedra,'Завод + Склад')==0 else 1)

# Вывод по докум-ам потребности
df['Вывод по документам потребности'] = df.apply(lambda x: formula_vivod_po_doc_potrebn(x),axis=1 )


df['Вывод по документам пополнения'] = df.apply(lambda x: formula_vivod_po_doc_popolneniya(x),axis=1 )

In [ ]:
df[(df['Вывод по документам потребности']!='OK')|(df['Вывод по документам пополнения']!='OK')]

In [ ]:
save_pyex(df=df[(df['Вывод по документам потребности']!='OK')|(df['Вывод по документам пополнения']!='OK')],path='C:\\Users\\Илья\\Desktop\\kazzinc\\Uploads1\\'+folder+'\\extr_xlsx\\111.xlsx')

In [ ]:
NB['Комплекс'].value_counts()

In [ ]:
ZNB1['Комплекс'].value_counts()

In [ ]:
ZNB['Комплекс'].value_counts()[0]

In [ ]:
ZNB1['Комплекс'].value_counts()[0]

In [ ]:
NB['Комплекс'].value_counts()['0']

In [ ]:
NB[NB['Комплекс']=='0']

In [ ]:
NB = NB[NB['Обозначение индикатора деблокирования']!='Тендер завершен']

In [ ]:
ZNB = ZNB[ZNB['Количество пополнения']>0]

In [ ]:
potr_with_ostatok_total['Завод']  = potr_with_ostatok_total['Завод+НН'].str.split(' #').apply(lambda x: x[0])
spr_zav_obl =  rev_to_dict(sprav_mdlv['Область ППМ'].astype('str').str.strip(),sprav_mdlv['Текст ОблПланирПотр'])
potr_with_ostatok_total['Имя Завода'] = potr_with_ostatok_total.apply(lambda x: insert_vpr(x['Завод'],spr_zav_obl,'Текст ОблПланирПотр')  if x['Область ППМ']=='' or x['Область ППМ']==0 else insert_vpr(x['Область ППМ'],spr_zav_obl,'Текст ОблПланирПотр'),axis = 1 )

potr_with_ostatok_total['стрЗапас'] = potr_with_ostatok_total['Страховой запас']
fr_bad = potr_with_ostatok_total[(potr_with_ostatok_total['Блокированный запас']>0)&(potr_with_ostatok_total['Распределение_остатака']>0)]
potr_with_ostatok_total.loc[fr_bad.index,['Распределение_остатака','Сумма_распреденного_остатка','Распределение_прихода','Сумма_распреденного_прихода','Распределение_расхода','Сумма_распреденного_расхода','avizot','стрЗапас']]    = 0
fr_bad.loc[:,['Блокированный запас','Ст-ть/блок. запаса']]=0
potr_with_ostatok_total = pd.concat([potr_with_ostatok_total,fr_bad],axis=0,ignore_index=True)
potr_with_ostatok_total['Комплекс'] = potr_with_ostatok_total['Комплекс'].astype('str')
potr_with_ostatok_total['Категория ТМЦ'] = potr_with_ostatok_total.apply(lambda x: func_cat_tmc(x),axis = 1)
ind____  = potr_with_ostatok_total[potr_with_ostatok_total['стрЗапас']>0].index
potr_with_ostatok_total.loc[ind____,'Категория ТМЦ'] = 'страховой запас'

In [ ]:
potr_with_ostatok_total['Категория ТМЦ'].unique()

In [ ]:
potr_with_ostatok_total['Сумма_распреденного_остатка'].sum()

In [ ]:
c_f0

In [ ]:
ffy = pd.read_csv(directory+'CSV.csv',sep = ';',parse_dates=['Дата потребности'],dtype={'Материал':'str','Завод':'str'})

In [ ]:
ffy.shape

In [ ]:
spr_for_obl_ppm = pd.read_excel('C:\\Users\\Илья\\Desktop\\kazzinc\\Uploads1\\0220\\extr_xlsx\\0.xlsx',sheet_name='sheet1')

In [ ]:
spr_for_obl_ppm['1/2020 Остаток Сумма'].sum()

In [ ]:
spr_for_obl_ppm.fillna(0,inplace = True)

In [ ]:
spr_for_obl_ppm.name = '001'
save_backup('f0_S.h5',[spr_for_obl_ppm])

In [ ]:
~spr_for_obl_ppm['Модуль'].isnull()

In [ ]:
gr = list(spr_for_obl_ppm[['Материал','Категория_запаса','Цена','Модуль','Заказ','Категория ТМЦ','БЕ','Комплекс','Завод','Код округа','КрТекстМатериала','Имя Завода','Базовая ЕИ',]].columns)

In [ ]:
agg = spr_for_obl_ppm.loc[:,'ПП кол-во 1/2020':].columns

In [ ]:
agg_m = dict((x,'sum')for x in agg)

In [ ]:
gr_df = spr_for_obl_ppm.groupby(gr).agg(agg_m).reset_index()

In [ ]:
gr_df['1/2020 Остаток Сумма'].sum()

In [ ]:
gr_df

In [ ]:
spr_for_obl_ppm[spr_for_obl_ppm['Цена']==1672.5]

In [ ]:
save_pyex(df=gr_df,path='C:\\Users\\Илья\\Desktop\\kazzinc\\Uploads1\\'+folder+'\\extr_xlsx\\gr.xlsx')

In [ ]:
folder = '0220'

In [ ]:
for ind, i in enumerate(np.array_split(gr_df,2)) :
    save_pyex(df=i,path='C:\\Users\\Илья\\Desktop\\kazzinc\\Uploads1\\'+folder+'\\extr_xlsx\\'+str(ind)+'.xlsx')

In [ ]:
ggm = pd.read_excel('C:\\Users\\Илья\\Desktop\\kazzinc\\0.xlsx',sheet_name='sheet1')

In [ ]:
df['Модуль'].unique()

In [ ]:
aaaa = df[df['Модуль']=='ХЗ'].shape

In [ ]:
aaaa[aaaa['Материал']=='030301150']

In [ ]:
aaaa

In [ ]:
potr_with_ostatok_total['Имя Завода'].value_counts()

In [ ]:
potr_with_ostatok_total[potr_with_ostatok_total['Материал']=='010101015']['Имя Завода'].unique()

In [ ]:
potr_with_ostatok_total[(potr_with_ostatok_total['Материал']=='010101015')&(potr_with_ostatok_total['Количество потребности']==0)&(potr_with_ostatok_total['Распределение_остатака']>0)]

In [ ]:
potr_with_ostatok_total['Завод']  = potr_with_ostatok_total['Завод+НН'].str.split(' #').apply(lambda x: x[0])
spr_zav_obl =  rev_to_dict(sprav_mdlv['Область ППМ'].astype('str').str.strip(),sprav_mdlv['Текст ОблПланирПотр'])


potr_with_ostatok_total['стрЗапас'] = potr_with_ostatok_total['Страховой запас']
fr_bad = potr_with_ostatok_total[(potr_with_ostatok_total['Блокированный запас']>0)&(potr_with_ostatok_total['Распределение_остатака']>0)]
potr_with_ostatok_total.loc[fr_bad.index,['Распределение_остатака','Сумма_распреденного_остатка','Распределение_прихода','Сумма_распреденного_прихода','Распределение_расхода','Сумма_распреденного_расхода','avizot','стрЗапас']]    = 0
fr_bad.loc[:,['Блокированный запас','Ст-ть/блок. запаса']]=0
potr_with_ostatok_total = pd.concat([potr_with_ostatok_total,fr_bad],axis=0,ignore_index=True)
potr_with_ostatok_total['Комплекс'] = potr_with_ostatok_total['Комплекс'].astype('str')
potr_with_ostatok_total['Категория ТМЦ'] = potr_with_ostatok_total.apply(lambda x: func_cat_tmc(x),axis = 1)

In [ ]:
yyyyuuu = ost_Q1[f_tr]

In [ ]:
yyyyuuu

In [ ]:
yyyyuuu[yyyyuuu['Материал']=='010101015']

In [ ]:
ost_Q1[ost_Q1['Материал']=='010101015']

In [ ]:
ost_Q[ost_Q['Материал']=='010101015']

In [ ]:
ost_with_block[ost_with_block['Материал']=='010101015']

In [ ]:
ff

In [ ]:
tottal_ost

In [ ]:
ost_Q1[f_tr]

In [ ]:
potr_with_ostatok_total[(potr_with_ostatok_total['Количество потребности']==0)&(potr_with_ostatok_total['Распределение_остатака']>0)]

In [ ]:
ost_Q1

In [ ]:
neraspr_ostat_tab

In [ ]:
neraspr_ostat_tab

In [ ]:
ost_copy1[(ost_copy1['осталось']>0)&(ost_copy1['Cвободный запас']>0)][['Комплекс','Завод+НН','Cвободный запас','Стоимость свободного остатка','Партия']].groupby(['Комплекс','Завод+НН','Партия']).sum().reset_index()

In [ ]:
save_pyex(df=ost_copy1[(ost_copy1['осталось']>0)&(ost_copy1['Cвободный запас']>0)][['Комплекс','Завод+НН','Cвободный запас','Стоимость свободного остатка','Партия']].groupby(['Комплекс','Завод+НН','Партия']).sum().reset_index(),path='C:\\Users\\Илья\\Desktop\\kazzinc\\Uploads1\\'+folder+'\\extr_xlsx\\444.xlsx')

In [ ]:
ZNB1_cop = ZNB1[ZNB1['Материал']=='070100241'].copy()

In [ ]:
potr_with_ostatok_total = get_in_backup(path_backup='backup_for_powerBI_test_no.h5',name='f0_vertical_02_20_')

In [ ]:
arr_copy = potr_copy_for_prihod[potr_copy_for_prihod['Материал']=='070100241'].copy()

In [ ]:

# управление механикой остатков(вызывать ее)
def prt_pzk(ost_st, potr_st, current_date, filters,all_months):
    #     группировка остатков по
    if all_months ==True:
        master_zav_nn = dict(tuple(potr_st[potr_st['month'] >= current_date].groupby('Заказчик + HH')))
    else:
        master_zav_nn = dict(tuple(potr_st[potr_st['month'] > current_date].groupby('Заказчик + HH')))
    pz_ost = ost_st
    # пз
#     print(master_zav_nn.keys())
    for pz_slave_row in pz_ost.index:
        df_master = ret_data_dict(pz_ost.at[pz_slave_row, 'Комплекс+Маттериал'], master_zav_nn)
        if df_master is None:
            df_master = pd.DataFrame()
        # если есть потребность у сцепки зав нн
        # pz_ost.at[pz_slave_row, 'осталось'] = 1600
        if len(df_master) > 0 and pz_ost.at[pz_slave_row, 'осталось'] > 0:
            df_master = df_master.sort_values('Дата потребности')
            # функция
            df_master = mex_prt_pzk(df_master, pz_ost, pz_slave_row, ost_st, potr_st)
            master_zav_nn[pz_ost.at[pz_slave_row, 'Комплекс+Маттериал']] = df_master
    if all_months ==True:
        master_zav_nn = dict(tuple(potr_st[potr_st['month'] >= current_date].groupby('Материал')))
    else:
        master_zav_nn = dict(tuple(potr_st[potr_st['month'] > current_date].groupby('Материал')))

    for pz_slave_row in pz_ost.index:
        if pz_ost.at[pz_slave_row, 'Комплекс'] == 'ALL1000':
            df_master = ret_data_dict(pz_ost.at[pz_slave_row, 'Материал'], master_zav_nn)
            if df_master is None:
                df_master = pd.DataFrame()
            # если есть потребность у сцепки зав нн
            # pz_ost.at[pz_slave_row, 'осталось'] = 1600
            if len(df_master) > 0 and pz_ost.at[pz_slave_row, 'осталось'] > 0:
                df_master = df_master[df_master['Комплекс'].isin(filters['one'])].sort_values('Дата потребности')
                # функция
                
                df_master = mex_prt_pzk(df_master, pz_ost, pz_slave_row, ost_st, potr_st)
                master_zav_nn[pz_ost.at[pz_slave_row, 'Материал']] = df_master


# мехника распределения остатков
def mex_prt_pzk(df_master, pz_ost, pz_slave_row, ost_st, potr_st):
#     print(df_master[['Распределение_остатака', 'Количество потребности', 'законтрактовано']])
    ostatok = pz_ost.at[pz_slave_row, 'Количество пополнения']
    len_it = len(df_master) - 1
    for ind, row_master_index in enumerate(df_master.index):

        # если покрыта вся потребность то ничего не делаем
        #         if df_master.at[row_master_index, 'Количество потребности'] == df_master.at[row_master_index, 'Распределение_прихода']:
        #             pass
        # если это последний элемент нужно записать то что осталось в остатке
        #  1 прибавляем в столбец Распределение_остатака гласного фрейма все то что осталось
        #  2 вставляем в лист остатков в столбец забрали
        #  3 вставлем в маленькие выборки аналогично
        #  4 рассчитываем сумму по цене и прибавляем к имеющейся в ячейке
        print(row_master_index)
        if len_it == ind and (df_master.at[row_master_index, 'Количество потребности'] > (df_master.at[row_master_index, 'Распределение_остатака'] + df_master.at[row_master_index, 'законтрактовано'])):
            #             price = ost_st.at[pz_slave_row, 'Сумма во ВВ'] / ost_st.at[pz_slave_row, 'Количество']
            #             summa = ostatok * price
            #             df_master.at[row_master_index, 'Сумма_распреденного_прихода'] = df_master.at[row_master_index, 'Сумма_распреденного_прихода'] + summa
            #             potr_st.at[row_master_index, 'Сумма_распреденного_прихода'] = potr_st.at[row_master_index, 'Сумма_распреденного_прихода'] + summa

            #             df_master.at[row_master_index, 'Распределение_прихода'] = df_master.at[row_master_index, 'Распределение_прихода'] + ostatok
            #             potr_st.at[row_master_index, 'Распределение_прихода'] = potr_st.at[row_master_index, 'Распределение_прихода'] + ostatok

            
            if (ostatok - (df_master.at[row_master_index, 'Количество потребности'] - df_master.at[row_master_index, 'Распределение_остатака']-df_master.at[row_master_index, 'законтрактовано']))<0:
                df_master.at[row_master_index, 'законтрактовано'] =df_master.at[row_master_index, 'законтрактовано']+ ostatok
                potr_st.at[row_master_index, 'законтрактовано'] = potr_st.at[row_master_index, 'законтрактовано']+ ostatok
                ostatok = 0
                pz_ost.at[pz_slave_row, 'осталось'] = ostatok
                ost_st.at[pz_slave_row, 'осталось'] = ostatok
#                 print(df_master[['Распределение_остатака', 'Количество потребности', 'законтрактовано']])
                return df_master
            else:
                a = df_master.at[row_master_index, 'законтрактовано']
                df_master.at[row_master_index, 'законтрактовано'] =df_master.at[row_master_index, 'законтрактовано']+(df_master.at[row_master_index, 'Количество потребности'] - df_master.at[row_master_index, 'Распределение_остатака']-a)
                potr_st.at[row_master_index, 'законтрактовано'] = potr_st.at[row_master_index, 'законтрактовано']+ (potr_st.at[row_master_index, 'Количество потребности'] - df_master.at[row_master_index, 'Распределение_остатака']-a)
                ostatok = ostatok - (df_master.at[row_master_index, 'Количество потребности'] - df_master.at[row_master_index, 'Распределение_остатака']-a)
#                 print(df_master[['Распределение_остатака', 'Количество потребности', 'avizot']])



                        # ostatok = 0
            # return df_master
        elif ostatok == 0:
            return df_master
        # если покарыта не вся потребность  и не последняя строка тогда и не хватает чтобы закрыть потребность:
        #  1 прибавляем в столбец  Распределение_остатака гласного фрейма потребностей только остаток
        #  2 зануляем остаток
        #  3 зануляем в листе остатков в столбец осталось
        #  4 вставлем в маленькие выборки аналогично
        #  5 рассчитываем сумму по цене и прибавляем к имеющейся в ячейке

        elif (df_master.at[row_master_index, 'Количество потребности'] > (df_master.at[row_master_index, 'Распределение_остатака'] + df_master.at[row_master_index, 'законтрактовано'])):



            if (ostatok - (df_master.at[row_master_index, 'Количество потребности'] - df_master.at[row_master_index, 'Распределение_остатака'] - df_master.at[row_master_index, 'законтрактовано']))<0:
                df_master.at[row_master_index, 'законтрактовано'] =df_master.at[row_master_index, 'законтрактовано']+ ostatok
                potr_st.at[row_master_index, 'законтрактовано'] = potr_st.at[row_master_index, 'законтрактовано']+ ostatok
                ostatok = 0
                pz_ost.at[pz_slave_row, 'осталось'] = 0
                ost_st.at[pz_slave_row, 'осталось'] = 0
#                 print(df_master[['Распределение_остатака', 'Количество потребности', 'законтрактовано']])
                return df_master
            else:
                a = df_master.at[row_master_index, 'законтрактовано']
                df_master.at[row_master_index, 'законтрактовано'] =df_master.at[row_master_index, 'законтрактовано']+(df_master.at[row_master_index, 'Количество потребности'] - df_master.at[row_master_index, 'Распределение_остатака']-a)
                potr_st.at[row_master_index, 'законтрактовано'] = potr_st.at[row_master_index, 'законтрактовано']+ (potr_st.at[row_master_index, 'Количество потребности'] - df_master.at[row_master_index, 'Распределение_остатака']-a)
                ostatok = ostatok - (df_master.at[row_master_index, 'Количество потребности'] - df_master.at[row_master_index, 'Распределение_остатака']-a)
#                 print(df_master[['Распределение_остатака', 'Количество потребности', 'avizot']])
#             return df_master
        if len_it == ind and (df_master.at[row_master_index, 'Количество потребности'] <= (df_master.at[row_master_index, 'Распределение_остатака'] + df_master.at[row_master_index, 'законтрактовано'])):
            df_master.at[row_master_index, 'законтрактовано'] = df_master.at[row_master_index, 'законтрактовано'] + 0
            potr_st.at[row_master_index, 'законтрактовано'] = potr_st.at[row_master_index, 'законтрактовано'] + 0
#             print(df_master[['Распределение_остатака', 'Количество потребности', 'законтрактовано']])
            return df_master

In [ ]:
filter_else['one']

In [ ]:
start_time = time.time()
potr_prt_pzk = arr_copy.copy()
prt_pzk_copy = ZNB1_cop.copy()
potr_prt_pzk.name = 'potr_prt_pzk'
prt_pzk_copy.name =  'prt_pzk_copy'
potr_prt_pzk['законтрактовано'] = 0.000000
prt_pzk_copy['осталось'] = prt_pzk_copy['Количество пополнения']
prt_pzk(ost_st=prt_pzk_copy,potr_st=potr_prt_pzk,filters=filter_else,current_date = current_date,all_months=True)
print("--- %s seconds ---" % (time.time() - start_time))

In [ ]:
arr_copy[['Количество потребности','Распределение_остатака','законтрактовано','Дата потребности']]

In [ ]:
pd.DataFrame  (arr_copy.loc[560660]).T

In [ ]:
potr_copy_for_prihod = get_in_backup(path_backup='tto.h5',name='02_20')

In [ ]:
ZNB1_cop['Количество пополнения'].sum()

In [ ]:
ZNB[ZNB['Материал']=='070100199']

In [ ]:
# проверка распределения остатка и формирование сцепки с основным листом
# работа пок онсолидаци остатков

ost_Q = ost_start[(ost_start['Cвободный запас']>0)&(ost_start['Особый запас']=='Q')]
ost_Q['Признак Q остатка'] = True

ost_Q1 = ost_start[(ost_start['Cвободный запас']==0)&(ost_start['Особый запас']!='Q')]    

ost_Q  = pd.concat([ost_Q,ost_Q1])
ost_Q.fillna(0,inplace=True)

ff = ost_block[['Завод+склад+партия+особый запас+НН','Блокированный запас','Завод+НН','Комплекс','Партия']]
ost_with_block  =  pd.merge(ost_copy1,ff,how='outer')
ost_with_block.fillna(0,inplace = True)

ost_copy1['Cвободный запас'] = ost_copy1['осталось']
ost_copy1['Стоимость свободного остатка'] = (ost_copy1['Ст-ть запаса в конце периода']    / ost_copy1['Кол-во запаса в конце периода']) * ost_copy1['осталось']
# 1составляющая нераспределенный остаток
neraspr_ostat_tab = ost_copy1[(ost_copy1['осталось']>0)&(ost_copy1['Cвободный запас']>0)][['Комплекс','Завод+НН','Cвободный запас','Стоимость свободного остатка','Партия']].groupby(['Комплекс','Завод+НН','Партия']).sum().reset_index()
neraspr_ostat_tab['Признак нераспределнного остатка'] = True
# 2составляющая нераспределенный остаток
ost_Q_tab = ost_Q[['Признак Q остатка','Комплекс','Завод+НН','Cвободный запас','Стоимость свободного остатка','Базовая ЕИ','Материал','Партия']].groupby(['Комплекс','Завод+НН','Партия']).sum().reset_index()
# ost_Q_tab['Признак Q остатка'] = 0


ost1 = neraspr_ostat_tab[['Комплекс','Завод+НН','Cвободный запас','Стоимость свободного остатка','Партия']]



# 11111111111111111111111
ost2 = ost_Q_tab[['Комплекс','Завод+НН','Cвободный запас','Стоимость свободного остатка','Партия']]
ost2['Признак Q остатка'] = True






one = pd.merge(ost_ner_q_t,neraspr_ostat_tab[['Комплекс','Завод+НН','Признак нераспределнного остатка','Партия']],how='left',on=['Комплекс','Завод+НН','Партия'])
one = one.groupby(['Комплекс','Завод+НН','Партия']).sum().reset_index()

# # 
two  = pd.merge(one,ost_Q_tab[['Комплекс','Завод+НН','Партия']],how='outer',on=['Комплекс','Завод+НН','Партия'])
stoim_bl_zap =  ost_start[['Комплекс','Завод+НН','Ст-ть/блок. запаса','Партия']].groupby(['Комплекс','Завод+НН','Партия']).sum().reset_index()


aa = stoim_bl_zap.groupby(['Комплекс','Завод+НН','Партия']).sum().reset_index()
bb = ost_with_block[['Комплекс','Завод+НН','Блокированный запас','Ст-ть/блок. запаса','Партия']]
bb['Ст-ть/блок. запаса'] = 0

ost_dlya_scepki = pd.merge(aa,bb,how='outer').fillna(0).groupby(['Комплекс','Завод+НН','Партия']).sum().reset_index()

tottal_ost  = pd.merge(two,ost_dlya_scepki,how='outer').fillna(0)

a = ost_Q[['Комплекс','Завод+НН','Признак Q остатка','Партия']]
tottal_ost = pd.concat([tottal_ost,a[a['Признак Q остатка']==True].drop_duplicates()])
# tottal_ost = pd.merge(tottal_ost,a[a['Признак Q остатка']==True].drop_duplicates(),how='left',on=['Завод+НН','Комплекс'])
tottal_ost.rename(columns={"Cвободный запас": "Распределение_остатака",'Стоимость свободного остатка':'Сумма_распреденного_остатка'},inplace=True)
ost2.rename(columns={"Cвободный запас": "Распределение_остатака",'Стоимость свободного остатка':'Сумма_распреденного_остатка'},inplace=True)
tottal_ost = pd.concat([tottal_ost,ost2])
tottal_ost = tottal_ost.reset_index(drop=True)

tottal_ost = tottal_ost.loc[tottal_ost[['Распределение_остатака','Ст-ть/блок. запаса','Сумма_распреденного_остатка','Партия']].dropna(how = 'all').index]


# tottal_ost = pd.concat([tottal_ost,ost2])

# potr_with_ostatok_total = pd.concat([potr_copy_for_prihod,tottal_ost]).fillna(0)


# работа по консолидации прихода



# raspr_prihod = prihod_copy[(prihod_copy['осталось']==0)&(prihod_copy['Количество']>0)]['Количество'].sum()
neraspr_prihod_tab =  prihod_copy[(prihod_copy['осталось']>0)&(prihod_copy['Количество']>0)]
Q_prihod_tab =   prihod_start[prihod_start['Особый запас']=='Q']

tab_ner_sc_prihod= neraspr_prihod_tab[['Комплекс','Завод+НН','Количество','Сумма во ВВ']].groupby(['Комплекс','Завод+НН']).sum().reset_index()
tab_ner_sc_prihod['Признак нераспределенного прихода'] = True
tab_Q_sc_prihod =   Q_prihod_tab[['Комплекс','Завод+НН','Количество','Сумма во ВВ']].groupby(['Комплекс','Завод+НН']).sum().reset_index()
tab_Q_sc_prihod['Признак Q прихода'] = True


tab_prihod = pd.concat([tab_ner_sc_prihod[['Комплекс','Завод+НН','Количество','Сумма во ВВ']]]).groupby(['Комплекс','Завод+НН']).sum().reset_index()
# ,tab_Q_sc_prihod[['Комплекс','Завод+НН','Количество','Сумма во ВВ']]

one_prihod = pd.merge(tab_prihod,tab_ner_sc_prihod[['Комплекс','Завод+НН','Признак нераспределенного прихода']],how='left')

# tab_prihod_with_cons = pd.merge(one_prihod,tab_Q_sc_prihod[['Комплекс','Завод+НН','Признак Q прихода']],how='left').fillna(0)

tab_prihod_with_cons = pd.concat([one_prihod,tab_Q_sc_prihod[['Комплекс','Завод+НН','Признак Q прихода']]])

tab_prihod_with_cons.rename(columns={"Количество": "Распределение_прихода",'Сумма во ВВ':'Сумма_распреденного_прихода'},inplace=True)

tab_Q_sc_prihod.rename(columns={"Количество": "Распределение_прихода",'Сумма во ВВ':'Сумма_распреденного_прихода'},inplace=True)
tab_prihod_with_cons = pd.concat([tab_Q_sc_prihod,tab_prihod_with_cons])
tab_prihod_with_cons = tab_prihod_with_cons.reset_index(drop=True)
tab_prihod_with_cons = tab_prihod_with_cons.loc[tab_prihod_with_cons[['Распределение_прихода','Сумма_распреденного_прихода']].dropna(how = 'all').index]



# работа по консолидации расхода

# raspr_rashod_tab = rashod_copy[(rashod_copy['осталось']==0)&(rashod_copy['Кол-во в ЕИ ввода']>0)]
neraspr_rashod_tab =  rashod_copy[(rashod_copy['осталось']>0)&(rashod_copy['Кол-во в ЕИ ввода']>0)]
Q_rashod_tab =  rashod_start[rashod_start['Особый запас']=='Q']

tab_ner_sc_rashod = neraspr_rashod_tab[['Комплекс','Завод+НН','Кол-во в ЕИ ввода','Сумма во ВВ']].groupby(['Комплекс','Завод+НН']).sum().reset_index()
tab_ner_sc_rashod['Признак нераспределенного расхода'] = True
tab_Q_sc_rashod =   Q_rashod_tab[['Комплекс','Завод+НН','Кол-во в ЕИ ввода','Сумма во ВВ']].groupby(['Комплекс','Завод+НН']).sum().reset_index()
tab_Q_sc_rashod['Признак Q расхода'] = True
tab_rashod = pd.concat([tab_ner_sc_rashod[['Комплекс','Завод+НН','Кол-во в ЕИ ввода','Сумма во ВВ']]]).groupby(['Комплекс','Завод+НН']).sum().reset_index()
one_rashod = pd.merge(tab_rashod,tab_ner_sc_rashod[['Комплекс','Завод+НН','Признак нераспределенного расхода']],how='left')

# tab_rashod_with_cons = pd.merge(one_rashod,tab_Q_sc_rashod[['Комплекс','Завод+НН','Признак Q расхода']],how='left').fillna(0)

tab_rashod_with_cons = pd.concat([one_rashod,tab_Q_sc_rashod[['Комплекс','Завод+НН','Признак Q расхода']]])


tab_rashod_with_cons.rename(columns={"Кол-во в ЕИ ввода": "Распределение_расхода",'Сумма во ВВ':'Сумма_распреденного_расхода'},inplace=True)


tab_Q_sc_rashod.rename(columns={"Кол-во в ЕИ ввода": "Распределение_расхода",'Сумма во ВВ':'Сумма_распреденного_расхода'},inplace=True)

tab_rashod_with_cons = pd.concat([tab_Q_sc_rashod,tab_rashod_with_cons])
# ,tab_Q_sc_rashod[['Комплекс','Завод+НН','Кол-во в ЕИ ввода','Сумма во ВВ']]

tab_rashod_with_cons = tab_rashod_with_cons.reset_index(drop=True)
tab_rashod_with_cons = tab_rashod_with_cons.loc[tab_rashod_with_cons[['Распределение_расхода','Сумма_распреденного_расхода']].dropna(how = 'all').index]


tab_rashod_with_cons['Q']  =tab_rashod_with_cons['Признак Q расхода']
tab_prihod_with_cons['Q'] = tab_prihod_with_cons['Признак Q прихода']
pre_low_part = pd.merge(tab_rashod_with_cons,tab_prihod_with_cons,how='outer',on=['Комплекс','Завод+НН','Q'])

tottal_ost['Q'] = tottal_ost['Признак Q остатка']

pre_low_part2 = pd.merge(pre_low_part,tottal_ost,how='outer',on=['Комплекс','Завод+НН','Q']).fillna(0)

arr_str_zap  = vst.groupby(['Завод+НН','Комплекс']).agg({'Количество потребности':'sum'}).reset_index()
arr_str_zap.rename(columns={'Количество потребности':'стрЗапас'},inplace=True)
arr_str_zap['Признак Q остатка'] = 0
pre_low_part3 = pd.merge(pre_low_part2,arr_str_zap,how='outer',on=['Комплекс','Завод+НН','Признак Q остатка']).fillna(0)

sprav_mat_cal['Завод+НН'] = sprav_mat_cal['Завод'].astype('str')+' #'+sprav_mat_cal['Материал'].astype('str')
sprav_mat_cal.drop_duplicates('Завод+НН',inplace=True)
sprav_mat_cal_soed = sprav_mat_cal[['Завод+НН','Имя плановика (завод)','Класс оценки','СтатусМтрл на заводе','Ведущий материал','Базовая ЕИ','Краткий текст материала']]
sprav_mat_cal_soed.rename(columns={'Имя плановика (завод)':'Плановик'},inplace=True)


pre_low_part3 = pd.merge(pre_low_part3,sprav_mat_cal_soed,how='left',on='Завод+НН')

potr_with_ostatok_total = pd.concat([potr_copy_for_prihod,pre_low_part3]).fillna(0)

# справочник для материала
mater = pd.concat([df[['Завод+НН','Материал']],NZ_start[['Завод+НН','Материал']],ZNB[['Завод+НН','Материал']],ostatok[['Завод+НН','Материал']],prihod[['Завод+НН','Материал']],rashod[['Завод+НН','Материал']]]).drop_duplicates()



# подключаем материал к общему фрейму
potr_with_ostatok_total = pd.merge(potr_with_ostatok_total.drop(['Материал'],axis=1),mater,how='left',on='Завод+НН').fillna(0)
# potr_with_ostatok_total = pd.merge(potr_with_ostatok_total.drop('БЕ',axis=1),BE,how='left',on='Завод+НН').fillna(0)

potr_with_ostatok_total.reset_index(inplace=True)


# красота








In [ ]:
# баланс количества остатка
int(potr_with_ostatok_total['Распределение_остатака'].sum()+potr_with_ostatok_total['Блокированный запас'].sum()-ostatok['Кол-во запаса в конце периода'].sum())

In [ ]:
# баланс суммы остатка
int(potr_with_ostatok_total['Ст-ть/блок. запаса'].sum() + potr_with_ostatok_total['Сумма_распреденного_остатка'].sum()-ostatok['Ст-ть запаса в конце периода'].sum())

In [ ]:
# баланс количества прихода
int(potr_with_ostatok_total['Распределение_прихода'].sum()-(raspr_prihod+neraspr_prihod+Q_prihod))

In [ ]:
#  баланс суммы прихода
int(potr_with_ostatok_total['Сумма_распреденного_прихода'].sum()-(raspr_prihod_sum+neraspr_prihod_sum+Q_prihod_sum))

In [ ]:
#  баланс суммы расхода
int(potr_with_ostatok_total['Сумма_распреденного_расхода'].sum()-(raspr_rashod_money+neraspr_rashod_money+Q_rashod_money))

In [ ]:
a = tottal_ost.copy()

In [ ]:
a['Распределение_остатака'].sum()

In [ ]:
a['Сумма_распреденного_остатка'].sum()

In [ ]:
a['Ст-ть/блок. запаса'].sum()

In [ ]:
tottal_ost

In [ ]:
# проверка распределения остатка и формирование сцепки с основным листом
# работа пок онсолидаци остатков

ost_Q = ost_start[(ost_start['Cвободный запас']>0)&(ost_start['Особый запас']=='Q')]
ost_Q['Признак Q остатка'] = True

ost_Q1 = ost_start[(ost_start['Cвободный запас']==0)&(ost_start['Особый запас']!='Q')]    

ost_Q  = pd.concat([ost_Q,ost_Q1])
ost_Q.fillna(0,inplace=True)

ff = ost_block[['Завод+склад+партия+особый запас+НН','Блокированный запас','Завод+НН','Комплекс','Партия']]
ost_with_block  =  pd.merge(ost_copy1,ff,how='outer')
ost_with_block.fillna(0,inplace = True)

ost_copy1['Cвободный запас'] = ost_copy1['осталось']
ost_copy1['Стоимость свободного остатка'] = (ost_copy1['Ст-ть запаса в конце периода']    / ost_copy1['Кол-во запаса в конце периода']) * ost_copy1['осталось']
# 1составляющая нераспределенный остаток
neraspr_ostat_tab = ost_copy1[(ost_copy1['осталось']>0)&(ost_copy1['Cвободный запас']>0)][['Комплекс','Завод+НН','Cвободный запас','Стоимость свободного остатка','Партия']].groupby(['Комплекс','Завод+НН','Партия']).sum().reset_index()
neraspr_ostat_tab['Признак нераспределнного остатка'] = True
# 2составляющая нераспределенный остаток
ost_Q_tab = ost_Q[['Признак Q остатка','Комплекс','Завод+НН','Cвободный запас','Стоимость свободного остатка','Базовая ЕИ','Материал','Партия']].groupby(['Комплекс','Завод+НН','Партия']).sum().reset_index()
# ost_Q_tab['Признак Q остатка'] = 0


ost1 = neraspr_ostat_tab[['Комплекс','Завод+НН','Cвободный запас','Стоимость свободного остатка','Партия']]



# 11111111111111111111111
ost2 = ost_Q_tab[['Комплекс','Завод+НН','Cвободный запас','Стоимость свободного остатка','Партия']]
ost2['Признак Q остатка'] = True






one = pd.merge(ost_ner_q_t,neraspr_ostat_tab[['Комплекс','Завод+НН','Признак нераспределнного остатка','Партия']],how='left',on=['Комплекс','Завод+НН','Партия'])
one = one.groupby(['Комплекс','Завод+НН','Партия']).sum().reset_index()

# # 
two  = pd.merge(one,ost_Q_tab[['Комплекс','Завод+НН','Партия']],how='outer',on=['Комплекс','Завод+НН','Партия'])
stoim_bl_zap =  ost_start[['Комплекс','Завод+НН','Ст-ть/блок. запаса','Партия']].groupby(['Комплекс','Завод+НН','Партия']).sum().reset_index()


aa = stoim_bl_zap.groupby(['Комплекс','Завод+НН','Партия']).sum().reset_index()
bb = ost_with_block[['Комплекс','Завод+НН','Блокированный запас','Ст-ть/блок. запаса','Партия']]
bb['Ст-ть/блок. запаса'] = 0

ost_dlya_scepki = pd.merge(aa,bb,how='outer').fillna(0).groupby(['Комплекс','Завод+НН','Партия']).sum().reset_index()

tottal_ost  = pd.merge(two,ost_dlya_scepki,how='outer').fillna(0)

a = ost_Q[['Комплекс','Завод+НН','Признак Q остатка','Партия']]
tottal_ost = pd.concat([tottal_ost,a[a['Признак Q остатка']==True].drop_duplicates()])
# tottal_ost = pd.merge(tottal_ost,a[a['Признак Q остатка']==True].drop_duplicates(),how='left',on=['Завод+НН','Комплекс'])
tottal_ost.rename(columns={"Cвободный запас": "Распределение_остатака",'Стоимость свободного остатка':'Сумма_распреденного_остатка'},inplace=True)
ost2.rename(columns={"Cвободный запас": "Распределение_остатака",'Стоимость свободного остатка':'Сумма_распреденного_остатка'},inplace=True)
tottal_ost = pd.concat([tottal_ost,ost2])
tottal_ost = tottal_ost.reset_index(drop=True)

tottal_ost = tottal_ost.loc[tottal_ost[['Распределение_остатака','Ст-ть/блок. запаса','Сумма_распреденного_остатка','Партия']].dropna(how = 'all').index]

f_tr = tottal_ost.apply(lambda x: True if x['Партия'][:2].isalpha() and x['Партия'][-2:].isalpha()==False else False,axis = 1)
for ind in tottal_ost[f_tr].index:
    st = tottal_ost.loc[ind,'Завод+НН'].split(' ')
    st[0] = tottal_ost.loc[ind,'Партия'][-4:]
    tottal_ost.loc[ind,'Завод+НН'] = ' '.join(st)
    
tottal_ost.drop('Партия',axis = 1,inplace = True)
tottal_ost.fillna(0,inplace = True)
ggttt = tottal_ost[f_tr].copy()
tottal_ost = tottal_ost.groupby(['Завод+НН','Комплекс','Признак Q остатка','Признак нераспределнного остатка']).sum().reset_index()


# raspr_prihod = prihod_copy[(prihod_copy['осталось']==0)&(prihod_copy['Количество']>0)]['Количество'].sum()
neraspr_prihod_tab =  prihod_copy[(prihod_copy['осталось']>0)&(prihod_copy['Количество']>0)]
Q_prihod_tab =   prihod_start[prihod_start['Особый запас']=='Q']

tab_ner_sc_prihod= neraspr_prihod_tab[['Комплекс','Завод+НН','Количество','Сумма во ВВ']].groupby(['Комплекс','Завод+НН']).sum().reset_index()
tab_ner_sc_prihod['Признак нераспределенного прихода'] = True
tab_Q_sc_prihod =   Q_prihod_tab[['Комплекс','Завод+НН','Количество','Сумма во ВВ']].groupby(['Комплекс','Завод+НН']).sum().reset_index()
tab_Q_sc_prihod['Признак Q прихода'] = True


tab_prihod = pd.concat([tab_ner_sc_prihod[['Комплекс','Завод+НН','Количество','Сумма во ВВ']]]).groupby(['Комплекс','Завод+НН']).sum().reset_index()
# ,tab_Q_sc_prihod[['Комплекс','Завод+НН','Количество','Сумма во ВВ']]

one_prihod = pd.merge(tab_prihod,tab_ner_sc_prihod[['Комплекс','Завод+НН','Признак нераспределенного прихода']],how='left')

# tab_prihod_with_cons = pd.merge(one_prihod,tab_Q_sc_prihod[['Комплекс','Завод+НН','Признак Q прихода']],how='left').fillna(0)

tab_prihod_with_cons = pd.concat([one_prihod,tab_Q_sc_prihod[['Комплекс','Завод+НН','Признак Q прихода']]])

tab_prihod_with_cons.rename(columns={"Количество": "Распределение_прихода",'Сумма во ВВ':'Сумма_распреденного_прихода'},inplace=True)

tab_Q_sc_prihod.rename(columns={"Количество": "Распределение_прихода",'Сумма во ВВ':'Сумма_распреденного_прихода'},inplace=True)
tab_prihod_with_cons = pd.concat([tab_Q_sc_prihod,tab_prihod_with_cons])
tab_prihod_with_cons = tab_prihod_with_cons.reset_index(drop=True)
tab_prihod_with_cons = tab_prihod_with_cons.loc[tab_prihod_with_cons[['Распределение_прихода','Сумма_распреденного_прихода']].dropna(how = 'all').index]



# работа по консолидации расхода

# raspr_rashod_tab = rashod_copy[(rashod_copy['осталось']==0)&(rashod_copy['Кол-во в ЕИ ввода']>0)]
neraspr_rashod_tab =  rashod_copy[(rashod_copy['осталось']>0)&(rashod_copy['Кол-во в ЕИ ввода']>0)]
Q_rashod_tab =  rashod_start[rashod_start['Особый запас']=='Q']

tab_ner_sc_rashod = neraspr_rashod_tab[['Комплекс','Завод+НН','Кол-во в ЕИ ввода','Сумма во ВВ']].groupby(['Комплекс','Завод+НН']).sum().reset_index()
tab_ner_sc_rashod['Признак нераспределенного расхода'] = True
tab_Q_sc_rashod =   Q_rashod_tab[['Комплекс','Завод+НН','Кол-во в ЕИ ввода','Сумма во ВВ']].groupby(['Комплекс','Завод+НН']).sum().reset_index()
tab_Q_sc_rashod['Признак Q расхода'] = True
tab_rashod = pd.concat([tab_ner_sc_rashod[['Комплекс','Завод+НН','Кол-во в ЕИ ввода','Сумма во ВВ']]]).groupby(['Комплекс','Завод+НН']).sum().reset_index()
one_rashod = pd.merge(tab_rashod,tab_ner_sc_rashod[['Комплекс','Завод+НН','Признак нераспределенного расхода']],how='left')

# tab_rashod_with_cons = pd.merge(one_rashod,tab_Q_sc_rashod[['Комплекс','Завод+НН','Признак Q расхода']],how='left').fillna(0)

tab_rashod_with_cons = pd.concat([one_rashod,tab_Q_sc_rashod[['Комплекс','Завод+НН','Признак Q расхода']]])


tab_rashod_with_cons.rename(columns={"Кол-во в ЕИ ввода": "Распределение_расхода",'Сумма во ВВ':'Сумма_распреденного_расхода'},inplace=True)


tab_Q_sc_rashod.rename(columns={"Кол-во в ЕИ ввода": "Распределение_расхода",'Сумма во ВВ':'Сумма_распреденного_расхода'},inplace=True)

tab_rashod_with_cons = pd.concat([tab_Q_sc_rashod,tab_rashod_with_cons])
# ,tab_Q_sc_rashod[['Комплекс','Завод+НН','Кол-во в ЕИ ввода','Сумма во ВВ']]

tab_rashod_with_cons = tab_rashod_with_cons.reset_index(drop=True)
tab_rashod_with_cons = tab_rashod_with_cons.loc[tab_rashod_with_cons[['Распределение_расхода','Сумма_распреденного_расхода']].dropna(how = 'all').index]


tab_rashod_with_cons['Q']  =tab_rashod_with_cons['Признак Q расхода']
tab_prihod_with_cons['Q'] = tab_prihod_with_cons['Признак Q прихода']
pre_low_part = pd.merge(tab_rashod_with_cons,tab_prihod_with_cons,how='outer',on=['Комплекс','Завод+НН','Q'])

tottal_ost['Q'] = tottal_ost['Признак Q остатка']

pre_low_part2 = pd.merge(pre_low_part,tottal_ost,how='outer',on=['Комплекс','Завод+НН','Q']).fillna(0)

arr_str_zap  = vst.groupby(['Завод+НН','Комплекс']).agg({'Количество потребности':'sum'}).reset_index()
arr_str_zap.rename(columns={'Количество потребности':'стрЗапас'},inplace=True)
arr_str_zap['Признак Q остатка'] = 0
pre_low_part3 = pd.merge(pre_low_part2,arr_str_zap,how='outer',on=['Комплекс','Завод+НН','Признак Q остатка']).fillna(0)

sprav_mat_cal['Завод+НН'] = sprav_mat_cal['Завод'].astype('str')+' #'+sprav_mat_cal['Материал'].astype('str')
sprav_mat_cal.drop_duplicates('Завод+НН',inplace=True)
sprav_mat_cal_soed = sprav_mat_cal[['Завод+НН','Имя плановика (завод)','Класс оценки','СтатусМтрл на заводе','Ведущий материал','Базовая ЕИ','Краткий текст материала']]
sprav_mat_cal_soed.rename(columns={'Имя плановика (завод)':'Плановик'},inplace=True)


pre_low_part3 = pd.merge(pre_low_part3,sprav_mat_cal_soed,how='left',on='Завод+НН')

potr_with_ostatok_total = pd.concat([potr_copy_for_prihod,pre_low_part3]).fillna(0)

# справочник для материала
mater = pd.concat([df[['Завод+НН','Материал']],NZ_start[['Завод+НН','Материал']],ZNB[['Завод+НН','Материал']],ostatok[['Завод+НН','Материал']],prihod[['Завод+НН','Материал']],rashod[['Завод+НН','Материал']]]).drop_duplicates()



# подключаем материал к общему фрейму
potr_with_ostatok_total = pd.merge(potr_with_ostatok_total.drop(['Материал'],axis=1),mater,how='left',on='Завод+НН').fillna(0)
# potr_with_ostatok_total = pd.merge(potr_with_ostatok_total.drop('БЕ',axis=1),BE,how='left',on='Завод+НН').fillna(0)

potr_with_ostatok_total.reset_index(inplace=True)


# красота


In [ ]:
tottal_ost

In [ ]:
tottal_ost['Распределение_остатака'].sum()

In [ ]:
tottal_ost

In [ ]:
f_tr = tottal_ost.apply(lambda x: True if x['Партия'][:2].isalpha() and x['Партия'][-2:].isalpha()==False else False,axis = 1)
for ind in tottal_ost[f_tr].index:
    st = tottal_ost.loc[ind,'Завод+НН'].split(' ')
    st[0] = tottal_ost.loc[ind,'Партия'][-4:]
    tottal_ost.loc[ind,'Завод+НН'] = ' '.join(st)
tottal_ost.fillna(0,inplace = True)
tottal_ost = tottal_ost.groupby(['Завод+НН','Комплекс','Признак Q остатка','Признак нераспределнного остатка','Q']).sum().reset_index()

In [ ]:
tottal_ost[f_tr]

In [ ]:
tottal_ost.drop('Партия',axis = 1,inplace = True)

In [ ]:
tottal_ost.info()

In [ ]:
tottal_ost.groupby(['Завод+НН','Комплекс','Признак Q остатка','Признак нераспределнного остатка','Q']).sum().reset_index()

In [ ]:
potr_with_ostatok_total[potr_with_ostatok_total['Материал']=='011001103']['Завод+НН'].unique()

In [ ]:
aassd = get_in_backup(path_backup='backup_for_powerBI_test_no.h5',name='f0_vertical_02_20_')

In [ ]:
aassd[aassd['Материал']=='011001207']['Завод+НН'].unique()

In [ ]:
ggttt

In [ ]:
save_pyex(df=ZNB,path='C:\\Users\\Илья\\Desktop\\kazzinc\\Uploads1\\'+folder+'\\extr_xlsx\\ZNB.xlsx')

In [ ]:
save_pyex(df=ZNB1,path='C:\\Users\\Илья\\Desktop\\kazzinc\\Uploads1\\'+folder+'\\extr_xlsx\\ZNB1.xlsx')

In [ ]:
save_pyex(df=NB,path='C:\\Users\\Илья\\Desktop\\kazzinc\\Uploads1\\'+folder+'\\extr_xlsx\\NB.xlsx')

In [ ]:
sprav_mat_cal_last_full[(sprav_mat_cal_last_full['Материал']=='430201525')&(sprav_mat_cal_last_full['Завод']=='4611')]

In [ ]:
df[['Завод+НН','Инициатор']].drop_duplicates()

In [ ]:
save_pyex(df=df[['Завод+НН','Инициатор']].drop_duplicates(),path='C:\\Users\\Илья\\Desktop\\kazzinc\\Uploads1\\'+folder+'\\extr_xlsx\\ggd.xlsx')

In [ ]:
c_f0[c_f0['Материал']=='910460017']

In [ ]:
potr_with_ostatok_total[potr_with_ostatok_total['Материал']=='910460017']

In [ ]:
potr_with_ostatok_total[potr_with_ostatok_total['Материал']=='910460017']

In [ ]:
trttr = tottal_ost[f_tr]

In [ ]:
trttr

In [ ]:
thtytth = pd.concat([tottal_ost,ost2])

In [ ]:
thtytth[thtytth['Завод+НН'].apply(lambda x: x.find('910460017')!=-1)]

In [ ]:
tottal_ost

In [ ]:
ost2

In [ ]:
ost2[ost2['Завод+НН'].apply(lambda x: x.find('910460017')!=-1)]

In [ ]:
ost_dlya_scepki[ost_dlya_scepki['Завод+НН'].apply(lambda x: x.find('910460017')!=-1)]

In [ ]:
df = get_in_backup(path_backup='backup.h5',name='extract')

In [ ]:
# KONH = pd.read_excel('C:\\Users\\Илья\\Desktop\\kazzinc\\KPI\\0120\\KONH.xlsx',sheet_name='Sheet1')
# KONP  = pd.read_excel('C:\\Users\\Илья\\Desktop\\kazzinc\\KPI\\0120\\KONP.xlsx',sheet_name='Sheet1')
# MARM = pd.read_excel('C:\\Users\\Илья\\Desktop\\kazzinc\\KPI\\0120\\MARM.xlsx',sheet_name='Sheet1')
# A900 = pd.read_excel('C:\\Users\\Илья\\Desktop\\kazzinc\\KPI\\0120\\A900.xlsx',sheet_name='Sheet1')
# A901 = pd.read_excel('C:\\Users\\Илья\\Desktop\\kazzinc\\KPI\\0120\\A901.xlsx',sheet_name='Sheet1')
# A902 = pd.read_excel('C:\\Users\\Илья\\Desktop\\kazzinc\\KPI\\0120\\A902.xlsx',sheet_name='Sheet1')
KONH = pd.read_excel('C:\\Users\\Илья\\Desktop\\kazzinc\\musor\\KONH.xlsx',sheet_name='Sheet1')
KONP  = pd.read_excel('C:\\Users\\Илья\\Desktop\\kazzinc\\musor\\KONP.xlsx',sheet_name='Sheet1')
MARM = pd.read_excel('C:\\Users\\Илья\\Desktop\\kazzinc\\musor\\MARM.xlsx',sheet_name='Sheet1')
A900 = pd.read_excel('C:\\Users\\Илья\\Desktop\\kazzinc\\musor\\A900.xlsx',sheet_name='Sheet1')
A901 = pd.read_excel('C:\\Users\\Илья\\Desktop\\kazzinc\\musor\\A901.xlsx',sheet_name='Sheet1')
A902 = pd.read_excel('C:\\Users\\Илья\\Desktop\\kazzinc\\musor\\A902.xlsx',sheet_name='Sheet1')

In [ ]:
def func_KONH(x):
    if x['Таблица']=='900':return x['Переменный ключ']
    elif x['Таблица']=='901':return x['Переменный ключ'][4:]
    elif x['Таблица']=='902':return x['Переменный ключ'][3:]
    else:return 'пусто'    
    
KONH.fillna(0,inplace=True)
KONP.fillna(0,inplace=True)
MARM.fillna(0,inplace=True)

KONP = KONP[KONP['Индикатор удаления']!='X']

to_str_KONH = ['Номер записи условий','Таблица','Переменный ключ','Вид условия']
for i in to_str_KONH:
    KONH[i] = KONH[i].apply(lambda x: str(x).replace('.0','')).astype('str')
    KONH[i] = KONH[i].apply(lambda x: '' if x=='0' else x)
    
to_str_KONP = ['Номер записи условий','Единица измерения','Вид условия','Валюта условия']
for i in to_str_KONP:
    KONP[i] = KONP[i].apply(lambda x: str(x).replace('.0','')).astype('str')
    KONP[i] = KONP[i].apply(lambda x: '' if x=='0' else x)
    
KONH = KONH[(KONH['Вид условия']=='ZP00')&((KONH['Таблица']=='900')|(KONH['Таблица']=='901')|(KONH['Таблица']=='902'))]   
KONH['Материал'] = KONH.apply(lambda x:func_KONH(x),axis = 1)

KONH['округ'] = KONH.apply(lambda x: x['Переменный ключ'][:2] if x['Таблица']=='902' else 'пусто',axis = 1)
KONH['Завод'] = KONH.apply(lambda x: x['Переменный ключ'][:4] if x['Таблица']=='901' else 'пусто',axis = 1)
KONH.reset_index(drop = True,inplace = True)


to_str_MARM = ['Материал','Альтернативная ЕИ']
for i in to_str_MARM:
    MARM[i] = MARM[i].apply(lambda x: str(x).replace('.0','')).astype('str')
    MARM[i] = MARM[i].apply(lambda x: '' if x=='0' else x)


for d in [A900,A901,A902]:
    d['Номер записи условий'] =  d['Номер записи условий'].apply(lambda x: str(x).replace('.0','')).astype('str')
    
fil = KONH['Номер записи условий'].isin(pd.concat([A900['Номер записи условий'],A901['Номер записи условий'],A902['Номер записи условий']]).drop_duplicates())
KONH = KONH[fil]  


In [ ]:
potr_with_ostatok_total1 = potr_with_ostatok_total[:1000].copy()

In [ ]:
potr_with_ostatok_total = parallel_calc_fast(potr_with_ostatok_total,KONH,current_date,KONP,MARM,375)

In [ ]:
import numpy as np
import time
from multiprocessing import Pool
def parallel_calc_fast(cons,KONH,current_date,KONP,MARM):
    if len([x for x in cons.columns if x=='Цена'])>0:
        cons.drop(['Цена'],axis = 1,inplace = True)
    data_frame = cons.copy()
    cons = cons[['Код округа','Материал','Завод','Базовая ЕИ']].drop_duplicates()
    cons.reset_index(inplace = True,drop = True)
    start_time = time.time()
    pool = Pool(processes=10)
    cons['Цена']=0
    cons.rename(columns={"Код округа": "округ"},inplace=True)
    KONH = KONH.reset_index(drop=True)
#     KONH_po = KONH.reset_index().set_index(['Материал',pd.Index(KONH['Действ. по'].apply(lambda x:str(x.year)))])
    KONH['key2'] = KONH['Материал']+KONH['Действ. по'].apply(lambda x: str(x.year))
#     KONH_po = KONH.reset_index().set_index(['key2'])

#     KONH_s = KONH.reset_index().set_index(['Материал',pd.Index(KONH['Действ. с'].apply(lambda x:str(x.year)))])
    KONH['key1'] = KONH['Материал']+KONH['Действ. с'].apply(lambda x: str(x.year))
#     KONH_s = KONH.reset_index().set_index(['key1'])

    KONH.rename(columns={"Номер записи условий": 'Номер_записи_условий'},inplace=True)
    KONH['sort'] = KONH['Номер_записи_условий'].astype('int')
    KONH.sort_values('sort',inplace = True,ascending = False)
    KONH_po = KONH.groupby('key2')
    KONH_s = KONH.groupby('key1')
    
    
    
    
#     KONP = KONP.set_index(['Номер записи условий'])
    KONP = KONP.groupby('Номер записи условий')
    MARM['key'] = MARM['Материал']+MARM['Альтернативная ЕИ']
#     MARM =  MARM.set_index(['Материал','Альтернативная ЕИ'])
    MARM = MARM.groupby('key')
    arrs =  np.array_split(cons,20)
    arrs =  [(arr,KONH_po,KONH_s,current_date,KONP,MARM) for arr in arrs]
    df_list = pool.map(func_read_ex.calc_price_super_fast, arrs)
    df = pd.concat(df_list)
    print("--- %s seconds ---" % (time.time() - start_time))
    pool.close()
    df.rename(columns={"округ": "Код округа"},inplace=True)
#     df = pd.concat([df.set_index(['Код округа','Материал','Завод','Базовая ЕИ']),data_frame.set_index(['Код округа','Материал','Завод','Базовая ЕИ'])],axis = 1)
    df = data_frame.merge(df,left_on=['Код округа','Материал','Завод','Базовая ЕИ'],right_on=['Код округа','Материал','Завод','Базовая ЕИ'],how='outer')
    
    df.reset_index(inplace = True,drop = True)
    
    df.drop(['номер записи условий','цена без учета коэф.пересчата','единица измерения закупки'],axis = 1,inplace = True)
    return df 

In [ ]:
potr_with_ostatok_total = potr_with_ostatok_total[potr_with_ostatok_total['Материал']=='122501050'].copy()

In [ ]:
a = potr_with_ostatok_total1.set_index(['Код округа','Материал','Завод','Базовая ЕИ'])

In [ ]:
b = potr_with_ostatok_total2.set_index(['Код округа','Материал','Завод','Базовая ЕИ'])

In [ ]:
if len([x for x in potr_with_ostatok_total1.columns if x=='Цена'])>0:
    

In [ ]:
potr_with_ostatok_total = get_in_backup(path_backup='backup_for_powerBI_test_no.h5',name='f0_vertical_02_20_')

In [ ]:
T001W = pd.read_excel(directory+'T001W.xlsx',sheet_name='Sheet1')
ZMM_E0078_MDLV = pd.read_excel(directory+'ZMM_E0078_MDLV.xlsx',sheet_name='Sheet1',dtype={'Область ППМ':'str','Код округа':'str'})

In [ ]:
spr_kod_okruga = rev_to_dict(ZMM_E0078_MDLV['Область ППМ'].apply(lambda x: str(x).replace('.0',''))\
                             .astype('str').str.strip(),ZMM_E0078_MDLV['Код округа'])
spr_kod_okruga1 = rev_to_dict(T001W['Завод'].apply(lambda x: str(x).replace('.0','')).astype('str'),\
            T001W['Код округа'].apply(lambda x: str(x).replace('.0','')).astype('str'))

potr_with_ostatok_total['Код округа'] = potr_with_ostatok_total.apply(lambda x: insert_vpr(x['Завод'],spr_kod_okruga1,'Код округа') if insert_vpr(x['Область ППМ'],spr_kod_okruga,'Код округа')==0 else insert_vpr(x['Область ППМ'],spr_kod_okruga,'Код округа'),axis = 1).astype('str')

In [ ]:
potr_with_ostatok_total['Код округа'].value_counts()

In [ ]:
for el in potr_with_ostatok_total['Код округа'].unique():
    save_pyex(df=potr_with_ostatok_total[potr_with_ostatok_total['Код округа']==el],path='C:\\Users\\Илья\\Desktop\\kazzinc\\Uploads1\\'+folder+'\\extr_xlsx\\'+ str(el)+'.xlsx')

In [ ]:
potr_with_ostatok_total[potr_with_ostatok_total['Код округа']=='nan']

In [ ]:
potr_with_ostatok_total['Код округа'].unique()

In [ ]:
df

In [ ]:
df = get_in_backup(path_backup='backup.h5',name='extract')

In [ ]:
potr_with_ostatok_total = potr_with_ostatok_total[:5000].copy()

In [ ]:
potr_with_ostatok_total['Материал'].unique()

In [ ]:
potr_with_ostatok_total[potr_with_ostatok_total['Код округа']=='KO']['Цена'].sum()

In [ ]:
potr_with_ostatok_total1 = potr_with_ostatok_total[:100000].copy()

In [ ]:
potr_with_ostatok_total1 = parallel_calc_fast(potr_with_ostatok_total1,KONH,current_date,KONP,MARM)

In [ ]:
KONH[(KONH['округ']!='пусто')|(KONH['Завод']!='пусто')]

In [ ]:
KONH['key2'] = KONH['Материал']+KONH['Действ. по'].apply(lambda x: str(x.year))
#     KONH_po = KONH.reset_index().set_index(['key2'])

#     KONH_s = KONH.reset_index().set_index(['Материал',pd.Index(KONH['Действ. с'].apply(lambda x:str(x.year)))])
KONH['key1'] = KONH['Материал']+KONH['Действ. с'].apply(lambda x: str(x.year))

In [ ]:
potr_with_ostatok_total[(potr_with_ostatok_total['Материал']=='360209004')&(potr_with_ostatok_total['Код округа']=='UK')]

In [ ]:
Konh_ = KONH_po.get_group('0601020032020')

In [ ]:
KONH.rename(columns={"Номер записи условий": 'Номер_записи_условий'},inplace=True)

In [ ]:
KONH[KONH['key2']=='2601320052020']

In [ ]:
KONH[KONH['key1']=='3602090042020']

In [ ]:
f_KONH_fast(KONH[KONH['key2']=='0501240012020'],'Завод','1301')

In [ ]:
def f_KONH_fast (KONH,name_col_KONH,value):
    i = 1
    if len(KONH)==0:
            return ('пусто','0')
    
    for row in KONH.itertuples():
        
        #if i ==len(KONH):
            #return ('пусто','0')
        if name_col_KONH=='округ':
            if row.округ == value:
                
                return (row.округ,row.Номер_записи_условий)
            i+=1
        elif name_col_KONH=='Завод':
            if row.Завод == value:
                
                return (row.Завод,row.Номер_записи_условий)
            i+=1
    return ('пусто','0')

In [ ]:
KONP[KONP['Номер записи условий']=='356373']

In [ ]:
KONP[KONP['Номер записи условий']=='356373']

In [ ]:
f_KONH_fast(KONH[KONH['key1']=='2601320052020'],'округ','KO')[1]

In [ ]:
MARM[MARM['Материал']=='060301010']

In [ ]:
start_time = time.time()
for row in KONH.itertuples():
    pass
print("--- %s seconds ---" % (time.time() - start_time))   

In [ ]:
start_time = time.time()
for row in KONH.iteritems():
    pass
print("--- %s seconds ---" % (time.time() - start_time))  

In [ ]:
start_time = time.time()
for row in KONH.itertuples():
    pass
print("--- %s seconds ---" % (time.time() - start_time))  

In [ ]:
start_time = time.time()
for row in KONH.index.values:
    pass
print("--- %s seconds ---" % (time.time() - start_time))  

In [ ]:
zav = KONH['Завод'].values
okr = KONH['округ'].values
number = KONH['Номер_записи_условий'].values

ind_ = KONH[:500].index.values

In [ ]:

def f_KONH_fast_(ind_,name_col_KONH,value,zav,okr,number):
    start_time = time.time()
    if len(ind_)==0:
            return
    for ind in ind_:
        if name_col_KONH=='округ':
            if okr[ind] == value:
                print("--- %s seconds ---" % (time.time() - start_time))   
                return (okr[ind],number[ind])
            
        elif name_col_KONH=='Завод':
            if zav[ind] == value:
                print("--- %s seconds ---" % (time.time() - start_time))   
                return (zav[ind],number[ind])
    print("--- %s seconds ---" % (time.time() - start_time))         
    return ('пусто','0')
        
        
    

In [ ]:
ind_ = KONH[KONH['key1']=='0601020032020'].index.values

In [ ]:
f_KONH_fast_(ind_,'округ','RM',zav,okr,number)

In [ ]:
f_KONH_fast_(ind_,'Завод','1402',zav,okr,number)

In [ ]:
okr[381328]

In [ ]:
len(okr)

In [ ]:
KONH = KONH.reset_index(drop=True)

In [ ]:
KONH

In [ ]:
def f_KONH_fast (KONH,name_col_KONH,value):
    start_time = time.time()
    i = 1
    if len(KONH)==0:
        return ('пусто','0')
    
    for row in KONH.itertuples():
        
        if name_col_KONH=='округ':
            if row.округ == value:
                print("--- %s seconds ---" % (time.time() - start_time))
                return (row.округ,row.Номер_записи_условий)
            i+=1
        elif name_col_KONH=='Завод':
            if row.Завод == value:
                print("--- %s seconds ---" % (time.time() - start_time))
                return (row.Завод,row.Номер_записи_условий)
            i+=1
    print("--- %s seconds ---" % (time.time() - start_time))
    return ('пусто','0')

In [ ]:
potr_with_ostatok_total1

In [ ]:
r = pd.DataFrame()

In [ ]:
r.index.values

In [ ]:
for i in r.index.values:
    pass

In [ ]:
potr_with_ostatok_total[potr_with_ostatok_total['Код округа']=='KO']['Цена'].sum()

In [ ]:
potr_with_ostatok_total[['Код округа','Материал','Завод']].drop_duplicates().reset_index(drop = True)

In [ ]:
KONP1  = pd.read_excel('C:\\Users\\Илья\\Desktop\\kazzinc\\musor\\KONP.xlsx',sheet_name='Sheet1')

In [ ]:
potr_with_ostatok_total1 = potr_with_ostatok_total[potr_with_ostatok_total['Материал']=='260132005'].copy()

In [ ]:
for ind, i in enumerate(np.array_split(potr_with_ostatok_total,3)) :
    save_pyex(df=i,path='C:\\Users\\Илья\\Desktop\\kazzinc\\Uploads1\\'+folder+'\\extr_xlsx\\'+str(ind)+'.xlsx')

In [ ]:
KONP['Индикатор удаления'].unique()

In [ ]:
KONP = KONP[KONP['Индикатор удаления']!='X']

In [ ]:
df

In [ ]:
potr_with_ostatok_total

In [ ]:
KONH

In [ ]:
def get_konp(KONP,val):
    try:
        # ar = KONP.xs(cons.loc[ind,'номер записи условий'])
        ar = KONP.get_group(val)
        

    except KeyError:
        ar = pd.DataFrame()   
        
    return ar

In [ ]:
KONP

In [ ]:
potr_with_ostatok_total1

In [ ]:
df = pd.read_csv(directory+'CSV.csv',sep = ';',parse_dates=['Дата потребности'],dtype={'Материал':'str','Завод':'str'})

In [ ]:
pd.DataFrame(df['Завод'].unique())  

In [ ]:
save_pyex(df=pd.DataFrame(df['Завод'].unique())  ,path='C:\\Users\\Илья\\Desktop\\kazzinc\\Uploads1\\'+folder+'\\extr_xlsx\\gmghmg.xlsx')

In [ ]:
potr_copy1['Распределение_остатака'].sum()

In [ ]:
potr_copy['Распределение_остатака'].sum()

In [ ]:
potr_copy1['Дата потребности'][603559]

In [ ]:
df

In [ ]:
df['month'] =    df.apply(lambda x: datetime.datetime(2199,1,1)  if x['Страховой запас']>=0 else x['month'],axis = 1)
df['Дата потребности'] =    df.apply(lambda x: datetime.datetime(2199,1,1)  if x['Страховой запас']>=0 else x['Дата потребности'],axis = 1)

In [ ]:
save_pyex(df=ost_start,path='C:\\Users\\Илья\\Desktop\\kazzinc\\Uploads1\\'+folder+'\\extr_xlsx\\gggg.xlsx')

In [ ]:
spr_komplex_ost = rev_to_dict(spr_ost['Завод']+' '+spr_ost['Склад']+' '+spr_ost['Партия'],spr_ost['Комплекс'])
ostatok['Комплекс'] = ostatok['Завод+Склад+Партия'].apply(lambda x: insert_vpr(x,spr_komplex_ost,'Комплекс') )

In [ ]:
ostatok[(ostatok['Материал']=='080402087')&(ostatok['Завод']=='1502')&(ostatok['Склад']=='1600')]

In [ ]:
spr_komplex_ost['1502 1600 FREEUK1251']

In [ ]:
potr_with_ostatok_total['Дата потребности'].unique()

In [ ]:
df['Дата потребности'].unique()

In [ ]:
KONP['Номер записи условий']

In [ ]:
df

In [ ]:
KONP

In [ ]:
df

In [ ]:
df = get_in_backup(path_backup='backup.h5',name='extract')

In [ ]:
df[(df['Вывод по документам пополнения']!=0)&(df['Страховой запас']==0)].shape

In [ ]:
df[(df['Вывод по документам пополнения']==0)&(df['Страховой запас']==0)]['Количество потребности'].sum()

In [ ]:
df = get_in_backup(path_backup='backup.h5',name='extract')

In [ ]:
df[df['Материал']=='445001556']

In [ ]:
df

In [ ]:
df_str_zap = get_in_backup(path_backup='backup.h5',name='df_str_zap')

In [ ]:
df_str_zap['Комплекс'].value_counts()

In [ ]:
df_str_zap['Заказчик + HH'].value_counts()

In [ ]:
df_str_zap['Заказчик + HH'] =df_str_zap['Комплекс'].astype('str')+' '+df_str_zap['Материал'].astype('str')

In [ ]:
ind = df_str_zap[(df_str_zap['Комплекс']==0)&(df_str_zap['Имя Завода']==0)].index

In [ ]:
df_str_zap.loc[ind,'Заказчик + HH'] = 0

In [ ]:
df_str_zap.name = 'df_str_zap'
save_backup('backup.h5',[df_str_zap])

In [ ]:
sprav_mat_cal_last_full = sprav_mat_cal_last_full[['Материал','Базовая ЕИ']].drop_duplicates().copy()
ZNB = ZNB.reset_index(drop = True)
a = []
for i in  ZNB.index:
    ar = sprav_mat_cal_last_full[(sprav_mat_cal_last_full['Материал']==ZNB.at[i,'Материал'])&(sprav_mat_cal_last_full['Базовая ЕИ']==ZNB.at[i,'ЕИ заказа на постав.'])].drop_duplicates().shape
    print(ar)
    if ar[0]==0:
        a.append((ZNB.at[i,'Материал'],ZNB.at[i,'ЕИ заказа на постав.']))
        
print(a)

In [ ]:
sprav_mat_cal_last_full = get_in_backup(path_backup='backup.h5',name='sprav_mat_cal_last_full')

In [ ]:
ZNB = pd.read_excel('E:\\ZNB.xlsx',sheet_name='Sheet1')

In [ ]:
ZNB[['Материал','ЕИ заказа на постав.']]

In [ ]:
a

In [ ]:
t = [x[0] for x in a]

In [ ]:
t

In [ ]:
b = [x[1] for x in a]

In [ ]:
pd.DataFrame({'mat':t,'ed':b}).drop_duplicates()

In [ ]:
save_pyex(df=pd.DataFrame({'mat':t,'ed':b}).drop_duplicates(),path='C:\\Users\\Илья\\Desktop\\kazzinc\\Uploads1\\'+folder+'\\extr_xlsx\\otkloneniya.xlsx')

In [ ]:
otkloneniya = pd.read_excel('E:\\otkloneniya.xlsx',sheet_name='sheet1')

In [ ]:
y = list(otkloneniya['mat'].apply(lambda x: '0'+str(x) if  len(str(x))<9 else x).astype('str').values)

In [ ]:
f0 = get_in_backup(path_backup='backup_obesp.h5',name='obesp_no_f_032020v2')

In [ ]:
final[final['Материал'].isin(y)]

In [ ]:
final = get_in_backup(path_backup='backup_obesp.h5',name='final_obesp_0320_v2')

In [ ]:
save_pyex(df=final[final['Материал'].isin(y)],path='C:\\Users\\Илья\\Desktop\\kazzinc\\Uploads1\\'+folder+'\\extr_xlsx\\otkloneniya1.xlsx')

In [8]:
directory = 'C:\\Users\\NKedrun\\PythonScriptWrapper_331b9d68-67b7-4571-9468-2168ecde6df5\\test.h5'

In [9]:
sprav_po_kompl = get_in_backup(path_backup=directory,name='test')

In [10]:
sprav_po_kompl

,Область ППМ,Код округа,Вид ОблПланирПотребн,Партия,Тип завода/области ППМ,Гора
0,1101000001,UK,2,UK1101,4,NaN
1,1101000002,RI,2,RI1101,4,NaN
2,1101000003,ZY,2,ZY1101,4,NaN
3,1101000004,AS,2,AS,4,NaN
4,1101000005,AL,2,AL,4,NaN
...,...,...,...,...,...,...
206,7891,AS,1,AS7891,4,NaN
207,7892,AL,1,AL7892,4,NaN
208,7893,AL,1,AL7893,4,NaN
209,7894000001,KG,2,KG7894,4,NaN


In [ ]:
backup.info()

In [2]:
import pandas as pd
def save_backup(path_backup,arrs):
    try:
        backup = pd.HDFStore(path_backup)
        for arr in arrs:
            backup[str(arr.name)] = arr
        backup.close()   
    except:
        backup.close()
dataset.name = 'test11'        
save_backup('test11.h5',[dataset])

NameError: name 'dataset' is not defined

In [6]:
df = pd.DataFrame(data={'col1': [1, 2], 'col2': [3, 4]})
df.to_parquet('df.parquet.gzip',
              compression='gzip',engine = 'pyarrow')  
pd.read_parquet('df.parquet.gzip')

,col1,col2
0,1,3
1,2,4


In [4]:
df.to_parquet?

In [7]:
pd.read_parquet('yy.parquet.gzip')

,date
0,Microsoft.OleDb.Date
1,Microsoft.OleDb.Date
